***GENERATED CODE FOR testdatapipeline PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def replaceValues(df, functionsData, applyOn):
    for columnData in applyOn:
        for functionData in functionsData:
            if functionData['replaceType'] == 'value':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, functionData['ReplaceWith'])
            elif functionData['replaceType'] == 'none':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", minValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", maxValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        "", functionData['ReplaceWith'])
                    df[columnData['columnName']] = df[columnData['columnName']].fillna(
                        functionData['ReplaceWith'])
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'max': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateofService', 'transformation_label': 'Extract Date'}], 'feature': 'DateofService', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateofService'}, {'feature_label': 'DateofService', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateofService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '91232.81', 'stddev': '6100.5', 'min': '67028', 'max': 'J9035', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '5.5', 'stddev': '12.4', 'min': '0000', 'max': 'RT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B58.01', 'max': 'T85.29XA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'C69.22', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'feature_label': 'PayerId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerId')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, path):
        df.to_csv("/dbfs" + eval(config)['url'], header=True, index=False)
        return True


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run testdatapipelineHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'Demodb2020', 'optionalDB': 'Demodb2020', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT * FROM\r\n(\r\n\t\t\tSELECT DISTINCT\r\n\t\t\tcd.ChargeDetailID, \r\n\t\t\tcd.DateofService, \r\n\t\t\tcd.Amount,\r\n\t\t\tcd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\n\t\t\tcptc.CPTCode,\r\n\t\t\tCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\t\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\t\tIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END), \r\n\t\t\tIsauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\t\t\tPredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\t\t\tMedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\t\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\t\t\td.DepartmentCategory,\r\n\t\t\ts.SpecialityCategory,\r\n\t\t\tlabel = 0 , \r\n\t\t\tt.PayerId\r\n\t\t\tfrom ChargeDetail cd\r\n\t\t\tinner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\n\t\t\tleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = e.PayerID  \r\n\t\t\tinner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\t\t\tinner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\n\t\t\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\t\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\t\tWHERE\r\n\t\t\tcd.DateOfService between '11/01/2020' and '12/31/2020' \r\n\t\t\tand cd.Amount >.01 and t.CloseDate is not null\r\n\t\t\tand trc.reasonType = 'Denial'  \r\n\t\t\tand tt.TransactionTypeCategory = 'Charges'\r\n\t\t\t\r\nUNION\r\n\r\nSELECT DISTINCT \r\ncd.ChargeDetailID, cd.DateOfService,  \r\ncd.Amount,\r\ncd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\ncptc.CPTCode,\r\nCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\ndc.DiagnosisCode as pdiagnosis1,\r\ndc2.DiagnosisCode as pdiagnosis2,\r\nIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END), \r\nIsauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END),   \r\nPredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\nMedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\nFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\nd.DepartmentCategory,\r\ns.SpecialityCategory,\r\nlabel = 1,\r\nt.PayerId\r\nfrom ChargeDetail cd\r\ninner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\nleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerID = e.PayerID\r\ninner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID  \r\ninner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\ninner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\ninner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\ninner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\ninner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\ninner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\ninner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\ninner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\ninner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\ninner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\nWHERE\r\ncd.DateOfService between '11/01/2020' and '12/31/2020' \r\nand cd.Amount >.01 and t.CloseDate is not null \r\nand trc.reasonType != 'Denial' \r\nand tt.TransactionTypeCategory = 'Charges' \r\n) as DND", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run testdatapipelineHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(sqlserver,spark,json.dumps( {"url": null, "source_attributes": {"host": "wsc-sqlserver1.database.windows.net", "port": "1433", "password": "G5gTW5rnKVeAZWXm", "database": "Demodb2020", "optionalDB": "Demodb2020", "user": "Numtra", "qtype": "query", "query": "SELECT * FROM\r\n(\r\n\t\t\tSELECT DISTINCT\r\n\t\t\tcd.ChargeDetailID, \r\n\t\t\tcd.DateofService, \r\n\t\t\tcd.Amount,\r\n\t\t\tcd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\n\t\t\tcptc.CPTCode,\r\n\t\t\tCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\t\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\t\tIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END), \r\n\t\t\tIsauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\t\t\tPredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\t\t\tMedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\t\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\t\t\td.DepartmentCategory,\r\n\t\t\ts.SpecialityCategory,\r\n\t\t\tlabel = 0 , \r\n\t\t\tt.PayerId\r\n\t\t\tfrom ChargeDetail cd\r\n\t\t\tinner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\n\t\t\tleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = e.PayerID  \r\n\t\t\tinner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\t\t\tinner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\n\t\t\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\t\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\t\tWHERE\r\n\t\t\tcd.DateOfService between '11/01/2020' and '12/31/2020' \r\n\t\t\tand cd.Amount >.01 and t.CloseDate is not null\r\n\t\t\tand trc.reasonType = 'Denial'  \r\n\t\t\tand tt.TransactionTypeCategory = 'Charges'\r\n\t\t\t\r\nUNION\r\n\r\nSELECT DISTINCT \r\ncd.ChargeDetailID, cd.DateOfService,  \r\ncd.Amount,\r\ncd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\ncptc.CPTCode,\r\nCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\ndc.DiagnosisCode as pdiagnosis1,\r\ndc2.DiagnosisCode as pdiagnosis2,\r\nIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END), \r\nIsauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END),   \r\nPredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\nMedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\nFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\nd.DepartmentCategory,\r\ns.SpecialityCategory,\r\nlabel = 1,\r\nt.PayerId\r\nfrom ChargeDetail cd\r\ninner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\nleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerID = e.PayerID\r\ninner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID  \r\ninner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\ninner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\ninner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\ninner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\ninner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\ninner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\ninner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\ninner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\ninner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\ninner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\nWHERE\r\ncd.DateOfService between '11/01/2020' and '12/31/2020' \r\nand cd.Amount >.01 and t.CloseDate is not null \r\nand trc.reasonType != 'Denial' \r\nand tt.TransactionTypeCategory = 'Charges' \r\n) as DND", "dbtype": "mssql", "is_header": "Use Header Line", "server_url": "/numtraPlatform/NumtraPlatformV2/uploads/platform/"}, "DataPrepFile": null, "data_source": "mssql", "startListenerOnly": 1, "FilePath": "/FileStore/platform/extra/60891549a187c956b337cf4c1619597314/0part.csv", "requestRatio": 40.0, "totalRows": 20104, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 21, "numberOfRows": 20104, "duplicateRowCount": 0, "stats": [{"column": "ChargeDetailID", "alias": "ChargeDetailID", "generated": 0, "type": "String", "max": "FFFFCA42-A3B8-448D-B94D-DD87D0C5C456", "min": "0008EE4F-60AB-4CC9-BF33-2BFC979E6942", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DateofService", "alias": "DateofService", "generated": 0, "type": "String", "max": "2020-12-31T00:00:00.000Z", "min": "2020-11-01T00:00:00.000Z", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Amount", "alias": "Amount", "generated": 0, "type": "real", "max": 19000.0, "min": 0.02, "mean": 362.9416260445681, "missing": 0.0, "stddev": 842.65}, {"column": "PatientBal", "alias": "PatientBal", "generated": 0, "type": "real", "max": 3600.0, "min": -1077.5, "mean": -1.0354785117389578, "missing": 0.0, "stddev": 43.79}, {"column": "InsuranceBal", "alias": "InsuranceBal", "generated": 0, "type": "real", "max": 3869.31, "min": -2926.14, "mean": 1.0199039992041383, "missing": 0.0, "stddev": 62.55}, {"column": "PrimaryInsuranceBal", "alias": "PrimaryInsuranceBal", "generated": 0, "type": "real", "max": 3000.0, "min": -2926.14, "mean": 0.6188838042180661, "missing": 0.0, "stddev": 48.13}, {"column": "CPTCode", "alias": "CPTCode", "generated": 0, "type": "String", "max": "NSF", "min": "65105", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "CptModifier", "alias": "CptModifier", "generated": 0, "type": "String", "max": "TC", "min": "0000", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "pdiagnosis1", "alias": "pdiagnosis1", "generated": 0, "type": "String", "max": "Z98.890", "min": "A51.43", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "pdiagnosis2", "alias": "pdiagnosis2", "generated": 0, "type": "String", "max": "Z98.890", "min": "A51.43", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "IsBillable", "alias": "IsBillable", "generated": 0, "type": "real", "max": 1.0, "min": 1.0, "mean": 1.0, "missing": 0.0, "stddev": 0.0}, {"column": "Isauthorized", "alias": "Isauthorized", "generated": 0, "type": "real", "max": 1.0, "min": 0.0, "mean": 0.00019896538002387584, "missing": 0.0, "stddev": 0.01}, {"column": "PredeterminationIndicator", "alias": "PredeterminationIndicator", "generated": 0, "type": "real", "max": 1.0, "min": 0.0, "mean": 0.00019896538002387584, "missing": 0.0, "stddev": 0.01}, {"column": "MedNecessityRequiredIndicator", "alias": "MedNecessityRequiredIndicator", "generated": 0, "type": "real", "max": 1.0, "min": 0.0, "mean": 0.00019896538002387584, "missing": 0.0, "stddev": 0.01}, {"column": "Age", "alias": "Age", "generated": 0, "type": "real", "max": 108.0, "min": 3.0, "mean": 68.79606048547552, "missing": 0.0, "stddev": 15.42}, {"column": "Gender", "alias": "Gender", "generated": 0, "type": "String", "max": "M", "min": "F", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DepartmentCategory", "alias": "DepartmentCategory", "generated": 0, "type": "String", "max": "Pulmonary", "min": "Cardiology", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "SpecialityCategory", "alias": "SpecialityCategory", "generated": 0, "type": "String", "max": "Ophtamology", "min": "Ophtamology", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "label", "alias": "label", "generated": 0, "type": "real", "max": 1.0, "min": 0.0, "mean": 0.9347393553521687, "missing": 0.0, "stddev": 0.25}, {"column": "PayerId", "alias": "PayerId", "generated": 0, "type": "String", "max": "FF58D17E-F6AC-4390-999A-AE5378E83883", "min": "012FD8A9-37BE-4D13-B808-7E6742223444", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Gender_Recoded", "alias": "Gender_Recoded", "generated": 1, "type": "String", "max": "2", "min": "1", "mean": "", "missing": 0.0, "stddev": ""}]}, "HasBasicStats": 1, "functionsApplied": [{"functionName": "replaceBy", "applyOn": [{"columnName": "Gender", "type": "String", "min": "F", "max": "M", "mean": "-"}], "functionsData": [{"replaceType": "value", "toReplace": "F", "replaceValueType": "byValue", "ReplaceWith": "1", "asNewColumn": 1, "newColumnName": "Gender_Recoded"}, {"replaceType": "value", "toReplace": "M", "replaceValueType": "byValue", "ReplaceWith": "2", "asNewColumn": 1, "newColumnName": "Gender_Recoded"}]}], "functionChanges": [{"columnName": "Gender", "functionName": "Replace Outliers", "Type": "String", "Parameters": [{"replaceType": "value", "toReplace": "F", "replaceValueType": "byValue", "ReplaceWith": "1", "asNewColumn": 1, "newColumnName": "Gender_Recoded"}, {"replaceType": "value", "toReplace": "M", "replaceValueType": "byValue", "ReplaceWith": "2", "asNewColumn": 1, "newColumnName": "Gender_Recoded"}]}], "fileheader": [{"field": "ChargeDetailID", "alias": "ChargeDetailID", "generated": 0, "position": 1, "type": "String"}, {"field": "DateofService", "alias": "DateofService", "generated": 0, "position": 2, "type": "String"}, {"field": "Amount", "alias": "Amount", "generated": 0, "position": 3, "type": "real"}, {"field": "PatientBal", "alias": "PatientBal", "generated": 0, "position": 4, "type": "real"}, {"field": "InsuranceBal", "alias": "InsuranceBal", "generated": 0, "position": 5, "type": "real"}, {"field": "PrimaryInsuranceBal", "alias": "PrimaryInsuranceBal", "generated": 0, "position": 6, "type": "real"}, {"field": "CPTCode", "alias": "CPTCode", "generated": 0, "position": 7, "type": "String"}, {"field": "CptModifier", "alias": "CptModifier", "generated": 0, "position": 8, "type": "String"}, {"field": "pdiagnosis1", "alias": "pdiagnosis1", "generated": 0, "position": 9, "type": "String"}, {"field": "pdiagnosis2", "alias": "pdiagnosis2", "generated": 0, "position": 10, "type": "String"}, {"field": "IsBillable", "alias": "IsBillable", "generated": 0, "position": 11, "type": "real"}, {"field": "Isauthorized", "alias": "Isauthorized", "generated": 0, "position": 12, "type": "real"}, {"field": "PredeterminationIndicator", "alias": "PredeterminationIndicator", "generated": 0, "position": 13, "type": "real"}, {"field": "MedNecessityRequiredIndicator", "alias": "MedNecessityRequiredIndicator", "generated": 0, "position": 14, "type": "real"}, {"field": "Age", "alias": "Age", "generated": 0, "position": 15, "type": "real"}, {"field": "Gender", "alias": "Gender", "generated": 0, "position": 16, "type": "String"}, {"field": "DepartmentCategory", "alias": "DepartmentCategory", "generated": 0, "position": 17, "type": "String"}, {"field": "SpecialityCategory", "alias": "SpecialityCategory", "generated": 0, "position": 18, "type": "String"}, {"field": "label", "alias": "label", "generated": 0, "position": 19, "type": "real"}, {"field": "PayerId", "alias": "PayerId", "generated": 0, "position": 20, "type": "String"}, {"field": "Gender_Recoded", "alias": "Gender_Recoded", "generated": 1, "position": 21, "type": "String"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run testdatapipelineHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "010ED4A3-EB64-4AE2-99CE-B458B59B538C", "max": "FFE0153F-9565-4FBB-9B1E-FBBA52624192", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "DateofService", "transformation_label": "Extract Date"}], "feature": "DateofService", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateofService"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "357.38", "stddev": "823.57", "min": "45.0", "max": "6688.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-1.54", "stddev": "18.4", "min": "-250.1", "max": "123.41", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PatientBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.27", "stddev": "3.94", "min": "0.0", "max": "76.68", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InsuranceBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PrimaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.15", "stddev": "3.43", "min": "0.0", "max": "76.68", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PrimaryInsuranceBal"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "91232.81", "stddev": "6100.5", "min": "67028", "max": "J9035", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "CptModifier", "transformation_label": "String Indexer"}], "feature": "CptModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "5.5", "stddev": "12.4", "min": "0000", "max": "RT", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B58.01", "max": "T85.29XA", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "C69.22", "max": "Z98.890", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsBillable", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IsBillable"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Isauthorized", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Isauthorized"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredeterminationIndicator", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredeterminationIndicator"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MedNecessityRequiredIndicator", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MedNecessityRequiredIndic..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "67.76", "stddev": "16.27", "min": "5.0", "max": "102.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Cardiology", "max": "Pulmonary", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Ophtamology", "max": "Ophtamology", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "label", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.91", "stddev": "0.29", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "label"}, {"transformationsData": [{"feature_label": "PayerId", "transformation_label": "String Indexer"}], "feature": "PayerId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "0375F08F-43D4-44BF-AB09-D3A109DAEAC2", "max": "F71F64B6-E8D9-4A9C-A7E3-A518F957DE52", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Gender_Recoded", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.45", "stddev": "0.5", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Gender_Recoded"}, {"feature": "ChargeDetailID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "249.5", "stddev": "144.48", "min": "0.0", "max": "499.0", "missing": "0"}, "updatedLabel": "ChargeDetailID_stringinde..."}, {"feature": "DateofService_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "14.58", "stddev": "8.27", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "DateofService_dayofmonth"}, {"feature": "DateofService_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "11.44", "stddev": "0.5", "min": "11", "max": "12", "missing": "0"}, "updatedLabel": "DateofService_month"}, {"feature": "DateofService_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "DateofService_year"}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "2.18", "stddev": "3.55", "min": "0.0", "max": "22.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "CptModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "0.42", "stddev": "0.98", "min": "0.0", "max": "6.0", "missing": "0"}, "updatedLabel": "CptModifier_stringindexer..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "33.73", "stddev": "38.52", "min": "0.0", "max": "148.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "35.52", "stddev": "41.1", "min": "0.0", "max": "155.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.45", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "0.91", "stddev": "1.2", "min": "0.0", "max": "7.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}, {"feature": "PayerId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "4.89", "stddev": "9.31", "min": "0.0", "max": "50.0", "missing": "0"}, "updatedLabel": "PayerId_stringindexer_tra..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run testdatapipelineHooks.ipynb
try:
	#sinkPreExecutionHook()

	dbfssink = DBFSConnector.put(df, "{'url': '/Surekha_DataPipe/undefined.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'source_attributes': {'AutoFE': {'FE': [{'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'max': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'transformationsData': [{'feature_label': 'DateofService', 'transformation_label': 'Extract Date'}], 'feature': 'DateofService', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False, 'updatedLabel': 'DateofService'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Amount', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '357.38', 'stddev': '823.57', 'min': '45.0', 'max': '6688.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'Amount'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PatientBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '-1.54', 'stddev': '18.4', 'min': '-250.1', 'max': '123.41', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientBal'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'InsuranceBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.27', 'stddev': '3.94', 'min': '0.0', 'max': '76.68', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'InsuranceBal'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PrimaryInsuranceBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.15', 'stddev': '3.43', 'min': '0.0', 'max': '76.68', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryInsuranceBal'}, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '91232.81', 'stddev': '6100.5', 'min': '67028', 'max': 'J9035', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '5.5', 'stddev': '12.4', 'min': '0000', 'max': 'RT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'B58.01', 'max': 'T85.29XA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'C69.22', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'IsBillable', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '1.0', 'stddev': '0.0', 'min': '1.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'IsBillable'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Isauthorized', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'Isauthorized'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PredeterminationIndicator', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'PredeterminationIndicator'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'MedNecessityRequiredIndicator', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'MedNecessityRequiredIndic...'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Age', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '67.76', 'stddev': '16.27', 'min': '5.0', 'max': '102.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'Age'}, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'label', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.91', 'stddev': '0.29', 'min': '0.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'label'}, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Gender_Recoded', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '500', 'mean': '1.45', 'stddev': '0.5', 'min': '1', 'max': '2', 'missing': '0'}, 'updatedLabel': 'Gender_Recoded'}, {'feature': 'ChargeDetailID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '249.5', 'stddev': '144.48', 'min': '0.0', 'max': '499.0', 'missing': '0'}, 'updatedLabel': 'ChargeDetailID_stringinde...'}, {'feature': 'DateofService_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '14.58', 'stddev': '8.27', 'min': '1', 'max': '31', 'missing': '0'}, 'updatedLabel': 'DateofService_dayofmonth'}, {'feature': 'DateofService_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '11.44', 'stddev': '0.5', 'min': '11', 'max': '12', 'missing': '0'}, 'updatedLabel': 'DateofService_month'}, {'feature': 'DateofService_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '2020.0', 'stddev': '0.0', 'min': '2020', 'max': '2020', 'missing': '0'}, 'updatedLabel': 'DateofService_year'}, {'feature': 'CPTCode_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '2.18', 'stddev': '3.55', 'min': '0.0', 'max': '22.0', 'missing': '0'}, 'updatedLabel': 'CPTCode_stringindexer_tra...'}, {'feature': 'CptModifier_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '0.42', 'stddev': '0.98', 'min': '0.0', 'max': '6.0', 'missing': '0'}, 'updatedLabel': 'CptModifier_stringindexer...'}, {'feature': 'pdiagnosis1_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '33.73', 'stddev': '38.52', 'min': '0.0', 'max': '148.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis1_stringindexer...'}, {'feature': 'pdiagnosis2_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '35.52', 'stddev': '41.1', 'min': '0.0', 'max': '155.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis2_stringindexer...'}, {'feature': 'Gender_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '500', 'mean': '0.45', 'stddev': '0.5', 'min': '0', 'max': '1', 'missing': '0'}, 'updatedLabel': 'Gender_stringindexer_tran...'}, {'feature': 'DepartmentCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '0.91', 'stddev': '1.2', 'min': '0.0', 'max': '7.0', 'missing': '0'}, 'updatedLabel': 'DepartmentCategory_string...'}, {'feature': 'SpecialityCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0'}, 'updatedLabel': 'SpecialityCategory_string...'}, {'feature': 'PayerId_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '4.89', 'stddev': '9.31', 'min': '0.0', 'max': '50.0', 'missing': '0'}, 'updatedLabel': 'PayerId_stringindexer_tra...'}], 'dataPercentage': '10', 'samplecount': 500, 'originalcount': 18968, 'FEGraph': [{'values': {'label': ['174DA322-870E-429F-A9A9-BE22340E9836', 'A1575911-0E83-4479-B182-D26EDAD439F6', 'A87EF1E9-E636-4742-8E2B-414477926782', '1FFF3E45-B9F4-4CFB-8580-C4B05D4822DD', '20BC5C22-2C34-494C-B75F-55BDD8B54263', '210DAE19-9257-4180-9057-E5817F555A88', '3FA21D4D-05B5-4C43-A33E-A01D20B45A99', 'DC65D7C3-0822-4E54-86EF-DC7639D38C62', '11327DC4-1599-46B2-A000-C05D135E1F91', '853B5BAB-0A5F-4B44-85E0-FEE0AA663796', '92F7294B-4F6C-41B5-A9BD-65CE72B27FC0', '97BB8C89-8E47-400B-B546-B662E9EDCA00', '9C7B2481-4469-4FB8-AC5D-18EB2D29F213', 'BCE331FC-BC9E-43B2-B9C3-1ADE95CF4081', '0391FBA9-7748-4F50-8CA2-FC4EB5116F6A', '88D923C8-5217-46AF-887E-CB88A2E6C3E3', '8AAE78A7-7725-4C1C-B55C-82B41A364A2B', 'B5C5BD20-64AE-4166-8115-14DA13624764', '343273F8-E598-4815-A700-0683D31F3291', '50F7ADC5-2166-4461-B01E-8D8FAA92304B', 'A9EC659A-ED07-4B6C-84CA-D4E45411C3C6', 'DCBDA7E0-A5F8-4D0F-836D-66187FEB5B6F', 'E907166F-CE0F-49D2-A04B-40734F1A6F0C', '6518F89E-CD31-4903-89E2-2F31BEB28508', '7B41BFB4-B1D7-47A9-9659-6E8319429423', '7E4D4049-B1BC-4135-8020-D69318B66B18', '94FEF155-5ED7-4DF9-836F-78557484B8E4', 'A6B1DE4F-2FBD-4AB9-B09E-77B3E8C44771', 'B9F344F0-CEE0-4EA2-A55C-274FAB39E9DB', 'BBE4E59F-B282-490E-8FCD-6C9F5E8069EB', 'EC206AC2-403A-4047-B82E-82C383E29CCE', '01867531-8532-4FD4-BE17-232F39CFA32D', '2ECD88B7-05A0-4959-8736-397F8CF74784', '524D7B98-2D7C-458D-8877-35CDE8F3D981', '54E3991B-1609-470D-8095-C14FF5A9444D', '8F8BDCC6-5B74-4EBA-A154-8576DE0FFC13', 'A28023B3-9D64-44D3-BD13-2A22A1350A24', '1B71D2B0-C473-4A8D-9FA0-76A0692C5102', '6CF3754A-BC46-430C-A116-9329959CD556', 'A6CED81D-7B1D-441A-8EFC-544E8D31FD26', 'A82270CF-0B59-4C62-877D-9FAF720B6E5D', 'D9A3771E-E97A-4EBE-BFA9-54577F445F21', '1AA261AC-C877-4F0D-8372-F268A4D5F90B', '2B385821-C105-42CA-84F0-CEF8BFD1681A', '9D5CF424-8DA6-4A2B-8716-FDB93B808F3C', 'A17DF3A2-8077-49B4-A114-11F77F67E7C9', '0E47C422-450E-4A04-9B90-680F296C9686', '46CB6A8C-362F-4345-941E-842F429B48B5', '8CD490B4-CEBE-4DFE-BA0A-C805ACDF0D34', '9F54BD25-7C67-454A-B100-CD9974B62C7D', 'A78BD078-2165-4C68-B398-9AB6E94C0FB6', 'C72EBDAD-1D1D-40D0-BA4D-A9CDD08430A6', 'DBFEC39B-37B2-4B37-9D95-A444C27A619C', 'E1C8F5F6-2DDF-4370-B40C-D86322919657', 'EB7C11AD-0A01-4359-B1AB-289373C669DC', '16062E35-A49E-4227-A3E6-E8DDBFEBA5D2', '20416F30-920B-4925-8642-C83F38ECFCEE', '642D42E4-275F-4E86-8378-21E2CD291224', 'FBB301BB-1560-415C-91B5-A6E8B0B26B6C', '4011D505-BE33-4A24-AD40-240BFDDB2766', '74FC2B0E-AEC7-4263-951F-14E72EF044DA', 'A5881100-AA89-4D5D-80AC-AE2419271C5D', 'D0667457-EF74-4FA4-91D9-7169C64A53DB', 'E16D36F3-C4D2-4C4B-A004-E06C823C7960', '20463C7A-45B7-4BD4-A3EA-53ACC03E93F1', '6E834EA0-8A83-4326-8FAE-80C6DEE3B045', '8607EC2B-26F1-4A40-9646-5990D37EC622', 'A050F893-D9BB-4FB6-9B4B-B7AA83355D60', 'C2F471A9-22FA-4A9A-A14C-49A5F292662D', 'D6F6C707-D42D-4485-A571-82EEF5002E64', 'E164830A-C381-4AEC-81EE-3B116E56B496', '2701DB53-B3A0-466C-BFCB-6684D7059DC4', '89D7AAB0-BF88-493E-A7AB-7D6D793273DB', '8A07AC76-56FD-4E13-BFEA-A724E4C2D603', '8E5FAFBE-87F5-4C26-88EA-002384A50B1D', 'E67C3D6F-A4D2-4EA9-B41A-E5BEB54E0209', 'AC68E17B-9EBB-45A3-BD22-60367A203515', 'D0BA312C-C42D-4695-A6E4-71096BE6854B', '50C6C060-08F2-4F75-91BE-EF1D34F6EDA9', 'C99C5CC6-DF6E-4024-88A6-04D326F6D6B2', '1A70E6C3-0B4F-4338-A15C-903F552EC30C', '279BE041-8831-43CF-A0EB-219F709A986F', '3E0A6D69-9B07-4BFC-B389-24E0C884701E', '9017B6B5-6C3F-42FC-ABD4-67A54E07D67F', 'F16597BA-3D84-47E3-AF6C-072ECBD33467', '2AA7F9ED-3694-4ADD-91A8-3EFB8F8054FB', '45F8D74C-DE28-4F0D-B781-AB81CCD5D5CD', 'D7B8BD7C-A3F9-422B-B491-59F236BADE4D', 'E9DB7A1D-67B8-4936-B7C7-FB071EA72907', '2B6E92D7-EA15-4F53-B133-2A1F9B1E958C', '86E8386B-2651-4928-8CD1-631CE24D0432', '97D21C13-7AFC-4865-888F-5E92CC51F2ED', 'A40F161A-A931-41D9-A65B-597347BC9D6A', 'C43925F7-AA9E-419B-B0DB-0A044E67C4B5', 'D05F93BC-037E-4BD7-8E65-FDA46391791F', '14167C30-8123-4DCF-B3C5-05D3F16B6391', '2022006D-F4D6-4F8A-B76D-1617C97083CA', '25EF819E-5F4C-43DA-BEBB-C4FAD71B0EF5', '8648E212-02DB-4D77-BB02-88B67C56A9DE', 'EDDC8ACA-6742-4C6C-A200-FA42DD89285D', '03E0DCDC-A071-4987-A7CE-268D9C3D017B', '161B0D6C-5A50-4EEE-9B2A-E4DE8D9EBC1C', '1BD4EAF6-F3F8-4369-93B3-4E6144A69CAD', '20AC5B30-ACB1-4714-9794-5454B92C1A23', '3FE9456A-15CB-4840-90CB-05F7DF34CACE', '42F5D00A-6F59-4F63-ACCA-9EFDA0DD9194', '6E80EF7D-E761-4720-A1BD-9E2CA7FDD21F', '734CA534-2CD7-4A77-9D03-CF2390E8B0D6', 'B0501331-13EF-4126-B052-44D67EDFE7F9', 'C0B29F0F-BBE1-47C7-B625-0989FDCC5288', '136261A4-52B0-43B5-B099-1EF929506A93', '252D8E17-A9E8-48F5-84D3-6DA9B452EE75', '5A8270A6-53BD-4916-8805-B762FF67BE20', '67841660-18E4-45DE-B386-285C53582762', '9C60FA6D-FAAF-46A9-A149-1F941986D089', 'D6FE7A69-6B2B-495F-A36A-789DF588F6D5', 'D9C0CCE1-6D5D-4DB7-8376-D8562C150145', '42B818D0-170D-4838-BC4A-7D22039C0F53', '7DC8A130-8030-4915-8BF5-54015B27DAB6', 'A3555B0C-5A99-4619-B6E8-9831D99F0210', 'AECE54B4-2AE0-44A1-9409-2BFB52D07BD2', 'BE7612D0-7548-49C7-800E-173BAF9DF290', '0EB85789-CE25-4CCC-850E-44D2C3496272', '45F8ABD8-1D29-495F-B63D-866175B861A0', 'A69FF747-5E5E-44A3-A2AE-806C56D75743', 'B1C24F17-5A6A-4D76-A42E-A273CF94898F', '41D99DEA-233E-47A6-98C0-8B6F522D1B70', '53BD3672-31BA-4CE7-B2F3-2AFFC29C5F6F', '55F86AF5-F804-4A84-90ED-12E4125E1A8D', '8C94AF61-5EC3-44D7-B4B3-BBEB64AD2A23', 'B52B5682-9815-4EC1-A594-912A9724EB74', '079F8DAC-D8C1-48FA-9743-BC2AA08E287C', '2D60073D-3E0E-4347-B0A6-7E0CE91E92FA', '99772A77-B764-48FF-9242-5700C9572B65', 'E45AB682-6C1F-4B84-A1AC-3D1CD79AC3A9', '51526EAE-3FB7-4C7E-9A33-26990B70EFFE', '540B2A5F-11A9-4F0F-8038-B5A1ECE5E583', '91D48DF5-F63A-466A-AFB9-2E65C4F2D44D', 'ADF5D273-DB58-4F7F-8DF0-9D9D9647DA16', 'F672AF9B-E361-4572-A9C0-8B0731F9CD3E', '34803FD1-6C40-4323-8657-F1FCC3E34990', '5327CFC0-5C48-435B-98EB-A76AAE858941', 'D2DF16C4-D3AB-4E5C-9350-8F6FDE2EDA8D', 'E2DBCD71-B901-41F3-97B1-ED71030C7F2F', '049BA1F1-F362-48BE-9393-C05CCEA8694B', '13002A76-350E-4525-BDA1-9F181E056F74', '167E1195-CC7A-4F8C-8D91-9577BCBEE8FB', '1F264D61-D5D7-4505-9AC2-25ACF275E3BD', '23BCDE27-9F80-48E4-B29D-348C871829B3', 'A71A1D2F-3F1A-4CA6-AA98-E5D6A855C250', 'B62DA993-E320-4CEE-9D23-B3238CA5759A', 'B6375BF2-EFA5-44C9-A05C-F71DACD5F8A8', 'C924A1C9-2E9A-4BF3-8BC7-25BFAEB02CE1', 'D2300410-689E-453C-895D-FC7FDA559759', 'F12A6DBF-FBB4-4C62-8EE0-0F677D58EC3B', 'FA9EEE47-7F76-4264-9A82-9B370BCF9534', 'FBBC8CD5-C35A-42E5-AEBA-C5F61050B726', '045700E8-F572-417C-A8DD-41E76C5C159A', '1A088AE8-D8BA-404D-97E5-9DB1DFB972F8', '1C109ACE-6610-4098-BE1A-C64F7805EBCA', '25251236-8157-4EE2-89CD-C5A600E278BE', 'A1818C97-82BE-4115-8C3C-3532CB8DE838', 'AC9C3FBE-B694-4A6F-9A6F-6CC4563C5C73', 'B5E35D7B-A0F8-4A47-B72F-1D372236FC63', 'C1119267-721A-4606-9D06-09FF240505EC', 'C3339BC7-4E2E-4E0E-822E-7EE8E489451C', 'CDDFE064-998C-4CDD-97A2-04647A33EE7A', 'D4930936-02DA-4DBB-A3B2-5391AC5D7153', 'FD8029C5-4ACA-4F97-8CE6-144AE49C8BCB', '4C3D48F4-860F-49EF-812D-75B08DC1359C', '58207A31-753A-42B1-86D8-E750AEA10079', 'EAF0CF6B-9569-41B6-8B77-6335F34B3D7E', '693B32F4-ECA0-4844-80DF-6D9C94DD57BA', '7E2F8EA5-4A1D-4490-A208-49CE684BB427', '83A30AD9-1329-4675-99E9-AF5ACBE98CFA', '84F4A744-F946-4B1E-99DF-20E0538C5F4F', '90C2C4BC-4F1D-4730-8078-6CF025EB493F', 'AE214CAC-A747-4549-B4E0-957075E94F38', 'BD337B93-0D12-4AE2-9566-35705B577186', 'CE11AF94-7EB4-4B98-BC57-51593FD36B72', '1F6C5DD9-4889-43B7-8768-BA1D2873322F', '322A11BB-2B77-480A-BABA-2FD89CDD0305', 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', '3342F086-2D91-4081-BEE0-C18AEAB0C981', 'C4FFF996-0FB6-427D-9266-BE10E9F5DAA9', '510C2305-6CB3-460D-84C7-9EB6281F8E9D', 'FA7AC95E-1961-4A7B-AC57-3FD3212BAF20', '04CCB591-8B59-4CCB-9EB0-5A4AECB3C705', '0D66C132-CD95-442B-800E-E19F2651E018', '0ECEF203-5A40-428B-A19E-BFA699806763', '2A1A46A7-1E50-41C1-ADD1-EBC0F82168F9', '6C6C6230-52AE-4590-AFA7-42A14991BD69', '71B8A2A8-14EC-4B19-A4DA-6DE30E8A6E30', '2B3AACB7-A045-4E46-A2DB-8B841AD87F65', '2FCDBCD0-49FC-4AA6-B249-041E476D766A', '3DF7C0B1-EA0D-4014-916E-E76984AFDE2A', '8A1DE8B3-2FDE-4581-AC94-4EA74A997CB0', 'F49D6604-F705-480F-BCB9-E6F16B76DACF', '31C70405-0F52-4BD8-A785-13AFD15809A6', '323DACC7-ED74-42A3-A1E6-21BC104AAD51', '35841CCE-C3B1-4652-B011-C44F79BB442D', '8875D93D-866D-4897-81C3-67DB4570B102', 'A7A12FBC-72F0-4216-8360-126AF826214C', 'BDAADB25-9498-4D4F-809E-4590BAEE53B5', '0438621D-9EC2-4530-B3E8-F0C68FD8295F', '55E3B0D0-8E40-48F3-854B-A8E3C893E803', '793B38EA-690C-456B-9597-5444E6D1499F', '899B4439-339E-4CC1-AC1E-4C1F4BA312EB', 'A4CD837B-6B10-47C0-B9DC-E8D6ADF73B88', 'CC817F76-EE46-4AD5-9911-86A53FB1DBAE', 'F1C5E648-7E44-43B6-A884-48A861BA52BB', '3D2ECD44-776B-437D-9DD6-936DD5B7A4BF', 'AE3A61BE-5820-45D3-BCD1-9982D0BA93EF', 'C94A98D6-223E-47E8-9589-5631EBCB55E9', '05BBEB22-65F8-4773-89A4-452C6FD9DDE9', '0C7AF11E-16DB-461A-A412-B711F0810E4F', '8745504F-A5BD-470D-ABAD-CDA68F032BE0', '8B63DA7A-BB50-450B-BA2A-E68DFFF3F4F6', '0BC312A3-FBB5-4F2E-9A5C-6BAC5566A285', '1BA29057-DE25-4EF0-A06C-94A526C557A3', '8E5DABE9-6BE2-4D98-BF87-D87C6A70C35B', '9AE67B15-8F75-4CED-8129-80E003318733', 'D84605C6-4883-46E5-8BE3-4C82081FC5E0', 'EF5E604D-077C-4903-95D7-5568576B7F96', '14710135-067E-4E9D-91B1-E52FD6B72796', 'ADC1E05E-8451-4C66-976B-3DBBB7103A27', '1BDC9A8F-39D7-43D8-B3A8-5E9457AEB7CE', '45BD84A3-1EE3-4B60-8352-D372DFEB677C', '6F66C6AD-3D4F-4EC2-9C71-D7580A768CF5', '74E7B927-7E80-423E-8EBE-94FC0B7E514F', '82948A9C-6345-49ED-AB80-DD62B36CE11F', 'BA941CAA-6F1A-4BBD-9D4A-42C905C8C41A', 'ED3A209D-7398-4255-A9FB-3712B16F9DC5', 'FBCA9AD8-4FC2-48A9-B059-0347A0EDA4E1', '142DD0C0-133D-4FCB-934A-C6C50D564E61', '803ED239-BF70-4FCD-B02C-B5977404BA98', '967ABAFC-DBB1-4878-89E6-2B32E0E8387C', 'B3EE5A6F-6BC4-4F1D-B196-1127D5A7FED1', 'B77643EB-41F3-4EF0-818C-B4617862C124', 'CF7B7502-BFFD-479E-87EF-8C30089033C8', '04A3E774-1DF2-461B-8DCA-90EAB47932A0', '1054BD81-AB8A-4FDC-9A87-F41814188945', '202111D3-0B41-42EF-8113-161C06700D53', '23863534-0389-4F1E-BDDC-96C82F56F4C0', '906E67BB-7D18-4CBE-8802-172E4D09F677', 'A930D4B2-2DE6-4134-A58E-36E88012A437', '043D822F-B7D6-433B-B8F7-3A0F58BFFC22', '37409C70-626D-4D18-BFD4-749375A6D10F', 'C1DDC816-520D-4175-8613-E726EAF36CBA', 'CCC6C19C-C3C3-4F0B-8D42-083ABB99F0E9', 'FF11B590-D529-4372-A722-252F47C1AA76', 'AF3AFE4B-9BD1-42B1-9D03-5031CDB276F7', 'D45A198D-C1A5-4A54-8BCE-CD4AD0B2FBCF', 'E7C8D3FA-DCE9-4622-96F0-2047722D8564', '399470D8-4590-411F-8652-380D7FB48A6F', 'A8670196-A390-42F7-B247-84D40DCE973F', 'F39EE67E-0589-4138-9AEF-261777DFC2B1', 'FA571DDD-0422-48A6-BE3F-1DFAABE49A5D', '010ED4A3-EB64-4AE2-99CE-B458B59B538C', '1EA4A9FD-2C9B-4C54-9296-841FA5F9F23D', '3DAA771F-294A-4395-9B6B-30B898BC01E7', '411875F3-AAAD-48AC-92E6-112579D42F27', '7A0B900D-FB81-4652-AC59-4D27FB1BF2DA', 'D1CA4C65-B76E-4F88-991D-E39CDBCB9A25', 'D501A556-9400-4BBE-876D-4E2269A69E50', 'E44DA795-311C-464F-A98C-AF783E9A8687', '8800B6CD-48B2-414A-AA85-2FB6E2F94F62', '0C738DFF-CDE2-446B-8605-885F7FFEA814', '387FB6B8-8FE5-4483-A401-B595EF6BE611', '9E48B685-ED16-456A-9BDA-73F1CDE529D5', 'FF73DA08-83EA-4F38-8153-B686DFBD480C', '8E785A6C-A77D-46F3-B74F-701B18C72B2B', 'B881C1B6-9321-4EB8-901E-843FB577ECDE', '6C01513E-DC0D-48F4-85F7-5DE34C582CA7', '88E69525-5461-45A9-879A-21A99C906710', '980CDD7C-B8FF-4C24-9EB8-E66EF9502A25', '29761757-EC8E-4BC8-9BD4-764810C0D435', '2E9CCC26-EE83-4CE0-BBE6-78198DE26939', '3596B626-33E4-4438-A2D7-C7B7F2884A61', '722ED05A-EAA6-41C2-AB96-ED5E231D9CA5', '8D12EF16-4731-4FDD-802D-73518A27A384', 'BB4D3CAB-1412-4339-A438-A8AFB1ECCA82', '113F2B98-6796-4055-8712-8147E996D2FB', '19635F03-1921-4E20-A610-C32DCD0A86E0', '1EDBDDB5-5D98-4516-87A4-135EB82D051F', '28A393A5-A7B5-475F-8710-5DFB463F120E', '36009550-373F-4179-BD6D-42DC2B91652B', '8EEEA997-D2B5-4EC9-A50E-B2CBD0094405', 'C295E50E-9D1F-4967-B8F4-CE7ACB0DD858', 'F3A2BB7A-DCFC-4740-AB2C-51B4DE18A84F', '077A557E-846D-453A-AE7D-9B3003FFA94F', '0EDBE7EE-67FD-48BA-86E9-F70C9C504642', 'B06CBEAD-36DB-426E-84CE-89B56FD46159', '1DEC7009-90C7-4C0B-8617-D9E805BD39DC', '65AAE408-BFF7-45D0-98E7-78DFDAEC64EC', '823ACF06-0F8B-4DC7-AB68-4DFE535C2F92', 'B86E4CA3-AF97-467C-80C1-A05484D3A866', 'D525418D-529A-470D-B010-7B90CD096743', '15695896-D50A-4650-8607-86092DE66F86', '724A277E-2530-4A79-AC3F-6E4467BB2688', '7C334D9D-8773-4B52-871C-7F4CE8FED784', 'A15234F5-28B1-425F-8338-F05A4BF34B5A', '0CD457A8-5460-4F98-8059-107464922EFC', '1CE42B39-5F00-4C37-A276-6DFEE0CB161F', '2AD5B7D3-9824-461C-8E4C-F5088C5A5061', '6295FB65-95AE-465C-A7BE-93CDEA6FCD41', '9E709F1E-38BF-4B10-B126-7C344A21DD11', '9EB1990C-6484-48F0-9B12-F05AA4B6815E', 'E2272BDD-653A-47F9-A8BA-EEEDBC803C1B', 'FB109A88-81CE-4832-B4AC-56DD97251649', '53B02D9D-4160-42AA-936D-FCBB76F49B95', '99D51AF1-7AD3-4DD6-96F0-25AA1E59972F', '07A0559D-6D61-4D02-9F83-0730224B6C93', '4D06240B-F581-412B-B7B2-5F3A1060CB47', 'D9501B03-2C0C-4096-A3F1-A4EBEC12D7AE', 'EC536097-D4FA-4221-9C08-0C6168F80D46', '11077BFE-4169-4F6F-A104-75B18FD0FF87', '2EB8DC5F-2F79-46F0-A78F-A59491C148F6', '36EF7301-7E02-412A-9E23-47987397A484', '620495B1-AE1E-4BEA-881B-AA8645DDAAB3', '73A89B2A-DEAD-456A-95A2-EA941FF2DBFB', '83B3ABC6-1E23-4471-9F1E-78EA18B3CCE0', '9231D20B-208F-4B7A-BEEA-28718633282D', '9FE9533D-C993-42CC-A477-DA0FD4E5FF72', 'DA09EFC9-6D5E-4472-8CF3-C9017C22A30E', 'F9A0096A-9F38-42FA-B2FF-20B3F570FEB8', '116B1537-FBEA-479D-8725-F3F98CE8696A', '496D2DE6-BB81-4484-AC58-3BB0D13B4E8E', '8D6F6E45-269B-4E19-AB45-130E4C2F8EE5', 'BF7CB069-0FD9-44DA-A277-EDEB8C31E102', '02705462-64DD-4C6E-AB7A-2CDDB023FE7A', '3882B031-F286-4F77-8AAE-3ECC1E13433E', '9C0A75B2-5E85-479D-B614-3C97418C4570', 'F5C916EC-F2CA-4136-9312-5A71933D7C4C', '846A3024-1033-46EB-9F06-0F3E17DAD4AA', 'CBA90F0E-ADB0-4F8F-A849-CD75DB7DD2A1', 'D7A3A337-59AB-4474-8D56-FF4B8402E128', '15527B5F-A9E1-4797-A410-9E324DD82868', '5649E152-6876-444A-97D3-66E0C38D7661', 'BB2FFBE2-24D7-4618-B3FE-DC8B3ECA14AC', 'E760489F-6EE4-4B15-9B6D-5B12F6A5E3DC', 'FAD56DDC-6281-4433-94D8-23FCB65FD823', '74492123-CD52-45B6-AAFB-CDD634C4C9BC', '79956D3A-C616-4A7A-9F06-BA0DC2806695', '9C582B3F-8717-4E03-84C9-F6EAAC3F4948', '394DE176-F74B-4877-A640-EF3C34905AE0', '3F563146-EE95-44B0-B130-41E8E1DE3030', '48C82871-65CB-40BD-98D3-6EB4CB2E0F61', '49B6D9F9-0BB5-43E9-B453-A0D4616A3689', '5C1B73F4-3EDD-41BE-9FEC-7BAC25366092', '9D0125FC-58A9-4CD0-A823-57B6464A308F', '9E726301-D189-4224-945B-2FBA8FA26A03', 'A5EE1C1B-278D-478B-AC1F-980E200CB843', 'C8D69246-8334-4EBF-954E-5AAFA7C1F65F', 'CAABED88-1591-4E9D-82C4-7926EA41DD52', 'DD23176B-512C-49D2-AC85-CFAA4256C72D', 'FAD99EF5-F102-4FC6-BCCC-ECDEF99456BB', '2653D41E-11AE-432B-993D-8CC4D4DAA1FA', '71E5F463-049A-42A7-A510-EACEC14E8301', '86D2CB7E-E54C-40E4-BCDD-C6D0DE7A6D7C', 'AB9BAF44-7980-4636-B844-488341E49BA9', 'F66CDB59-6144-4776-B129-5FEED24970C3', 'F829CBA3-AD5D-4CF8-BE1F-3FDC44044AF7', '244D35E3-5D74-4F5C-BB72-5860E1D4561C', '2C3E981A-853E-45E7-87BD-799325176251', '3B1A2F21-FFBF-4454-B0FB-BCBF1CEA4FE3', '49FF4EFE-6A51-44EA-BDDA-27F6A4663C04', '6BD754C1-07F9-4C5D-A1B8-B77B284AFC7F', '71A70101-0BDF-43F3-B580-4DB046CB1EBD', '8076FF62-D8A1-4309-9EAD-F4B83E85DB1D', 'DB05D823-A5CC-4AD0-87B0-6FBF69B7493E', 'DC1BE341-E3CB-4EBF-B968-F5764D826DC7', 'DCF767F0-7167-41FC-B981-296FBE63588B', '46445DC3-6A22-4664-8F2F-C73C4F9B7010', '9F250BA0-F703-4F1D-8AD6-82F2F4E58B70', 'BCCDC778-B59A-44F9-B1B6-9B67462C4B0A', 'CF8CDC47-5AD3-442E-9F3C-07CF84EC35A5', 'EADDE82D-5C40-4BF2-8C07-A91EEAAF8EC5', 'F32643CC-08F3-4108-B5D1-4A881B590328', 'C9139FC7-C65B-4F97-B7CC-56F810FD45DF', 'ECA40D88-F3E1-4286-8064-FD546E3C844E', 'FDBE8690-4E2C-4BD4-991A-74F5F3CC2872', '04A27415-9D35-49E4-A5F5-305DFD917D5F', '04EA280C-12AB-4842-9913-E97DFDDF508F', '0CB92950-EE3F-424C-AA0A-445F2D738518', '29737DE0-22D6-4D41-870C-DA59BE4A4E75', 'AA77712E-61BA-45A8-8E10-28057191D585', 'D271CCF9-CBA9-4A48-8202-5ED6FE135BDA', 'F488446B-42F0-4C47-B313-AE5AD0AD4C08', '0DC89466-2E6B-4DC7-B913-E5BBE5B168A0', '27DEFE26-1A3A-4654-A273-799CEA02BB33', '3D48B549-0C2E-4EC1-AA68-CD7DE250D65A', '790CB21B-A420-4C48-8EBC-67AAC5413D69', '9DD3CB15-B27B-4F87-A3C8-A9FC8D1F56B5', '2F71902A-768E-4A58-BE77-2AB583872514', '305A3131-EFD4-420C-88B8-361D358D9857', '65920EF9-C672-4D37-9A6B-795B0F656EDD', '736AECC6-FFE6-48E7-9F0D-4AF0CDC06DED', '9DEA15FC-C28D-42CB-8283-AC5B84B7FCEF', '0B1BA7BE-21C6-4B0E-A25F-C198051FD597', '0E9EF90A-1AEA-403E-9058-D6F60A449238', '235B02A5-8AC3-47BD-86BB-CAA1A63C317B', '48A25AFB-573A-4213-9B1C-E3654825F634', '49C499CF-DD82-4195-B4F3-F889BE7E0BC4', '691933B0-0D3D-4355-B453-79E791D6DC36', '83B52DD3-F13E-450B-84A7-CAA79A556718', '842AC936-21A9-4F15-8BF5-ACF6D4756EB0', 'A3CC38B5-605E-476B-A987-2AC63C81F539', 'C3A37CEA-E1D8-4DB2-837C-D1041863EC7F', 'CF82712C-91A7-46D2-9F6D-19A1BC8A1A0D', '485494F4-D20A-4303-89BB-F09C5646A9EA', '4FEA26C5-DD7F-4A65-B859-D5FECFF66684', 'B48620D1-DD13-471B-936F-AE9970B850B6', 'BBF82897-F57E-484D-8159-C4E14BBECB5C', 'F0B25A0F-D965-460E-AA37-EA676B8BE27F', '279D822D-CECC-407C-8F7D-DA683C10C7B1', '27E0E710-3B33-4DEC-BED0-FDB55F95EEE1', '8793C097-7DC2-47B0-8639-CBD108B34F5F', '1D2CA217-3B28-4C61-89C2-7C55BAB37BDD', '3A4437B5-45EE-4734-BF9C-C7BA808C123B', '13872B75-E219-4B25-9F7E-24F021CF0B48', '377FADE6-202C-4841-BDD4-82367D36097C', '6A29D415-E299-4812-B8CD-8CD471F28115', '84A66A4F-CB48-4BA8-B11C-F2EA34C4A1A4', 'D7B0792D-0B5F-4CAA-9787-237DFECCA98B', '6D22E643-C022-4637-A81F-22A0DB410A33', '7C771536-6EAB-4FEA-B848-E077662307D7', 'DD0C3E18-53F4-4001-9334-BEC408540A3E', 'EA11E355-7A8A-4BAD-9066-1FC3E824CBC5', '267E6CA7-6B3A-4582-B01B-9DD2C4AC8DEE', '43B4ED70-4EAA-4C2A-AB60-F4A3848BF5E9', '7D59A1A0-3C61-4F7A-AD1E-0EF39B392E83', 'FF80B585-D64F-45DC-9CE9-59C51F3001D2', '324408CC-B86D-49E1-A0DE-99A7EE03276E', '3E505BF0-6673-4C9F-996F-64E07E72E60C', '5FFAD31C-B0EF-4A88-B7EA-2F02D7B336D5', '8CC0B44A-2E34-417F-A084-3E2F067F6966', 'B29EEE24-30E4-4B06-9FA3-777B1E99463D', 'C84DCC7E-7D84-433A-8D23-74F8BB79EE40', 'E776DC1F-10CF-4EC1-A151-05EC29E6C03C', '40F1314D-EBF6-49BC-B1A8-674F07F92039', '66E28365-DDB9-4F59-916F-2B171C5D93AB', '88539DEA-C521-4D06-A758-7FEBC99D5C5E', '8CCEE88C-C36B-4DE3-88D7-00D78C1D9B0A', 'C0AC5031-0382-4350-82C5-5B80634DEEAC', 'E5775C4C-F86F-49D2-8B58-22ACA6668352', '08683FBF-CD1F-42D9-A442-EF696A6D7C83', '289AA716-A1D3-48FF-B830-4BF9DD4A4FB7', '28D97A0D-03AE-4405-BFD3-66E00EDAE377', '55F62337-F077-4BA9-87D8-4C7AC0736913', '56A0F3CD-F2AA-4492-A3F3-D8694240FF34', 'B16FA6E7-E985-411C-AA06-A1F7E6BBCEFB', '0BB8A5C4-7444-4DEE-A3AC-2E0CFF7A3477', '1DB8D73F-61E5-4065-9342-592BE88E5981', '4C323EC0-5AB8-4CAF-9381-0011A0141AD7', '07BDF3AA-3B6E-4650-BDFC-596FE687C33F', '3F664203-58B4-411D-835A-667F32E1AF1E', '94F9C8EF-40F1-4CB4-B8F1-71EA8B1482D9', 'A56274A7-1DCD-46CB-84FC-83FD3C26472D', 'BDB2ADB4-356A-40AF-9E57-C6ECA2EB1235', 'F28706C5-DDB5-4FC3-9F7D-4896DB86AF18', '3DA05017-5FD5-4CBE-9CC5-956876EECBE6', '6AD7EB61-BE8E-49A3-929C-CE2CA6C95779', '8B79B6F2-FE78-4775-8668-2A35ECA963E5', '98AEC5FB-BE93-4E59-A4A9-9533419B73B6', 'A24C0F04-7684-4CC9-BFBF-CD7E01137187', 'C22535F6-D4C8-4B58-B0C4-BA24CE26B9E7', 'D9115251-3465-4EF5-B9B3-54B0EAF82045', '24253A09-E06B-4C51-A3AD-0709BD0DA4B5', '449D29CA-9C67-4DA9-95FE-C1531DA5DE4C', '77C723D8-92FE-487C-825B-93C8605DC74E', '8BAEB725-2E28-4DE9-B911-836846DF6F11', 'D89C117A-4B7C-471A-94A0-1F6F022193BB', 'E0D39756-5A76-4FA1-BA20-C957C3486DFC', '0A6DE37A-0F94-43B3-84E3-7590769F820E', '0F43FFB3-7DF2-4156-BD39-A6C0667C7392', '08AD40B0-B651-4069-AA92-9A64FC246B17', '0EC59014-45A0-45B7-AB3D-12367ECD4174', '13E519C2-73BD-451A-A651-55B67924A9B6', '26C382E6-F716-4F7F-A72C-A6C7150C2066', '989C36A9-F0D6-48B5-8D26-F5A56E610792', 'C4ADE72E-AD16-4711-886D-AB997CCB5E71', 'DE90AF3B-500B-42BC-B1F8-D4E5971DD853', '03A51C92-3977-42BE-B158-53B124029BCC', '176F9200-4582-49AB-80EB-558534EC297C', '2861F7ED-8D48-436D-9AD0-FEF67A864566', '7157C3A4-F8EE-42C2-864E-C9E7FD5C62C6', 'FB5D5A33-BA7C-4140-AB6B-9AC8E94FABA6', '3AAEBB1A-A489-4699-A7B4-F237F130B7EE', 'A3646A59-090D-4D1F-AE88-16B127AF81ED', 'A90B0781-610E-4FE4-80ED-BD9F6ABDE174', 'AF591C35-31A5-45DD-9399-292D82D794AF', 'D06DE332-BDE9-4D63-8959-BF68C5A40297', '23AEEFFE-19BA-487C-A1D5-A88964FF3BC1', '341B166A-3402-4A75-AD95-D57FDDA27DC9', '45D50939-3539-478D-AABE-C57C6791CF1F', '4A1D1603-893E-46B7-92C1-8C3069A4B63D', '502690F0-D587-4D27-B3A7-5B9679F7E217', '97E40160-D786-4460-B2D1-FCFFE2DB599F', 'A372D150-FF7E-4BA6-9349-DDD6451A1084'], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'ChargeDetailID', 'count': 500, 'min': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'max': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192'}, {'values': {'label': ['2020-12-18 00:00:00', '2020-12-13 00:00:00', '2020-11-16 00:00:00', '2020-11-27 00:00:00', '2020-12-03 00:00:00', '2020-11-02 00:00:00', '2020-12-14 00:00:00', '2020-11-28 00:00:00', '2020-11-30 00:00:00', '2020-11-26 00:00:00', '2020-11-09 00:00:00', '2020-12-19 00:00:00', '2020-12-04 00:00:00', '2020-11-15 00:00:00', '2020-11-20 00:00:00', '2020-12-05 00:00:00', '2020-12-06 00:00:00', '2020-12-07 00:00:00', '2020-11-06 00:00:00', '2020-11-13 00:00:00', '2020-12-11 00:00:00', '2020-12-17 00:00:00', '2020-11-12 00:00:00', '2020-11-07 00:00:00', '2020-11-14 00:00:00', '2020-11-29 00:00:00', '2020-12-12 00:00:00', '2020-12-10 00:00:00', '2020-12-21 00:00:00', '2020-11-21 00:00:00', '2020-12-24 00:00:00', '2020-11-05 00:00:00', '2020-11-19 00:00:00', '2020-12-20 00:00:00', '2020-11-08 00:00:00', '2020-12-27 00:00:00', '2020-11-01 00:00:00', '2020-12-26 00:00:00', '2020-12-28 00:00:00', '2020-12-31 00:00:00'], 'data': [15, 15, 23, 16, 19, 12, 17, 14, 17, 10, 11, 13, 12, 14, 12, 9, 14, 21, 16, 19, 18, 7, 15, 14, 14, 17, 12, 8, 10, 8, 5, 9, 14, 12, 15, 5, 9, 1, 5, 3]}, 'label': 'DateofService', 'count': 40, 'min': '', 'max': ''}, {'values': {'label': ['45.0 - 709.3', '709.3 - 1373.6', '1373.6 - 2037.9', '2037.9 - 2702.2', '2702.2 - 3366.5', '3366.5 - 4030.8', '4030.8 - 4695.1', '4695.1 - 5359.4', '5359.4 - 6023.7', '6023.7 - 6688.0'], 'data': [467, 13, 5, 0, 3, 5, 0, 0, 6, 1]}, 'label': 'Amount', 'count': 10, 'min': 45, 'max': 6688}, {'values': {'label': ['-250.0 - -212.7', '-212.7 - -175.4', '-175.4 - -138.1', '-138.1 - -100.8', '-100.8 - -63.5', '-63.5 - -26.2', '-26.2 - 11.1', '11.1 - 48.4', '48.4 - 85.7', '85.7 - 123.0'], 'data': [1, 0, 2, 0, 2, 3, 489, 0, 2, 1]}, 'label': 'PatientBal', 'count': 10, 'min': -250, 'max': 123}, {'values': {'label': ['0.0', '24.0', '76.0', '36.0'], 'data': [497, 1, 1, 1]}, 'label': 'InsuranceBal', 'count': 4, 'min': '0.0', 'max': '76.68'}, {'values': {'label': ['0.0', '76.0'], 'data': [499, 1]}, 'label': 'PrimaryInsuranceBal', 'count': 2, 'min': '0.0', 'max': '76.68'}, {'values': {'label': ['92014', '92134', '92250', '67028', '99204', '99354', '99215', '92012', 'J0178', '92225', '92226', '92004', '76512', '67036', '92235', 'J2778PF3', '99213', '99203', '67113', 'J7312', '92018', 'J9035', '99205'], 'data': [156, 185, 35, 22, 28, 2, 3, 12, 7, 7, 6, 6, 3, 2, 10, 9, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'CPTCode', 'count': 23, 'min': '67028', 'max': 'J9035'}, {'values': {'label': ['25', '0000', 'LT', 'RT', '50', '26', '57'], 'data': [70, 385, 13, 15, 13, 1, 3]}, 'label': 'CptModifier', 'count': 7, 'min': '0000', 'max': 'RT'}, {'values': {'label': ['H35.3112', 'H33.312', 'H35.711', 'H35.3211', 'E11.3313', 'H33.311', 'E11.3413', 'H33.101', 'H35.3121', 'H30.021', 'H35.3132', 'E11.3512', 'H30.91', 'E11.3513', 'E11.3391', 'H35.3231', 'H35.033', 'H35.413', 'H35.61', 'E10.3511', 'C69.32', 'H34.8110', 'H35.372', 'E11.3511', 'H33.301', 'C69.31', 'E11.3411', 'D31.31', 'H35.352', 'H43.813', 'H35.3134', 'H20.013', 'H35.3221', 'H35.3122', 'H35.351', 'B58.01', 'H34.231', 'H34.8310', 'H25.813', 'H33.001', 'H35.3113', 'H43.821', 'H35.3130', 'H34.8120', 'H35.411', 'H44.2A2', 'H33.002', 'H35.373', 'H35.371', 'H35.3222', 'H35.3212', 'H25.13', 'E10.3513', 'H35.051', 'H35.362', 'D31.22', 'H44.113', 'H43.11', 'H35.343', 'H43.811', 'H30.23', 'H33.313', 'H53.9', 'H30.133', 'E11.3311', 'H47.012', 'H34.8330', 'H47.10', 'H35.353', 'H20.9', 'H30.93', 'H35.3131', 'H35.3124', 'H31.113', 'H35.341', 'E10.3292', 'H33.22', 'H43.812', 'H35.053', 'H40.051', 'E11.3213', 'H35.342', 'H35.3133', 'H35.052', 'H20.12', 'H43.10', 'H35.3213', 'H35.3114', 'H53.8', 'E11.3212', 'H33.321', 'C79.32', 'H35.89', 'H30.22', 'E11.3211', 'H34.8312', 'H35.721', 'H35.433', 'H35.412', 'H40.033', 'H34.8320', 'H43.13', 'H47.211', 'H35.461', 'H43.823', 'H57.11', 'E11.3312', 'H15.121', 'H35.3111', 'H31.091', 'H44.2D2', 'H33.322', 'E11.3292', 'H44.111', 'H43.12', 'H33.103', 'H21.01', 'H04.123', 'H31.011', 'H35.54', 'H43.392', 'H35.3110', 'C69.21', 'H59.021', 'H43.22', 'H11.31', 'H20.012', 'H34.8130', 'H35.073', 'H35.023', 'E10.3391', 'H35.713', 'H31.8', 'H35.63', 'D31.32', 'H33.042', 'T85.29XA', 'H33.323', 'H40.053', 'H44.23', 'H10.45', 'E11.9', 'T66.XXXA', 'H43.391', 'H35.101', 'H30.013', 'H35.50', 'Q14.1', 'H35.712'], 'data': [21, 3, 5, 20, 14, 2, 8, 1, 1, 1, 13, 5, 2, 38, 1, 30, 15, 2, 1, 2, 5, 7, 11, 14, 2, 1, 4, 3, 3, 6, 5, 1, 3, 7, 8, 1, 1, 5, 1, 6, 2, 3, 3, 4, 2, 1, 5, 11, 9, 2, 8, 2, 1, 1, 1, 1, 1, 2, 1, 4, 2, 1, 2, 1, 4, 1, 1, 1, 6, 2, 2, 10, 2, 1, 3, 1, 1, 4, 1, 3, 3, 3, 3, 2, 2, 1, 2, 3, 1, 2, 3, 1, 2, 1, 5, 3, 2, 1, 3, 1, 3, 1, 1, 1, 3, 1, 4, 1, 4, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'pdiagnosis1', 'count': 149, 'min': 'B58.01', 'max': 'T85.29XA'}, {'values': {'label': ['H35.373', 'H35.411', 'H35.721', 'H35.3122', 'H43.811', 'E11.9', 'Z79.4', 'H43.823', 'H43.813', 'H35.351', 'H35.70', 'H43.822', 'E11.3312', 'H35.723', 'H34.8320', 'H52.13', 'H35.3221', 'Z71.89', 'H35.432', 'H43.821', 'H35.371', 'H35.61', 'H35.412', 'H35.372', 'H20.9', 'H21.543', 'H43.812', 'H35.3211', 'H30.101', 'H43.89', 'H35.033', 'H04.123', 'H25.811', 'H35.463', 'H35.3212', 'H35.3132', 'H35.461', 'H44.22', 'H35.032', 'H40.021', 'H35.341', 'H35.352', 'Z98.890', 'H34.8322', 'H35.711', 'E11.3311', 'H35.3133', 'H30.113', 'H35.3124', 'S01.111D', 'H47.211', 'E11.3412', 'Z79.899', 'D31.31', 'H35.353', 'Z96.1', 'H40.052', 'H35.3130', 'H35.3134', 'H40.013', 'H33.312', 'H43.12', 'D31.32', 'H34.8120', 'H43.11', 'H33.311', 'H31.092', 'H33.331', 'H30.93', 'H33.42', 'H47.10', 'H40.1110', 'H21.542', 'H35.3223', 'E11.3513', 'H59.022', 'E11.3292', 'E11.3592', 'H35.3131', 'H47.011', 'H35.3220', 'H35.022', 'H25.813', 'H35.3120', 'E11.3293', 'H35.342', 'H35.3112', 'H35.52', 'H25.11', 'H43.392', 'H43.23', 'H35.413', 'H40.053', 'H35.722', 'H43.13', 'H35.3121', 'E11.3511', 'H43.393', 'H40.1130', 'E11.3291', 'H33.41', 'H33.321', 'E11.3213', 'G45.9', 'E11.3532', 'H35.22', 'I10', 'H31.012', 'H31.011', 'H30.92', 'H47.091', 'H11.32', 'H33.43', 'H35.433', 'K50.918', 'H35.40', 'H40.043', 'H35.052', 'T85.398A', 'H11.31', 'H35.81', 'E11.3391', 'H44.111', 'H25.13', 'E11.3212', 'H01.003', 'T66.XXXS', 'H40.051', 'H33.322', 'H43.391', 'G89.11', 'H35.713', 'C69.22', 'H27.131', 'H35.89', 'S05.11XA', 'H26.222', 'H33.002', 'H35.3222', 'E11.3392', 'H40.42X0', 'E10.3312', 'H44.701', 'E11.3512', 'H44.23', 'H25.9', 'H33.003', 'T66.XXXA', 'H11.001', 'H35.3231', 'H31.091', 'H33.001', 'H31.422', 'H44.2A3', 'H47.233', 'H31.002'], 'data': [20, 2, 3, 9, 14, 6, 47, 7, 22, 7, 4, 6, 5, 4, 9, 2, 23, 4, 2, 4, 8, 3, 4, 10, 4, 1, 12, 2, 1, 3, 25, 2, 2, 2, 1, 2, 2, 1, 1, 1, 3, 8, 3, 3, 1, 1, 1, 1, 3, 1, 1, 4, 2, 5, 10, 3, 1, 1, 3, 1, 3, 4, 3, 4, 4, 3, 1, 1, 2, 1, 1, 1, 1, 3, 3, 1, 2, 3, 3, 1, 1, 1, 1, 2, 2, 4, 1, 1, 1, 3, 1, 7, 1, 2, 2, 3, 2, 2, 2, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'pdiagnosis2', 'count': 156, 'min': 'C69.22', 'max': 'Z98.890'}, {'values': {'label': ['1.0'], 'data': [500]}, 'label': 'IsBillable', 'count': 1, 'min': '1.0', 'max': '1.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'Isauthorized', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'PredeterminationIndicator', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'MedNecessityRequiredIndicator', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['5.0 - 14.7', '14.7 - 24.4', '24.4 - 34.1', '34.1 - 43.8', '43.8 - 53.5', '53.5 - 63.2', '63.2 - 72.9', '72.9 - 82.6', '82.6 - 92.3', '92.3 - 102.0'], 'data': [4, 6, 11, 20, 44, 75, 134, 120, 72, 14]}, 'label': 'Age', 'count': 10, 'min': 5, 'max': 102}, {'values': {'label': ['M', 'F'], 'data': [225, 275]}, 'label': 'Gender', 'count': 2, 'min': 'F', 'max': 'M'}, {'values': {'label': ['Pulmonary', 'Emergency', 'Cardiology', 'Orthopedic', 'Pediatrics', 'Physical Therapy', 'Ophtamology', 'Gastroenterology'], 'data': [172, 233, 39, 36, 3, 7, 9, 1]}, 'label': 'DepartmentCategory', 'count': 8, 'min': 'Cardiology', 'max': 'Pulmonary'}, {'values': {'label': ['Ophtamology'], 'data': [500]}, 'label': 'SpecialityCategory', 'count': 1, 'min': 'Ophtamology', 'max': 'Ophtamology'}, {'values': {'label': ['1.0', '0.0'], 'data': [455, 45]}, 'label': 'label', 'count': 2, 'min': '0.0', 'max': '1.0'}, {'values': {'label': ['B03B74CE-C223-4B1E-802B-5F5F76BFE64C', '105C1090-6ED0-4189-B07E-4C22940D948F', 'AD96772F-54C7-4750-86B1-1BD565B55244', '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', '4502FB31-5B01-4EEF-BB9E-2D396D96A6FC', '32E7EC13-C493-40B1-A999-6AC94E7FA3B9', '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', '0A471D4C-3977-4BF5-8039-5064519FDCD4', '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'BCA515C2-1B14-4060-9E24-B80BF9260648', '2CD6534B-F8F6-4CF9-8DEB-CCEF9EF87F9A', 'A7CD19DF-074A-47D3-958D-2B5E00671A3D', '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', '84222BD3-605E-43EA-9A84-7C0169BE0B0D', '4CED3627-B1D6-4530-87D7-9609525FB29C', '041FE834-D4EA-40F1-A495-4B94A97ACD4D', 'F3FB074E-423D-4F7F-A817-90C0551007A4', '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', '8351B681-269A-4AE6-936A-FF10AFAFD629', 'E459B9E9-8650-4DA2-91F6-00B77900FC50', '2EAF4989-97B8-4F1B-9259-03E37314202F', 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'F473817F-7515-46B7-97F9-348C88D7C348', '0BC878EE-8F48-415E-9233-25A7C28264DB', '789050C0-D1C3-40AD-8942-E67329E2F890', '193BA5E7-F797-4FD1-A9AB-27590E630FCB', '5EA62C5D-D9C0-4AFE-ACAF-4AD36F6B1057', '72F5084C-9DF9-4F0A-8BA1-A9B40DFCC9C6', '330E2DD7-352E-4E5C-9669-AE796ABFE5DC', 'E0FE1EC8-51D4-430D-8B8B-E2F9966875F8', 'A4D682CE-4D10-4765-AE88-0D5D9E1862E7', '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'C4A6C186-B867-46C5-A163-5AF23FE61DED', 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'BCCB5D33-476A-4FAC-BB0C-5C7479803EB4', '3BEA3D9A-B5EF-4B67-A4AB-868B7BE2C0A1', '0779204D-FC67-444E-97E3-83CB62A9D0A2', 'EDD37436-0E9D-45B5-8C46-A22CA763970A', '8CA1878F-1BD4-4595-9AFF-3FF6967CD5CB', '315C773B-4EF1-4164-AB43-6817D993329F', 'AAB72015-9704-40ED-A2E7-87468E793D48', '20C2761F-16B5-42B7-8C93-82A112735F02', '835BC6C3-FEDE-4AF9-8B88-91628A98389C', 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', 'D1C4AE8C-46B5-464F-91E1-A39C5C76CC25'], 'data': [15, 1, 30, 199, 25, 81, 2, 2, 16, 6, 37, 3, 1, 1, 1, 6, 3, 1, 2, 5, 12, 3, 3, 2, 2, 4, 2, 6, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'PayerId', 'count': 51, 'min': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52'}, {'values': {'label': ['2.0', '1.0'], 'data': [225, 275]}, 'label': 'Gender_Recoded', 'count': 2, 'min': '1', 'max': '2'}, {'values': {'label': ['0.0 - 49.9', '49.9 - 99.8', '99.8 - 149.7', '149.7 - 199.6', '199.6 - 249.5', '249.5 - 299.4', '299.4 - 349.3', '349.3 - 399.2', '399.2 - 449.1', '449.1 - 499.0'], 'data': [50, 50, 50, 50, 50, 50, 50, 50, 50, 50]}, 'label': 'ChargeDetailID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 499}, {'values': {'label': ['1 - 4', '4 - 7', '7 - 10', '10 - 13', '13 - 16', '16 - 19', '19 - 22', '22 - 25', '25 - 28', '28 - 31'], 'data': [40, 60, 61, 53, 79, 45, 69, 5, 32, 56]}, 'label': 'DateofService_dayofmonth', 'count': 10, 'min': 1, 'max': 31}, {'values': {'label': ['12', '11'], 'data': [221, 279]}, 'label': 'DateofService_month', 'count': 2, 'min': '11', 'max': '12'}, {'values': {'label': ['2020'], 'data': [500]}, 'label': 'DateofService_year', 'count': 1, 'min': '2020', 'max': '2020'}, {'values': {'label': ['0.0 - 2.2', '2.2 - 4.4', '4.4 - 6.6', '6.6 - 8.8', '8.8 - 11.0', '11.0 - 13.2', '13.2 - 15.4', '15.4 - 17.6', '17.6 - 19.8', '19.8 - 22.0'], 'data': [376, 50, 22, 16, 13, 12, 4, 2, 2, 3]}, 'label': 'CPTCode_stringindexer_transform', 'count': 10, 'min': 0, 'max': 22}, {'values': {'label': ['1.0', '0.0', '4.0', '2.0', '3.0', '6.0', '5.0'], 'data': [70, 385, 13, 15, 13, 1, 3]}, 'label': 'CptModifier_stringindexer_transform', 'count': 7, 'min': '0.0', 'max': '6.0'}, {'values': {'label': ['0.0 - 14.8', '14.8 - 29.6', '29.6 - 44.4', '44.4 - 59.2', '59.2 - 74.0', '74.0 - 88.8', '88.8 - 103.6', '103.6 - 118.4', '118.4 - 133.2', '133.2 - 148.0'], 'data': [230, 79, 49, 36, 28, 18, 15, 15, 15, 15]}, 'label': 'pdiagnosis1_stringindexer_transform', 'count': 10, 'min': 0, 'max': 148}, {'values': {'label': ['0.0 - 15.5', '15.5 - 31.0', '31.0 - 46.5', '46.5 - 62.0', '62.0 - 77.5', '77.5 - 93.0', '93.0 - 108.5', '108.5 - 124.0', '124.0 - 139.5', '139.5 - 155.0'], 'data': [238, 66, 51, 35, 32, 15, 16, 15, 16, 16]}, 'label': 'pdiagnosis2_stringindexer_transform', 'count': 10, 'min': 0, 'max': 155}, {'values': {'label': ['1', '0'], 'data': [225, 275]}, 'label': 'Gender_stringindexer_transform', 'count': 2, 'min': '0', 'max': '1'}, {'values': {'label': ['1.0', '0.0', '2.0', '3.0', '6.0', '5.0', '4.0', '7.0'], 'data': [172, 233, 39, 36, 3, 7, 9, 1]}, 'label': 'DepartmentCategory_stringindexer_transform', 'count': 8, 'min': '0.0', 'max': '7.0'}, {'values': {'label': ['0'], 'data': [500]}, 'label': 'SpecialityCategory_stringindexer_transform', 'count': 1, 'min': '0', 'max': '0'}, {'values': {'label': ['0.0 - 5.0', '5.0 - 10.0', '10.0 - 15.0', '15.0 - 20.0', '20.0 - 25.0', '25.0 - 30.0', '30.0 - 35.0', '35.0 - 40.0', '40.0 - 45.0', '45.0 - 50.0'], 'data': [372, 55, 21, 12, 10, 9, 5, 5, 5, 6]}, 'label': 'PayerId_stringindexer_transform', 'count': 10, 'min': 0, 'max': 50}], 'FESamplePath': '/FileStore/platform/sampleData/60891770a187c956b337cf50_AutoFE/part-00000-tid-3917072144446901006-f58791a3-edf4-48b1-a470-3dde18d61c22-172002-1-c000.csv', 'manualfesamplepath': '/FileStore/platform/sampleData/60891770a187c956b337cf50_ManualFE/part-00000-tid-7570618811626959673-94990ba1-2b39-4e6c-ac98-d7d939c2d4f4-172005-1-c000.csv', 'stageId': '60891770a187c956b337cf50', 'stage_id': '60891770a187c956b337cf50', 'status': 1, 'autoFE_flag': True, 'userid': '60406edac5c640163a69c2d6', 'message': 'Successfully executed', 'process_type': 'Auto FE', 'sampledata': [{'ChargeDetailID': '174DA322-870E-429F-A9A9-BE22340E9836', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '89.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '407.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': 'A1575911-0E83-4479-B182-D26EDAD439F6', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.312', 'pdiagnosis2': 'H35.411', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '105C1090-6ED0-4189-B07E-4C22940D948F', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '420.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '45.0', 'pdiagnosis2_stringindexer_transform': '72.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '49.0'}, {'ChargeDetailID': 'A87EF1E9-E636-4742-8E2B-414477926782', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.711', 'pdiagnosis2': 'H35.721', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '39.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '287.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '24.0', 'pdiagnosis2_stringindexer_transform': '45.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '1FFF3E45-B9F4-4CFB-8580-C4B05D4822DD', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '467.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '10.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '20BC5C22-2C34-494C-B75F-55BDD8B54263', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '463.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '210DAE19-9257-4180-9057-E5817F555A88', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.311', 'pdiagnosis2': 'E11.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '41.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '54.0', 'pdiagnosis2_stringindexer_transform': '16.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '3FA21D4D-05B5-4C43-A33E-A01D20B45A99', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '53.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '42.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '13.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': 'DC65D7C3-0822-4E54-86EF-DC7639D38C62', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H33.101', 'pdiagnosis2': 'H43.823', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '57.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '151.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '125.0', 'pdiagnosis2_stringindexer_transform': '15.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '11327DC4-1599-46B2-A000-C05D135E1F91', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3121', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '80.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '4502FB31-5B01-4EEF-BB9E-2D396D96A6FC', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '244.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '79.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '26.0'}, {'ChargeDetailID': '853B5BAB-0A5F-4B44-85E0-FEE0AA663796', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H30.021', 'pdiagnosis2': 'H35.351', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '32E7EC13-C493-40B1-A999-6AC94E7FA3B9', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '262.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '117.0', 'pdiagnosis2_stringindexer_transform': '13.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '19.0'}, {'ChargeDetailID': '92F7294B-4F6C-41B5-A9BD-65CE72B27FC0', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'E11.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '77.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '281.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '16.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '97BB8C89-8E47-400B-B546-B662E9EDCA00', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': 'LT', 'pdiagnosis1': 'E11.3512', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '50.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '445.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '26.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': '9C7B2481-4469-4FB8-AC5D-18EB2D29F213', 'DateofService': '2020-11-26T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H30.91', 'pdiagnosis2': 'H35.70', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '166.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '52.0', 'pdiagnosis2_stringindexer_transform': '32.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '8.0'}, {'ChargeDetailID': 'BCE331FC-BC9E-43B2-B9C3-1ADE95CF4081', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.822', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '48.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '284.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '17.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '0391FBA9-7748-4F50-8CA2-FC4EB5116F6A', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3391', 'pdiagnosis2': 'E11.3312', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '21.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '118.0', 'pdiagnosis2_stringindexer_transform': '18.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '88D923C8-5217-46AF-887E-CB88A2E6C3E3', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.723', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '88.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '469.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '22.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '8AAE78A7-7725-4C1C-B55C-82B41A364A2B', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H34.8320', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '48.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '384.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '9.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'B5C5BD20-64AE-4166-8115-14DA13624764', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '-172.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.413', 'pdiagnosis2': 'H52.13', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '336.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '63.0', 'pdiagnosis2_stringindexer_transform': '73.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '343273F8-E598-4815-A700-0683D31F3291', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.61', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '219.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '110.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '50F7ADC5-2166-4461-B01E-8D8FAA92304B', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E10.3511', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '54.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '326.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '67.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'A9EC659A-ED07-4B6C-84CA-D4E45411C3C6', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99354', 'CptModifier': '0000', 'pdiagnosis1': 'C69.32', 'pdiagnosis2': 'Z71.89', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '58.0', 'Gender': 'M', 'DepartmentCategory': 'Pediatrics', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '242.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '15.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '23.0', 'pdiagnosis2_stringindexer_transform': '31.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '6.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'DCBDA7E0-A5F8-4D0F-836D-66187FEB5B6F', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H35.432', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '48.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '471.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '15.0', 'pdiagnosis2_stringindexer_transform': '75.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'E907166F-CE0F-49D2-A04B-40734F1A6F0C', 'DateofService': '2020-12-05T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H43.821', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '62.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '24.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '6518F89E-CD31-4903-89E2-2F31BEB28508', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '310.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '7B41BFB4-B1D7-47A9-9659-6E8319429423', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E10.3511', 'pdiagnosis2': 'H35.371', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '53.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'BCA515C2-1B14-4060-9E24-B80BF9260648', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '325.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '67.0', 'pdiagnosis2_stringindexer_transform': '12.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '14.0'}, {'ChargeDetailID': '7E4D4049-B1BC-4135-8020-D69318B66B18', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.61', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '59.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '414.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '42.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '94FEF155-5ED7-4DF9-836F-78557484B8E4', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H33.301', 'pdiagnosis2': 'H35.412', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '58.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '364.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '55.0', 'pdiagnosis2_stringindexer_transform': '23.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'A6B1DE4F-2FBD-4AB9-B09E-77B3E8C44771', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '250.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99215', 'CptModifier': '25', 'pdiagnosis1': 'C69.31', 'pdiagnosis2': 'Z71.89', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '159.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '12.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '94.0', 'pdiagnosis2_stringindexer_transform': '31.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': 'B9F344F0-CEE0-4EA2-A55C-274FAB39E9DB', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E11.3411', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '2CD6534B-F8F6-4CF9-8DEB-CCEF9EF87F9A', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '171.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '31.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '35.0'}, {'ChargeDetailID': 'BBE4E59F-B282-490E-8FCD-6C9F5E8069EB', 'DateofService': '2020-12-17T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'D31.31', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '240.0', 'DateofService_dayofmonth': '17', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '48.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'EC206AC2-403A-4047-B82E-82C383E29CCE', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.352', 'pdiagnosis2': 'H35.372', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '77.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '167.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '44.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '01867531-8532-4FD4-BE17-232F39CFA32D', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.371', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '296.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '19.0', 'pdiagnosis2_stringindexer_transform': '12.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '2ECD88B7-05A0-4959-8736-397F8CF74784', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H20.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7CD19DF-074A-47D3-958D-2B5E00671A3D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '371.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '29.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '29.0'}, {'ChargeDetailID': '524D7B98-2D7C-458D-8877-35CDE8F3D981', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3134', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '3.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '21.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '46.0'}, {'ChargeDetailID': '54E3991B-1609-470D-8095-C14FF5A9444D', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H20.013', 'pdiagnosis2': 'H21.543', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '31.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '395.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '135.0', 'pdiagnosis2_stringindexer_transform': '86.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '8F8BDCC6-5B74-4EBA-A154-8576DE0FFC13', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '453.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A28023B3-9D64-44D3-BD13-2A22A1350A24', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3221', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '89.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '152.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '43.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '1B71D2B0-C473-4A8D-9FA0-76A0692C5102', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '6000.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '56.0', 'Gender': 'M', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '169.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '9.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '5.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '6CF3754A-BC46-430C-A116-9329959CD556', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.3211', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '96.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '16.0', 'pdiagnosis2_stringindexer_transform': '76.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '9.0'}, {'ChargeDetailID': 'A6CED81D-7B1D-441A-8EFC-544E8D31FD26', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '389.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '10.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A82270CF-0B59-4C62-877D-9FAF720B6E5D', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H30.101', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '50.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '129.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '14.0', 'pdiagnosis2_stringindexer_transform': '145.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'D9A3771E-E97A-4EBE-BFA9-54577F445F21', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'B58.01', 'pdiagnosis2': 'H43.89', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '43.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '308.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '138.0', 'pdiagnosis2_stringindexer_transform': '48.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '1AA261AC-C877-4F0D-8372-F268A4D5F90B', 'DateofService': '2020-12-10T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.231', 'pdiagnosis2': 'E11.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '4CED3627-B1D6-4530-87D7-9609525FB29C', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '71.0', 'DateofService_dayofmonth': '10', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '98.0', 'pdiagnosis2_stringindexer_transform': '16.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '13.0'}, {'ChargeDetailID': '2B385821-C105-42CA-84F0-CEF8BFD1681A', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '462.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '25.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '9D5CF424-8DA6-4A2B-8716-FDB93B808F3C', 'DateofService': '2020-12-21T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H25.813', 'pdiagnosis2': 'H04.123', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '444.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '112.0', 'pdiagnosis2_stringindexer_transform': '59.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A17DF3A2-8077-49B4-A114-11F77F67E7C9', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '142.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '25.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '0E47C422-450E-4A04-9B90-680F296C9686', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H25.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '480.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '17.0', 'pdiagnosis2_stringindexer_transform': '71.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': '46CB6A8C-362F-4345-941E-842F429B48B5', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3113', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '92.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '232.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '56.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '8CD490B4-CEBE-4DFE-BA0A-C805ACDF0D34', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3113', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '75.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '373.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '56.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '9F54BD25-7C67-454A-B100-CD9974B62C7D', 'DateofService': '2020-11-21T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.463', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '39.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '337.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '19.0', 'pdiagnosis2_stringindexer_transform': '58.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': 'A78BD078-2165-4C68-B398-9AB6E94C0FB6', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '041FE834-D4EA-40F1-A495-4B94A97ACD4D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '304.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '10.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '40.0'}, {'ChargeDetailID': 'C72EBDAD-1D1D-40D0-BA4D-A9CDD08430A6', 'DateofService': '2020-12-24T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '56.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '379.0', 'DateofService_dayofmonth': '24', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'DBFEC39B-37B2-4B37-9D95-A444C27A619C', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.821', 'pdiagnosis2': 'H35.3212', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '33.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '38.0', 'pdiagnosis2_stringindexer_transform': '83.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'E1C8F5F6-2DDF-4370-B40C-D86322919657', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '264.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'EB7C11AD-0A01-4359-B1AB-289373C669DC', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '44.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '440.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '16062E35-A49E-4227-A3E6-E8DDBFEBA5D2', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3130', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '90.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '417.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '39.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '20416F30-920B-4925-8642-C83F38ECFCEE', 'DateofService': '2020-11-05T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8120', 'pdiagnosis2': 'H35.3132', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '234.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '29.0', 'pdiagnosis2_stringindexer_transform': '65.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '642D42E4-275F-4E86-8378-21E2CD291224', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '50.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92225', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.411', 'pdiagnosis2': 'H35.461', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '31.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '305.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '8.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '74.0', 'pdiagnosis2_stringindexer_transform': '74.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': 'FBB301BB-1560-415C-91B5-A6E8B0B26B6C', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.351', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '88.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '123.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '13.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '4011D505-BE33-4A24-AD40-240BFDDB2766', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H43.821', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '298.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '24.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '17.0'}, {'ChargeDetailID': '74FC2B0E-AEC7-4263-951F-14E72EF044DA', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H44.2A2', 'pdiagnosis2': 'H44.22', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '32.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '66.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '134.0', 'pdiagnosis2_stringindexer_transform': '105.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'A5881100-AA89-4D5D-80AC-AE2419271C5D', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.721', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '96.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '339.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '45.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D0667457-EF74-4FA4-91D9-7169C64A53DB', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H34.8320', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '478.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '9.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '11.0'}, {'ChargeDetailID': 'E16D36F3-C4D2-4C4B-A004-E06C823C7960', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H35.032', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '224.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '20.0', 'pdiagnosis2_stringindexer_transform': '120.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '20463C7A-45B7-4BD4-A3EA-53ACC03E93F1', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '431.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '6E834EA0-8A83-4326-8FAE-80C6DEE3B045', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H40.021', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '272.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '11.0', 'pdiagnosis2_stringindexer_transform': '110.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': '8607EC2B-26F1-4A40-9646-5990D37EC622', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3222', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '438.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '76.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'A050F893-D9BB-4FB6-9B4B-B7AA83355D60', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.341', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '195.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '12.0', 'pdiagnosis2_stringindexer_transform': '46.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C2F471A9-22FA-4A9A-A14C-49A5F292662D', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H35.352', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '93.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '11.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D6F6C707-D42D-4485-A571-82EEF5002E64', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H35.371', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '290.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '14.0', 'pdiagnosis2_stringindexer_transform': '12.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'E164830A-C381-4AEC-81EE-3B116E56B496', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H25.13', 'pdiagnosis2': 'Z98.890', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '48.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '342.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '72.0', 'pdiagnosis2_stringindexer_transform': '50.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '2701DB53-B3A0-466C-BFCB-6684D7059DC4', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '89.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '59.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '89D7AAB0-BF88-493E-A7AB-7D6D793273DB', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '18.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '19.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '8A07AC76-56FD-4E13-BFEA-A724E4C2D603', 'DateofService': '2020-11-05T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H34.8322', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '77.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '190.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '43.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '8E5FAFBE-87F5-4C26-88EA-002384A50B1D', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.723', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '87.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '340.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '22.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': 'E67C3D6F-A4D2-4EA9-B41A-E5BEB54E0209', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H43.822', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '13.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '17.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '16.0'}, {'ChargeDetailID': 'AC68E17B-9EBB-45A3-BD22-60367A203515', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E10.3513', 'pdiagnosis2': 'H35.371', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '40.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '8351B681-269A-4AE6-936A-FF10AFAFD629', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '163.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '147.0', 'pdiagnosis2_stringindexer_transform': '12.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '15.0'}, {'ChargeDetailID': 'D0BA312C-C42D-4695-A6E4-71096BE6854B', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.051', 'pdiagnosis2': 'H35.711', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '59.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '449.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '86.0', 'pdiagnosis2_stringindexer_transform': '106.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '50C6C060-08F2-4F75-91BE-EF1D34F6EDA9', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H43.821', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '133.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '24.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C99C5CC6-DF6E-4024-88A6-04D326F6D6B2', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3512', 'pdiagnosis2': 'E11.3311', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'E459B9E9-8650-4DA2-91F6-00B77900FC50', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '168.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '26.0', 'pdiagnosis2_stringindexer_transform': '148.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '27.0'}, {'ChargeDetailID': '1A70E6C3-0B4F-4338-A15C-903F552EC30C', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '84.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '346.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '279BE041-8831-43CF-A0EB-219F709A986F', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.362', 'pdiagnosis2': 'H35.351', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '419.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '103.0', 'pdiagnosis2_stringindexer_transform': '13.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '3E0A6D69-9B07-4BFC-B389-24E0C884701E', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '45.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92226', 'CptModifier': 'LT', 'pdiagnosis1': 'D31.22', 'pdiagnosis2': 'H34.8320', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '42.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '35.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '11.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '101.0', 'pdiagnosis2_stringindexer_transform': '9.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': '9017B6B5-6C3F-42FC-ABD4-67A54E07D67F', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.3133', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '87.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '497.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '93.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'F16597BA-3D84-47E3-AF6C-072ECBD33467', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H44.113', 'pdiagnosis2': 'H30.113', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '26.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '341.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '129.0', 'pdiagnosis2_stringindexer_transform': '119.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '2AA7F9ED-3694-4ADD-91A8-3EFB8F8054FB', 'DateofService': '2020-12-05T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '24.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.711', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '19.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '24.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '45F8D74C-DE28-4F0D-B781-AB81CCD5D5CD', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3124', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '91.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '226.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '12.0', 'pdiagnosis2_stringindexer_transform': '38.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'D7B8BD7C-A3F9-422B-B491-59F236BADE4D', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '112.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'E9DB7A1D-67B8-4936-B7C7-FB071EA72907', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8120', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '72.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '29.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '2B6E92D7-EA15-4F53-B133-2A1F9B1E958C', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H35.341', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '75.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '292.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '17.0', 'pdiagnosis2_stringindexer_transform': '46.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '86E8386B-2651-4928-8CD1-631CE24D0432', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3124', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '95.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '362.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '38.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '97D21C13-7AFC-4865-888F-5E92CC51F2ED', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '246.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92004', 'CptModifier': '25', 'pdiagnosis1': 'H43.11', 'pdiagnosis2': 'S01.111D', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '125.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '10.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '64.0', 'pdiagnosis2_stringindexer_transform': '117.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A40F161A-A931-41D9-A65B-597347BC9D6A', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '25', 'pdiagnosis1': 'H35.3221', 'pdiagnosis2': 'H47.211', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '91.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '439.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '43.0', 'pdiagnosis2_stringindexer_transform': '102.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C43925F7-AA9E-419B-B0DB-0A044E67C4B5', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3412', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '59.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '76.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '20.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D05F93BC-037E-4BD7-8E65-FDA46391791F', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.343', 'pdiagnosis2': 'Z79.899', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '250.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '90.0', 'pdiagnosis2_stringindexer_transform': '69.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '14167C30-8123-4DCF-B3C5-05D3F16B6391', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '383.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '76512', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.711', 'pdiagnosis2': 'D31.31', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '198.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '13.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '24.0', 'pdiagnosis2_stringindexer_transform': '19.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '2022006D-F4D6-4F8A-B76D-1617C97083CA', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H35.353', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '187.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '25EF819E-5F4C-43DA-BEBB-C4FAD71B0EF5', 'DateofService': '2020-12-27T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.822', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '54.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '2EAF4989-97B8-4F1B-9259-03E37314202F', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '10.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '17.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '22.0'}, {'ChargeDetailID': '8648E212-02DB-4D77-BB02-88B67C56A9DE', 'DateofService': '2020-12-27T00:00:00.000Z', 'Amount': '246.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92004', 'CptModifier': '25', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'Z96.1', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '82.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '294.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '10.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '40.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'EDDC8ACA-6742-4C6C-A200-FA42DD89285D', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H40.052', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '413.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '13.0', 'pdiagnosis2_stringindexer_transform': '139.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '03E0DCDC-A071-4987-A7CE-268D9C3D017B', 'DateofService': '2020-12-27T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '-40.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '89.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '381.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '12.0'}, {'ChargeDetailID': '161B0D6C-5A50-4EEE-9B2A-E4DE8D9EBC1C', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '48.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '51.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '15.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '1BD4EAF6-F3F8-4369-93B3-4E6144A69CAD', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.811', 'pdiagnosis2': 'H35.3130', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '180.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '28.0', 'pdiagnosis2_stringindexer_transform': '107.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': '20AC5B30-ACB1-4714-9794-5454B92C1A23', 'DateofService': '2020-11-01T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3222', 'pdiagnosis2': 'H35.3134', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '91.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '205.0', 'DateofService_dayofmonth': '1', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '76.0', 'pdiagnosis2_stringindexer_transform': '35.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '3FE9456A-15CB-4840-90CB-05F7DF34CACE', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H40.013', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '87.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '8.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '85.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '42F5D00A-6F59-4F63-ACCA-9EFDA0DD9194', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H35.353', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '80.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '101.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '6E80EF7D-E761-4720-A1BD-9E2CA7FDD21F', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'D31.31', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '89.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '14.0', 'pdiagnosis2_stringindexer_transform': '19.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '734CA534-2CD7-4A77-9D03-CF2390E8B0D6', 'DateofService': '2020-12-10T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H30.23', 'pdiagnosis2': 'H35.353', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '41.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '136.0', 'DateofService_dayofmonth': '10', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '75.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': 'B0501331-13EF-4126-B052-44D67EDFE7F9', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H35.372', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '318.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C0B29F0F-BBE1-47C7-B625-0989FDCC5288', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H33.312', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '488.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '17.0', 'pdiagnosis2_stringindexer_transform': '51.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '20.0'}, {'ChargeDetailID': '136261A4-52B0-43B5-B099-1EF929506A93', 'DateofService': '2020-12-17T00:00:00.000Z', 'Amount': '3348.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67036', 'CptModifier': 'LT', 'pdiagnosis1': 'H30.23', 'pdiagnosis2': 'H43.12', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '14.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '46.0', 'DateofService_dayofmonth': '17', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '14.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '75.0', 'pdiagnosis2_stringindexer_transform': '21.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '252D8E17-A9E8-48F5-84D3-6DA9B452EE75', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'D31.32', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '412.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '49.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '5A8270A6-53BD-4916-8805-B762FF67BE20', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H34.8120', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '283.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '26.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '10.0'}, {'ChargeDetailID': '67841660-18E4-45DE-B386-285C53582762', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '97.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '374.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '12.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '9C60FA6D-FAAF-46A9-A149-1F941986D089', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H33.313', 'pdiagnosis2': 'H43.11', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '46.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'F473817F-7515-46B7-97F9-348C88D7C348', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '221.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '146.0', 'pdiagnosis2_stringindexer_transform': '25.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '21.0'}, {'ChargeDetailID': 'D6FE7A69-6B2B-495F-A36A-789DF588F6D5', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '84.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '68.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D9C0CCE1-6D5D-4DB7-8376-D8562C150145', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '468.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '13.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '42B818D0-170D-4838-BC4A-7D22039C0F53', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '303.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '14.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '7DC8A130-8030-4915-8BF5-54015B27DAB6', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H25.13', 'pdiagnosis2': 'H33.311', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '36.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '81.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '72.0', 'pdiagnosis2_stringindexer_transform': '36.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'A3555B0C-5A99-4619-B6E8-9831D99F0210', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '54.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '476.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'AECE54B4-2AE0-44A1-9409-2BFB52D07BD2', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H34.8322', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '84.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '433.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '43.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'BE7612D0-7548-49C7-800E-173BAF9DF290', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '50.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92225', 'CptModifier': 'LT', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H31.092', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '34.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '282.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '8.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '153.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '0EB85789-CE25-4CCC-850E-44D2C3496272', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H53.9', 'pdiagnosis2': 'H33.331', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '41.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '494.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '60.0', 'pdiagnosis2_stringindexer_transform': '142.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '9.0'}, {'ChargeDetailID': '45F8ABD8-1D29-495F-B63D-866175B861A0', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.723', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '88.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '127.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '22.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A69FF747-5E5E-44A3-A2AE-806C56D75743', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H30.133', 'pdiagnosis2': 'H30.93', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '39.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '2EAF4989-97B8-4F1B-9259-03E37314202F', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '85.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '105.0', 'pdiagnosis2_stringindexer_transform': '66.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '22.0'}, {'ChargeDetailID': 'B1C24F17-5A6A-4D76-A42E-A273CF94898F', 'DateofService': '2020-12-26T00:00:00.000Z', 'Amount': '3000.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3311', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '82.0', 'Gender': 'M', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '79.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '9.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '32.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '5.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '41D99DEA-233E-47A6-98C0-8B6F522D1B70', 'DateofService': '2020-12-28T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '53.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '70.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '53BD3672-31BA-4CE7-B2F3-2AFFC29C5F6F', 'DateofService': '2020-12-21T00:00:00.000Z', 'Amount': '45.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92226', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H35.351', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '77.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '58.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '11.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '11.0', 'pdiagnosis2_stringindexer_transform': '13.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '55F86AF5-F804-4A84-90ED-12E4125E1A8D', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.821', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '377.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '38.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '12.0'}, {'ChargeDetailID': '8C94AF61-5EC3-44D7-B4B3-BBEB64AD2A23', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H33.42', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '32.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '474.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '84.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'B52B5682-9815-4EC1-A594-912A9724EB74', 'DateofService': '2020-11-01T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H35.341', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '322.0', 'DateofService_dayofmonth': '1', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '46.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '079F8DAC-D8C1-48FA-9743-BC2AA08E287C', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '53.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '2D60073D-3E0E-4347-B0A6-7E0CE91E92FA', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H47.012', 'pdiagnosis2': 'H47.10', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '86.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '410.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '136.0', 'pdiagnosis2_stringindexer_transform': '134.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '99772A77-B764-48FF-9242-5700C9572B65', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '50.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92225', 'CptModifier': 'LT', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '165.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '8.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'E45AB682-6C1F-4B84-A1AC-3D1CD79AC3A9', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '295.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '51526EAE-3FB7-4C7E-9A33-26990B70EFFE', 'DateofService': '2020-12-10T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H40.1110', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '77.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '238.0', 'DateofService_dayofmonth': '10', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '81.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '540B2A5F-11A9-4F0F-8038-B5A1ECE5E583', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8330', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '330.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '99.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '91D48DF5-F63A-466A-AFB9-2E65C4F2D44D', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H34.8320', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '89.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '436.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '9.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'ADF5D273-DB58-4F7F-8DF0-9D9D9647DA16', 'DateofService': '2020-12-05T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H47.10', 'pdiagnosis2': 'H21.542', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '60.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '0BC878EE-8F48-415E-9233-25A7C28264DB', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '110.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '127.0', 'pdiagnosis2_stringindexer_transform': '99.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '30.0'}, {'ChargeDetailID': 'F672AF9B-E361-4572-A9C0-8B0731F9CD3E', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.711', 'pdiagnosis2': 'H35.3211', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '53.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '177.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '24.0', 'pdiagnosis2_stringindexer_transform': '76.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '34803FD1-6C40-4323-8657-F1FCC3E34990', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '45.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92226', 'CptModifier': 'LT', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.352', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '435.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '11.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '16.0', 'pdiagnosis2_stringindexer_transform': '11.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '5327CFC0-5C48-435B-98EB-A76AAE858941', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3223', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '102.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '210.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '37.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D2DF16C4-D3AB-4E5C-9350-8F6FDE2EDA8D', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '153.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '14.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'E2DBCD71-B901-41F3-97B1-ED71030C7F2F', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '92.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '191.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '16.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '049BA1F1-F362-48BE-9393-C05CCEA8694B', 'DateofService': '2020-12-24T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '46.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '208.0', 'DateofService_dayofmonth': '24', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '25.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '8.0'}, {'ChargeDetailID': '13002A76-350E-4525-BDA1-9F181E056F74', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'Z79.899', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '63.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '19.0', 'pdiagnosis2_stringindexer_transform': '69.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '167E1195-CC7A-4F8C-8D91-9577BCBEE8FB', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '286.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'E11.3513', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '124.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '6.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '18.0', 'pdiagnosis2_stringindexer_transform': '39.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '1F264D61-D5D7-4505-9AC2-25ACF275E3BD', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H20.9', 'pdiagnosis2': 'H59.022', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '197.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '69.0', 'pdiagnosis2_stringindexer_transform': '90.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '23BCDE27-9F80-48E4-B29D-348C871829B3', 'DateofService': '2020-12-10T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H30.93', 'pdiagnosis2': 'E11.3292', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '55.0', 'DateofService_dayofmonth': '10', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '70.0', 'pdiagnosis2_stringindexer_transform': '57.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '10.0'}, {'ChargeDetailID': 'A71A1D2F-3F1A-4CA6-AA98-E5D6A855C250', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '94.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'B62DA993-E320-4CEE-9D23-B3238CA5759A', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '285.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': 'B6375BF2-EFA5-44C9-A05C-F71DACD5F8A8', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '2.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C924A1C9-2E9A-4BF3-8BC7-25BFAEB02CE1', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '87.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D2300410-689E-453C-895D-FC7FDA559759', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3592', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '451.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '44.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'F12A6DBF-FBB4-4C62-8EE0-0F677D58EC3B', 'DateofService': '2020-12-21T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '316.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': 'FA9EEE47-7F76-4264-9A82-9B370BCF9534', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3124', 'pdiagnosis2': 'H35.352', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '100.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '57.0', 'pdiagnosis2_stringindexer_transform': '11.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'FBBC8CD5-C35A-42E5-AEBA-C5F61050B726', 'DateofService': '2020-12-24T00:00:00.000Z', 'Amount': '6000.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3412', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'F', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '82.0', 'DateofService_dayofmonth': '24', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '9.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '20.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '5.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '10.0'}, {'ChargeDetailID': '045700E8-F572-417C-A8DD-41E76C5C159A', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '90.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '401.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '1A088AE8-D8BA-404D-97E5-9DB1DFB972F8', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H31.113', 'pdiagnosis2': 'H35.3131', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '286.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '95.0', 'pdiagnosis2_stringindexer_transform': '47.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '1C109ACE-6610-4098-BE1A-C64F7805EBCA', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.341', 'pdiagnosis2': 'H35.3132', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '91.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '41.0', 'pdiagnosis2_stringindexer_transform': '65.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '25251236-8157-4EE2-89CD-C5A600E278BE', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.352', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '89.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '256.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '11.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A1818C97-82BE-4115-8C3C-3532CB8DE838', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E10.3292', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '309.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '91.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'AC9C3FBE-B694-4A6F-9A6F-6CC4563C5C73', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '365.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'B5E35D7B-A0F8-4A47-B72F-1D372236FC63', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '45.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92226', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H35.411', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '43.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '789050C0-D1C3-40AD-8942-E67329E2F890', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '229.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '11.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '11.0', 'pdiagnosis2_stringindexer_transform': '72.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '36.0'}, {'ChargeDetailID': 'C1119267-721A-4606-9D06-09FF240505EC', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '202.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '14.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C3339BC7-4E2E-4E0E-822E-7EE8E489451C', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.353', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '15.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'CDDFE064-998C-4CDD-97A2-04647A33EE7A', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H47.011', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '77.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '69.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '146.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D4930936-02DA-4DBB-A3B2-5391AC5D7153', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '6000.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'F', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'BCA515C2-1B14-4060-9E24-B80BF9260648', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '23.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '9.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '5.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '14.0'}, {'ChargeDetailID': 'FD8029C5-4ACA-4F97-8CE6-144AE49C8BCB', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '1800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3411', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '51.0', 'Gender': 'F', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '64.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '7.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '31.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '4.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': '4C3D48F4-860F-49EF-812D-75B08DC1359C', 'DateofService': '2020-11-05T00:00:00.000Z', 'Amount': '1800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3512', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '60.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '7.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '26.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '4.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '58207A31-753A-42B1-86D8-E750AEA10079', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.811', 'pdiagnosis2': 'H35.371', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '80.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '479.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '28.0', 'pdiagnosis2_stringindexer_transform': '12.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'EAF0CF6B-9569-41B6-8B77-6335F34B3D7E', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '193BA5E7-F797-4FD1-A9AB-27590E630FCB', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '275.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '48.0'}, {'ChargeDetailID': '693B32F4-ECA0-4844-80DF-6D9C94DD57BA', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3220', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '427.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '100.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '7E2F8EA5-4A1D-4490-A208-49CE684BB427', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '26', 'pdiagnosis1': 'H33.22', 'pdiagnosis2': 'H35.022', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '7.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '5EA62C5D-D9C0-4AFE-ACAF-4AD36F6B1057', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '162.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '6.0', 'pdiagnosis1_stringindexer_transform': '120.0', 'pdiagnosis2_stringindexer_transform': '108.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '24.0'}, {'ChargeDetailID': '83A30AD9-1329-4675-99E9-AF5ACBE98CFA', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.812', 'pdiagnosis2': 'H25.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '425.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '33.0', 'pdiagnosis2_stringindexer_transform': '113.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '84F4A744-F946-4B1E-99DF-20E0538C5F4F', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '80.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '291.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '90C2C4BC-4F1D-4730-8078-6CF025EB493F', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.372', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '121.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '16.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'AE214CAC-A747-4549-B4E0-957075E94F38', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H33.311', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '320.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '17.0', 'pdiagnosis2_stringindexer_transform': '36.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': 'BD337B93-0D12-4AE2-9566-35705B577186', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3120', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '113.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '68.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '9.0'}, {'ChargeDetailID': 'CE11AF94-7EB4-4B98-BC57-51593FD36B72', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.721', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '184.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '45.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '1F6C5DD9-4889-43B7-8768-BA1D2873322F', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'E11.3293', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '175.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '60.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '322A11BB-2B77-480A-BABA-2FD89CDD0305', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H35.342', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '74.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '33.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '286.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '94.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '442.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '6.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '3342F086-2D91-4081-BEE0-C18AEAB0C981', 'DateofService': '2020-11-26T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '355.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C4FFF996-0FB6-427D-9266-BE10E9F5DAA9', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '1800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3311', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '59.0', 'Gender': 'F', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '12.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '7.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '32.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '4.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '510C2305-6CB3-460D-84C7-9EB6281F8E9D', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.053', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '57.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '254.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '141.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': 'FA7AC95E-1961-4A7B-AC57-3FD3212BAF20', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H40.051', 'pdiagnosis2': 'H35.3112', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '7.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '47.0', 'pdiagnosis2_stringindexer_transform': '127.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '04CCB591-8B59-4CCB-9EB0-5A4AECB3C705', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '-10.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3213', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '98.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '42.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '0D66C132-CD95-442B-800E-E19F2651E018', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H34.8320', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '80.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '428.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '9.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '0ECEF203-5A40-428B-A19E-BFA699806763', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3134', 'pdiagnosis2': 'H35.52', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '72F5084C-9DF9-4F0A-8BA1-A9B40DFCC9C6', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '253.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '21.0', 'pdiagnosis2_stringindexer_transform': '80.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '32.0'}, {'ChargeDetailID': '2A1A46A7-1E50-41C1-ADD1-EBC0F82168F9', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3592', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '84.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '387.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '44.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '6C6C6230-52AE-4590-AFA7-42A14991BD69', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.342', 'pdiagnosis2': 'H25.11', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '299.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '49.0', 'pdiagnosis2_stringindexer_transform': '91.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': '71B8A2A8-14EC-4B19-A4DA-6DE30E8A6E30', 'DateofService': '2020-12-05T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'H43.392', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '255.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '18.0', 'pdiagnosis2_stringindexer_transform': '41.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '2B3AACB7-A045-4E46-A2DB-8B841AD87F65', 'DateofService': '2020-12-05T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'Z96.1', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '84.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '17.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '15.0', 'pdiagnosis2_stringindexer_transform': '40.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '2FCDBCD0-49FC-4AA6-B249-041E476D766A', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '46.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '349.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '13.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '3DF7C0B1-EA0D-4014-916E-E76984AFDE2A', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3133', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '11.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '35.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '8A1DE8B3-2FDE-4581-AC94-4EA74A997CB0', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.821', 'pdiagnosis2': 'H35.371', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '32.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '38.0', 'pdiagnosis2_stringindexer_transform': '12.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '11.0'}, {'ChargeDetailID': 'F49D6604-F705-480F-BCB9-E6F16B76DACF', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.23', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '91.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '378.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '79.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': '31C70405-0F52-4BD8-A785-13AFD15809A6', 'DateofService': '2020-12-17T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H35.413', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '23.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '323.0', 'DateofService_dayofmonth': '17', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '20.0', 'pdiagnosis2_stringindexer_transform': '14.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '323DACC7-ED74-42A3-A1E6-21BC104AAD51', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.052', 'pdiagnosis2': 'H35.352', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '46.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '366.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '53.0', 'pdiagnosis2_stringindexer_transform': '11.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '35841CCE-C3B1-4652-B011-C44F79BB442D', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H40.053', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '75.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '196.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '104.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': '8875D93D-866D-4897-81C3-67DB4570B102', 'DateofService': '2020-12-17T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3133', 'pdiagnosis2': 'H35.722', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '28.0', 'DateofService_dayofmonth': '17', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '35.0', 'pdiagnosis2_stringindexer_transform': '61.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A7A12FBC-72F0-4216-8360-126AF826214C', 'DateofService': '2020-12-21T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3133', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '271.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '35.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'BDAADB25-9498-4D4F-809E-4590BAEE53B5', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3412', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '472.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '20.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '0438621D-9EC2-4530-B3E8-F0C68FD8295F', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.11', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '77.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '25.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '55E3B0D0-8E40-48F3-854B-A8E3C893E803', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '149.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '793B38EA-690C-456B-9597-5444E6D1499F', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '130.0', 'PatientBal': '-10.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99213', 'CptModifier': '0000', 'pdiagnosis1': 'H20.12', 'pdiagnosis2': 'H43.89', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '122.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '20.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '73.0', 'pdiagnosis2_stringindexer_transform': '48.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '899B4439-339E-4CC1-AC1E-4C1F4BA312EB', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '84.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '249.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A4CD837B-6B10-47C0-B9DC-E8D6ADF73B88', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.13', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '144.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '67.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'CC817F76-EE46-4AD5-9911-86A53FB1DBAE', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3213', 'pdiagnosis2': 'H35.353', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '416.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '42.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'F1C5E648-7E44-43B6-A884-48A861BA52BB', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H35.351', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '48.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '434.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '11.0', 'pdiagnosis2_stringindexer_transform': '13.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '3D2ECD44-776B-437D-9DD6-936DD5B7A4BF', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3121', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '329.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '34.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '16.0'}, {'ChargeDetailID': 'AE3A61BE-5820-45D3-BCD1-9982D0BA93EF', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '223.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '12.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C94A98D6-223E-47E8-9589-5631EBCB55E9', 'DateofService': '2020-11-05T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '88.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '321.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '05BBEB22-65F8-4773-89A4-452C6FD9DDE9', 'DateofService': '2020-12-21T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '464.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': '0C7AF11E-16DB-461A-A412-B711F0810E4F', 'DateofService': '2020-11-01T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H43.10', 'pdiagnosis2': 'E11.3511', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '52.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '183.0', 'DateofService_dayofmonth': '1', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '124.0', 'pdiagnosis2_stringindexer_transform': '64.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '8745504F-A5BD-470D-ABAD-CDA68F032BE0', 'DateofService': '2020-12-21T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3213', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '97.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '301.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '61.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '8B63DA7A-BB50-450B-BA2A-E68DFFF3F4F6', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.723', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '57.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '22.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '0BC312A3-FBB5-4F2E-9A5C-6BAC5566A285', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H34.8320', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '91.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '344.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '9.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '1BA29057-DE25-4EF0-A06C-94A526C557A3', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '45.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '8E5DABE9-6BE2-4D98-BF87-D87C6A70C35B', 'DateofService': '2020-11-05T00:00:00.000Z', 'Amount': '286.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3114', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '128.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '6.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '46.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '9AE67B15-8F75-4CED-8129-80E003318733', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '-250.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '50.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '132.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '15.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': 'D84605C6-4883-46E5-8BE3-4C82081FC5E0', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H43.393', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '257.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '11.0', 'pdiagnosis2_stringindexer_transform': '56.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '10.0'}, {'ChargeDetailID': 'EF5E604D-077C-4903-95D7-5568576B7F96', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '289.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '14710135-067E-4E9D-91B1-E52FD6B72796', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '254.0', 'PatientBal': '-99.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99203', 'CptModifier': '0000', 'pdiagnosis1': 'H53.8', 'pdiagnosis2': 'H43.393', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '41.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '333.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '22.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '121.0', 'pdiagnosis2_stringindexer_transform': '56.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': 'ADC1E05E-8451-4C66-976B-3DBBB7103A27', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3124', 'pdiagnosis2': 'H40.1130', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '97.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '4CED3627-B1D6-4530-87D7-9609525FB29C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '370.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '57.0', 'pdiagnosis2_stringindexer_transform': '62.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '13.0'}, {'ChargeDetailID': '1BDC9A8F-39D7-43D8-B3A8-5E9457AEB7CE', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3212', 'pdiagnosis2': 'E11.3291', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '458.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '62.0', 'pdiagnosis2_stringindexer_transform': '87.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '45BD84A3-1EE3-4B60-8352-D372DFEB677C', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'H33.41', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '49.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '27.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '6F66C6AD-3D4F-4EC2-9C71-D7580A768CF5', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '3600.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'M', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '4.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '7.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '4.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '11.0'}, {'ChargeDetailID': '74E7B927-7E80-423E-8EBE-94FC0B7E514F', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H35.352', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '359.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '11.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '82948A9C-6345-49ED-AB80-DD62B36CE11F', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '-14.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H34.8120', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '225.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '26.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'BA941CAA-6F1A-4BBD-9D4A-42C905C8C41A', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '-93.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H43.812', 'pdiagnosis2': 'H43.821', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '241.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '33.0', 'pdiagnosis2_stringindexer_transform': '24.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'ED3A209D-7398-4255-A9FB-3712B16F9DC5', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '-170.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H33.321', 'pdiagnosis2': 'H35.412', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '53.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '111.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '40.0', 'pdiagnosis2_stringindexer_transform': '23.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': 'FBCA9AD8-4FC2-48A9-B059-0347A0EDA4E1', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '6688.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67113', 'CptModifier': 'RT', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H33.321', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '88.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '19.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '17.0', 'pdiagnosis2_stringindexer_transform': '28.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '142DD0C0-133D-4FCB-934A-C6C50D564E61', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'C79.32', 'pdiagnosis2': 'H04.123', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '481.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '82.0', 'pdiagnosis2_stringindexer_transform': '59.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '803ED239-BF70-4FCD-B02C-B5977404BA98', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '75.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '176.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': '967ABAFC-DBB1-4878-89E6-2B32E0E8387C', 'DateofService': '2020-12-21T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'C69.32', 'pdiagnosis2': 'D31.32', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '53.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '459.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '23.0', 'pdiagnosis2_stringindexer_transform': '49.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'B3EE5A6F-6BC4-4F1D-B196-1127D5A7FED1', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '286.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'E11.3213', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '29.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '6.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '135.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'B77643EB-41F3-4EF0-818C-B4617862C124', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.89', 'pdiagnosis2': 'H33.321', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '47.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '327.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '68.0', 'pdiagnosis2_stringindexer_transform': '28.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': 'CF7B7502-BFFD-479E-87EF-8C30089033C8', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H35.371', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '84.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '13.0', 'pdiagnosis2_stringindexer_transform': '12.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '04A3E774-1DF2-461B-8DCA-90EAB47932A0', 'DateofService': '2020-12-21T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H53.9', 'pdiagnosis2': 'G45.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '52.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '330E2DD7-352E-4E5C-9669-AE796ABFE5DC', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '313.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '60.0', 'pdiagnosis2_stringindexer_transform': '132.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '18.0'}, {'ChargeDetailID': '1054BD81-AB8A-4FDC-9A87-F41814188945', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H30.22', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '411.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '128.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '202111D3-0B41-42EF-8113-161C06700D53', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3211', 'pdiagnosis2': 'E11.3292', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '392.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '22.0', 'pdiagnosis2_stringindexer_transform': '57.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '23863534-0389-4F1E-BDDC-96C82F56F4C0', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3121', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '75.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '324.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '34.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '906E67BB-7D18-4CBE-8802-172E4D09F677', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '48.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '269.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': 'A930D4B2-2DE6-4134-A58E-36E88012A437', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8312', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '148.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '37.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': '043D822F-B7D6-433B-B8F7-3A0F58BFFC22', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'H35.70', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '147.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '18.0', 'pdiagnosis2_stringindexer_transform': '32.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '37409C70-626D-4D18-BFD4-749375A6D10F', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3212', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '314.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '62.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '10.0'}, {'ChargeDetailID': 'C1DDC816-520D-4175-8613-E726EAF36CBA', 'DateofService': '2020-12-10T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.721', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '252.0', 'DateofService_dayofmonth': '10', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '71.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'CCC6C19C-C3C3-4F0B-8D42-083ABB99F0E9', 'DateofService': '2020-11-21T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'E11.3532', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '47.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '421.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '96.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'FF11B590-D529-4372-A722-252F47C1AA76', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H43.822', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '404.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '17.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'AF3AFE4B-9BD1-42B1-9D03-5031CDB276F7', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.433', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '345.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '83.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D45A198D-C1A5-4A54-8BCE-CD4AD0B2FBCF', 'DateofService': '2020-11-26T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.812', 'pdiagnosis2': 'H33.312', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '135.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '33.0', 'pdiagnosis2_stringindexer_transform': '51.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'E7C8D3FA-DCE9-4622-96F0-2047722D8564', 'DateofService': '2020-11-05T00:00:00.000Z', 'Amount': '50.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92225', 'CptModifier': 'LT', 'pdiagnosis1': 'H35.412', 'pdiagnosis2': 'H43.822', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '361.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '8.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '36.0', 'pdiagnosis2_stringindexer_transform': '17.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '399470D8-4590-411F-8652-380D7FB48A6F', 'DateofService': '2020-12-31T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H40.033', 'pdiagnosis2': 'E11.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '105.0', 'DateofService_dayofmonth': '31', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '139.0', 'pdiagnosis2_stringindexer_transform': '16.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'A8670196-A390-42F7-B247-84D40DCE973F', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8320', 'pdiagnosis2': 'H35.22', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '44.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '357.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '34.0', 'pdiagnosis2_stringindexer_transform': '141.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'F39EE67E-0589-4138-9AEF-261777DFC2B1', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3223', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '89.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '172.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '37.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'FA571DDD-0422-48A6-BE3F-1DFAABE49A5D', 'DateofService': '2020-11-01T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3211', 'pdiagnosis2': 'E11.3312', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '47.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '118.0', 'DateofService_dayofmonth': '1', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '22.0', 'pdiagnosis2_stringindexer_transform': '18.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '286.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '90.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '4502FB31-5B01-4EEF-BB9E-2D396D96A6FC', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '193.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '6.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '26.0'}, {'ChargeDetailID': '1EA4A9FD-2C9B-4C54-9296-841FA5F9F23D', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '50.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H43.392', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '55.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '8351B681-269A-4AE6-936A-FF10AFAFD629', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '338.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '14.0', 'pdiagnosis2_stringindexer_transform': '41.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '15.0'}, {'ChargeDetailID': '3DAA771F-294A-4395-9B6B-30B898BC01E7', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3130', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '477.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '39.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': '411875F3-AAAD-48AC-92E6-112579D42F27', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H30.91', 'pdiagnosis2': 'I10', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '447.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '52.0', 'pdiagnosis2_stringindexer_transform': '94.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '7A0B900D-FB81-4652-AC59-4D27FB1BF2DA', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '-10.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3124', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '91.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '243.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '12.0', 'pdiagnosis2_stringindexer_transform': '38.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'D1CA4C65-B76E-4F88-991D-E39CDBCB9A25', 'DateofService': '2020-12-05T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.411', 'pdiagnosis2': 'H35.372', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '47.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '83.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '74.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': 'D501A556-9400-4BBE-876D-4E2269A69E50', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '48.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '181.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': 'E44DA795-311C-464F-A98C-AF783E9A8687', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3121', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '403.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '12.0', 'pdiagnosis2_stringindexer_transform': '34.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '8800B6CD-48B2-414A-AA85-2FB6E2F94F62', 'DateofService': '2020-12-21T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.13', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'E0FE1EC8-51D4-430D-8B8B-E2F9966875F8', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '270.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '122.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '25.0'}, {'ChargeDetailID': '0C738DFF-CDE2-446B-8605-885F7FFEA814', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H47.211', 'pdiagnosis2': 'H31.012', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '155.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '107.0', 'pdiagnosis2_stringindexer_transform': '89.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '387FB6B8-8FE5-4483-A401-B595EF6BE611', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'H31.011', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '86.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '397.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '18.0', 'pdiagnosis2_stringindexer_transform': '128.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '9E48B685-ED16-456A-9BDA-73F1CDE529D5', 'DateofService': '2020-11-21T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.461', 'pdiagnosis2': 'H30.92', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '38.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '388.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '143.0', 'pdiagnosis2_stringindexer_transform': '150.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'FF73DA08-83EA-4F38-8153-B686DFBD480C', 'DateofService': '2020-12-27T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3120', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '56.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '68.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '9.0'}, {'ChargeDetailID': '8E785A6C-A77D-46F3-B74F-701B18C72B2B', 'DateofService': '2020-11-26T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.823', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '77.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '103.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '50.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'B881C1B6-9321-4EB8-901E-843FB577ECDE', 'DateofService': '2020-12-05T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3223', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '369.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '37.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '6C01513E-DC0D-48F4-85F7-5DE34C582CA7', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '246.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '13.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '88E69525-5461-45A9-879A-21A99C906710', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '286.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H47.091', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '154.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '6.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '98.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '980CDD7C-B8FF-4C24-9EB8-E66EF9502A25', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.811', 'pdiagnosis2': 'H40.1130', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '91.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '293.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '28.0', 'pdiagnosis2_stringindexer_transform': '62.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '29761757-EC8E-4BC8-9BD4-764810C0D435', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H11.32', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '77.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '178.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '122.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '2E9CCC26-EE83-4CE0-BBE6-78198DE26939', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H57.11', 'pdiagnosis2': 'H33.43', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '57.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '32E7EC13-C493-40B1-A999-6AC94E7FA3B9', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '192.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '140.0', 'pdiagnosis2_stringindexer_transform': '144.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '19.0'}, {'ChargeDetailID': '3596B626-33E4-4438-A2D7-C7B7F2884A61', 'DateofService': '2020-12-24T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.433', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '170.0', 'DateofService_dayofmonth': '24', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '54.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '722ED05A-EAA6-41C2-AB96-ED5E231D9CA5', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.433', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '75.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '90.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '16.0', 'pdiagnosis2_stringindexer_transform': '54.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '8D12EF16-4731-4FDD-802D-73518A27A384', 'DateofService': '2020-12-10T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.3134', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '86.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A4D682CE-4D10-4765-AE88-0D5D9E1862E7', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '24.0', 'DateofService_dayofmonth': '10', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '35.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '23.0'}, {'ChargeDetailID': 'BB4D3CAB-1412-4339-A438-A8AFB1ECCA82', 'DateofService': '2020-12-10T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3312', 'pdiagnosis2': 'E11.3511', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '93.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '315.0', 'DateofService_dayofmonth': '10', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '27.0', 'pdiagnosis2_stringindexer_transform': '64.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '113F2B98-6796-4055-8712-8147E996D2FB', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.89', 'pdiagnosis2': 'H30.93', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '350.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '68.0', 'pdiagnosis2_stringindexer_transform': '66.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '19635F03-1921-4E20-A610-C32DCD0A86E0', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H15.121', 'pdiagnosis2': 'K50.918', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '56.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '80.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '111.0', 'pdiagnosis2_stringindexer_transform': '97.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '1EDBDDB5-5D98-4516-87A4-135EB82D051F', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'E459B9E9-8650-4DA2-91F6-00B77900FC50', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '441.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '27.0'}, {'ChargeDetailID': '28A393A5-A7B5-475F-8710-5DFB463F120E', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H34.8320', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '160.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '15.0', 'pdiagnosis2_stringindexer_transform': '9.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '36009550-373F-4179-BD6D-42DC2B91652B', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H35.40', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '89.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '372.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '55.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '8EEEA997-D2B5-4EC9-A50E-B2CBD0094405', 'DateofService': '2020-11-01T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.341', 'pdiagnosis2': 'H40.043', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '88.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '146.0', 'DateofService_dayofmonth': '1', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '41.0', 'pdiagnosis2_stringindexer_transform': '88.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': 'C295E50E-9D1F-4967-B8F4-CE7ACB0DD858', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '400.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '18.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'F3A2BB7A-DCFC-4740-AB2C-51B4DE18A84F', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '1800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3211', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '82.0', 'Gender': 'F', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '391.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '7.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '22.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '4.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '077A557E-846D-453A-AE7D-9B3003FFA94F', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3111', 'pdiagnosis2': 'H35.352', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '31.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '30.0', 'pdiagnosis2_stringindexer_transform': '11.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '28.0'}, {'ChargeDetailID': '0EDBE7EE-67FD-48BA-86E9-F70C9C504642', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H33.41', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '394.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '27.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': 'B06CBEAD-36DB-426E-84CE-89B56FD46159', 'DateofService': '2020-12-05T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.823', 'pdiagnosis2': 'H35.052', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '406.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '50.0', 'pdiagnosis2_stringindexer_transform': '82.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '1DEC7009-90C7-4C0B-8617-D9E805BD39DC', 'DateofService': '2020-11-26T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '60.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'T85.398A', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '231.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '14.0', 'pdiagnosis2_stringindexer_transform': '121.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '65AAE408-BFF7-45D0-98E7-78DFDAEC64EC', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H40.051', 'pdiagnosis2': 'H35.412', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '306.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '47.0', 'pdiagnosis2_stringindexer_transform': '23.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': '823ACF06-0F8B-4DC7-AB68-4DFE535C2F92', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3311', 'pdiagnosis2': 'E11.3412', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '116.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '32.0', 'pdiagnosis2_stringindexer_transform': '20.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'B86E4CA3-AF97-467C-80C1-A05484D3A866', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '286.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '390.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '6.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '8.0'}, {'ChargeDetailID': 'D525418D-529A-470D-B010-7B90CD096743', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '25', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3312', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '115.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '18.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '15695896-D50A-4650-8607-86092DE66F86', 'DateofService': '2020-12-31T00:00:00.000Z', 'Amount': '50.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92225', 'CptModifier': 'RT', 'pdiagnosis1': 'H31.091', 'pdiagnosis2': 'H35.461', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '47.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '212.0', 'DateofService_dayofmonth': '31', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '8.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '126.0', 'pdiagnosis2_stringindexer_transform': '74.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '724A277E-2530-4A79-AC3F-6E4467BB2688', 'DateofService': '2020-12-05T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3114', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '89.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '266.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '46.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '11.0'}, {'ChargeDetailID': '7C334D9D-8773-4B52-871C-7F4CE8FED784', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '233.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A15234F5-28B1-425F-8338-F05A4BF34B5A', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3512', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '59.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '385.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '26.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '8.0'}, {'ChargeDetailID': '0CD457A8-5460-4F98-8059-107464922EFC', 'DateofService': '2020-12-28T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '86.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '258.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '1CE42B39-5F00-4C37-A276-6DFEE0CB161F', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '42.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '277.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': '2AD5B7D3-9824-461C-8E4C-F5088C5A5061', 'DateofService': '2020-12-28T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '328.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '6295FB65-95AE-465C-A7BE-93CDEA6FCD41', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '75.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '382.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '9E709F1E-38BF-4B10-B126-7C344A21DD11', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3134', 'pdiagnosis2': 'H11.31', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '82.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '130.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '21.0', 'pdiagnosis2_stringindexer_transform': '103.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '9EB1990C-6484-48F0-9B12-F05AA4B6815E', 'DateofService': '2020-11-21T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.81', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '41.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '335.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '147.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'E2272BDD-653A-47F9-A8BA-EEEDBC803C1B', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '88.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '22.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '10.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'FB109A88-81CE-4832-B4AC-56DD97251649', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '45.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92226', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'E11.3391', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '47.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '11.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '123.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '53B02D9D-4160-42AA-936D-FCBB76F49B95', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H43.823', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '36.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '15.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '99D51AF1-7AD3-4DD6-96F0-25AA1E59972F', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3221', 'pdiagnosis2': 'H44.111', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '235.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '43.0', 'pdiagnosis2_stringindexer_transform': '136.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '07A0559D-6D61-4D02-9F83-0730224B6C93', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H44.2D2', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '58.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'C4A6C186-B867-46C5-A163-5AF23FE61DED', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '297.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '142.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '38.0'}, {'ChargeDetailID': '4D06240B-F581-412B-B7B2-5F3A1060CB47', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.342', 'pdiagnosis2': 'H35.372', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '50.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '49.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': 'D9501B03-2C0C-4096-A3F1-A4EBEC12D7AE', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '25', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H25.13', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '251.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '30.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'EC536097-D4FA-4221-9C08-0C6168F80D46', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3134', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '96.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '114.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '21.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '11077BFE-4169-4F6F-A104-75B18FD0FF87', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'E11.3212', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '56.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '263.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '15.0', 'pdiagnosis2_stringindexer_transform': '101.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '2EB8DC5F-2F79-46F0-A78F-A59491C148F6', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.413', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '218.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '63.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '36EF7301-7E02-412A-9E23-47987397A484', 'DateofService': '2020-12-28T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '186.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '620495B1-AE1E-4BEA-881B-AA8645DDAAB3', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H01.003', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '91.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '4CED3627-B1D6-4530-87D7-9609525FB29C', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '39.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '137.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '13.0'}, {'ChargeDetailID': '73A89B2A-DEAD-456A-95A2-EA941FF2DBFB', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': 'LT', 'pdiagnosis1': 'E11.3312', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '185.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '27.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '10.0'}, {'ChargeDetailID': '83B3ABC6-1E23-4471-9F1E-78EA18B3CCE0', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '57', 'pdiagnosis1': 'H33.322', 'pdiagnosis2': 'H35.413', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '50.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '228.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '5.0', 'pdiagnosis1_stringindexer_transform': '51.0', 'pdiagnosis2_stringindexer_transform': '14.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '9231D20B-208F-4B7A-BEEA-28718633282D', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '25', 'pdiagnosis1': 'H35.052', 'pdiagnosis2': 'T66.XXXS', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '85.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '213.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '53.0', 'pdiagnosis2_stringindexer_transform': '112.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '31.0'}, {'ChargeDetailID': '9FE9533D-C993-42CC-A477-DA0FD4E5FF72', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'Z98.890', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '65.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '50.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'DA09EFC9-6D5E-4472-8CF3-C9017C22A30E', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '3885.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J7312', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3512', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'M', 'DepartmentCategory': 'Gastroenterology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '443.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '16.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '26.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '7.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'F9A0096A-9F38-42FA-B2FF-20B3F570FEB8', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E11.3292', 'pdiagnosis2': 'D31.31', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '89.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '73.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '88.0', 'pdiagnosis2_stringindexer_transform': '19.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '116B1537-FBEA-479D-8725-F3F98CE8696A', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H35.352', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '227.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '11.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '496D2DE6-BB81-4484-AC58-3BB0D13B4E8E', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '87.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '399.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': '8D6F6E45-269B-4E19-AB45-130E4C2F8EE5', 'DateofService': '2020-11-26T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H44.111', 'pdiagnosis2': 'H40.051', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '179.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '130.0', 'pdiagnosis2_stringindexer_transform': '151.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'BF7CB069-0FD9-44DA-A277-EDEB8C31E102', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.12', 'pdiagnosis2': 'H25.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '351.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '58.0', 'pdiagnosis2_stringindexer_transform': '71.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '02705462-64DD-4C6E-AB7A-2CDDB023FE7A', 'DateofService': '2020-11-01T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H33.103', 'pdiagnosis2': 'H35.413', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '34.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '393.0', 'DateofService_dayofmonth': '1', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '81.0', 'pdiagnosis2_stringindexer_transform': '14.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '3882B031-F286-4F77-8AAE-3ECC1E13433E', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H33.322', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '15.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '457.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '20.0', 'pdiagnosis2_stringindexer_transform': '53.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '9C0A75B2-5E85-479D-B614-3C97418C4570', 'DateofService': '2020-12-17T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H35.722', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '380.0', 'DateofService_dayofmonth': '17', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '61.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'F5C916EC-F2CA-4136-9312-5A71933D7C4C', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3592', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '38.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '383.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '44.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '846A3024-1033-46EB-9F06-0F3E17DAD4AA', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H35.353', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'BCCB5D33-476A-4FAC-BB0C-5C7479803EB4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '311.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '44.0'}, {'ChargeDetailID': 'CBA90F0E-ADB0-4F8F-A849-CD75DB7DD2A1', 'DateofService': '2020-11-05T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H30.93', 'pdiagnosis2': 'H43.391', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '490.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '70.0', 'pdiagnosis2_stringindexer_transform': '114.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'D7A3A337-59AB-4474-8D56-FF4B8402E128', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'E11.3293', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '54.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '60.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '15527B5F-A9E1-4797-A410-9E324DD82868', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.11', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '58.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '9.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '25.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': '5649E152-6876-444A-97D3-66E0C38D7661', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3114', 'pdiagnosis2': 'H35.342', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '131.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '46.0', 'pdiagnosis2_stringindexer_transform': '33.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'BB2FFBE2-24D7-4618-B3FE-DC8B3ECA14AC', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '60.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '495.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'E760489F-6EE4-4B15-9B6D-5B12F6A5E3DC', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.11', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '261.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '25.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'FAD56DDC-6281-4433-94D8-23FCB65FD823', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H21.01', 'pdiagnosis2': 'G89.11', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '409.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '85.0', 'pdiagnosis2_stringindexer_transform': '129.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '74492123-CD52-45B6-AAFB-CDD634C4C9BC', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '356.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '79956D3A-C616-4A7A-9F06-BA0DC2806695', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3213', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '91.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A4D682CE-4D10-4765-AE88-0D5D9E1862E7', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '473.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '61.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '23.0'}, {'ChargeDetailID': '9C582B3F-8717-4E03-84C9-F6EAAC3F4948', 'DateofService': '2020-11-01T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H04.123', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '174.0', 'DateofService_dayofmonth': '1', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '65.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '394DE176-F74B-4877-A640-EF3C34905AE0', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '141.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '3F563146-EE95-44B0-B130-41E8E1DE3030', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H31.011', 'pdiagnosis2': 'E11.3513', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '215.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '119.0', 'pdiagnosis2_stringindexer_transform': '39.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '48C82871-65CB-40BD-98D3-6EB4CB2E0F61', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '56.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '8351B681-269A-4AE6-936A-FF10AFAFD629', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '486.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '11.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '15.0'}, {'ChargeDetailID': '49B6D9F9-0BB5-43E9-B453-A0D4616A3689', 'DateofService': '2020-12-24T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H35.372', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '119.0', 'DateofService_dayofmonth': '24', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '20.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '5C1B73F4-3EDD-41BE-9FEC-7BAC25366092', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.54', 'pdiagnosis2': 'H35.432', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '46.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '482.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '133.0', 'pdiagnosis2_stringindexer_transform': '75.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '9D0125FC-58A9-4CD0-A823-57B6464A308F', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '93.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '278.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '9E726301-D189-4224-945B-2FBA8FA26A03', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'D31.31', 'pdiagnosis2': 'H35.713', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '236.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '48.0', 'pdiagnosis2_stringindexer_transform': '155.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'A5EE1C1B-278D-478B-AC1F-980E200CB843', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '10.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.392', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '51.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '3BEA3D9A-B5EF-4B67-A4AB-868B7BE2C0A1', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '158.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '66.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '43.0'}, {'ChargeDetailID': 'C8D69246-8334-4EBF-954E-5AAFA7C1F65F', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '300.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'CAABED88-1591-4E9D-82C4-7926EA41DD52', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '250.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99215', 'CptModifier': '25', 'pdiagnosis1': 'C69.32', 'pdiagnosis2': 'Z71.89', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '60.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '134.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '12.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '23.0', 'pdiagnosis2_stringindexer_transform': '31.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'DD23176B-512C-49D2-AC85-CFAA4256C72D', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.721', 'pdiagnosis2': 'H43.392', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '55.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '343.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '71.0', 'pdiagnosis2_stringindexer_transform': '41.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'FAD99EF5-F102-4FC6-BCCC-ECDEF99456BB', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H35.353', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '6.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '2653D41E-11AE-432B-993D-8CC4D4DAA1FA', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '286.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '57.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '150.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '6.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '71E5F463-049A-42A7-A510-EACEC14E8301', 'DateofService': '2020-11-26T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3110', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '77.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '216.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '80.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '86D2CB7E-E54C-40E4-BCDD-C6D0DE7A6D7C', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3134', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '87.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '423.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '21.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'AB9BAF44-7980-4636-B844-488341E49BA9', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H35.342', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '46.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '475.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '8.0', 'pdiagnosis2_stringindexer_transform': '33.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'F66CDB59-6144-4776-B129-5FEED24970C3', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '27.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'F829CBA3-AD5D-4CF8-BE1F-3FDC44044AF7', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.40', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '90.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '34.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '55.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '244D35E3-5D74-4F5C-BB72-5860E1D4561C', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '246.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92004', 'CptModifier': '57', 'pdiagnosis1': 'H33.312', 'pdiagnosis2': 'H52.13', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '32.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '207.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '10.0', 'CptModifier_stringindexer_transform': '5.0', 'pdiagnosis1_stringindexer_transform': '45.0', 'pdiagnosis2_stringindexer_transform': '73.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '2C3E981A-853E-45E7-87BD-799325176251', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '-2.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H25.13', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '55.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '14.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '30.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '3B1A2F21-FFBF-4454-B0FB-BCBF1CEA4FE3', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3312', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '54.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '334.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '18.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '20.0'}, {'ChargeDetailID': '49FF4EFE-6A51-44EA-BDDA-27F6A4663C04', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H35.3131', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '95.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '47.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '6BD754C1-07F9-4C5D-A1B8-B77B284AFC7F', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92018', 'CptModifier': '0000', 'pdiagnosis1': 'C69.21', 'pdiagnosis2': 'C69.22', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '5.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '0779204D-FC67-444E-97E3-83CB62A9D0A2', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '44.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '21.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '59.0', 'pdiagnosis2_stringindexer_transform': '63.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '41.0'}, {'ChargeDetailID': '71A70101-0BDF-43F3-B580-4DB046CB1EBD', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '3348.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67036', 'CptModifier': 'RT', 'pdiagnosis1': 'H59.021', 'pdiagnosis2': 'H27.131', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '405.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '14.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '144.0', 'pdiagnosis2_stringindexer_transform': '140.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '8076FF62-D8A1-4309-9EAD-F4B83E85DB1D', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'E11.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '38.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '16.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'DB05D823-A5CC-4AD0-87B0-6FBF69B7493E', 'DateofService': '2020-12-31T00:00:00.000Z', 'Amount': '246.0', 'PatientBal': '-63.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92004', 'CptModifier': '25', 'pdiagnosis1': 'H43.11', 'pdiagnosis2': 'H25.13', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '47.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '363.0', 'DateofService_dayofmonth': '31', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '10.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '64.0', 'pdiagnosis2_stringindexer_transform': '30.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'DC1BE341-E3CB-4EBF-B968-F5764D826DC7', 'DateofService': '2020-11-21T00:00:00.000Z', 'Amount': '3600.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'M', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '126.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '7.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '4.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'DCF767F0-7167-41FC-B981-296FBE63588B', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3111', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '95.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '0.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '30.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '46445DC3-6A22-4664-8F2F-C73C4F9B7010', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J9035', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3311', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Pediatrics', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '157.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '18.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '32.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '6.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '9F250BA0-F703-4F1D-8AD6-82F2F4E58B70', 'DateofService': '2020-12-17T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '484.0', 'DateofService_dayofmonth': '17', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'BCCDC778-B59A-44F9-B1B6-9B67462C4B0A', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8320', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': 'BCA515C2-1B14-4060-9E24-B80BF9260648', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '418.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '34.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '14.0'}, {'ChargeDetailID': 'CF8CDC47-5AD3-442E-9F3C-07CF84EC35A5', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H35.89', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '437.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '109.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': 'EADDE82D-5C40-4BF2-8C07-A91EEAAF8EC5', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '25', 'pdiagnosis1': 'D31.31', 'pdiagnosis2': 'D31.32', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '59.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '498.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '48.0', 'pdiagnosis2_stringindexer_transform': '49.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': 'F32643CC-08F3-4108-B5D1-4A881B590328', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.371', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '83.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '247.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '12.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C9139FC7-C65B-4F97-B7CC-56F810FD45DF', 'DateofService': '2020-11-21T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.22', 'pdiagnosis2': 'H35.372', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '75.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '102.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'ECA40D88-F3E1-4286-8064-FD546E3C844E', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '246.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92004', 'CptModifier': '0000', 'pdiagnosis1': 'H11.31', 'pdiagnosis2': 'S05.11XA', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '25.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '145.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '10.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '108.0', 'pdiagnosis2_stringindexer_transform': '124.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'FDBE8690-4E2C-4BD4-991A-74F5F3CC2872', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H33.311', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '209.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '36.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '04A27415-9D35-49E4-A5F5-305DFD917D5F', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '108.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '04EA280C-12AB-4842-9913-E97DFDDF508F', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '383.0', 'PatientBal': '0.0', 'InsuranceBal': '76.0', 'PrimaryInsuranceBal': '76.0', 'CPTCode': '76512', 'CptModifier': 'LT', 'pdiagnosis1': 'H20.012', 'pdiagnosis2': 'H26.222', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': 'EDD37436-0E9D-45B5-8C46-A22CA763970A', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '461.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '13.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '145.0', 'pdiagnosis2_stringindexer_transform': '131.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '33.0'}, {'ChargeDetailID': '0CB92950-EE3F-424C-AA0A-445F2D738518', 'DateofService': '2020-11-26T00:00:00.000Z', 'Amount': '286.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8130', 'pdiagnosis2': 'H43.12', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '43.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '206.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '6.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '115.0', 'pdiagnosis2_stringindexer_transform': '21.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': '29737DE0-22D6-4D41-870C-DA59BE4A4E75', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '237.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '7.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'AA77712E-61BA-45A8-8E10-28057191D585', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '260.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '17.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '8.0'}, {'ChargeDetailID': 'D271CCF9-CBA9-4A48-8202-5ED6FE135BDA', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.372', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '85.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '92.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '16.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'F488446B-42F0-4C47-B313-AE5AD0AD4C08', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '1800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3211', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'M', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '273.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '7.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '22.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '4.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '0DC89466-2E6B-4DC7-B913-E5BBE5B168A0', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '86.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '27DEFE26-1A3A-4654-A273-799CEA02BB33', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H33.002', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '49.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '452.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '52.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '3D48B549-0C2E-4EC1-AA68-CD7DE250D65A', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '77.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '104.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '790CB21B-A420-4C48-8EBC-67AAC5413D69', 'DateofService': '2020-11-26T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.822', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '16.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '17.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '9DD3CB15-B27B-4F87-A3C8-A9FC8D1F56B5', 'DateofService': '2020-12-20T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.301', 'pdiagnosis2': 'H33.321', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '245.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '55.0', 'pdiagnosis2_stringindexer_transform': '28.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '9.0'}, {'ChargeDetailID': '2F71902A-768E-4A58-BE77-2AB583872514', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.351', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '85.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '182.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '13.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '305A3131-EFD4-420C-88B8-361D358D9857', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.341', 'pdiagnosis2': 'H34.8320', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '201.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '41.0', 'pdiagnosis2_stringindexer_transform': '9.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '65920EF9-C672-4D37-9A6B-795B0F656EDD', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3222', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '84.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '466.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '77.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '736AECC6-FFE6-48E7-9F0D-4AF0CDC06DED', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'D31.31', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '408.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '16.0', 'pdiagnosis2_stringindexer_transform': '19.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '9DEA15FC-C28D-42CB-8283-AC5B84B7FCEF', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'E11.3392', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '138.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '149.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '0B1BA7BE-21C6-4B0E-A25F-C198051FD597', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '123.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.073', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '43.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '84.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '0E9EF90A-1AEA-403E-9058-D6F60A449238', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '90.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '217.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '2.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '235B02A5-8AC3-47BD-86BB-CAA1A63C317B', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H20.9', 'pdiagnosis2': 'H40.42X0', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '56.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '67.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '69.0', 'pdiagnosis2_stringindexer_transform': '126.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '48A25AFB-573A-4213-9B1C-E3654825F634', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.023', 'pdiagnosis2': 'H43.823', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '47.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '455.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '78.0', 'pdiagnosis2_stringindexer_transform': '15.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': '49C499CF-DD82-4195-B4F3-F889BE7E0BC4', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E10.3391', 'pdiagnosis2': 'E10.3312', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '33.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '454.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '106.0', 'pdiagnosis2_stringindexer_transform': '78.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '691933B0-0D3D-4355-B453-79E791D6DC36', 'DateofService': '2020-11-26T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H40.051', 'pdiagnosis2': 'H44.701', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '18.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '99.0', 'DateofService_dayofmonth': '26', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '47.0', 'pdiagnosis2_stringindexer_transform': '115.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '28.0'}, {'ChargeDetailID': '83B52DD3-F13E-450B-84A7-CAA79A556718', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '6000.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3411', 'pdiagnosis2': 'E11.3512', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'F', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '164.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '9.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '31.0', 'pdiagnosis2_stringindexer_transform': '70.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '5.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '842AC936-21A9-4F15-8BF5-ACF6D4756EB0', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.713', 'pdiagnosis2': 'H35.70', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '44.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '354.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '148.0', 'pdiagnosis2_stringindexer_transform': '32.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'A3CC38B5-605E-476B-A987-2AC63C81F539', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.321', 'pdiagnosis2': 'H33.002', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '332.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '40.0', 'pdiagnosis2_stringindexer_transform': '52.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C3A37CEA-E1D8-4DB2-837C-D1041863EC7F', 'DateofService': '2020-12-19T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.322', 'pdiagnosis2': 'H35.413', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '50.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '276.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '51.0', 'pdiagnosis2_stringindexer_transform': '14.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '5.0'}, {'ChargeDetailID': 'CF82712C-91A7-46D2-9F6D-19A1BC8A1A0D', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H44.23', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '82.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '102.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '19.0', 'pdiagnosis2_stringindexer_transform': '138.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '485494F4-D20A-4303-89BB-F09C5646A9EA', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H20.12', 'pdiagnosis2': 'H43.89', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '430.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '73.0', 'pdiagnosis2_stringindexer_transform': '48.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '4FEA26C5-DD7F-4A65-B859-D5FECFF66684', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '250.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99215', 'CptModifier': '25', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'D31.31', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '97.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '499.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '12.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '19.0', 'pdiagnosis2_stringindexer_transform': '19.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'B48620D1-DD13-471B-936F-AE9970B850B6', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '342.0', 'PatientBal': '-15.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99205', 'CptModifier': '25', 'pdiagnosis1': 'H31.8', 'pdiagnosis2': 'H25.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '8CA1878F-1BD4-4595-9AFF-3FF6967CD5CB', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '259.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '17.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '123.0', 'pdiagnosis2_stringindexer_transform': '130.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '50.0'}, {'ChargeDetailID': 'BBF82897-F57E-484D-8159-C4E14BBECB5C', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '52.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'F0B25A0F-D965-460E-AA37-EA676B8BE27F', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E11.3312', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '88.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '348.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '27.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '279D822D-CECC-407C-8F7D-DA683C10C7B1', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.353', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '424.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '17.0'}, {'ChargeDetailID': '27E0E710-3B33-4DEC-BED0-FDB55F95EEE1', 'DateofService': '2020-11-28T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.342', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '173.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '49.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '8793C097-7DC2-47B0-8639-CBD108B34F5F', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.711', 'pdiagnosis2': 'H35.372', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '54.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '25.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '24.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '1D2CA217-3B28-4C61-89C2-7C55BAB37BDD', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H35.413', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '139.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '14.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '3A4437B5-45EE-4734-BF9C-C7BA808C123B', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.352', 'pdiagnosis2': 'H43.823', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '432.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '44.0', 'pdiagnosis2_stringindexer_transform': '15.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '13872B75-E219-4B25-9F7E-24F021CF0B48', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '-38.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '25', 'pdiagnosis1': 'H35.63', 'pdiagnosis2': 'H35.3134', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '97.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '37.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '92.0', 'pdiagnosis2_stringindexer_transform': '35.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '377FADE6-202C-4841-BDD4-82367D36097C', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H25.13', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'E0FE1EC8-51D4-430D-8B8B-E2F9966875F8', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '489.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '10.0', 'pdiagnosis2_stringindexer_transform': '30.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '25.0'}, {'ChargeDetailID': '6A29D415-E299-4812-B8CD-8CD471F28115', 'DateofService': '2020-12-10T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H43.12', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '84.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '358.0', 'DateofService_dayofmonth': '10', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '21.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '84A66A4F-CB48-4BA8-B11C-F2EA34C4A1A4', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '6000.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3411', 'pdiagnosis2': 'E11.3512', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'F', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '398.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '9.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '31.0', 'pdiagnosis2_stringindexer_transform': '70.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '5.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D7B0792D-0B5F-4CAA-9787-237DFECCA98B', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.70', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '5.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '32.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '6D22E643-C022-4637-A81F-22A0DB410A33', 'DateofService': '2020-11-29T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3211', 'pdiagnosis2': 'H34.8120', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '220.0', 'DateofService_dayofmonth': '29', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '22.0', 'pdiagnosis2_stringindexer_transform': '26.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '7C771536-6EAB-4FEA-B848-E077662307D7', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': 'LT', 'pdiagnosis1': 'E11.3312', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '367.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '27.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'DD0C3E18-53F4-4001-9334-BEC408540A3E', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H33.003', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '26.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '11.0', 'pdiagnosis2_stringindexer_transform': '154.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'EA11E355-7A8A-4BAD-9066-1FC3E824CBC5', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'C69.32', 'pdiagnosis2': 'T66.XXXA', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '54.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '188.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '23.0', 'pdiagnosis2_stringindexer_transform': '116.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '267E6CA7-6B3A-4582-B01B-9DD2C4AC8DEE', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.811', 'pdiagnosis2': 'H35.413', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '315C773B-4EF1-4164-AB43-6817D993329F', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '222.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '28.0', 'pdiagnosis2_stringindexer_transform': '14.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '42.0'}, {'ChargeDetailID': '43B4ED70-4EAA-4C2A-AB60-F4A3848BF5E9', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '25', 'pdiagnosis1': 'D31.32', 'pdiagnosis2': 'H35.3122', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '75.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '1.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '132.0', 'pdiagnosis2_stringindexer_transform': '10.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '7D59A1A0-3C61-4F7A-AD1E-0EF39B392E83', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '53.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '143.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'FF80B585-D64F-45DC-9CE9-59C51F3001D2', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H34.8322', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '85.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AAB72015-9704-40ED-A2E7-87468E793D48', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '30.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '12.0', 'pdiagnosis2_stringindexer_transform': '43.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '47.0'}, {'ChargeDetailID': '324408CC-B86D-49E1-A0DE-99A7EE03276E', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8120', 'pdiagnosis2': 'H20.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '40.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '156.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '29.0', 'pdiagnosis2_stringindexer_transform': '29.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '3E505BF0-6673-4C9F-996F-64E07E72E60C', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '106.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '5FFAD31C-B0EF-4A88-B7EA-2F02D7B336D5', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H43.823', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '20.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '15.0', 'pdiagnosis2_stringindexer_transform': '15.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '8CC0B44A-2E34-417F-A084-3E2F067F6966', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '383.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '76512', 'CptModifier': 'LT', 'pdiagnosis1': 'H33.042', 'pdiagnosis2': 'Z98.890', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '24.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '330E2DD7-352E-4E5C-9669-AE796ABFE5DC', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '248.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '13.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '89.0', 'pdiagnosis2_stringindexer_transform': '50.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '18.0'}, {'ChargeDetailID': 'B29EEE24-30E4-4B06-9FA3-777B1E99463D', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.811', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '415.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '5.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C84DCC7E-7D84-433A-8D23-74F8BB79EE40', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '25', 'pdiagnosis1': 'H43.12', 'pdiagnosis2': 'H35.413', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '78.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '58.0', 'pdiagnosis2_stringindexer_transform': '14.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'E776DC1F-10CF-4EC1-A151-05EC29E6C03C', 'DateofService': '2020-11-02T00:00:00.000Z', 'Amount': '6000.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '43.0', 'Gender': 'F', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '204.0', 'DateofService_dayofmonth': '2', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '9.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '5.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '40F1314D-EBF6-49BC-B1A8-674F07F92039', 'DateofService': '2020-11-01T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.353', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '92.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '267.0', 'DateofService_dayofmonth': '1', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '66E28365-DDB9-4F59-916F-2B171C5D93AB', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H04.123', 'pdiagnosis2': 'H35.412', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '59.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '20C2761F-16B5-42B7-8C93-82A112735F02', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '40.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '65.0', 'pdiagnosis2_stringindexer_transform': '23.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '34.0'}, {'ChargeDetailID': '88539DEA-C521-4D06-A758-7FEBC99D5C5E', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8312', 'pdiagnosis2': 'H34.8120', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '426.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '37.0', 'pdiagnosis2_stringindexer_transform': '26.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '16.0'}, {'ChargeDetailID': '8CCEE88C-C36B-4DE3-88D7-00D78C1D9B0A', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.321', 'pdiagnosis2': 'H33.312', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '59.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '274.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '40.0', 'pdiagnosis2_stringindexer_transform': '51.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'C0AC5031-0382-4350-82C5-5B80634DEEAC', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8312', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '54.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '117.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '37.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'E5775C4C-F86F-49D2-8B58-22ACA6668352', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'E11.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '65.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '302.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '16.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '08683FBF-CD1F-42D9-A442-EF696A6D7C83', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '70.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '496.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '20.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '289AA716-A1D3-48FF-B830-4BF9DD4A4FB7', 'DateofService': '2020-12-18T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'T85.29XA', 'pdiagnosis2': 'H20.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '54.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '493.0', 'DateofService_dayofmonth': '18', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '116.0', 'pdiagnosis2_stringindexer_transform': '29.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '9.0'}, {'ChargeDetailID': '28D97A0D-03AE-4405-BFD3-66E00EDAE377', 'DateofService': '2020-11-06T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '85.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '107.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '55F62337-F077-4BA9-87D8-4C7AC0736913', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H33.323', 'pdiagnosis2': 'H35.463', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '23.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '835BC6C3-FEDE-4AF9-8B88-91628A98389C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '465.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '96.0', 'pdiagnosis2_stringindexer_transform': '58.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '45.0'}, {'ChargeDetailID': '56A0F3CD-F2AA-4492-A3F3-D8694240FF34', 'DateofService': '2020-11-19T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.823', 'pdiagnosis2': 'H11.001', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '230.0', 'DateofService_dayofmonth': '19', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '50.0', 'pdiagnosis2_stringindexer_transform': '118.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': 'B16FA6E7-E985-411C-AA06-A1F7E6BBCEFB', 'DateofService': '2020-11-21T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'C69.21', 'pdiagnosis2': 'C69.22', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '7.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '5EA62C5D-D9C0-4AFE-ACAF-4AD36F6B1057', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '203.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '59.0', 'pdiagnosis2_stringindexer_transform': '63.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '24.0'}, {'ChargeDetailID': '0BB8A5C4-7444-4DEE-A3AC-2E0CFF7A3477', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '61.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '1DB8D73F-61E5-4065-9342-592BE88E5981', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H40.053', 'pdiagnosis2': 'H43.823', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '194.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '109.0', 'pdiagnosis2_stringindexer_transform': '15.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '4C323EC0-5AB8-4CAF-9381-0011A0141AD7', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H44.23', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '37.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '487.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '131.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '37.0'}, {'ChargeDetailID': '07BDF3AA-3B6E-4650-BDFC-596FE687C33F', 'DateofService': '2020-11-05T00:00:00.000Z', 'Amount': '50.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92225', 'CptModifier': 'LT', 'pdiagnosis1': 'H35.412', 'pdiagnosis2': 'H33.322', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '48.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D1C4AE8C-46B5-464F-91E1-A39C5C76CC25', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '307.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '8.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '36.0', 'pdiagnosis2_stringindexer_transform': '53.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '39.0'}, {'ChargeDetailID': '3F664203-58B4-411D-835A-667F32E1AF1E', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '57', 'pdiagnosis1': 'H33.311', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '211.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '5.0', 'pdiagnosis1_stringindexer_transform': '54.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '94F9C8EF-40F1-4CB4-B8F1-71EA8B1482D9', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '246.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92004', 'CptModifier': '0000', 'pdiagnosis1': 'H43.392', 'pdiagnosis2': 'H33.321', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '386.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '10.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '66.0', 'pdiagnosis2_stringindexer_transform': '28.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A56274A7-1DCD-46CB-84FC-83FD3C26472D', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '-25.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H10.45', 'pdiagnosis2': 'H33.41', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '52.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '429.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '100.0', 'pdiagnosis2_stringindexer_transform': '27.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': 'BDB2ADB4-356A-40AF-9E57-C6ECA2EB1235', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3111', 'pdiagnosis2': 'H35.373', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '82.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '470.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '30.0', 'pdiagnosis2_stringindexer_transform': '4.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'F28706C5-DDB5-4FC3-9F7D-4896DB86AF18', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E11.9', 'pdiagnosis2': 'H35.3231', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '456.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '97.0', 'pdiagnosis2_stringindexer_transform': '125.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '3DA05017-5FD5-4CBE-9CC5-956876EECBE6', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3130', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '375.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '39.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '6AD7EB61-BE8E-49A3-929C-CE2CA6C95779', 'DateofService': '2020-12-06T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '64.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '161.0', 'DateofService_dayofmonth': '6', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '5.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '3.0'}, {'ChargeDetailID': '8B79B6F2-FE78-4775-8668-2A35ECA963E5', 'DateofService': '2020-12-21T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'T66.XXXA', 'pdiagnosis2': 'H35.351', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '58.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '347.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '114.0', 'pdiagnosis2_stringindexer_transform': '13.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '98AEC5FB-BE93-4E59-A4A9-9533419B73B6', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '312.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'A24C0F04-7684-4CC9-BFBF-CD7E01137187', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8120', 'pdiagnosis2': 'H35.372', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '265.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '29.0', 'pdiagnosis2_stringindexer_transform': '7.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'C22535F6-D4C8-4B58-B0C4-BA24CE26B9E7', 'DateofService': '2020-11-16T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.61', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '101.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '450.0', 'DateofService_dayofmonth': '16', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '25.0', 'pdiagnosis2_stringindexer_transform': '42.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D9115251-3465-4EF5-B9B3-54B0EAF82045', 'DateofService': '2020-11-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '36.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H43.823', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '57.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '189.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '13.0', 'pdiagnosis2_stringindexer_transform': '15.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '24253A09-E06B-4C51-A3AD-0709BD0DA4B5', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H31.091', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'F473817F-7515-46B7-97F9-348C88D7C348', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '360.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '133.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '21.0'}, {'ChargeDetailID': '449D29CA-9C67-4DA9-95FE-C1531DA5DE4C', 'DateofService': '2020-12-13T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H34.8320', 'pdiagnosis2': 'H43.812', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '68.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '279.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '34.0', 'pdiagnosis2_stringindexer_transform': '6.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '77C723D8-92FE-487C-825B-93C8605DC74E', 'DateofService': '2020-12-04T00:00:00.000Z', 'Amount': '800.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3213', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '239.0', 'DateofService_dayofmonth': '4', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '3.0', 'pdiagnosis1_stringindexer_transform': '42.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '8BAEB725-2E28-4DE9-B911-836846DF6F11', 'DateofService': '2020-11-21T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '90.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '368.0', 'DateofService_dayofmonth': '21', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '10.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D89C117A-4B7C-471A-94A0-1F6F022193BB', 'DateofService': '2020-12-17T00:00:00.000Z', 'Amount': '162.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.412', 'pdiagnosis2': 'H33.001', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '20.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '109.0', 'DateofService_dayofmonth': '17', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '2.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '36.0', 'pdiagnosis2_stringindexer_transform': '92.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'E0D39756-5A76-4FA1-BA20-C957C3486DFC', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '50.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92225', 'CptModifier': 'LT', 'pdiagnosis1': 'H43.812', 'pdiagnosis2': 'H43.12', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '120.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '8.0', 'CptModifier_stringindexer_transform': '4.0', 'pdiagnosis1_stringindexer_transform': '33.0', 'pdiagnosis2_stringindexer_transform': '21.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '0A6DE37A-0F94-43B3-84E3-7590769F820E', 'DateofService': '2020-11-13T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3222', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '90.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '48.0', 'DateofService_dayofmonth': '13', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '77.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '12.0'}, {'ChargeDetailID': '0F43FFB3-7DF2-4156-BD39-A6C0667C7392', 'DateofService': '2020-11-15T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.391', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '63.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '353.0', 'DateofService_dayofmonth': '15', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '1.0', 'pdiagnosis1_stringindexer_transform': '113.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '08AD40B0-B651-4069-AA92-9A64FC246B17', 'DateofService': '2020-11-07T00:00:00.000Z', 'Amount': '45.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92226', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.101', 'pdiagnosis2': 'H33.41', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '28.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '97.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '11.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '137.0', 'pdiagnosis2_stringindexer_transform': '27.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '0EC59014-45A0-45B7-AB3D-12367ECD4174', 'DateofService': '2020-12-07T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H31.422', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '72.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '319.0', 'DateofService_dayofmonth': '7', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '111.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '13E519C2-73BD-451A-A651-55B67924A9B6', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '71.0', 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '317.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '4.0', 'CptModifier_stringindexer_transform': '2.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '2.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '11.0'}, {'ChargeDetailID': '26C382E6-F716-4F7F-A72C-A6C7150C2066', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H30.013', 'pdiagnosis2': 'H44.2A3', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '61.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '376.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '87.0', 'pdiagnosis2_stringindexer_transform': '143.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '4.0'}, {'ChargeDetailID': '989C36A9-F0D6-48B5-8D26-F5A56E610792', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.61', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '73.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '485.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '25.0', 'pdiagnosis2_stringindexer_transform': '42.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': 'C4ADE72E-AD16-4711-886D-AB997CCB5E71', 'DateofService': '2020-11-05T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3111', 'pdiagnosis2': 'H35.3122', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '492.0', 'DateofService_dayofmonth': '5', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '30.0', 'pdiagnosis2_stringindexer_transform': '10.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'DE90AF3B-500B-42BC-B1F8-D4E5971DD853', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'H35.3131', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '81.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '140.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '18.0', 'pdiagnosis2_stringindexer_transform': '47.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '03A51C92-3977-42BE-B158-53B124029BCC', 'DateofService': '2020-11-20T00:00:00.000Z', 'Amount': '118.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.342', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '288.0', 'DateofService_dayofmonth': '20', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '5.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '33.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '176F9200-4582-49AB-80EB-558534EC297C', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'E11.3513', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '491.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '11.0', 'pdiagnosis2_stringindexer_transform': '39.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '12.0'}, {'ChargeDetailID': '2861F7ED-8D48-436D-9AD0-FEF67A864566', 'DateofService': '2020-11-01T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H35.353', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '88.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '137.0', 'DateofService_dayofmonth': '1', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '9.0', 'pdiagnosis2_stringindexer_transform': '8.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': '7157C3A4-F8EE-42C2-864E-C9E7FD5C62C6', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '3600.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'Z79.4', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '74.0', 'Gender': 'M', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '460.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '7.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '13.0', 'pdiagnosis2_stringindexer_transform': '0.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '4.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '2.0'}, {'ChargeDetailID': 'FB5D5A33-BA7C-4140-AB6B-9AC8E94FABA6', 'DateofService': '2020-12-14T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.50', 'pdiagnosis2': 'H35.3221', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '199.0', 'DateofService_dayofmonth': '14', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '104.0', 'pdiagnosis2_stringindexer_transform': '2.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '3AAEBB1A-A489-4699-A7B4-F237F130B7EE', 'DateofService': '2020-12-11T00:00:00.000Z', 'Amount': '286.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H34.8320', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '56.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '446.0', 'DateofService_dayofmonth': '11', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '6.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '4.0', 'pdiagnosis2_stringindexer_transform': '9.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': 'A3646A59-090D-4D1F-AE88-16B127AF81ED', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H33.312', 'pdiagnosis2': 'Z96.1', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '200.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '45.0', 'pdiagnosis2_stringindexer_transform': '40.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '8.0'}, {'ChargeDetailID': 'A90B0781-610E-4FE4-80ED-BD9F6ABDE174', 'DateofService': '2020-11-09T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.352', 'pdiagnosis2': 'H20.9', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '76.0', 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '448.0', 'DateofService_dayofmonth': '9', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '44.0', 'pdiagnosis2_stringindexer_transform': '29.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': 'AF591C35-31A5-45DD-9399-292D82D794AF', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3122', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '78.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '396.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '12.0', 'pdiagnosis2_stringindexer_transform': '10.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'D06DE332-BDE9-4D63-8959-BF68C5A40297', 'DateofService': '2020-11-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.813', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '91.0', 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '331.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '1.0', 'pdiagnosis2_stringindexer_transform': '3.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': '23AEEFFE-19BA-487C-A1D5-A88964FF3BC1', 'DateofService': '2020-12-27T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '69.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '422.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '3.0', 'pdiagnosis2_stringindexer_transform': '10.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '341B166A-3402-4A75-AD95-D57FDDA27DC9', 'DateofService': '2020-12-12T00:00:00.000Z', 'Amount': '148.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H35.033', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '66.0', 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '214.0', 'DateofService_dayofmonth': '12', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '0.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '11.0', 'pdiagnosis2_stringindexer_transform': '1.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '0.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': '45D50939-3539-478D-AABE-C57C6791CF1F', 'DateofService': '2020-11-08T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'Q14.1', 'pdiagnosis2': 'H47.233', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '36.0', 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': '1.0', 'ChargeDetailID_stringindexer_transform': '402.0', 'DateofService_dayofmonth': '8', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '93.0', 'pdiagnosis2_stringindexer_transform': '95.0', 'Gender_stringindexer_transform': '0', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '7.0'}, {'ChargeDetailID': '4A1D1603-893E-46B7-92C1-8C3069A4B63D', 'DateofService': '2020-11-27T00:00:00.000Z', 'Amount': '400.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99354', 'CptModifier': '0000', 'pdiagnosis1': 'C69.32', 'pdiagnosis2': 'Z71.89', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '79.0', 'Gender': 'M', 'DepartmentCategory': 'Pediatrics', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '268.0', 'DateofService_dayofmonth': '27', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '15.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '23.0', 'pdiagnosis2_stringindexer_transform': '31.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '6.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '6.0'}, {'ChargeDetailID': '502690F0-D587-4D27-B3A7-5B9679F7E217', 'DateofService': '2020-12-28T00:00:00.000Z', 'Amount': '3600.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3312', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '67.0', 'Gender': 'M', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '280.0', 'DateofService_dayofmonth': '28', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '7.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '6.0', 'pdiagnosis2_stringindexer_transform': '18.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '4.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}, {'ChargeDetailID': '97E40160-D786-4460-B2D1-FCFFE2DB599F', 'DateofService': '2020-11-30T00:00:00.000Z', 'Amount': '212.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.13', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '62.0', 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': '0.0', 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '483.0', 'DateofService_dayofmonth': '30', 'DateofService_month': '11', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '1.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '0.0', 'pdiagnosis2_stringindexer_transform': '67.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '1.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '0.0'}, {'ChargeDetailID': 'A372D150-FF7E-4BA6-9349-DDD6451A1084', 'DateofService': '2020-12-03T00:00:00.000Z', 'Amount': '300.0', 'PatientBal': '0.0', 'InsuranceBal': '0.0', 'PrimaryInsuranceBal': '0.0', 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.712', 'pdiagnosis2': 'H31.002', 'IsBillable': '1.0', 'Isauthorized': '0.0', 'PredeterminationIndicator': '0.0', 'MedNecessityRequiredIndicator': '0.0', 'Age': '46.0', 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': '1.0', 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': '2.0', 'ChargeDetailID_stringindexer_transform': '352.0', 'DateofService_dayofmonth': '3', 'DateofService_month': '12', 'DateofService_year': '2020', 'CPTCode_stringindexer_transform': '3.0', 'CptModifier_stringindexer_transform': '0.0', 'pdiagnosis1_stringindexer_transform': '77.0', 'pdiagnosis2_stringindexer_transform': '152.0', 'Gender_stringindexer_transform': '1', 'DepartmentCategory_stringindexer_transform': '3.0', 'SpecialityCategory_stringindexer_transform': '0', 'PayerId_stringindexer_transform': '1.0'}], 'attributesFT': [{'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'max': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'DateofService', 'transformation_label': 'Extract Date'}], 'feature': 'DateofService', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Amount', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '357.38', 'stddev': '823.57', 'min': '45.0', 'max': '6688.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PatientBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '-1.54', 'stddev': '18.4', 'min': '-250.1', 'max': '123.41', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'InsuranceBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.27', 'stddev': '3.94', 'min': '0.0', 'max': '76.68', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PrimaryInsuranceBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.15', 'stddev': '3.43', 'min': '0.0', 'max': '76.68', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '91232.81', 'stddev': '6100.5', 'min': '67028', 'max': 'J9035', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '5.5', 'stddev': '12.4', 'min': '0000', 'max': 'RT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'B58.01', 'max': 'T85.29XA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'C69.22', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'IsBillable', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '1.0', 'stddev': '0.0', 'min': '1.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Isauthorized', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PredeterminationIndicator', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'MedNecessityRequiredIndicator', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Age', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '67.76', 'stddev': '16.27', 'min': '5.0', 'max': '102.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'label', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.91', 'stddev': '0.29', 'min': '0.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Gender_Recoded', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '500', 'mean': '1.45', 'stddev': '0.5', 'min': '1', 'max': '2', 'missing': '0'}}, {'feature': 'ChargeDetailID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '249.5', 'stddev': '144.48', 'min': '0.0', 'max': '499.0', 'missing': '0'}}, {'feature': 'DateofService_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '14.58', 'stddev': '8.27', 'min': '1', 'max': '31', 'missing': '0'}}, {'feature': 'DateofService_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '11.44', 'stddev': '0.5', 'min': '11', 'max': '12', 'missing': '0'}}, {'feature': 'DateofService_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '2020.0', 'stddev': '0.0', 'min': '2020', 'max': '2020', 'missing': '0'}}, {'feature': 'CPTCode_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '2.18', 'stddev': '3.55', 'min': '0.0', 'max': '22.0', 'missing': '0'}}, {'feature': 'CptModifier_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '0.42', 'stddev': '0.98', 'min': '0.0', 'max': '6.0', 'missing': '0'}}, {'feature': 'pdiagnosis1_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '33.73', 'stddev': '38.52', 'min': '0.0', 'max': '148.0', 'missing': '0'}}, {'feature': 'pdiagnosis2_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '35.52', 'stddev': '41.1', 'min': '0.0', 'max': '155.0', 'missing': '0'}}, {'feature': 'Gender_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '500', 'mean': '0.45', 'stddev': '0.5', 'min': '0', 'max': '1', 'missing': '0'}}, {'feature': 'DepartmentCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '0.91', 'stddev': '1.2', 'min': '0.0', 'max': '7.0', 'missing': '0'}}, {'feature': 'SpecialityCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0'}}, {'feature': 'PayerId_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '4.89', 'stddev': '9.31', 'min': '0.0', 'max': '50.0', 'missing': '0'}}], 'path_AutoSample': '/FileStore/platform/sampleData/60891770a187c956b337cf50_AutoFE/part-00000-tid-3917072144446901006-f58791a3-edf4-48b1-a470-3dde18d61c22-172002-1-c000.csv', 'listOfGraph': [{'values': {'label': ['174DA322-870E-429F-A9A9-BE22340E9836', 'A1575911-0E83-4479-B182-D26EDAD439F6', 'A87EF1E9-E636-4742-8E2B-414477926782', '1FFF3E45-B9F4-4CFB-8580-C4B05D4822DD', '20BC5C22-2C34-494C-B75F-55BDD8B54263', '210DAE19-9257-4180-9057-E5817F555A88', '3FA21D4D-05B5-4C43-A33E-A01D20B45A99', 'DC65D7C3-0822-4E54-86EF-DC7639D38C62', '11327DC4-1599-46B2-A000-C05D135E1F91', '853B5BAB-0A5F-4B44-85E0-FEE0AA663796', '92F7294B-4F6C-41B5-A9BD-65CE72B27FC0', '97BB8C89-8E47-400B-B546-B662E9EDCA00', '9C7B2481-4469-4FB8-AC5D-18EB2D29F213', 'BCE331FC-BC9E-43B2-B9C3-1ADE95CF4081', '0391FBA9-7748-4F50-8CA2-FC4EB5116F6A', '88D923C8-5217-46AF-887E-CB88A2E6C3E3', '8AAE78A7-7725-4C1C-B55C-82B41A364A2B', 'B5C5BD20-64AE-4166-8115-14DA13624764', '343273F8-E598-4815-A700-0683D31F3291', '50F7ADC5-2166-4461-B01E-8D8FAA92304B', 'A9EC659A-ED07-4B6C-84CA-D4E45411C3C6', 'DCBDA7E0-A5F8-4D0F-836D-66187FEB5B6F', 'E907166F-CE0F-49D2-A04B-40734F1A6F0C', '6518F89E-CD31-4903-89E2-2F31BEB28508', '7B41BFB4-B1D7-47A9-9659-6E8319429423', '7E4D4049-B1BC-4135-8020-D69318B66B18', '94FEF155-5ED7-4DF9-836F-78557484B8E4', 'A6B1DE4F-2FBD-4AB9-B09E-77B3E8C44771', 'B9F344F0-CEE0-4EA2-A55C-274FAB39E9DB', 'BBE4E59F-B282-490E-8FCD-6C9F5E8069EB', 'EC206AC2-403A-4047-B82E-82C383E29CCE', '01867531-8532-4FD4-BE17-232F39CFA32D', '2ECD88B7-05A0-4959-8736-397F8CF74784', '524D7B98-2D7C-458D-8877-35CDE8F3D981', '54E3991B-1609-470D-8095-C14FF5A9444D', '8F8BDCC6-5B74-4EBA-A154-8576DE0FFC13', 'A28023B3-9D64-44D3-BD13-2A22A1350A24', '1B71D2B0-C473-4A8D-9FA0-76A0692C5102', '6CF3754A-BC46-430C-A116-9329959CD556', 'A6CED81D-7B1D-441A-8EFC-544E8D31FD26', 'A82270CF-0B59-4C62-877D-9FAF720B6E5D', 'D9A3771E-E97A-4EBE-BFA9-54577F445F21', '1AA261AC-C877-4F0D-8372-F268A4D5F90B', '2B385821-C105-42CA-84F0-CEF8BFD1681A', '9D5CF424-8DA6-4A2B-8716-FDB93B808F3C', 'A17DF3A2-8077-49B4-A114-11F77F67E7C9', '0E47C422-450E-4A04-9B90-680F296C9686', '46CB6A8C-362F-4345-941E-842F429B48B5', '8CD490B4-CEBE-4DFE-BA0A-C805ACDF0D34', '9F54BD25-7C67-454A-B100-CD9974B62C7D', 'A78BD078-2165-4C68-B398-9AB6E94C0FB6', 'C72EBDAD-1D1D-40D0-BA4D-A9CDD08430A6', 'DBFEC39B-37B2-4B37-9D95-A444C27A619C', 'E1C8F5F6-2DDF-4370-B40C-D86322919657', 'EB7C11AD-0A01-4359-B1AB-289373C669DC', '16062E35-A49E-4227-A3E6-E8DDBFEBA5D2', '20416F30-920B-4925-8642-C83F38ECFCEE', '642D42E4-275F-4E86-8378-21E2CD291224', 'FBB301BB-1560-415C-91B5-A6E8B0B26B6C', '4011D505-BE33-4A24-AD40-240BFDDB2766', '74FC2B0E-AEC7-4263-951F-14E72EF044DA', 'A5881100-AA89-4D5D-80AC-AE2419271C5D', 'D0667457-EF74-4FA4-91D9-7169C64A53DB', 'E16D36F3-C4D2-4C4B-A004-E06C823C7960', '20463C7A-45B7-4BD4-A3EA-53ACC03E93F1', '6E834EA0-8A83-4326-8FAE-80C6DEE3B045', '8607EC2B-26F1-4A40-9646-5990D37EC622', 'A050F893-D9BB-4FB6-9B4B-B7AA83355D60', 'C2F471A9-22FA-4A9A-A14C-49A5F292662D', 'D6F6C707-D42D-4485-A571-82EEF5002E64', 'E164830A-C381-4AEC-81EE-3B116E56B496', '2701DB53-B3A0-466C-BFCB-6684D7059DC4', '89D7AAB0-BF88-493E-A7AB-7D6D793273DB', '8A07AC76-56FD-4E13-BFEA-A724E4C2D603', '8E5FAFBE-87F5-4C26-88EA-002384A50B1D', 'E67C3D6F-A4D2-4EA9-B41A-E5BEB54E0209', 'AC68E17B-9EBB-45A3-BD22-60367A203515', 'D0BA312C-C42D-4695-A6E4-71096BE6854B', '50C6C060-08F2-4F75-91BE-EF1D34F6EDA9', 'C99C5CC6-DF6E-4024-88A6-04D326F6D6B2', '1A70E6C3-0B4F-4338-A15C-903F552EC30C', '279BE041-8831-43CF-A0EB-219F709A986F', '3E0A6D69-9B07-4BFC-B389-24E0C884701E', '9017B6B5-6C3F-42FC-ABD4-67A54E07D67F', 'F16597BA-3D84-47E3-AF6C-072ECBD33467', '2AA7F9ED-3694-4ADD-91A8-3EFB8F8054FB', '45F8D74C-DE28-4F0D-B781-AB81CCD5D5CD', 'D7B8BD7C-A3F9-422B-B491-59F236BADE4D', 'E9DB7A1D-67B8-4936-B7C7-FB071EA72907', '2B6E92D7-EA15-4F53-B133-2A1F9B1E958C', '86E8386B-2651-4928-8CD1-631CE24D0432', '97D21C13-7AFC-4865-888F-5E92CC51F2ED', 'A40F161A-A931-41D9-A65B-597347BC9D6A', 'C43925F7-AA9E-419B-B0DB-0A044E67C4B5', 'D05F93BC-037E-4BD7-8E65-FDA46391791F', '14167C30-8123-4DCF-B3C5-05D3F16B6391', '2022006D-F4D6-4F8A-B76D-1617C97083CA', '25EF819E-5F4C-43DA-BEBB-C4FAD71B0EF5', '8648E212-02DB-4D77-BB02-88B67C56A9DE', 'EDDC8ACA-6742-4C6C-A200-FA42DD89285D', '03E0DCDC-A071-4987-A7CE-268D9C3D017B', '161B0D6C-5A50-4EEE-9B2A-E4DE8D9EBC1C', '1BD4EAF6-F3F8-4369-93B3-4E6144A69CAD', '20AC5B30-ACB1-4714-9794-5454B92C1A23', '3FE9456A-15CB-4840-90CB-05F7DF34CACE', '42F5D00A-6F59-4F63-ACCA-9EFDA0DD9194', '6E80EF7D-E761-4720-A1BD-9E2CA7FDD21F', '734CA534-2CD7-4A77-9D03-CF2390E8B0D6', 'B0501331-13EF-4126-B052-44D67EDFE7F9', 'C0B29F0F-BBE1-47C7-B625-0989FDCC5288', '136261A4-52B0-43B5-B099-1EF929506A93', '252D8E17-A9E8-48F5-84D3-6DA9B452EE75', '5A8270A6-53BD-4916-8805-B762FF67BE20', '67841660-18E4-45DE-B386-285C53582762', '9C60FA6D-FAAF-46A9-A149-1F941986D089', 'D6FE7A69-6B2B-495F-A36A-789DF588F6D5', 'D9C0CCE1-6D5D-4DB7-8376-D8562C150145', '42B818D0-170D-4838-BC4A-7D22039C0F53', '7DC8A130-8030-4915-8BF5-54015B27DAB6', 'A3555B0C-5A99-4619-B6E8-9831D99F0210', 'AECE54B4-2AE0-44A1-9409-2BFB52D07BD2', 'BE7612D0-7548-49C7-800E-173BAF9DF290', '0EB85789-CE25-4CCC-850E-44D2C3496272', '45F8ABD8-1D29-495F-B63D-866175B861A0', 'A69FF747-5E5E-44A3-A2AE-806C56D75743', 'B1C24F17-5A6A-4D76-A42E-A273CF94898F', '41D99DEA-233E-47A6-98C0-8B6F522D1B70', '53BD3672-31BA-4CE7-B2F3-2AFFC29C5F6F', '55F86AF5-F804-4A84-90ED-12E4125E1A8D', '8C94AF61-5EC3-44D7-B4B3-BBEB64AD2A23', 'B52B5682-9815-4EC1-A594-912A9724EB74', '079F8DAC-D8C1-48FA-9743-BC2AA08E287C', '2D60073D-3E0E-4347-B0A6-7E0CE91E92FA', '99772A77-B764-48FF-9242-5700C9572B65', 'E45AB682-6C1F-4B84-A1AC-3D1CD79AC3A9', '51526EAE-3FB7-4C7E-9A33-26990B70EFFE', '540B2A5F-11A9-4F0F-8038-B5A1ECE5E583', '91D48DF5-F63A-466A-AFB9-2E65C4F2D44D', 'ADF5D273-DB58-4F7F-8DF0-9D9D9647DA16', 'F672AF9B-E361-4572-A9C0-8B0731F9CD3E', '34803FD1-6C40-4323-8657-F1FCC3E34990', '5327CFC0-5C48-435B-98EB-A76AAE858941', 'D2DF16C4-D3AB-4E5C-9350-8F6FDE2EDA8D', 'E2DBCD71-B901-41F3-97B1-ED71030C7F2F', '049BA1F1-F362-48BE-9393-C05CCEA8694B', '13002A76-350E-4525-BDA1-9F181E056F74', '167E1195-CC7A-4F8C-8D91-9577BCBEE8FB', '1F264D61-D5D7-4505-9AC2-25ACF275E3BD', '23BCDE27-9F80-48E4-B29D-348C871829B3', 'A71A1D2F-3F1A-4CA6-AA98-E5D6A855C250', 'B62DA993-E320-4CEE-9D23-B3238CA5759A', 'B6375BF2-EFA5-44C9-A05C-F71DACD5F8A8', 'C924A1C9-2E9A-4BF3-8BC7-25BFAEB02CE1', 'D2300410-689E-453C-895D-FC7FDA559759', 'F12A6DBF-FBB4-4C62-8EE0-0F677D58EC3B', 'FA9EEE47-7F76-4264-9A82-9B370BCF9534', 'FBBC8CD5-C35A-42E5-AEBA-C5F61050B726', '045700E8-F572-417C-A8DD-41E76C5C159A', '1A088AE8-D8BA-404D-97E5-9DB1DFB972F8', '1C109ACE-6610-4098-BE1A-C64F7805EBCA', '25251236-8157-4EE2-89CD-C5A600E278BE', 'A1818C97-82BE-4115-8C3C-3532CB8DE838', 'AC9C3FBE-B694-4A6F-9A6F-6CC4563C5C73', 'B5E35D7B-A0F8-4A47-B72F-1D372236FC63', 'C1119267-721A-4606-9D06-09FF240505EC', 'C3339BC7-4E2E-4E0E-822E-7EE8E489451C', 'CDDFE064-998C-4CDD-97A2-04647A33EE7A', 'D4930936-02DA-4DBB-A3B2-5391AC5D7153', 'FD8029C5-4ACA-4F97-8CE6-144AE49C8BCB', '4C3D48F4-860F-49EF-812D-75B08DC1359C', '58207A31-753A-42B1-86D8-E750AEA10079', 'EAF0CF6B-9569-41B6-8B77-6335F34B3D7E', '693B32F4-ECA0-4844-80DF-6D9C94DD57BA', '7E2F8EA5-4A1D-4490-A208-49CE684BB427', '83A30AD9-1329-4675-99E9-AF5ACBE98CFA', '84F4A744-F946-4B1E-99DF-20E0538C5F4F', '90C2C4BC-4F1D-4730-8078-6CF025EB493F', 'AE214CAC-A747-4549-B4E0-957075E94F38', 'BD337B93-0D12-4AE2-9566-35705B577186', 'CE11AF94-7EB4-4B98-BC57-51593FD36B72', '1F6C5DD9-4889-43B7-8768-BA1D2873322F', '322A11BB-2B77-480A-BABA-2FD89CDD0305', 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', '3342F086-2D91-4081-BEE0-C18AEAB0C981', 'C4FFF996-0FB6-427D-9266-BE10E9F5DAA9', '510C2305-6CB3-460D-84C7-9EB6281F8E9D', 'FA7AC95E-1961-4A7B-AC57-3FD3212BAF20', '04CCB591-8B59-4CCB-9EB0-5A4AECB3C705', '0D66C132-CD95-442B-800E-E19F2651E018', '0ECEF203-5A40-428B-A19E-BFA699806763', '2A1A46A7-1E50-41C1-ADD1-EBC0F82168F9', '6C6C6230-52AE-4590-AFA7-42A14991BD69', '71B8A2A8-14EC-4B19-A4DA-6DE30E8A6E30', '2B3AACB7-A045-4E46-A2DB-8B841AD87F65', '2FCDBCD0-49FC-4AA6-B249-041E476D766A', '3DF7C0B1-EA0D-4014-916E-E76984AFDE2A', '8A1DE8B3-2FDE-4581-AC94-4EA74A997CB0', 'F49D6604-F705-480F-BCB9-E6F16B76DACF', '31C70405-0F52-4BD8-A785-13AFD15809A6', '323DACC7-ED74-42A3-A1E6-21BC104AAD51', '35841CCE-C3B1-4652-B011-C44F79BB442D', '8875D93D-866D-4897-81C3-67DB4570B102', 'A7A12FBC-72F0-4216-8360-126AF826214C', 'BDAADB25-9498-4D4F-809E-4590BAEE53B5', '0438621D-9EC2-4530-B3E8-F0C68FD8295F', '55E3B0D0-8E40-48F3-854B-A8E3C893E803', '793B38EA-690C-456B-9597-5444E6D1499F', '899B4439-339E-4CC1-AC1E-4C1F4BA312EB', 'A4CD837B-6B10-47C0-B9DC-E8D6ADF73B88', 'CC817F76-EE46-4AD5-9911-86A53FB1DBAE', 'F1C5E648-7E44-43B6-A884-48A861BA52BB', '3D2ECD44-776B-437D-9DD6-936DD5B7A4BF', 'AE3A61BE-5820-45D3-BCD1-9982D0BA93EF', 'C94A98D6-223E-47E8-9589-5631EBCB55E9', '05BBEB22-65F8-4773-89A4-452C6FD9DDE9', '0C7AF11E-16DB-461A-A412-B711F0810E4F', '8745504F-A5BD-470D-ABAD-CDA68F032BE0', '8B63DA7A-BB50-450B-BA2A-E68DFFF3F4F6', '0BC312A3-FBB5-4F2E-9A5C-6BAC5566A285', '1BA29057-DE25-4EF0-A06C-94A526C557A3', '8E5DABE9-6BE2-4D98-BF87-D87C6A70C35B', '9AE67B15-8F75-4CED-8129-80E003318733', 'D84605C6-4883-46E5-8BE3-4C82081FC5E0', 'EF5E604D-077C-4903-95D7-5568576B7F96', '14710135-067E-4E9D-91B1-E52FD6B72796', 'ADC1E05E-8451-4C66-976B-3DBBB7103A27', '1BDC9A8F-39D7-43D8-B3A8-5E9457AEB7CE', '45BD84A3-1EE3-4B60-8352-D372DFEB677C', '6F66C6AD-3D4F-4EC2-9C71-D7580A768CF5', '74E7B927-7E80-423E-8EBE-94FC0B7E514F', '82948A9C-6345-49ED-AB80-DD62B36CE11F', 'BA941CAA-6F1A-4BBD-9D4A-42C905C8C41A', 'ED3A209D-7398-4255-A9FB-3712B16F9DC5', 'FBCA9AD8-4FC2-48A9-B059-0347A0EDA4E1', '142DD0C0-133D-4FCB-934A-C6C50D564E61', '803ED239-BF70-4FCD-B02C-B5977404BA98', '967ABAFC-DBB1-4878-89E6-2B32E0E8387C', 'B3EE5A6F-6BC4-4F1D-B196-1127D5A7FED1', 'B77643EB-41F3-4EF0-818C-B4617862C124', 'CF7B7502-BFFD-479E-87EF-8C30089033C8', '04A3E774-1DF2-461B-8DCA-90EAB47932A0', '1054BD81-AB8A-4FDC-9A87-F41814188945', '202111D3-0B41-42EF-8113-161C06700D53', '23863534-0389-4F1E-BDDC-96C82F56F4C0', '906E67BB-7D18-4CBE-8802-172E4D09F677', 'A930D4B2-2DE6-4134-A58E-36E88012A437', '043D822F-B7D6-433B-B8F7-3A0F58BFFC22', '37409C70-626D-4D18-BFD4-749375A6D10F', 'C1DDC816-520D-4175-8613-E726EAF36CBA', 'CCC6C19C-C3C3-4F0B-8D42-083ABB99F0E9', 'FF11B590-D529-4372-A722-252F47C1AA76', 'AF3AFE4B-9BD1-42B1-9D03-5031CDB276F7', 'D45A198D-C1A5-4A54-8BCE-CD4AD0B2FBCF', 'E7C8D3FA-DCE9-4622-96F0-2047722D8564', '399470D8-4590-411F-8652-380D7FB48A6F', 'A8670196-A390-42F7-B247-84D40DCE973F', 'F39EE67E-0589-4138-9AEF-261777DFC2B1', 'FA571DDD-0422-48A6-BE3F-1DFAABE49A5D', '010ED4A3-EB64-4AE2-99CE-B458B59B538C', '1EA4A9FD-2C9B-4C54-9296-841FA5F9F23D', '3DAA771F-294A-4395-9B6B-30B898BC01E7', '411875F3-AAAD-48AC-92E6-112579D42F27', '7A0B900D-FB81-4652-AC59-4D27FB1BF2DA', 'D1CA4C65-B76E-4F88-991D-E39CDBCB9A25', 'D501A556-9400-4BBE-876D-4E2269A69E50', 'E44DA795-311C-464F-A98C-AF783E9A8687', '8800B6CD-48B2-414A-AA85-2FB6E2F94F62', '0C738DFF-CDE2-446B-8605-885F7FFEA814', '387FB6B8-8FE5-4483-A401-B595EF6BE611', '9E48B685-ED16-456A-9BDA-73F1CDE529D5', 'FF73DA08-83EA-4F38-8153-B686DFBD480C', '8E785A6C-A77D-46F3-B74F-701B18C72B2B', 'B881C1B6-9321-4EB8-901E-843FB577ECDE', '6C01513E-DC0D-48F4-85F7-5DE34C582CA7', '88E69525-5461-45A9-879A-21A99C906710', '980CDD7C-B8FF-4C24-9EB8-E66EF9502A25', '29761757-EC8E-4BC8-9BD4-764810C0D435', '2E9CCC26-EE83-4CE0-BBE6-78198DE26939', '3596B626-33E4-4438-A2D7-C7B7F2884A61', '722ED05A-EAA6-41C2-AB96-ED5E231D9CA5', '8D12EF16-4731-4FDD-802D-73518A27A384', 'BB4D3CAB-1412-4339-A438-A8AFB1ECCA82', '113F2B98-6796-4055-8712-8147E996D2FB', '19635F03-1921-4E20-A610-C32DCD0A86E0', '1EDBDDB5-5D98-4516-87A4-135EB82D051F', '28A393A5-A7B5-475F-8710-5DFB463F120E', '36009550-373F-4179-BD6D-42DC2B91652B', '8EEEA997-D2B5-4EC9-A50E-B2CBD0094405', 'C295E50E-9D1F-4967-B8F4-CE7ACB0DD858', 'F3A2BB7A-DCFC-4740-AB2C-51B4DE18A84F', '077A557E-846D-453A-AE7D-9B3003FFA94F', '0EDBE7EE-67FD-48BA-86E9-F70C9C504642', 'B06CBEAD-36DB-426E-84CE-89B56FD46159', '1DEC7009-90C7-4C0B-8617-D9E805BD39DC', '65AAE408-BFF7-45D0-98E7-78DFDAEC64EC', '823ACF06-0F8B-4DC7-AB68-4DFE535C2F92', 'B86E4CA3-AF97-467C-80C1-A05484D3A866', 'D525418D-529A-470D-B010-7B90CD096743', '15695896-D50A-4650-8607-86092DE66F86', '724A277E-2530-4A79-AC3F-6E4467BB2688', '7C334D9D-8773-4B52-871C-7F4CE8FED784', 'A15234F5-28B1-425F-8338-F05A4BF34B5A', '0CD457A8-5460-4F98-8059-107464922EFC', '1CE42B39-5F00-4C37-A276-6DFEE0CB161F', '2AD5B7D3-9824-461C-8E4C-F5088C5A5061', '6295FB65-95AE-465C-A7BE-93CDEA6FCD41', '9E709F1E-38BF-4B10-B126-7C344A21DD11', '9EB1990C-6484-48F0-9B12-F05AA4B6815E', 'E2272BDD-653A-47F9-A8BA-EEEDBC803C1B', 'FB109A88-81CE-4832-B4AC-56DD97251649', '53B02D9D-4160-42AA-936D-FCBB76F49B95', '99D51AF1-7AD3-4DD6-96F0-25AA1E59972F', '07A0559D-6D61-4D02-9F83-0730224B6C93', '4D06240B-F581-412B-B7B2-5F3A1060CB47', 'D9501B03-2C0C-4096-A3F1-A4EBEC12D7AE', 'EC536097-D4FA-4221-9C08-0C6168F80D46', '11077BFE-4169-4F6F-A104-75B18FD0FF87', '2EB8DC5F-2F79-46F0-A78F-A59491C148F6', '36EF7301-7E02-412A-9E23-47987397A484', '620495B1-AE1E-4BEA-881B-AA8645DDAAB3', '73A89B2A-DEAD-456A-95A2-EA941FF2DBFB', '83B3ABC6-1E23-4471-9F1E-78EA18B3CCE0', '9231D20B-208F-4B7A-BEEA-28718633282D', '9FE9533D-C993-42CC-A477-DA0FD4E5FF72', 'DA09EFC9-6D5E-4472-8CF3-C9017C22A30E', 'F9A0096A-9F38-42FA-B2FF-20B3F570FEB8', '116B1537-FBEA-479D-8725-F3F98CE8696A', '496D2DE6-BB81-4484-AC58-3BB0D13B4E8E', '8D6F6E45-269B-4E19-AB45-130E4C2F8EE5', 'BF7CB069-0FD9-44DA-A277-EDEB8C31E102', '02705462-64DD-4C6E-AB7A-2CDDB023FE7A', '3882B031-F286-4F77-8AAE-3ECC1E13433E', '9C0A75B2-5E85-479D-B614-3C97418C4570', 'F5C916EC-F2CA-4136-9312-5A71933D7C4C', '846A3024-1033-46EB-9F06-0F3E17DAD4AA', 'CBA90F0E-ADB0-4F8F-A849-CD75DB7DD2A1', 'D7A3A337-59AB-4474-8D56-FF4B8402E128', '15527B5F-A9E1-4797-A410-9E324DD82868', '5649E152-6876-444A-97D3-66E0C38D7661', 'BB2FFBE2-24D7-4618-B3FE-DC8B3ECA14AC', 'E760489F-6EE4-4B15-9B6D-5B12F6A5E3DC', 'FAD56DDC-6281-4433-94D8-23FCB65FD823', '74492123-CD52-45B6-AAFB-CDD634C4C9BC', '79956D3A-C616-4A7A-9F06-BA0DC2806695', '9C582B3F-8717-4E03-84C9-F6EAAC3F4948', '394DE176-F74B-4877-A640-EF3C34905AE0', '3F563146-EE95-44B0-B130-41E8E1DE3030', '48C82871-65CB-40BD-98D3-6EB4CB2E0F61', '49B6D9F9-0BB5-43E9-B453-A0D4616A3689', '5C1B73F4-3EDD-41BE-9FEC-7BAC25366092', '9D0125FC-58A9-4CD0-A823-57B6464A308F', '9E726301-D189-4224-945B-2FBA8FA26A03', 'A5EE1C1B-278D-478B-AC1F-980E200CB843', 'C8D69246-8334-4EBF-954E-5AAFA7C1F65F', 'CAABED88-1591-4E9D-82C4-7926EA41DD52', 'DD23176B-512C-49D2-AC85-CFAA4256C72D', 'FAD99EF5-F102-4FC6-BCCC-ECDEF99456BB', '2653D41E-11AE-432B-993D-8CC4D4DAA1FA', '71E5F463-049A-42A7-A510-EACEC14E8301', '86D2CB7E-E54C-40E4-BCDD-C6D0DE7A6D7C', 'AB9BAF44-7980-4636-B844-488341E49BA9', 'F66CDB59-6144-4776-B129-5FEED24970C3', 'F829CBA3-AD5D-4CF8-BE1F-3FDC44044AF7', '244D35E3-5D74-4F5C-BB72-5860E1D4561C', '2C3E981A-853E-45E7-87BD-799325176251', '3B1A2F21-FFBF-4454-B0FB-BCBF1CEA4FE3', '49FF4EFE-6A51-44EA-BDDA-27F6A4663C04', '6BD754C1-07F9-4C5D-A1B8-B77B284AFC7F', '71A70101-0BDF-43F3-B580-4DB046CB1EBD', '8076FF62-D8A1-4309-9EAD-F4B83E85DB1D', 'DB05D823-A5CC-4AD0-87B0-6FBF69B7493E', 'DC1BE341-E3CB-4EBF-B968-F5764D826DC7', 'DCF767F0-7167-41FC-B981-296FBE63588B', '46445DC3-6A22-4664-8F2F-C73C4F9B7010', '9F250BA0-F703-4F1D-8AD6-82F2F4E58B70', 'BCCDC778-B59A-44F9-B1B6-9B67462C4B0A', 'CF8CDC47-5AD3-442E-9F3C-07CF84EC35A5', 'EADDE82D-5C40-4BF2-8C07-A91EEAAF8EC5', 'F32643CC-08F3-4108-B5D1-4A881B590328', 'C9139FC7-C65B-4F97-B7CC-56F810FD45DF', 'ECA40D88-F3E1-4286-8064-FD546E3C844E', 'FDBE8690-4E2C-4BD4-991A-74F5F3CC2872', '04A27415-9D35-49E4-A5F5-305DFD917D5F', '04EA280C-12AB-4842-9913-E97DFDDF508F', '0CB92950-EE3F-424C-AA0A-445F2D738518', '29737DE0-22D6-4D41-870C-DA59BE4A4E75', 'AA77712E-61BA-45A8-8E10-28057191D585', 'D271CCF9-CBA9-4A48-8202-5ED6FE135BDA', 'F488446B-42F0-4C47-B313-AE5AD0AD4C08', '0DC89466-2E6B-4DC7-B913-E5BBE5B168A0', '27DEFE26-1A3A-4654-A273-799CEA02BB33', '3D48B549-0C2E-4EC1-AA68-CD7DE250D65A', '790CB21B-A420-4C48-8EBC-67AAC5413D69', '9DD3CB15-B27B-4F87-A3C8-A9FC8D1F56B5', '2F71902A-768E-4A58-BE77-2AB583872514', '305A3131-EFD4-420C-88B8-361D358D9857', '65920EF9-C672-4D37-9A6B-795B0F656EDD', '736AECC6-FFE6-48E7-9F0D-4AF0CDC06DED', '9DEA15FC-C28D-42CB-8283-AC5B84B7FCEF', '0B1BA7BE-21C6-4B0E-A25F-C198051FD597', '0E9EF90A-1AEA-403E-9058-D6F60A449238', '235B02A5-8AC3-47BD-86BB-CAA1A63C317B', '48A25AFB-573A-4213-9B1C-E3654825F634', '49C499CF-DD82-4195-B4F3-F889BE7E0BC4', '691933B0-0D3D-4355-B453-79E791D6DC36', '83B52DD3-F13E-450B-84A7-CAA79A556718', '842AC936-21A9-4F15-8BF5-ACF6D4756EB0', 'A3CC38B5-605E-476B-A987-2AC63C81F539', 'C3A37CEA-E1D8-4DB2-837C-D1041863EC7F', 'CF82712C-91A7-46D2-9F6D-19A1BC8A1A0D', '485494F4-D20A-4303-89BB-F09C5646A9EA', '4FEA26C5-DD7F-4A65-B859-D5FECFF66684', 'B48620D1-DD13-471B-936F-AE9970B850B6', 'BBF82897-F57E-484D-8159-C4E14BBECB5C', 'F0B25A0F-D965-460E-AA37-EA676B8BE27F', '279D822D-CECC-407C-8F7D-DA683C10C7B1', '27E0E710-3B33-4DEC-BED0-FDB55F95EEE1', '8793C097-7DC2-47B0-8639-CBD108B34F5F', '1D2CA217-3B28-4C61-89C2-7C55BAB37BDD', '3A4437B5-45EE-4734-BF9C-C7BA808C123B', '13872B75-E219-4B25-9F7E-24F021CF0B48', '377FADE6-202C-4841-BDD4-82367D36097C', '6A29D415-E299-4812-B8CD-8CD471F28115', '84A66A4F-CB48-4BA8-B11C-F2EA34C4A1A4', 'D7B0792D-0B5F-4CAA-9787-237DFECCA98B', '6D22E643-C022-4637-A81F-22A0DB410A33', '7C771536-6EAB-4FEA-B848-E077662307D7', 'DD0C3E18-53F4-4001-9334-BEC408540A3E', 'EA11E355-7A8A-4BAD-9066-1FC3E824CBC5', '267E6CA7-6B3A-4582-B01B-9DD2C4AC8DEE', '43B4ED70-4EAA-4C2A-AB60-F4A3848BF5E9', '7D59A1A0-3C61-4F7A-AD1E-0EF39B392E83', 'FF80B585-D64F-45DC-9CE9-59C51F3001D2', '324408CC-B86D-49E1-A0DE-99A7EE03276E', '3E505BF0-6673-4C9F-996F-64E07E72E60C', '5FFAD31C-B0EF-4A88-B7EA-2F02D7B336D5', '8CC0B44A-2E34-417F-A084-3E2F067F6966', 'B29EEE24-30E4-4B06-9FA3-777B1E99463D', 'C84DCC7E-7D84-433A-8D23-74F8BB79EE40', 'E776DC1F-10CF-4EC1-A151-05EC29E6C03C', '40F1314D-EBF6-49BC-B1A8-674F07F92039', '66E28365-DDB9-4F59-916F-2B171C5D93AB', '88539DEA-C521-4D06-A758-7FEBC99D5C5E', '8CCEE88C-C36B-4DE3-88D7-00D78C1D9B0A', 'C0AC5031-0382-4350-82C5-5B80634DEEAC', 'E5775C4C-F86F-49D2-8B58-22ACA6668352', '08683FBF-CD1F-42D9-A442-EF696A6D7C83', '289AA716-A1D3-48FF-B830-4BF9DD4A4FB7', '28D97A0D-03AE-4405-BFD3-66E00EDAE377', '55F62337-F077-4BA9-87D8-4C7AC0736913', '56A0F3CD-F2AA-4492-A3F3-D8694240FF34', 'B16FA6E7-E985-411C-AA06-A1F7E6BBCEFB', '0BB8A5C4-7444-4DEE-A3AC-2E0CFF7A3477', '1DB8D73F-61E5-4065-9342-592BE88E5981', '4C323EC0-5AB8-4CAF-9381-0011A0141AD7', '07BDF3AA-3B6E-4650-BDFC-596FE687C33F', '3F664203-58B4-411D-835A-667F32E1AF1E', '94F9C8EF-40F1-4CB4-B8F1-71EA8B1482D9', 'A56274A7-1DCD-46CB-84FC-83FD3C26472D', 'BDB2ADB4-356A-40AF-9E57-C6ECA2EB1235', 'F28706C5-DDB5-4FC3-9F7D-4896DB86AF18', '3DA05017-5FD5-4CBE-9CC5-956876EECBE6', '6AD7EB61-BE8E-49A3-929C-CE2CA6C95779', '8B79B6F2-FE78-4775-8668-2A35ECA963E5', '98AEC5FB-BE93-4E59-A4A9-9533419B73B6', 'A24C0F04-7684-4CC9-BFBF-CD7E01137187', 'C22535F6-D4C8-4B58-B0C4-BA24CE26B9E7', 'D9115251-3465-4EF5-B9B3-54B0EAF82045', '24253A09-E06B-4C51-A3AD-0709BD0DA4B5', '449D29CA-9C67-4DA9-95FE-C1531DA5DE4C', '77C723D8-92FE-487C-825B-93C8605DC74E', '8BAEB725-2E28-4DE9-B911-836846DF6F11', 'D89C117A-4B7C-471A-94A0-1F6F022193BB', 'E0D39756-5A76-4FA1-BA20-C957C3486DFC', '0A6DE37A-0F94-43B3-84E3-7590769F820E', '0F43FFB3-7DF2-4156-BD39-A6C0667C7392', '08AD40B0-B651-4069-AA92-9A64FC246B17', '0EC59014-45A0-45B7-AB3D-12367ECD4174', '13E519C2-73BD-451A-A651-55B67924A9B6', '26C382E6-F716-4F7F-A72C-A6C7150C2066', '989C36A9-F0D6-48B5-8D26-F5A56E610792', 'C4ADE72E-AD16-4711-886D-AB997CCB5E71', 'DE90AF3B-500B-42BC-B1F8-D4E5971DD853', '03A51C92-3977-42BE-B158-53B124029BCC', '176F9200-4582-49AB-80EB-558534EC297C', '2861F7ED-8D48-436D-9AD0-FEF67A864566', '7157C3A4-F8EE-42C2-864E-C9E7FD5C62C6', 'FB5D5A33-BA7C-4140-AB6B-9AC8E94FABA6', '3AAEBB1A-A489-4699-A7B4-F237F130B7EE', 'A3646A59-090D-4D1F-AE88-16B127AF81ED', 'A90B0781-610E-4FE4-80ED-BD9F6ABDE174', 'AF591C35-31A5-45DD-9399-292D82D794AF', 'D06DE332-BDE9-4D63-8959-BF68C5A40297', '23AEEFFE-19BA-487C-A1D5-A88964FF3BC1', '341B166A-3402-4A75-AD95-D57FDDA27DC9', '45D50939-3539-478D-AABE-C57C6791CF1F', '4A1D1603-893E-46B7-92C1-8C3069A4B63D', '502690F0-D587-4D27-B3A7-5B9679F7E217', '97E40160-D786-4460-B2D1-FCFFE2DB599F', 'A372D150-FF7E-4BA6-9349-DDD6451A1084'], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'ChargeDetailID', 'count': 500, 'min': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'max': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192'}, {'values': {'label': ['2020-12-18 00:00:00', '2020-12-13 00:00:00', '2020-11-16 00:00:00', '2020-11-27 00:00:00', '2020-12-03 00:00:00', '2020-11-02 00:00:00', '2020-12-14 00:00:00', '2020-11-28 00:00:00', '2020-11-30 00:00:00', '2020-11-26 00:00:00', '2020-11-09 00:00:00', '2020-12-19 00:00:00', '2020-12-04 00:00:00', '2020-11-15 00:00:00', '2020-11-20 00:00:00', '2020-12-05 00:00:00', '2020-12-06 00:00:00', '2020-12-07 00:00:00', '2020-11-06 00:00:00', '2020-11-13 00:00:00', '2020-12-11 00:00:00', '2020-12-17 00:00:00', '2020-11-12 00:00:00', '2020-11-07 00:00:00', '2020-11-14 00:00:00', '2020-11-29 00:00:00', '2020-12-12 00:00:00', '2020-12-10 00:00:00', '2020-12-21 00:00:00', '2020-11-21 00:00:00', '2020-12-24 00:00:00', '2020-11-05 00:00:00', '2020-11-19 00:00:00', '2020-12-20 00:00:00', '2020-11-08 00:00:00', '2020-12-27 00:00:00', '2020-11-01 00:00:00', '2020-12-26 00:00:00', '2020-12-28 00:00:00', '2020-12-31 00:00:00'], 'data': [15, 15, 23, 16, 19, 12, 17, 14, 17, 10, 11, 13, 12, 14, 12, 9, 14, 21, 16, 19, 18, 7, 15, 14, 14, 17, 12, 8, 10, 8, 5, 9, 14, 12, 15, 5, 9, 1, 5, 3]}, 'label': 'DateofService', 'count': 40, 'min': '', 'max': ''}, {'values': {'label': ['45.0 - 709.3', '709.3 - 1373.6', '1373.6 - 2037.9', '2037.9 - 2702.2', '2702.2 - 3366.5', '3366.5 - 4030.8', '4030.8 - 4695.1', '4695.1 - 5359.4', '5359.4 - 6023.7', '6023.7 - 6688.0'], 'data': [467, 13, 5, 0, 3, 5, 0, 0, 6, 1]}, 'label': 'Amount', 'count': 10, 'min': 45, 'max': 6688}, {'values': {'label': ['-250.0 - -212.7', '-212.7 - -175.4', '-175.4 - -138.1', '-138.1 - -100.8', '-100.8 - -63.5', '-63.5 - -26.2', '-26.2 - 11.1', '11.1 - 48.4', '48.4 - 85.7', '85.7 - 123.0'], 'data': [1, 0, 2, 0, 2, 3, 489, 0, 2, 1]}, 'label': 'PatientBal', 'count': 10, 'min': -250, 'max': 123}, {'values': {'label': ['0.0', '24.0', '76.0', '36.0'], 'data': [497, 1, 1, 1]}, 'label': 'InsuranceBal', 'count': 4, 'min': '0.0', 'max': '76.68'}, {'values': {'label': ['0.0', '76.0'], 'data': [499, 1]}, 'label': 'PrimaryInsuranceBal', 'count': 2, 'min': '0.0', 'max': '76.68'}, {'values': {'label': ['92014', '92134', '92250', '67028', '99204', '99354', '99215', '92012', 'J0178', '92225', '92226', '92004', '76512', '67036', '92235', 'J2778PF3', '99213', '99203', '67113', 'J7312', '92018', 'J9035', '99205'], 'data': [156, 185, 35, 22, 28, 2, 3, 12, 7, 7, 6, 6, 3, 2, 10, 9, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'CPTCode', 'count': 23, 'min': '67028', 'max': 'J9035'}, {'values': {'label': ['25', '0000', 'LT', 'RT', '50', '26', '57'], 'data': [70, 385, 13, 15, 13, 1, 3]}, 'label': 'CptModifier', 'count': 7, 'min': '0000', 'max': 'RT'}, {'values': {'label': ['H35.3112', 'H33.312', 'H35.711', 'H35.3211', 'E11.3313', 'H33.311', 'E11.3413', 'H33.101', 'H35.3121', 'H30.021', 'H35.3132', 'E11.3512', 'H30.91', 'E11.3513', 'E11.3391', 'H35.3231', 'H35.033', 'H35.413', 'H35.61', 'E10.3511', 'C69.32', 'H34.8110', 'H35.372', 'E11.3511', 'H33.301', 'C69.31', 'E11.3411', 'D31.31', 'H35.352', 'H43.813', 'H35.3134', 'H20.013', 'H35.3221', 'H35.3122', 'H35.351', 'B58.01', 'H34.231', 'H34.8310', 'H25.813', 'H33.001', 'H35.3113', 'H43.821', 'H35.3130', 'H34.8120', 'H35.411', 'H44.2A2', 'H33.002', 'H35.373', 'H35.371', 'H35.3222', 'H35.3212', 'H25.13', 'E10.3513', 'H35.051', 'H35.362', 'D31.22', 'H44.113', 'H43.11', 'H35.343', 'H43.811', 'H30.23', 'H33.313', 'H53.9', 'H30.133', 'E11.3311', 'H47.012', 'H34.8330', 'H47.10', 'H35.353', 'H20.9', 'H30.93', 'H35.3131', 'H35.3124', 'H31.113', 'H35.341', 'E10.3292', 'H33.22', 'H43.812', 'H35.053', 'H40.051', 'E11.3213', 'H35.342', 'H35.3133', 'H35.052', 'H20.12', 'H43.10', 'H35.3213', 'H35.3114', 'H53.8', 'E11.3212', 'H33.321', 'C79.32', 'H35.89', 'H30.22', 'E11.3211', 'H34.8312', 'H35.721', 'H35.433', 'H35.412', 'H40.033', 'H34.8320', 'H43.13', 'H47.211', 'H35.461', 'H43.823', 'H57.11', 'E11.3312', 'H15.121', 'H35.3111', 'H31.091', 'H44.2D2', 'H33.322', 'E11.3292', 'H44.111', 'H43.12', 'H33.103', 'H21.01', 'H04.123', 'H31.011', 'H35.54', 'H43.392', 'H35.3110', 'C69.21', 'H59.021', 'H43.22', 'H11.31', 'H20.012', 'H34.8130', 'H35.073', 'H35.023', 'E10.3391', 'H35.713', 'H31.8', 'H35.63', 'D31.32', 'H33.042', 'T85.29XA', 'H33.323', 'H40.053', 'H44.23', 'H10.45', 'E11.9', 'T66.XXXA', 'H43.391', 'H35.101', 'H30.013', 'H35.50', 'Q14.1', 'H35.712'], 'data': [21, 3, 5, 20, 14, 2, 8, 1, 1, 1, 13, 5, 2, 38, 1, 30, 15, 2, 1, 2, 5, 7, 11, 14, 2, 1, 4, 3, 3, 6, 5, 1, 3, 7, 8, 1, 1, 5, 1, 6, 2, 3, 3, 4, 2, 1, 5, 11, 9, 2, 8, 2, 1, 1, 1, 1, 1, 2, 1, 4, 2, 1, 2, 1, 4, 1, 1, 1, 6, 2, 2, 10, 2, 1, 3, 1, 1, 4, 1, 3, 3, 3, 3, 2, 2, 1, 2, 3, 1, 2, 3, 1, 2, 1, 5, 3, 2, 1, 3, 1, 3, 1, 1, 1, 3, 1, 4, 1, 4, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'pdiagnosis1', 'count': 149, 'min': 'B58.01', 'max': 'T85.29XA'}, {'values': {'label': ['H35.373', 'H35.411', 'H35.721', 'H35.3122', 'H43.811', 'E11.9', 'Z79.4', 'H43.823', 'H43.813', 'H35.351', 'H35.70', 'H43.822', 'E11.3312', 'H35.723', 'H34.8320', 'H52.13', 'H35.3221', 'Z71.89', 'H35.432', 'H43.821', 'H35.371', 'H35.61', 'H35.412', 'H35.372', 'H20.9', 'H21.543', 'H43.812', 'H35.3211', 'H30.101', 'H43.89', 'H35.033', 'H04.123', 'H25.811', 'H35.463', 'H35.3212', 'H35.3132', 'H35.461', 'H44.22', 'H35.032', 'H40.021', 'H35.341', 'H35.352', 'Z98.890', 'H34.8322', 'H35.711', 'E11.3311', 'H35.3133', 'H30.113', 'H35.3124', 'S01.111D', 'H47.211', 'E11.3412', 'Z79.899', 'D31.31', 'H35.353', 'Z96.1', 'H40.052', 'H35.3130', 'H35.3134', 'H40.013', 'H33.312', 'H43.12', 'D31.32', 'H34.8120', 'H43.11', 'H33.311', 'H31.092', 'H33.331', 'H30.93', 'H33.42', 'H47.10', 'H40.1110', 'H21.542', 'H35.3223', 'E11.3513', 'H59.022', 'E11.3292', 'E11.3592', 'H35.3131', 'H47.011', 'H35.3220', 'H35.022', 'H25.813', 'H35.3120', 'E11.3293', 'H35.342', 'H35.3112', 'H35.52', 'H25.11', 'H43.392', 'H43.23', 'H35.413', 'H40.053', 'H35.722', 'H43.13', 'H35.3121', 'E11.3511', 'H43.393', 'H40.1130', 'E11.3291', 'H33.41', 'H33.321', 'E11.3213', 'G45.9', 'E11.3532', 'H35.22', 'I10', 'H31.012', 'H31.011', 'H30.92', 'H47.091', 'H11.32', 'H33.43', 'H35.433', 'K50.918', 'H35.40', 'H40.043', 'H35.052', 'T85.398A', 'H11.31', 'H35.81', 'E11.3391', 'H44.111', 'H25.13', 'E11.3212', 'H01.003', 'T66.XXXS', 'H40.051', 'H33.322', 'H43.391', 'G89.11', 'H35.713', 'C69.22', 'H27.131', 'H35.89', 'S05.11XA', 'H26.222', 'H33.002', 'H35.3222', 'E11.3392', 'H40.42X0', 'E10.3312', 'H44.701', 'E11.3512', 'H44.23', 'H25.9', 'H33.003', 'T66.XXXA', 'H11.001', 'H35.3231', 'H31.091', 'H33.001', 'H31.422', 'H44.2A3', 'H47.233', 'H31.002'], 'data': [20, 2, 3, 9, 14, 6, 47, 7, 22, 7, 4, 6, 5, 4, 9, 2, 23, 4, 2, 4, 8, 3, 4, 10, 4, 1, 12, 2, 1, 3, 25, 2, 2, 2, 1, 2, 2, 1, 1, 1, 3, 8, 3, 3, 1, 1, 1, 1, 3, 1, 1, 4, 2, 5, 10, 3, 1, 1, 3, 1, 3, 4, 3, 4, 4, 3, 1, 1, 2, 1, 1, 1, 1, 3, 3, 1, 2, 3, 3, 1, 1, 1, 1, 2, 2, 4, 1, 1, 1, 3, 1, 7, 1, 2, 2, 3, 2, 2, 2, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'pdiagnosis2', 'count': 156, 'min': 'C69.22', 'max': 'Z98.890'}, {'values': {'label': ['1.0'], 'data': [500]}, 'label': 'IsBillable', 'count': 1, 'min': '1.0', 'max': '1.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'Isauthorized', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'PredeterminationIndicator', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'MedNecessityRequiredIndicator', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['5.0 - 14.7', '14.7 - 24.4', '24.4 - 34.1', '34.1 - 43.8', '43.8 - 53.5', '53.5 - 63.2', '63.2 - 72.9', '72.9 - 82.6', '82.6 - 92.3', '92.3 - 102.0'], 'data': [4, 6, 11, 20, 44, 75, 134, 120, 72, 14]}, 'label': 'Age', 'count': 10, 'min': 5, 'max': 102}, {'values': {'label': ['M', 'F'], 'data': [225, 275]}, 'label': 'Gender', 'count': 2, 'min': 'F', 'max': 'M'}, {'values': {'label': ['Pulmonary', 'Emergency', 'Cardiology', 'Orthopedic', 'Pediatrics', 'Physical Therapy', 'Ophtamology', 'Gastroenterology'], 'data': [172, 233, 39, 36, 3, 7, 9, 1]}, 'label': 'DepartmentCategory', 'count': 8, 'min': 'Cardiology', 'max': 'Pulmonary'}, {'values': {'label': ['Ophtamology'], 'data': [500]}, 'label': 'SpecialityCategory', 'count': 1, 'min': 'Ophtamology', 'max': 'Ophtamology'}, {'values': {'label': ['1.0', '0.0'], 'data': [455, 45]}, 'label': 'label', 'count': 2, 'min': '0.0', 'max': '1.0'}, {'values': {'label': ['B03B74CE-C223-4B1E-802B-5F5F76BFE64C', '105C1090-6ED0-4189-B07E-4C22940D948F', 'AD96772F-54C7-4750-86B1-1BD565B55244', '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', '4502FB31-5B01-4EEF-BB9E-2D396D96A6FC', '32E7EC13-C493-40B1-A999-6AC94E7FA3B9', '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', '0A471D4C-3977-4BF5-8039-5064519FDCD4', '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'BCA515C2-1B14-4060-9E24-B80BF9260648', '2CD6534B-F8F6-4CF9-8DEB-CCEF9EF87F9A', 'A7CD19DF-074A-47D3-958D-2B5E00671A3D', '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', '84222BD3-605E-43EA-9A84-7C0169BE0B0D', '4CED3627-B1D6-4530-87D7-9609525FB29C', '041FE834-D4EA-40F1-A495-4B94A97ACD4D', 'F3FB074E-423D-4F7F-A817-90C0551007A4', '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', '8351B681-269A-4AE6-936A-FF10AFAFD629', 'E459B9E9-8650-4DA2-91F6-00B77900FC50', '2EAF4989-97B8-4F1B-9259-03E37314202F', 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'F473817F-7515-46B7-97F9-348C88D7C348', '0BC878EE-8F48-415E-9233-25A7C28264DB', '789050C0-D1C3-40AD-8942-E67329E2F890', '193BA5E7-F797-4FD1-A9AB-27590E630FCB', '5EA62C5D-D9C0-4AFE-ACAF-4AD36F6B1057', '72F5084C-9DF9-4F0A-8BA1-A9B40DFCC9C6', '330E2DD7-352E-4E5C-9669-AE796ABFE5DC', 'E0FE1EC8-51D4-430D-8B8B-E2F9966875F8', 'A4D682CE-4D10-4765-AE88-0D5D9E1862E7', '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'C4A6C186-B867-46C5-A163-5AF23FE61DED', 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'BCCB5D33-476A-4FAC-BB0C-5C7479803EB4', '3BEA3D9A-B5EF-4B67-A4AB-868B7BE2C0A1', '0779204D-FC67-444E-97E3-83CB62A9D0A2', 'EDD37436-0E9D-45B5-8C46-A22CA763970A', '8CA1878F-1BD4-4595-9AFF-3FF6967CD5CB', '315C773B-4EF1-4164-AB43-6817D993329F', 'AAB72015-9704-40ED-A2E7-87468E793D48', '20C2761F-16B5-42B7-8C93-82A112735F02', '835BC6C3-FEDE-4AF9-8B88-91628A98389C', 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', 'D1C4AE8C-46B5-464F-91E1-A39C5C76CC25'], 'data': [15, 1, 30, 199, 25, 81, 2, 2, 16, 6, 37, 3, 1, 1, 1, 6, 3, 1, 2, 5, 12, 3, 3, 2, 2, 4, 2, 6, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'PayerId', 'count': 51, 'min': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52'}, {'values': {'label': ['2.0', '1.0'], 'data': [225, 275]}, 'label': 'Gender_Recoded', 'count': 2, 'min': '1', 'max': '2'}, {'values': {'label': ['0.0 - 49.9', '49.9 - 99.8', '99.8 - 149.7', '149.7 - 199.6', '199.6 - 249.5', '249.5 - 299.4', '299.4 - 349.3', '349.3 - 399.2', '399.2 - 449.1', '449.1 - 499.0'], 'data': [50, 50, 50, 50, 50, 50, 50, 50, 50, 50]}, 'label': 'ChargeDetailID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 499}, {'values': {'label': ['1 - 4', '4 - 7', '7 - 10', '10 - 13', '13 - 16', '16 - 19', '19 - 22', '22 - 25', '25 - 28', '28 - 31'], 'data': [40, 60, 61, 53, 79, 45, 69, 5, 32, 56]}, 'label': 'DateofService_dayofmonth', 'count': 10, 'min': 1, 'max': 31}, {'values': {'label': ['12', '11'], 'data': [221, 279]}, 'label': 'DateofService_month', 'count': 2, 'min': '11', 'max': '12'}, {'values': {'label': ['2020'], 'data': [500]}, 'label': 'DateofService_year', 'count': 1, 'min': '2020', 'max': '2020'}, {'values': {'label': ['0.0 - 2.2', '2.2 - 4.4', '4.4 - 6.6', '6.6 - 8.8', '8.8 - 11.0', '11.0 - 13.2', '13.2 - 15.4', '15.4 - 17.6', '17.6 - 19.8', '19.8 - 22.0'], 'data': [376, 50, 22, 16, 13, 12, 4, 2, 2, 3]}, 'label': 'CPTCode_stringindexer_transform', 'count': 10, 'min': 0, 'max': 22}, {'values': {'label': ['1.0', '0.0', '4.0', '2.0', '3.0', '6.0', '5.0'], 'data': [70, 385, 13, 15, 13, 1, 3]}, 'label': 'CptModifier_stringindexer_transform', 'count': 7, 'min': '0.0', 'max': '6.0'}, {'values': {'label': ['0.0 - 14.8', '14.8 - 29.6', '29.6 - 44.4', '44.4 - 59.2', '59.2 - 74.0', '74.0 - 88.8', '88.8 - 103.6', '103.6 - 118.4', '118.4 - 133.2', '133.2 - 148.0'], 'data': [230, 79, 49, 36, 28, 18, 15, 15, 15, 15]}, 'label': 'pdiagnosis1_stringindexer_transform', 'count': 10, 'min': 0, 'max': 148}, {'values': {'label': ['0.0 - 15.5', '15.5 - 31.0', '31.0 - 46.5', '46.5 - 62.0', '62.0 - 77.5', '77.5 - 93.0', '93.0 - 108.5', '108.5 - 124.0', '124.0 - 139.5', '139.5 - 155.0'], 'data': [238, 66, 51, 35, 32, 15, 16, 15, 16, 16]}, 'label': 'pdiagnosis2_stringindexer_transform', 'count': 10, 'min': 0, 'max': 155}, {'values': {'label': ['1', '0'], 'data': [225, 275]}, 'label': 'Gender_stringindexer_transform', 'count': 2, 'min': '0', 'max': '1'}, {'values': {'label': ['1.0', '0.0', '2.0', '3.0', '6.0', '5.0', '4.0', '7.0'], 'data': [172, 233, 39, 36, 3, 7, 9, 1]}, 'label': 'DepartmentCategory_stringindexer_transform', 'count': 8, 'min': '0.0', 'max': '7.0'}, {'values': {'label': ['0'], 'data': [500]}, 'label': 'SpecialityCategory_stringindexer_transform', 'count': 1, 'min': '0', 'max': '0'}, {'values': {'label': ['0.0 - 5.0', '5.0 - 10.0', '10.0 - 15.0', '15.0 - 20.0', '20.0 - 25.0', '25.0 - 30.0', '30.0 - 35.0', '35.0 - 40.0', '40.0 - 45.0', '45.0 - 50.0'], 'data': [372, 55, 21, 12, 10, 9, 5, 5, 5, 6]}, 'label': 'PayerId_stringindexer_transform', 'count': 10, 'min': 0, 'max': 50}], 'path_ManualSample': '/FileStore/platform/sampleData/60891770a187c956b337cf50_ManualFE/part-00000-tid-7570618811626959673-94990ba1-2b39-4e6c-ac98-d7d939c2d4f4-172005-1-c000.csv', 'graphdata': {'ChargeDetailID': {'label': ['174DA322-870E-429F-A9A9-BE22340E9836', 'A1575911-0E83-4479-B182-D26EDAD439F6', 'A87EF1E9-E636-4742-8E2B-414477926782', '1FFF3E45-B9F4-4CFB-8580-C4B05D4822DD', '20BC5C22-2C34-494C-B75F-55BDD8B54263', '210DAE19-9257-4180-9057-E5817F555A88', '3FA21D4D-05B5-4C43-A33E-A01D20B45A99', 'DC65D7C3-0822-4E54-86EF-DC7639D38C62', '11327DC4-1599-46B2-A000-C05D135E1F91', '853B5BAB-0A5F-4B44-85E0-FEE0AA663796', '92F7294B-4F6C-41B5-A9BD-65CE72B27FC0', '97BB8C89-8E47-400B-B546-B662E9EDCA00', '9C7B2481-4469-4FB8-AC5D-18EB2D29F213', 'BCE331FC-BC9E-43B2-B9C3-1ADE95CF4081', '0391FBA9-7748-4F50-8CA2-FC4EB5116F6A', '88D923C8-5217-46AF-887E-CB88A2E6C3E3', '8AAE78A7-7725-4C1C-B55C-82B41A364A2B', 'B5C5BD20-64AE-4166-8115-14DA13624764', '343273F8-E598-4815-A700-0683D31F3291', '50F7ADC5-2166-4461-B01E-8D8FAA92304B', 'A9EC659A-ED07-4B6C-84CA-D4E45411C3C6', 'DCBDA7E0-A5F8-4D0F-836D-66187FEB5B6F', 'E907166F-CE0F-49D2-A04B-40734F1A6F0C', '6518F89E-CD31-4903-89E2-2F31BEB28508', '7B41BFB4-B1D7-47A9-9659-6E8319429423', '7E4D4049-B1BC-4135-8020-D69318B66B18', '94FEF155-5ED7-4DF9-836F-78557484B8E4', 'A6B1DE4F-2FBD-4AB9-B09E-77B3E8C44771', 'B9F344F0-CEE0-4EA2-A55C-274FAB39E9DB', 'BBE4E59F-B282-490E-8FCD-6C9F5E8069EB', 'EC206AC2-403A-4047-B82E-82C383E29CCE', '01867531-8532-4FD4-BE17-232F39CFA32D', '2ECD88B7-05A0-4959-8736-397F8CF74784', '524D7B98-2D7C-458D-8877-35CDE8F3D981', '54E3991B-1609-470D-8095-C14FF5A9444D', '8F8BDCC6-5B74-4EBA-A154-8576DE0FFC13', 'A28023B3-9D64-44D3-BD13-2A22A1350A24', '1B71D2B0-C473-4A8D-9FA0-76A0692C5102', '6CF3754A-BC46-430C-A116-9329959CD556', 'A6CED81D-7B1D-441A-8EFC-544E8D31FD26', 'A82270CF-0B59-4C62-877D-9FAF720B6E5D', 'D9A3771E-E97A-4EBE-BFA9-54577F445F21', '1AA261AC-C877-4F0D-8372-F268A4D5F90B', '2B385821-C105-42CA-84F0-CEF8BFD1681A', '9D5CF424-8DA6-4A2B-8716-FDB93B808F3C', 'A17DF3A2-8077-49B4-A114-11F77F67E7C9', '0E47C422-450E-4A04-9B90-680F296C9686', '46CB6A8C-362F-4345-941E-842F429B48B5', '8CD490B4-CEBE-4DFE-BA0A-C805ACDF0D34', '9F54BD25-7C67-454A-B100-CD9974B62C7D', 'A78BD078-2165-4C68-B398-9AB6E94C0FB6', 'C72EBDAD-1D1D-40D0-BA4D-A9CDD08430A6', 'DBFEC39B-37B2-4B37-9D95-A444C27A619C', 'E1C8F5F6-2DDF-4370-B40C-D86322919657', 'EB7C11AD-0A01-4359-B1AB-289373C669DC', '16062E35-A49E-4227-A3E6-E8DDBFEBA5D2', '20416F30-920B-4925-8642-C83F38ECFCEE', '642D42E4-275F-4E86-8378-21E2CD291224', 'FBB301BB-1560-415C-91B5-A6E8B0B26B6C', '4011D505-BE33-4A24-AD40-240BFDDB2766', '74FC2B0E-AEC7-4263-951F-14E72EF044DA', 'A5881100-AA89-4D5D-80AC-AE2419271C5D', 'D0667457-EF74-4FA4-91D9-7169C64A53DB', 'E16D36F3-C4D2-4C4B-A004-E06C823C7960', '20463C7A-45B7-4BD4-A3EA-53ACC03E93F1', '6E834EA0-8A83-4326-8FAE-80C6DEE3B045', '8607EC2B-26F1-4A40-9646-5990D37EC622', 'A050F893-D9BB-4FB6-9B4B-B7AA83355D60', 'C2F471A9-22FA-4A9A-A14C-49A5F292662D', 'D6F6C707-D42D-4485-A571-82EEF5002E64', 'E164830A-C381-4AEC-81EE-3B116E56B496', '2701DB53-B3A0-466C-BFCB-6684D7059DC4', '89D7AAB0-BF88-493E-A7AB-7D6D793273DB', '8A07AC76-56FD-4E13-BFEA-A724E4C2D603', '8E5FAFBE-87F5-4C26-88EA-002384A50B1D', 'E67C3D6F-A4D2-4EA9-B41A-E5BEB54E0209', 'AC68E17B-9EBB-45A3-BD22-60367A203515', 'D0BA312C-C42D-4695-A6E4-71096BE6854B', '50C6C060-08F2-4F75-91BE-EF1D34F6EDA9', 'C99C5CC6-DF6E-4024-88A6-04D326F6D6B2', '1A70E6C3-0B4F-4338-A15C-903F552EC30C', '279BE041-8831-43CF-A0EB-219F709A986F', '3E0A6D69-9B07-4BFC-B389-24E0C884701E', '9017B6B5-6C3F-42FC-ABD4-67A54E07D67F', 'F16597BA-3D84-47E3-AF6C-072ECBD33467', '2AA7F9ED-3694-4ADD-91A8-3EFB8F8054FB', '45F8D74C-DE28-4F0D-B781-AB81CCD5D5CD', 'D7B8BD7C-A3F9-422B-B491-59F236BADE4D', 'E9DB7A1D-67B8-4936-B7C7-FB071EA72907', '2B6E92D7-EA15-4F53-B133-2A1F9B1E958C', '86E8386B-2651-4928-8CD1-631CE24D0432', '97D21C13-7AFC-4865-888F-5E92CC51F2ED', 'A40F161A-A931-41D9-A65B-597347BC9D6A', 'C43925F7-AA9E-419B-B0DB-0A044E67C4B5', 'D05F93BC-037E-4BD7-8E65-FDA46391791F', '14167C30-8123-4DCF-B3C5-05D3F16B6391', '2022006D-F4D6-4F8A-B76D-1617C97083CA', '25EF819E-5F4C-43DA-BEBB-C4FAD71B0EF5', '8648E212-02DB-4D77-BB02-88B67C56A9DE', 'EDDC8ACA-6742-4C6C-A200-FA42DD89285D', '03E0DCDC-A071-4987-A7CE-268D9C3D017B', '161B0D6C-5A50-4EEE-9B2A-E4DE8D9EBC1C', '1BD4EAF6-F3F8-4369-93B3-4E6144A69CAD', '20AC5B30-ACB1-4714-9794-5454B92C1A23', '3FE9456A-15CB-4840-90CB-05F7DF34CACE', '42F5D00A-6F59-4F63-ACCA-9EFDA0DD9194', '6E80EF7D-E761-4720-A1BD-9E2CA7FDD21F', '734CA534-2CD7-4A77-9D03-CF2390E8B0D6', 'B0501331-13EF-4126-B052-44D67EDFE7F9', 'C0B29F0F-BBE1-47C7-B625-0989FDCC5288', '136261A4-52B0-43B5-B099-1EF929506A93', '252D8E17-A9E8-48F5-84D3-6DA9B452EE75', '5A8270A6-53BD-4916-8805-B762FF67BE20', '67841660-18E4-45DE-B386-285C53582762', '9C60FA6D-FAAF-46A9-A149-1F941986D089', 'D6FE7A69-6B2B-495F-A36A-789DF588F6D5', 'D9C0CCE1-6D5D-4DB7-8376-D8562C150145', '42B818D0-170D-4838-BC4A-7D22039C0F53', '7DC8A130-8030-4915-8BF5-54015B27DAB6', 'A3555B0C-5A99-4619-B6E8-9831D99F0210', 'AECE54B4-2AE0-44A1-9409-2BFB52D07BD2', 'BE7612D0-7548-49C7-800E-173BAF9DF290', '0EB85789-CE25-4CCC-850E-44D2C3496272', '45F8ABD8-1D29-495F-B63D-866175B861A0', 'A69FF747-5E5E-44A3-A2AE-806C56D75743', 'B1C24F17-5A6A-4D76-A42E-A273CF94898F', '41D99DEA-233E-47A6-98C0-8B6F522D1B70', '53BD3672-31BA-4CE7-B2F3-2AFFC29C5F6F', '55F86AF5-F804-4A84-90ED-12E4125E1A8D', '8C94AF61-5EC3-44D7-B4B3-BBEB64AD2A23', 'B52B5682-9815-4EC1-A594-912A9724EB74', '079F8DAC-D8C1-48FA-9743-BC2AA08E287C', '2D60073D-3E0E-4347-B0A6-7E0CE91E92FA', '99772A77-B764-48FF-9242-5700C9572B65', 'E45AB682-6C1F-4B84-A1AC-3D1CD79AC3A9', '51526EAE-3FB7-4C7E-9A33-26990B70EFFE', '540B2A5F-11A9-4F0F-8038-B5A1ECE5E583', '91D48DF5-F63A-466A-AFB9-2E65C4F2D44D', 'ADF5D273-DB58-4F7F-8DF0-9D9D9647DA16', 'F672AF9B-E361-4572-A9C0-8B0731F9CD3E', '34803FD1-6C40-4323-8657-F1FCC3E34990', '5327CFC0-5C48-435B-98EB-A76AAE858941', 'D2DF16C4-D3AB-4E5C-9350-8F6FDE2EDA8D', 'E2DBCD71-B901-41F3-97B1-ED71030C7F2F', '049BA1F1-F362-48BE-9393-C05CCEA8694B', '13002A76-350E-4525-BDA1-9F181E056F74', '167E1195-CC7A-4F8C-8D91-9577BCBEE8FB', '1F264D61-D5D7-4505-9AC2-25ACF275E3BD', '23BCDE27-9F80-48E4-B29D-348C871829B3', 'A71A1D2F-3F1A-4CA6-AA98-E5D6A855C250', 'B62DA993-E320-4CEE-9D23-B3238CA5759A', 'B6375BF2-EFA5-44C9-A05C-F71DACD5F8A8', 'C924A1C9-2E9A-4BF3-8BC7-25BFAEB02CE1', 'D2300410-689E-453C-895D-FC7FDA559759', 'F12A6DBF-FBB4-4C62-8EE0-0F677D58EC3B', 'FA9EEE47-7F76-4264-9A82-9B370BCF9534', 'FBBC8CD5-C35A-42E5-AEBA-C5F61050B726', '045700E8-F572-417C-A8DD-41E76C5C159A', '1A088AE8-D8BA-404D-97E5-9DB1DFB972F8', '1C109ACE-6610-4098-BE1A-C64F7805EBCA', '25251236-8157-4EE2-89CD-C5A600E278BE', 'A1818C97-82BE-4115-8C3C-3532CB8DE838', 'AC9C3FBE-B694-4A6F-9A6F-6CC4563C5C73', 'B5E35D7B-A0F8-4A47-B72F-1D372236FC63', 'C1119267-721A-4606-9D06-09FF240505EC', 'C3339BC7-4E2E-4E0E-822E-7EE8E489451C', 'CDDFE064-998C-4CDD-97A2-04647A33EE7A', 'D4930936-02DA-4DBB-A3B2-5391AC5D7153', 'FD8029C5-4ACA-4F97-8CE6-144AE49C8BCB', '4C3D48F4-860F-49EF-812D-75B08DC1359C', '58207A31-753A-42B1-86D8-E750AEA10079', 'EAF0CF6B-9569-41B6-8B77-6335F34B3D7E', '693B32F4-ECA0-4844-80DF-6D9C94DD57BA', '7E2F8EA5-4A1D-4490-A208-49CE684BB427', '83A30AD9-1329-4675-99E9-AF5ACBE98CFA', '84F4A744-F946-4B1E-99DF-20E0538C5F4F', '90C2C4BC-4F1D-4730-8078-6CF025EB493F', 'AE214CAC-A747-4549-B4E0-957075E94F38', 'BD337B93-0D12-4AE2-9566-35705B577186', 'CE11AF94-7EB4-4B98-BC57-51593FD36B72', '1F6C5DD9-4889-43B7-8768-BA1D2873322F', '322A11BB-2B77-480A-BABA-2FD89CDD0305', 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', '3342F086-2D91-4081-BEE0-C18AEAB0C981', 'C4FFF996-0FB6-427D-9266-BE10E9F5DAA9', '510C2305-6CB3-460D-84C7-9EB6281F8E9D', 'FA7AC95E-1961-4A7B-AC57-3FD3212BAF20', '04CCB591-8B59-4CCB-9EB0-5A4AECB3C705', '0D66C132-CD95-442B-800E-E19F2651E018', '0ECEF203-5A40-428B-A19E-BFA699806763', '2A1A46A7-1E50-41C1-ADD1-EBC0F82168F9', '6C6C6230-52AE-4590-AFA7-42A14991BD69', '71B8A2A8-14EC-4B19-A4DA-6DE30E8A6E30', '2B3AACB7-A045-4E46-A2DB-8B841AD87F65', '2FCDBCD0-49FC-4AA6-B249-041E476D766A', '3DF7C0B1-EA0D-4014-916E-E76984AFDE2A', '8A1DE8B3-2FDE-4581-AC94-4EA74A997CB0', 'F49D6604-F705-480F-BCB9-E6F16B76DACF', '31C70405-0F52-4BD8-A785-13AFD15809A6', '323DACC7-ED74-42A3-A1E6-21BC104AAD51', '35841CCE-C3B1-4652-B011-C44F79BB442D', '8875D93D-866D-4897-81C3-67DB4570B102', 'A7A12FBC-72F0-4216-8360-126AF826214C', 'BDAADB25-9498-4D4F-809E-4590BAEE53B5', '0438621D-9EC2-4530-B3E8-F0C68FD8295F', '55E3B0D0-8E40-48F3-854B-A8E3C893E803', '793B38EA-690C-456B-9597-5444E6D1499F', '899B4439-339E-4CC1-AC1E-4C1F4BA312EB', 'A4CD837B-6B10-47C0-B9DC-E8D6ADF73B88', 'CC817F76-EE46-4AD5-9911-86A53FB1DBAE', 'F1C5E648-7E44-43B6-A884-48A861BA52BB', '3D2ECD44-776B-437D-9DD6-936DD5B7A4BF', 'AE3A61BE-5820-45D3-BCD1-9982D0BA93EF', 'C94A98D6-223E-47E8-9589-5631EBCB55E9', '05BBEB22-65F8-4773-89A4-452C6FD9DDE9', '0C7AF11E-16DB-461A-A412-B711F0810E4F', '8745504F-A5BD-470D-ABAD-CDA68F032BE0', '8B63DA7A-BB50-450B-BA2A-E68DFFF3F4F6', '0BC312A3-FBB5-4F2E-9A5C-6BAC5566A285', '1BA29057-DE25-4EF0-A06C-94A526C557A3', '8E5DABE9-6BE2-4D98-BF87-D87C6A70C35B', '9AE67B15-8F75-4CED-8129-80E003318733', 'D84605C6-4883-46E5-8BE3-4C82081FC5E0', 'EF5E604D-077C-4903-95D7-5568576B7F96', '14710135-067E-4E9D-91B1-E52FD6B72796', 'ADC1E05E-8451-4C66-976B-3DBBB7103A27', '1BDC9A8F-39D7-43D8-B3A8-5E9457AEB7CE', '45BD84A3-1EE3-4B60-8352-D372DFEB677C', '6F66C6AD-3D4F-4EC2-9C71-D7580A768CF5', '74E7B927-7E80-423E-8EBE-94FC0B7E514F', '82948A9C-6345-49ED-AB80-DD62B36CE11F', 'BA941CAA-6F1A-4BBD-9D4A-42C905C8C41A', 'ED3A209D-7398-4255-A9FB-3712B16F9DC5', 'FBCA9AD8-4FC2-48A9-B059-0347A0EDA4E1', '142DD0C0-133D-4FCB-934A-C6C50D564E61', '803ED239-BF70-4FCD-B02C-B5977404BA98', '967ABAFC-DBB1-4878-89E6-2B32E0E8387C', 'B3EE5A6F-6BC4-4F1D-B196-1127D5A7FED1', 'B77643EB-41F3-4EF0-818C-B4617862C124', 'CF7B7502-BFFD-479E-87EF-8C30089033C8', '04A3E774-1DF2-461B-8DCA-90EAB47932A0', '1054BD81-AB8A-4FDC-9A87-F41814188945', '202111D3-0B41-42EF-8113-161C06700D53', '23863534-0389-4F1E-BDDC-96C82F56F4C0', '906E67BB-7D18-4CBE-8802-172E4D09F677', 'A930D4B2-2DE6-4134-A58E-36E88012A437', '043D822F-B7D6-433B-B8F7-3A0F58BFFC22', '37409C70-626D-4D18-BFD4-749375A6D10F', 'C1DDC816-520D-4175-8613-E726EAF36CBA', 'CCC6C19C-C3C3-4F0B-8D42-083ABB99F0E9', 'FF11B590-D529-4372-A722-252F47C1AA76', 'AF3AFE4B-9BD1-42B1-9D03-5031CDB276F7', 'D45A198D-C1A5-4A54-8BCE-CD4AD0B2FBCF', 'E7C8D3FA-DCE9-4622-96F0-2047722D8564', '399470D8-4590-411F-8652-380D7FB48A6F', 'A8670196-A390-42F7-B247-84D40DCE973F', 'F39EE67E-0589-4138-9AEF-261777DFC2B1', 'FA571DDD-0422-48A6-BE3F-1DFAABE49A5D', '010ED4A3-EB64-4AE2-99CE-B458B59B538C', '1EA4A9FD-2C9B-4C54-9296-841FA5F9F23D', '3DAA771F-294A-4395-9B6B-30B898BC01E7', '411875F3-AAAD-48AC-92E6-112579D42F27', '7A0B900D-FB81-4652-AC59-4D27FB1BF2DA', 'D1CA4C65-B76E-4F88-991D-E39CDBCB9A25', 'D501A556-9400-4BBE-876D-4E2269A69E50', 'E44DA795-311C-464F-A98C-AF783E9A8687', '8800B6CD-48B2-414A-AA85-2FB6E2F94F62', '0C738DFF-CDE2-446B-8605-885F7FFEA814', '387FB6B8-8FE5-4483-A401-B595EF6BE611', '9E48B685-ED16-456A-9BDA-73F1CDE529D5', 'FF73DA08-83EA-4F38-8153-B686DFBD480C', '8E785A6C-A77D-46F3-B74F-701B18C72B2B', 'B881C1B6-9321-4EB8-901E-843FB577ECDE', '6C01513E-DC0D-48F4-85F7-5DE34C582CA7', '88E69525-5461-45A9-879A-21A99C906710', '980CDD7C-B8FF-4C24-9EB8-E66EF9502A25', '29761757-EC8E-4BC8-9BD4-764810C0D435', '2E9CCC26-EE83-4CE0-BBE6-78198DE26939', '3596B626-33E4-4438-A2D7-C7B7F2884A61', '722ED05A-EAA6-41C2-AB96-ED5E231D9CA5', '8D12EF16-4731-4FDD-802D-73518A27A384', 'BB4D3CAB-1412-4339-A438-A8AFB1ECCA82', '113F2B98-6796-4055-8712-8147E996D2FB', '19635F03-1921-4E20-A610-C32DCD0A86E0', '1EDBDDB5-5D98-4516-87A4-135EB82D051F', '28A393A5-A7B5-475F-8710-5DFB463F120E', '36009550-373F-4179-BD6D-42DC2B91652B', '8EEEA997-D2B5-4EC9-A50E-B2CBD0094405', 'C295E50E-9D1F-4967-B8F4-CE7ACB0DD858', 'F3A2BB7A-DCFC-4740-AB2C-51B4DE18A84F', '077A557E-846D-453A-AE7D-9B3003FFA94F', '0EDBE7EE-67FD-48BA-86E9-F70C9C504642', 'B06CBEAD-36DB-426E-84CE-89B56FD46159', '1DEC7009-90C7-4C0B-8617-D9E805BD39DC', '65AAE408-BFF7-45D0-98E7-78DFDAEC64EC', '823ACF06-0F8B-4DC7-AB68-4DFE535C2F92', 'B86E4CA3-AF97-467C-80C1-A05484D3A866', 'D525418D-529A-470D-B010-7B90CD096743', '15695896-D50A-4650-8607-86092DE66F86', '724A277E-2530-4A79-AC3F-6E4467BB2688', '7C334D9D-8773-4B52-871C-7F4CE8FED784', 'A15234F5-28B1-425F-8338-F05A4BF34B5A', '0CD457A8-5460-4F98-8059-107464922EFC', '1CE42B39-5F00-4C37-A276-6DFEE0CB161F', '2AD5B7D3-9824-461C-8E4C-F5088C5A5061', '6295FB65-95AE-465C-A7BE-93CDEA6FCD41', '9E709F1E-38BF-4B10-B126-7C344A21DD11', '9EB1990C-6484-48F0-9B12-F05AA4B6815E', 'E2272BDD-653A-47F9-A8BA-EEEDBC803C1B', 'FB109A88-81CE-4832-B4AC-56DD97251649', '53B02D9D-4160-42AA-936D-FCBB76F49B95', '99D51AF1-7AD3-4DD6-96F0-25AA1E59972F', '07A0559D-6D61-4D02-9F83-0730224B6C93', '4D06240B-F581-412B-B7B2-5F3A1060CB47', 'D9501B03-2C0C-4096-A3F1-A4EBEC12D7AE', 'EC536097-D4FA-4221-9C08-0C6168F80D46', '11077BFE-4169-4F6F-A104-75B18FD0FF87', '2EB8DC5F-2F79-46F0-A78F-A59491C148F6', '36EF7301-7E02-412A-9E23-47987397A484', '620495B1-AE1E-4BEA-881B-AA8645DDAAB3', '73A89B2A-DEAD-456A-95A2-EA941FF2DBFB', '83B3ABC6-1E23-4471-9F1E-78EA18B3CCE0', '9231D20B-208F-4B7A-BEEA-28718633282D', '9FE9533D-C993-42CC-A477-DA0FD4E5FF72', 'DA09EFC9-6D5E-4472-8CF3-C9017C22A30E', 'F9A0096A-9F38-42FA-B2FF-20B3F570FEB8', '116B1537-FBEA-479D-8725-F3F98CE8696A', '496D2DE6-BB81-4484-AC58-3BB0D13B4E8E', '8D6F6E45-269B-4E19-AB45-130E4C2F8EE5', 'BF7CB069-0FD9-44DA-A277-EDEB8C31E102', '02705462-64DD-4C6E-AB7A-2CDDB023FE7A', '3882B031-F286-4F77-8AAE-3ECC1E13433E', '9C0A75B2-5E85-479D-B614-3C97418C4570', 'F5C916EC-F2CA-4136-9312-5A71933D7C4C', '846A3024-1033-46EB-9F06-0F3E17DAD4AA', 'CBA90F0E-ADB0-4F8F-A849-CD75DB7DD2A1', 'D7A3A337-59AB-4474-8D56-FF4B8402E128', '15527B5F-A9E1-4797-A410-9E324DD82868', '5649E152-6876-444A-97D3-66E0C38D7661', 'BB2FFBE2-24D7-4618-B3FE-DC8B3ECA14AC', 'E760489F-6EE4-4B15-9B6D-5B12F6A5E3DC', 'FAD56DDC-6281-4433-94D8-23FCB65FD823', '74492123-CD52-45B6-AAFB-CDD634C4C9BC', '79956D3A-C616-4A7A-9F06-BA0DC2806695', '9C582B3F-8717-4E03-84C9-F6EAAC3F4948', '394DE176-F74B-4877-A640-EF3C34905AE0', '3F563146-EE95-44B0-B130-41E8E1DE3030', '48C82871-65CB-40BD-98D3-6EB4CB2E0F61', '49B6D9F9-0BB5-43E9-B453-A0D4616A3689', '5C1B73F4-3EDD-41BE-9FEC-7BAC25366092', '9D0125FC-58A9-4CD0-A823-57B6464A308F', '9E726301-D189-4224-945B-2FBA8FA26A03', 'A5EE1C1B-278D-478B-AC1F-980E200CB843', 'C8D69246-8334-4EBF-954E-5AAFA7C1F65F', 'CAABED88-1591-4E9D-82C4-7926EA41DD52', 'DD23176B-512C-49D2-AC85-CFAA4256C72D', 'FAD99EF5-F102-4FC6-BCCC-ECDEF99456BB', '2653D41E-11AE-432B-993D-8CC4D4DAA1FA', '71E5F463-049A-42A7-A510-EACEC14E8301', '86D2CB7E-E54C-40E4-BCDD-C6D0DE7A6D7C', 'AB9BAF44-7980-4636-B844-488341E49BA9', 'F66CDB59-6144-4776-B129-5FEED24970C3', 'F829CBA3-AD5D-4CF8-BE1F-3FDC44044AF7', '244D35E3-5D74-4F5C-BB72-5860E1D4561C', '2C3E981A-853E-45E7-87BD-799325176251', '3B1A2F21-FFBF-4454-B0FB-BCBF1CEA4FE3', '49FF4EFE-6A51-44EA-BDDA-27F6A4663C04', '6BD754C1-07F9-4C5D-A1B8-B77B284AFC7F', '71A70101-0BDF-43F3-B580-4DB046CB1EBD', '8076FF62-D8A1-4309-9EAD-F4B83E85DB1D', 'DB05D823-A5CC-4AD0-87B0-6FBF69B7493E', 'DC1BE341-E3CB-4EBF-B968-F5764D826DC7', 'DCF767F0-7167-41FC-B981-296FBE63588B', '46445DC3-6A22-4664-8F2F-C73C4F9B7010', '9F250BA0-F703-4F1D-8AD6-82F2F4E58B70', 'BCCDC778-B59A-44F9-B1B6-9B67462C4B0A', 'CF8CDC47-5AD3-442E-9F3C-07CF84EC35A5', 'EADDE82D-5C40-4BF2-8C07-A91EEAAF8EC5', 'F32643CC-08F3-4108-B5D1-4A881B590328', 'C9139FC7-C65B-4F97-B7CC-56F810FD45DF', 'ECA40D88-F3E1-4286-8064-FD546E3C844E', 'FDBE8690-4E2C-4BD4-991A-74F5F3CC2872', '04A27415-9D35-49E4-A5F5-305DFD917D5F', '04EA280C-12AB-4842-9913-E97DFDDF508F', '0CB92950-EE3F-424C-AA0A-445F2D738518', '29737DE0-22D6-4D41-870C-DA59BE4A4E75', 'AA77712E-61BA-45A8-8E10-28057191D585', 'D271CCF9-CBA9-4A48-8202-5ED6FE135BDA', 'F488446B-42F0-4C47-B313-AE5AD0AD4C08', '0DC89466-2E6B-4DC7-B913-E5BBE5B168A0', '27DEFE26-1A3A-4654-A273-799CEA02BB33', '3D48B549-0C2E-4EC1-AA68-CD7DE250D65A', '790CB21B-A420-4C48-8EBC-67AAC5413D69', '9DD3CB15-B27B-4F87-A3C8-A9FC8D1F56B5', '2F71902A-768E-4A58-BE77-2AB583872514', '305A3131-EFD4-420C-88B8-361D358D9857', '65920EF9-C672-4D37-9A6B-795B0F656EDD', '736AECC6-FFE6-48E7-9F0D-4AF0CDC06DED', '9DEA15FC-C28D-42CB-8283-AC5B84B7FCEF', '0B1BA7BE-21C6-4B0E-A25F-C198051FD597', '0E9EF90A-1AEA-403E-9058-D6F60A449238', '235B02A5-8AC3-47BD-86BB-CAA1A63C317B', '48A25AFB-573A-4213-9B1C-E3654825F634', '49C499CF-DD82-4195-B4F3-F889BE7E0BC4', '691933B0-0D3D-4355-B453-79E791D6DC36', '83B52DD3-F13E-450B-84A7-CAA79A556718', '842AC936-21A9-4F15-8BF5-ACF6D4756EB0', 'A3CC38B5-605E-476B-A987-2AC63C81F539', 'C3A37CEA-E1D8-4DB2-837C-D1041863EC7F', 'CF82712C-91A7-46D2-9F6D-19A1BC8A1A0D', '485494F4-D20A-4303-89BB-F09C5646A9EA', '4FEA26C5-DD7F-4A65-B859-D5FECFF66684', 'B48620D1-DD13-471B-936F-AE9970B850B6', 'BBF82897-F57E-484D-8159-C4E14BBECB5C', 'F0B25A0F-D965-460E-AA37-EA676B8BE27F', '279D822D-CECC-407C-8F7D-DA683C10C7B1', '27E0E710-3B33-4DEC-BED0-FDB55F95EEE1', '8793C097-7DC2-47B0-8639-CBD108B34F5F', '1D2CA217-3B28-4C61-89C2-7C55BAB37BDD', '3A4437B5-45EE-4734-BF9C-C7BA808C123B', '13872B75-E219-4B25-9F7E-24F021CF0B48', '377FADE6-202C-4841-BDD4-82367D36097C', '6A29D415-E299-4812-B8CD-8CD471F28115', '84A66A4F-CB48-4BA8-B11C-F2EA34C4A1A4', 'D7B0792D-0B5F-4CAA-9787-237DFECCA98B', '6D22E643-C022-4637-A81F-22A0DB410A33', '7C771536-6EAB-4FEA-B848-E077662307D7', 'DD0C3E18-53F4-4001-9334-BEC408540A3E', 'EA11E355-7A8A-4BAD-9066-1FC3E824CBC5', '267E6CA7-6B3A-4582-B01B-9DD2C4AC8DEE', '43B4ED70-4EAA-4C2A-AB60-F4A3848BF5E9', '7D59A1A0-3C61-4F7A-AD1E-0EF39B392E83', 'FF80B585-D64F-45DC-9CE9-59C51F3001D2', '324408CC-B86D-49E1-A0DE-99A7EE03276E', '3E505BF0-6673-4C9F-996F-64E07E72E60C', '5FFAD31C-B0EF-4A88-B7EA-2F02D7B336D5', '8CC0B44A-2E34-417F-A084-3E2F067F6966', 'B29EEE24-30E4-4B06-9FA3-777B1E99463D', 'C84DCC7E-7D84-433A-8D23-74F8BB79EE40', 'E776DC1F-10CF-4EC1-A151-05EC29E6C03C', '40F1314D-EBF6-49BC-B1A8-674F07F92039', '66E28365-DDB9-4F59-916F-2B171C5D93AB', '88539DEA-C521-4D06-A758-7FEBC99D5C5E', '8CCEE88C-C36B-4DE3-88D7-00D78C1D9B0A', 'C0AC5031-0382-4350-82C5-5B80634DEEAC', 'E5775C4C-F86F-49D2-8B58-22ACA6668352', '08683FBF-CD1F-42D9-A442-EF696A6D7C83', '289AA716-A1D3-48FF-B830-4BF9DD4A4FB7', '28D97A0D-03AE-4405-BFD3-66E00EDAE377', '55F62337-F077-4BA9-87D8-4C7AC0736913', '56A0F3CD-F2AA-4492-A3F3-D8694240FF34', 'B16FA6E7-E985-411C-AA06-A1F7E6BBCEFB', '0BB8A5C4-7444-4DEE-A3AC-2E0CFF7A3477', '1DB8D73F-61E5-4065-9342-592BE88E5981', '4C323EC0-5AB8-4CAF-9381-0011A0141AD7', '07BDF3AA-3B6E-4650-BDFC-596FE687C33F', '3F664203-58B4-411D-835A-667F32E1AF1E', '94F9C8EF-40F1-4CB4-B8F1-71EA8B1482D9', 'A56274A7-1DCD-46CB-84FC-83FD3C26472D', 'BDB2ADB4-356A-40AF-9E57-C6ECA2EB1235', 'F28706C5-DDB5-4FC3-9F7D-4896DB86AF18', '3DA05017-5FD5-4CBE-9CC5-956876EECBE6', '6AD7EB61-BE8E-49A3-929C-CE2CA6C95779', '8B79B6F2-FE78-4775-8668-2A35ECA963E5', '98AEC5FB-BE93-4E59-A4A9-9533419B73B6', 'A24C0F04-7684-4CC9-BFBF-CD7E01137187', 'C22535F6-D4C8-4B58-B0C4-BA24CE26B9E7', 'D9115251-3465-4EF5-B9B3-54B0EAF82045', '24253A09-E06B-4C51-A3AD-0709BD0DA4B5', '449D29CA-9C67-4DA9-95FE-C1531DA5DE4C', '77C723D8-92FE-487C-825B-93C8605DC74E', '8BAEB725-2E28-4DE9-B911-836846DF6F11', 'D89C117A-4B7C-471A-94A0-1F6F022193BB', 'E0D39756-5A76-4FA1-BA20-C957C3486DFC', '0A6DE37A-0F94-43B3-84E3-7590769F820E', '0F43FFB3-7DF2-4156-BD39-A6C0667C7392', '08AD40B0-B651-4069-AA92-9A64FC246B17', '0EC59014-45A0-45B7-AB3D-12367ECD4174', '13E519C2-73BD-451A-A651-55B67924A9B6', '26C382E6-F716-4F7F-A72C-A6C7150C2066', '989C36A9-F0D6-48B5-8D26-F5A56E610792', 'C4ADE72E-AD16-4711-886D-AB997CCB5E71', 'DE90AF3B-500B-42BC-B1F8-D4E5971DD853', '03A51C92-3977-42BE-B158-53B124029BCC', '176F9200-4582-49AB-80EB-558534EC297C', '2861F7ED-8D48-436D-9AD0-FEF67A864566', '7157C3A4-F8EE-42C2-864E-C9E7FD5C62C6', 'FB5D5A33-BA7C-4140-AB6B-9AC8E94FABA6', '3AAEBB1A-A489-4699-A7B4-F237F130B7EE', 'A3646A59-090D-4D1F-AE88-16B127AF81ED', 'A90B0781-610E-4FE4-80ED-BD9F6ABDE174', 'AF591C35-31A5-45DD-9399-292D82D794AF', 'D06DE332-BDE9-4D63-8959-BF68C5A40297', '23AEEFFE-19BA-487C-A1D5-A88964FF3BC1', '341B166A-3402-4A75-AD95-D57FDDA27DC9', '45D50939-3539-478D-AABE-C57C6791CF1F', '4A1D1603-893E-46B7-92C1-8C3069A4B63D', '502690F0-D587-4D27-B3A7-5B9679F7E217', '97E40160-D786-4460-B2D1-FCFFE2DB599F', 'A372D150-FF7E-4BA6-9349-DDD6451A1084'], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'DateofService': {'label': ['2020-12-18 00:00:00', '2020-12-13 00:00:00', '2020-11-16 00:00:00', '2020-11-27 00:00:00', '2020-12-03 00:00:00', '2020-11-02 00:00:00', '2020-12-14 00:00:00', '2020-11-28 00:00:00', '2020-11-30 00:00:00', '2020-11-26 00:00:00', '2020-11-09 00:00:00', '2020-12-19 00:00:00', '2020-12-04 00:00:00', '2020-11-15 00:00:00', '2020-11-20 00:00:00', '2020-12-05 00:00:00', '2020-12-06 00:00:00', '2020-12-07 00:00:00', '2020-11-06 00:00:00', '2020-11-13 00:00:00', '2020-12-11 00:00:00', '2020-12-17 00:00:00', '2020-11-12 00:00:00', '2020-11-07 00:00:00', '2020-11-14 00:00:00', '2020-11-29 00:00:00', '2020-12-12 00:00:00', '2020-12-10 00:00:00', '2020-12-21 00:00:00', '2020-11-21 00:00:00', '2020-12-24 00:00:00', '2020-11-05 00:00:00', '2020-11-19 00:00:00', '2020-12-20 00:00:00', '2020-11-08 00:00:00', '2020-12-27 00:00:00', '2020-11-01 00:00:00', '2020-12-26 00:00:00', '2020-12-28 00:00:00', '2020-12-31 00:00:00'], 'data': [15, 15, 23, 16, 19, 12, 17, 14, 17, 10, 11, 13, 12, 14, 12, 9, 14, 21, 16, 19, 18, 7, 15, 14, 14, 17, 12, 8, 10, 8, 5, 9, 14, 12, 15, 5, 9, 1, 5, 3]}, 'Amount': {'label': ['45.0 - 709.3', '709.3 - 1373.6', '1373.6 - 2037.9', '2037.9 - 2702.2', '2702.2 - 3366.5', '3366.5 - 4030.8', '4030.8 - 4695.1', '4695.1 - 5359.4', '5359.4 - 6023.7', '6023.7 - 6688.0'], 'data': [467, 13, 5, 0, 3, 5, 0, 0, 6, 1]}, 'PatientBal': {'label': ['-250.0 - -212.7', '-212.7 - -175.4', '-175.4 - -138.1', '-138.1 - -100.8', '-100.8 - -63.5', '-63.5 - -26.2', '-26.2 - 11.1', '11.1 - 48.4', '48.4 - 85.7', '85.7 - 123.0'], 'data': [1, 0, 2, 0, 2, 3, 489, 0, 2, 1]}, 'InsuranceBal': {'label': ['0.0', '24.0', '76.0', '36.0'], 'data': [497, 1, 1, 1]}, 'PrimaryInsuranceBal': {'label': ['0.0', '76.0'], 'data': [499, 1]}, 'CPTCode': {'label': ['92014', '92134', '92250', '67028', '99204', '99354', '99215', '92012', 'J0178', '92225', '92226', '92004', '76512', '67036', '92235', 'J2778PF3', '99213', '99203', '67113', 'J7312', '92018', 'J9035', '99205'], 'data': [156, 185, 35, 22, 28, 2, 3, 12, 7, 7, 6, 6, 3, 2, 10, 9, 1, 1, 1, 1, 1, 1, 1]}, 'CptModifier': {'label': ['25', '0000', 'LT', 'RT', '50', '26', '57'], 'data': [70, 385, 13, 15, 13, 1, 3]}, 'pdiagnosis1': {'label': ['H35.3112', 'H33.312', 'H35.711', 'H35.3211', 'E11.3313', 'H33.311', 'E11.3413', 'H33.101', 'H35.3121', 'H30.021', 'H35.3132', 'E11.3512', 'H30.91', 'E11.3513', 'E11.3391', 'H35.3231', 'H35.033', 'H35.413', 'H35.61', 'E10.3511', 'C69.32', 'H34.8110', 'H35.372', 'E11.3511', 'H33.301', 'C69.31', 'E11.3411', 'D31.31', 'H35.352', 'H43.813', 'H35.3134', 'H20.013', 'H35.3221', 'H35.3122', 'H35.351', 'B58.01', 'H34.231', 'H34.8310', 'H25.813', 'H33.001', 'H35.3113', 'H43.821', 'H35.3130', 'H34.8120', 'H35.411', 'H44.2A2', 'H33.002', 'H35.373', 'H35.371', 'H35.3222', 'H35.3212', 'H25.13', 'E10.3513', 'H35.051', 'H35.362', 'D31.22', 'H44.113', 'H43.11', 'H35.343', 'H43.811', 'H30.23', 'H33.313', 'H53.9', 'H30.133', 'E11.3311', 'H47.012', 'H34.8330', 'H47.10', 'H35.353', 'H20.9', 'H30.93', 'H35.3131', 'H35.3124', 'H31.113', 'H35.341', 'E10.3292', 'H33.22', 'H43.812', 'H35.053', 'H40.051', 'E11.3213', 'H35.342', 'H35.3133', 'H35.052', 'H20.12', 'H43.10', 'H35.3213', 'H35.3114', 'H53.8', 'E11.3212', 'H33.321', 'C79.32', 'H35.89', 'H30.22', 'E11.3211', 'H34.8312', 'H35.721', 'H35.433', 'H35.412', 'H40.033', 'H34.8320', 'H43.13', 'H47.211', 'H35.461', 'H43.823', 'H57.11', 'E11.3312', 'H15.121', 'H35.3111', 'H31.091', 'H44.2D2', 'H33.322', 'E11.3292', 'H44.111', 'H43.12', 'H33.103', 'H21.01', 'H04.123', 'H31.011', 'H35.54', 'H43.392', 'H35.3110', 'C69.21', 'H59.021', 'H43.22', 'H11.31', 'H20.012', 'H34.8130', 'H35.073', 'H35.023', 'E10.3391', 'H35.713', 'H31.8', 'H35.63', 'D31.32', 'H33.042', 'T85.29XA', 'H33.323', 'H40.053', 'H44.23', 'H10.45', 'E11.9', 'T66.XXXA', 'H43.391', 'H35.101', 'H30.013', 'H35.50', 'Q14.1', 'H35.712'], 'data': [21, 3, 5, 20, 14, 2, 8, 1, 1, 1, 13, 5, 2, 38, 1, 30, 15, 2, 1, 2, 5, 7, 11, 14, 2, 1, 4, 3, 3, 6, 5, 1, 3, 7, 8, 1, 1, 5, 1, 6, 2, 3, 3, 4, 2, 1, 5, 11, 9, 2, 8, 2, 1, 1, 1, 1, 1, 2, 1, 4, 2, 1, 2, 1, 4, 1, 1, 1, 6, 2, 2, 10, 2, 1, 3, 1, 1, 4, 1, 3, 3, 3, 3, 2, 2, 1, 2, 3, 1, 2, 3, 1, 2, 1, 5, 3, 2, 1, 3, 1, 3, 1, 1, 1, 3, 1, 4, 1, 4, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'pdiagnosis2': {'label': ['H35.373', 'H35.411', 'H35.721', 'H35.3122', 'H43.811', 'E11.9', 'Z79.4', 'H43.823', 'H43.813', 'H35.351', 'H35.70', 'H43.822', 'E11.3312', 'H35.723', 'H34.8320', 'H52.13', 'H35.3221', 'Z71.89', 'H35.432', 'H43.821', 'H35.371', 'H35.61', 'H35.412', 'H35.372', 'H20.9', 'H21.543', 'H43.812', 'H35.3211', 'H30.101', 'H43.89', 'H35.033', 'H04.123', 'H25.811', 'H35.463', 'H35.3212', 'H35.3132', 'H35.461', 'H44.22', 'H35.032', 'H40.021', 'H35.341', 'H35.352', 'Z98.890', 'H34.8322', 'H35.711', 'E11.3311', 'H35.3133', 'H30.113', 'H35.3124', 'S01.111D', 'H47.211', 'E11.3412', 'Z79.899', 'D31.31', 'H35.353', 'Z96.1', 'H40.052', 'H35.3130', 'H35.3134', 'H40.013', 'H33.312', 'H43.12', 'D31.32', 'H34.8120', 'H43.11', 'H33.311', 'H31.092', 'H33.331', 'H30.93', 'H33.42', 'H47.10', 'H40.1110', 'H21.542', 'H35.3223', 'E11.3513', 'H59.022', 'E11.3292', 'E11.3592', 'H35.3131', 'H47.011', 'H35.3220', 'H35.022', 'H25.813', 'H35.3120', 'E11.3293', 'H35.342', 'H35.3112', 'H35.52', 'H25.11', 'H43.392', 'H43.23', 'H35.413', 'H40.053', 'H35.722', 'H43.13', 'H35.3121', 'E11.3511', 'H43.393', 'H40.1130', 'E11.3291', 'H33.41', 'H33.321', 'E11.3213', 'G45.9', 'E11.3532', 'H35.22', 'I10', 'H31.012', 'H31.011', 'H30.92', 'H47.091', 'H11.32', 'H33.43', 'H35.433', 'K50.918', 'H35.40', 'H40.043', 'H35.052', 'T85.398A', 'H11.31', 'H35.81', 'E11.3391', 'H44.111', 'H25.13', 'E11.3212', 'H01.003', 'T66.XXXS', 'H40.051', 'H33.322', 'H43.391', 'G89.11', 'H35.713', 'C69.22', 'H27.131', 'H35.89', 'S05.11XA', 'H26.222', 'H33.002', 'H35.3222', 'E11.3392', 'H40.42X0', 'E10.3312', 'H44.701', 'E11.3512', 'H44.23', 'H25.9', 'H33.003', 'T66.XXXA', 'H11.001', 'H35.3231', 'H31.091', 'H33.001', 'H31.422', 'H44.2A3', 'H47.233', 'H31.002'], 'data': [20, 2, 3, 9, 14, 6, 47, 7, 22, 7, 4, 6, 5, 4, 9, 2, 23, 4, 2, 4, 8, 3, 4, 10, 4, 1, 12, 2, 1, 3, 25, 2, 2, 2, 1, 2, 2, 1, 1, 1, 3, 8, 3, 3, 1, 1, 1, 1, 3, 1, 1, 4, 2, 5, 10, 3, 1, 1, 3, 1, 3, 4, 3, 4, 4, 3, 1, 1, 2, 1, 1, 1, 1, 3, 3, 1, 2, 3, 3, 1, 1, 1, 1, 2, 2, 4, 1, 1, 1, 3, 1, 7, 1, 2, 2, 3, 2, 2, 2, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'IsBillable': {'label': ['1.0'], 'data': [500]}, 'Isauthorized': {'label': ['0.0'], 'data': [500]}, 'PredeterminationIndicator': {'label': ['0.0'], 'data': [500]}, 'MedNecessityRequiredIndicator': {'label': ['0.0'], 'data': [500]}, 'Age': {'label': ['5.0 - 14.7', '14.7 - 24.4', '24.4 - 34.1', '34.1 - 43.8', '43.8 - 53.5', '53.5 - 63.2', '63.2 - 72.9', '72.9 - 82.6', '82.6 - 92.3', '92.3 - 102.0'], 'data': [4, 6, 11, 20, 44, 75, 134, 120, 72, 14]}, 'Gender': {'label': ['M', 'F'], 'data': [225, 275]}, 'DepartmentCategory': {'label': ['Pulmonary', 'Emergency', 'Cardiology', 'Orthopedic', 'Pediatrics', 'Physical Therapy', 'Ophtamology', 'Gastroenterology'], 'data': [172, 233, 39, 36, 3, 7, 9, 1]}, 'SpecialityCategory': {'label': ['Ophtamology'], 'data': [500]}, 'label': {'label': ['1.0', '0.0'], 'data': [455, 45]}, 'PayerId': {'label': ['B03B74CE-C223-4B1E-802B-5F5F76BFE64C', '105C1090-6ED0-4189-B07E-4C22940D948F', 'AD96772F-54C7-4750-86B1-1BD565B55244', '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', '4502FB31-5B01-4EEF-BB9E-2D396D96A6FC', '32E7EC13-C493-40B1-A999-6AC94E7FA3B9', '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', '0A471D4C-3977-4BF5-8039-5064519FDCD4', '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'BCA515C2-1B14-4060-9E24-B80BF9260648', '2CD6534B-F8F6-4CF9-8DEB-CCEF9EF87F9A', 'A7CD19DF-074A-47D3-958D-2B5E00671A3D', '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', '84222BD3-605E-43EA-9A84-7C0169BE0B0D', '4CED3627-B1D6-4530-87D7-9609525FB29C', '041FE834-D4EA-40F1-A495-4B94A97ACD4D', 'F3FB074E-423D-4F7F-A817-90C0551007A4', '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', '8351B681-269A-4AE6-936A-FF10AFAFD629', 'E459B9E9-8650-4DA2-91F6-00B77900FC50', '2EAF4989-97B8-4F1B-9259-03E37314202F', 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'F473817F-7515-46B7-97F9-348C88D7C348', '0BC878EE-8F48-415E-9233-25A7C28264DB', '789050C0-D1C3-40AD-8942-E67329E2F890', '193BA5E7-F797-4FD1-A9AB-27590E630FCB', '5EA62C5D-D9C0-4AFE-ACAF-4AD36F6B1057', '72F5084C-9DF9-4F0A-8BA1-A9B40DFCC9C6', '330E2DD7-352E-4E5C-9669-AE796ABFE5DC', 'E0FE1EC8-51D4-430D-8B8B-E2F9966875F8', 'A4D682CE-4D10-4765-AE88-0D5D9E1862E7', '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'C4A6C186-B867-46C5-A163-5AF23FE61DED', 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'BCCB5D33-476A-4FAC-BB0C-5C7479803EB4', '3BEA3D9A-B5EF-4B67-A4AB-868B7BE2C0A1', '0779204D-FC67-444E-97E3-83CB62A9D0A2', 'EDD37436-0E9D-45B5-8C46-A22CA763970A', '8CA1878F-1BD4-4595-9AFF-3FF6967CD5CB', '315C773B-4EF1-4164-AB43-6817D993329F', 'AAB72015-9704-40ED-A2E7-87468E793D48', '20C2761F-16B5-42B7-8C93-82A112735F02', '835BC6C3-FEDE-4AF9-8B88-91628A98389C', 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', 'D1C4AE8C-46B5-464F-91E1-A39C5C76CC25'], 'data': [15, 1, 30, 199, 25, 81, 2, 2, 16, 6, 37, 3, 1, 1, 1, 6, 3, 1, 2, 5, 12, 3, 3, 2, 2, 4, 2, 6, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'Gender_Recoded': {'label': ['2.0', '1.0'], 'data': [225, 275]}, 'ChargeDetailID_stringindexer_transform': {'label': ['0.0 - 49.9', '49.9 - 99.8', '99.8 - 149.7', '149.7 - 199.6', '199.6 - 249.5', '249.5 - 299.4', '299.4 - 349.3', '349.3 - 399.2', '399.2 - 449.1', '449.1 - 499.0'], 'data': [50, 50, 50, 50, 50, 50, 50, 50, 50, 50]}, 'DateofService_dayofmonth': {'label': ['1 - 4', '4 - 7', '7 - 10', '10 - 13', '13 - 16', '16 - 19', '19 - 22', '22 - 25', '25 - 28', '28 - 31'], 'data': [40, 60, 61, 53, 79, 45, 69, 5, 32, 56]}, 'DateofService_month': {'label': ['12', '11'], 'data': [221, 279]}, 'DateofService_year': {'label': ['2020'], 'data': [500]}, 'CPTCode_stringindexer_transform': {'label': ['0.0 - 2.2', '2.2 - 4.4', '4.4 - 6.6', '6.6 - 8.8', '8.8 - 11.0', '11.0 - 13.2', '13.2 - 15.4', '15.4 - 17.6', '17.6 - 19.8', '19.8 - 22.0'], 'data': [376, 50, 22, 16, 13, 12, 4, 2, 2, 3]}, 'CptModifier_stringindexer_transform': {'label': ['1.0', '0.0', '4.0', '2.0', '3.0', '6.0', '5.0'], 'data': [70, 385, 13, 15, 13, 1, 3]}, 'pdiagnosis1_stringindexer_transform': {'label': ['0.0 - 14.8', '14.8 - 29.6', '29.6 - 44.4', '44.4 - 59.2', '59.2 - 74.0', '74.0 - 88.8', '88.8 - 103.6', '103.6 - 118.4', '118.4 - 133.2', '133.2 - 148.0'], 'data': [230, 79, 49, 36, 28, 18, 15, 15, 15, 15]}, 'pdiagnosis2_stringindexer_transform': {'label': ['0.0 - 15.5', '15.5 - 31.0', '31.0 - 46.5', '46.5 - 62.0', '62.0 - 77.5', '77.5 - 93.0', '93.0 - 108.5', '108.5 - 124.0', '124.0 - 139.5', '139.5 - 155.0'], 'data': [238, 66, 51, 35, 32, 15, 16, 15, 16, 16]}, 'Gender_stringindexer_transform': {'label': ['1', '0'], 'data': [225, 275]}, 'DepartmentCategory_stringindexer_transform': {'label': ['1.0', '0.0', '2.0', '3.0', '6.0', '5.0', '4.0', '7.0'], 'data': [172, 233, 39, 36, 3, 7, 9, 1]}, 'SpecialityCategory_stringindexer_transform': {'label': ['0'], 'data': [500]}, 'PayerId_stringindexer_transform': {'label': ['0.0 - 5.0', '5.0 - 10.0', '10.0 - 15.0', '15.0 - 20.0', '20.0 - 25.0', '25.0 - 30.0', '30.0 - 35.0', '35.0 - 40.0', '40.0 - 45.0', '45.0 - 50.0'], 'data': [372, 55, 21, 12, 10, 9, 5, 5, 5, 6]}}, 'vislinegraphdata': {'ChargeDetailID': {'name': 'ChargeDetailID', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'DateofService': {'name': 'DateofService', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'Amount': {'name': 'Amount', 'series': [{'name': '1', 'value': 212}, {'name': '2', 'value': 212}, {'name': '3', 'value': 212}, {'name': '4', 'value': 148}, {'name': '5', 'value': 148}, {'name': '6', 'value': 212}, {'name': '7', 'value': 148}, {'name': '8', 'value': 162}, {'name': '9', 'value': 212}, {'name': '10', 'value': 212}, {'name': '11', 'value': 212}, {'name': '12', 'value': 400}, {'name': '13', 'value': 162}, {'name': '14', 'value': 148}, {'name': '15', 'value': 162}, {'name': '16', 'value': 148}, {'name': '17', 'value': 148}, {'name': '18', 'value': 300}, {'name': '19', 'value': 148}, {'name': '20', 'value': 400}, {'name': '21', 'value': 400}, {'name': '22', 'value': 162}, {'name': '23', 'value': 212}, {'name': '24', 'value': 400}, {'name': '25', 'value': 148}, {'name': '26', 'value': 148}, {'name': '27', 'value': 162}, {'name': '28', 'value': 250}, {'name': '29', 'value': 400}, {'name': '30', 'value': 212}, {'name': '31', 'value': 212}, {'name': '32', 'value': 148}, {'name': '33', 'value': 212}, {'name': '34', 'value': 212}, {'name': '35', 'value': 118}, {'name': '36', 'value': 148}, {'name': '37', 'value': 148}, {'name': '38', 'value': 6000}, {'name': '39', 'value': 162}, {'name': '40', 'value': 148}, {'name': '41', 'value': 212}, {'name': '42', 'value': 212}, {'name': '43', 'value': 148}, {'name': '44', 'value': 148}, {'name': '45', 'value': 300}, {'name': '46', 'value': 148}, {'name': '47', 'value': 118}, {'name': '48', 'value': 212}, {'name': '49', 'value': 148}, {'name': '50', 'value': 212}, {'name': '51', 'value': 148}, {'name': '52', 'value': 212}, {'name': '53', 'value': 212}, {'name': '54', 'value': 148}, {'name': '55', 'value': 800}, {'name': '56', 'value': 148}, {'name': '57', 'value': 162}, {'name': '58', 'value': 50}, {'name': '59', 'value': 148}, {'name': '60', 'value': 148}, {'name': '61', 'value': 118}, {'name': '62', 'value': 148}, {'name': '63', 'value': 148}, {'name': '64', 'value': 212}, {'name': '65', 'value': 148}, {'name': '66', 'value': 212}, {'name': '67', 'value': 212}, {'name': '68', 'value': 212}, {'name': '69', 'value': 212}, {'name': '70', 'value': 148}, {'name': '71', 'value': 300}, {'name': '72', 'value': 148}, {'name': '73', 'value': 300}, {'name': '74', 'value': 212}, {'name': '75', 'value': 148}, {'name': '76', 'value': 212}, {'name': '77', 'value': 148}, {'name': '78', 'value': 300}, {'name': '79', 'value': 212}, {'name': '80', 'value': 212}, {'name': '81', 'value': 148}, {'name': '82', 'value': 148}, {'name': '83', 'value': 45}, {'name': '84', 'value': 148}, {'name': '85', 'value': 212}, {'name': '86', 'value': 212}, {'name': '87', 'value': 148}, {'name': '88', 'value': 148}, {'name': '89', 'value': 118}, {'name': '90', 'value': 212}, {'name': '91', 'value': 148}, {'name': '92', 'value': 246}, {'name': '93', 'value': 300}, {'name': '94', 'value': 148}, {'name': '95', 'value': 148}, {'name': '96', 'value': 383}, {'name': '97', 'value': 212}, {'name': '98', 'value': 148}, {'name': '99', 'value': 246}, {'name': '100', 'value': 148}, {'name': '101', 'value': 212}, {'name': '102', 'value': 162}, {'name': '103', 'value': 212}, {'name': '104', 'value': 212}, {'name': '105', 'value': 212}, {'name': '106', 'value': 148}, {'name': '107', 'value': 212}, {'name': '108', 'value': 212}, {'name': '109', 'value': 148}, {'name': '110', 'value': 212}, {'name': '111', 'value': 3348}, {'name': '112', 'value': 162}, {'name': '113', 'value': 148}, {'name': '114', 'value': 148}, {'name': '115', 'value': 162}, {'name': '116', 'value': 148}, {'name': '117', 'value': 162}, {'name': '118', 'value': 212}, {'name': '119', 'value': 212}, {'name': '120', 'value': 148}, {'name': '121', 'value': 148}, {'name': '122', 'value': 50}, {'name': '123', 'value': 212}, {'name': '124', 'value': 148}, {'name': '125', 'value': 162}, {'name': '126', 'value': 3000}, {'name': '127', 'value': 800}, {'name': '128', 'value': 45}, {'name': '129', 'value': 148}, {'name': '130', 'value': 148}, {'name': '131', 'value': 162}, {'name': '132', 'value': 800}, {'name': '133', 'value': 212}, {'name': '134', 'value': 50}, {'name': '135', 'value': 148}, {'name': '136', 'value': 212}, {'name': '137', 'value': 148}, {'name': '138', 'value': 148}, {'name': '139', 'value': 212}, {'name': '140', 'value': 212}, {'name': '141', 'value': 45}, {'name': '142', 'value': 148}, {'name': '143', 'value': 148}, {'name': '144', 'value': 148}, {'name': '145', 'value': 148}, {'name': '146', 'value': 148}, {'name': '147', 'value': 286}, {'name': '148', 'value': 212}, {'name': '149', 'value': 212}, {'name': '150', 'value': 148}, {'name': '151', 'value': 148}, {'name': '152', 'value': 148}, {'name': '153', 'value': 148}, {'name': '154', 'value': 148}, {'name': '155', 'value': 148}, {'name': '156', 'value': 212}, {'name': '157', 'value': 6000}, {'name': '158', 'value': 148}, {'name': '159', 'value': 212}, {'name': '160', 'value': 148}, {'name': '161', 'value': 148}, {'name': '162', 'value': 212}, {'name': '163', 'value': 148}, {'name': '164', 'value': 45}, {'name': '165', 'value': 148}, {'name': '166', 'value': 148}, {'name': '167', 'value': 148}, {'name': '168', 'value': 6000}, {'name': '169', 'value': 1800}, {'name': '170', 'value': 1800}, {'name': '171', 'value': 212}, {'name': '172', 'value': 148}, {'name': '173', 'value': 148}, {'name': '174', 'value': 162}, {'name': '175', 'value': 212}, {'name': '176', 'value': 148}, {'name': '177', 'value': 148}, {'name': '178', 'value': 162}, {'name': '179', 'value': 148}, {'name': '180', 'value': 148}, {'name': '181', 'value': 300}, {'name': '182', 'value': 212}, {'name': '183', 'value': 286}, {'name': '184', 'value': 300}, {'name': '185', 'value': 1800}, {'name': '186', 'value': 148}, {'name': '187', 'value': 212}, {'name': '188', 'value': 212}, {'name': '189', 'value': 148}, {'name': '190', 'value': 212}, {'name': '191', 'value': 148}, {'name': '192', 'value': 212}, {'name': '193', 'value': 148}, {'name': '194', 'value': 212}, {'name': '195', 'value': 148}, {'name': '196', 'value': 212}, {'name': '197', 'value': 212}, {'name': '198', 'value': 212}, {'name': '199', 'value': 300}, {'name': '200', 'value': 148}, {'name': '201', 'value': 212}, {'name': '202', 'value': 148}, {'name': '203', 'value': 212}, {'name': '204', 'value': 800}, {'name': '205', 'value': 162}, {'name': '206', 'value': 148}, {'name': '207', 'value': 130}, {'name': '208', 'value': 148}, {'name': '209', 'value': 162}, {'name': '210', 'value': 148}, {'name': '211', 'value': 212}, {'name': '212', 'value': 148}, {'name': '213', 'value': 148}, {'name': '214', 'value': 148}, {'name': '215', 'value': 212}, {'name': '216', 'value': 162}, {'name': '217', 'value': 148}, {'name': '218', 'value': 148}, {'name': '219', 'value': 148}, {'name': '220', 'value': 148}, {'name': '221', 'value': 286}, {'name': '222', 'value': 300}, {'name': '223', 'value': 148}, {'name': '224', 'value': 148}, {'name': '225', 'value': 254}, {'name': '226', 'value': 212}, {'name': '227', 'value': 212}, {'name': '228', 'value': 212}, {'name': '229', 'value': 3600}, {'name': '230', 'value': 212}, {'name': '231', 'value': 148}, {'name': '232', 'value': 300}, {'name': '233', 'value': 300}, {'name': '234', 'value': 6688}, {'name': '235', 'value': 212}, {'name': '236', 'value': 148}, {'name': '237', 'value': 162}, {'name': '238', 'value': 286}, {'name': '239', 'value': 162}, {'name': '240', 'value': 148}, {'name': '241', 'value': 212}, {'name': '242', 'value': 212}, {'name': '243', 'value': 148}, {'name': '244', 'value': 148}, {'name': '245', 'value': 162}, {'name': '246', 'value': 162}, {'name': '247', 'value': 148}, {'name': '248', 'value': 148}, {'name': '249', 'value': 212}, {'name': '250', 'value': 300}, {'name': '251', 'value': 212}, {'name': '252', 'value': 212}, {'name': '253', 'value': 212}, {'name': '254', 'value': 50}, {'name': '255', 'value': 300}, {'name': '256', 'value': 148}, {'name': '257', 'value': 148}, {'name': '258', 'value': 148}, {'name': '259', 'value': 286}, {'name': '260', 'value': 212}, {'name': '261', 'value': 148}, {'name': '262', 'value': 212}, {'name': '263', 'value': 212}, {'name': '264', 'value': 162}, {'name': '265', 'value': 800}, {'name': '266', 'value': 212}, {'name': '267', 'value': 212}, {'name': '268', 'value': 212}, {'name': '269', 'value': 212}, {'name': '270', 'value': 162}, {'name': '271', 'value': 148}, {'name': '272', 'value': 148}, {'name': '273', 'value': 148}, {'name': '274', 'value': 162}, {'name': '275', 'value': 286}, {'name': '276', 'value': 212}, {'name': '277', 'value': 148}, {'name': '278', 'value': 212}, {'name': '279', 'value': 148}, {'name': '280', 'value': 148}, {'name': '281', 'value': 148}, {'name': '282', 'value': 212}, {'name': '283', 'value': 212}, {'name': '284', 'value': 212}, {'name': '285', 'value': 148}, {'name': '286', 'value': 148}, {'name': '287', 'value': 212}, {'name': '288', 'value': 212}, {'name': '289', 'value': 162}, {'name': '290', 'value': 1800}, {'name': '291', 'value': 148}, {'name': '292', 'value': 148}, {'name': '293', 'value': 148}, {'name': '294', 'value': 118}, {'name': '295', 'value': 148}, {'name': '296', 'value': 148}, {'name': '297', 'value': 286}, {'name': '298', 'value': 300}, {'name': '299', 'value': 50}, {'name': '300', 'value': 212}, {'name': '301', 'value': 148}, {'name': '302', 'value': 148}, {'name': '303', 'value': 148}, {'name': '304', 'value': 800}, {'name': '305', 'value': 212}, {'name': '306', 'value': 800}, {'name': '307', 'value': 212}, {'name': '308', 'value': 148}, {'name': '309', 'value': 148}, {'name': '310', 'value': 45}, {'name': '311', 'value': 212}, {'name': '312', 'value': 148}, {'name': '313', 'value': 212}, {'name': '314', 'value': 212}, {'name': '315', 'value': 300}, {'name': '316', 'value': 212}, {'name': '317', 'value': 148}, {'name': '318', 'value': 212}, {'name': '319', 'value': 800}, {'name': '320', 'value': 212}, {'name': '321', 'value': 400}, {'name': '322', 'value': 300}, {'name': '323', 'value': 300}, {'name': '324', 'value': 212}, {'name': '325', 'value': 3885}, {'name': '326', 'value': 212}, {'name': '327', 'value': 148}, {'name': '328', 'value': 148}, {'name': '329', 'value': 118}, {'name': '330', 'value': 212}, {'name': '331', 'value': 162}, {'name': '332', 'value': 212}, {'name': '333', 'value': 148}, {'name': '334', 'value': 148}, {'name': '335', 'value': 212}, {'name': '336', 'value': 162}, {'name': '337', 'value': 148}, {'name': '338', 'value': 148}, {'name': '339', 'value': 212}, {'name': '340', 'value': 148}, {'name': '341', 'value': 148}, {'name': '342', 'value': 212}, {'name': '343', 'value': 148}, {'name': '344', 'value': 148}, {'name': '345', 'value': 212}, {'name': '346', 'value': 800}, {'name': '347', 'value': 162}, {'name': '348', 'value': 148}, {'name': '349', 'value': 212}, {'name': '350', 'value': 212}, {'name': '351', 'value': 118}, {'name': '352', 'value': 148}, {'name': '353', 'value': 212}, {'name': '354', 'value': 400}, {'name': '355', 'value': 250}, {'name': '356', 'value': 148}, {'name': '357', 'value': 148}, {'name': '358', 'value': 286}, {'name': '359', 'value': 148}, {'name': '360', 'value': 212}, {'name': '361', 'value': 212}, {'name': '362', 'value': 212}, {'name': '363', 'value': 148}, {'name': '364', 'value': 246}, {'name': '365', 'value': 212}, {'name': '366', 'value': 800}, {'name': '367', 'value': 148}, {'name': '368', 'value': 400}, {'name': '369', 'value': 3348}, {'name': '370', 'value': 162}, {'name': '371', 'value': 246}, {'name': '372', 'value': 3600}, {'name': '373', 'value': 212}, {'name': '374', 'value': 300}, {'name': '375', 'value': 148}, {'name': '376', 'value': 148}, {'name': '377', 'value': 212}, {'name': '378', 'value': 300}, {'name': '379', 'value': 148}, {'name': '380', 'value': 212}, {'name': '381', 'value': 246}, {'name': '382', 'value': 212}, {'name': '383', 'value': 148}, {'name': '384', 'value': 383}, {'name': '385', 'value': 286}, {'name': '386', 'value': 300}, {'name': '387', 'value': 212}, {'name': '388', 'value': 148}, {'name': '389', 'value': 1800}, {'name': '390', 'value': 148}, {'name': '391', 'value': 212}, {'name': '392', 'value': 400}, {'name': '393', 'value': 148}, {'name': '394', 'value': 212}, {'name': '395', 'value': 212}, {'name': '396', 'value': 148}, {'name': '397', 'value': 148}, {'name': '398', 'value': 212}, {'name': '399', 'value': 148}, {'name': '400', 'value': 212}, {'name': '401', 'value': 148}, {'name': '402', 'value': 212}, {'name': '403', 'value': 212}, {'name': '404', 'value': 212}, {'name': '405', 'value': 212}, {'name': '406', 'value': 6000}, {'name': '407', 'value': 300}, {'name': '408', 'value': 212}, {'name': '409', 'value': 212}, {'name': '410', 'value': 148}, {'name': '411', 'value': 212}, {'name': '412', 'value': 250}, {'name': '413', 'value': 342}, {'name': '414', 'value': 148}, {'name': '415', 'value': 212}, {'name': '416', 'value': 212}, {'name': '417', 'value': 212}, {'name': '418', 'value': 148}, {'name': '419', 'value': 212}, {'name': '420', 'value': 212}, {'name': '421', 'value': 118}, {'name': '422', 'value': 212}, {'name': '423', 'value': 148}, {'name': '424', 'value': 6000}, {'name': '425', 'value': 148}, {'name': '426', 'value': 148}, {'name': '427', 'value': 400}, {'name': '428', 'value': 148}, {'name': '429', 'value': 148}, {'name': '430', 'value': 212}, {'name': '431', 'value': 118}, {'name': '432', 'value': 148}, {'name': '433', 'value': 148}, {'name': '434', 'value': 118}, {'name': '435', 'value': 148}, {'name': '436', 'value': 148}, {'name': '437', 'value': 383}, {'name': '438', 'value': 148}, {'name': '439', 'value': 118}, {'name': '440', 'value': 6000}, {'name': '441', 'value': 148}, {'name': '442', 'value': 212}, {'name': '443', 'value': 148}, {'name': '444', 'value': 212}, {'name': '445', 'value': 300}, {'name': '446', 'value': 162}, {'name': '447', 'value': 212}, {'name': '448', 'value': 212}, {'name': '449', 'value': 800}, {'name': '450', 'value': 300}, {'name': '451', 'value': 212}, {'name': '452', 'value': 162}, {'name': '453', 'value': 148}, {'name': '454', 'value': 148}, {'name': '455', 'value': 212}, {'name': '456', 'value': 50}, {'name': '457', 'value': 300}, {'name': '458', 'value': 246}, {'name': '459', 'value': 212}, {'name': '460', 'value': 162}, {'name': '461', 'value': 212}, {'name': '462', 'value': 148}, {'name': '463', 'value': 800}, {'name': '464', 'value': 162}, {'name': '465', 'value': 148}, {'name': '466', 'value': 148}, {'name': '467', 'value': 212}, {'name': '468', 'value': 148}, {'name': '469', 'value': 300}, {'name': '470', 'value': 212}, {'name': '471', 'value': 800}, {'name': '472', 'value': 148}, {'name': '473', 'value': 162}, {'name': '474', 'value': 50}, {'name': '475', 'value': 148}, {'name': '476', 'value': 212}, {'name': '477', 'value': 45}, {'name': '478', 'value': 212}, {'name': '479', 'value': 400}, {'name': '480', 'value': 212}, {'name': '481', 'value': 212}, {'name': '482', 'value': 148}, {'name': '483', 'value': 212}, {'name': '484', 'value': 118}, {'name': '485', 'value': 300}, {'name': '486', 'value': 212}, {'name': '487', 'value': 3600}, {'name': '488', 'value': 148}, {'name': '489', 'value': 286}, {'name': '490', 'value': 148}, {'name': '491', 'value': 212}, {'name': '492', 'value': 148}, {'name': '493', 'value': 148}, {'name': '494', 'value': 148}, {'name': '495', 'value': 148}, {'name': '496', 'value': 300}, {'name': '497', 'value': 400}, {'name': '498', 'value': 3600}, {'name': '499', 'value': 212}, {'name': '500', 'value': 300}]}, 'PatientBal': {'name': 'PatientBal', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': -172}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': -40}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': -10}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': -10}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': -250}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': -99}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': -14}, {'name': '232', 'value': -93}, {'name': '233', 'value': -170}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 50}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': -10}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 60}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 10}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': -2}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': -63}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 123}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': -15}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': -38}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': -25}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'InsuranceBal': {'name': 'InsuranceBal', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 24}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 76}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 36}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'PrimaryInsuranceBal': {'name': 'PrimaryInsuranceBal', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 76}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'CPTCode': {'name': 'CPTCode', 'series': [{'name': '1', 'value': 92014}, {'name': '2', 'value': 92014}, {'name': '3', 'value': 92014}, {'name': '4', 'value': 92134}, {'name': '5', 'value': 92134}, {'name': '6', 'value': 92014}, {'name': '7', 'value': 92134}, {'name': '8', 'value': 92250}, {'name': '9', 'value': 92014}, {'name': '10', 'value': 92014}, {'name': '11', 'value': 92014}, {'name': '12', 'value': 67028}, {'name': '13', 'value': 92250}, {'name': '14', 'value': 92134}, {'name': '15', 'value': 92250}, {'name': '16', 'value': 92134}, {'name': '17', 'value': 92134}, {'name': '18', 'value': 99204}, {'name': '19', 'value': 92134}, {'name': '20', 'value': 67028}, {'name': '21', 'value': 99354}, {'name': '22', 'value': 92250}, {'name': '23', 'value': 92014}, {'name': '24', 'value': 67028}, {'name': '25', 'value': 92134}, {'name': '26', 'value': 92134}, {'name': '27', 'value': 92250}, {'name': '28', 'value': 99215}, {'name': '29', 'value': 67028}, {'name': '30', 'value': 92014}, {'name': '31', 'value': 92014}, {'name': '32', 'value': 92134}, {'name': '33', 'value': 92014}, {'name': '34', 'value': 92014}, {'name': '35', 'value': 92012}, {'name': '36', 'value': 92134}, {'name': '37', 'value': 92134}, {'name': '38', 'value': 0}, {'name': '39', 'value': 92250}, {'name': '40', 'value': 92134}, {'name': '41', 'value': 92014}, {'name': '42', 'value': 92014}, {'name': '43', 'value': 92134}, {'name': '44', 'value': 92134}, {'name': '45', 'value': 99204}, {'name': '46', 'value': 92134}, {'name': '47', 'value': 92012}, {'name': '48', 'value': 92014}, {'name': '49', 'value': 92134}, {'name': '50', 'value': 92014}, {'name': '51', 'value': 92134}, {'name': '52', 'value': 92014}, {'name': '53', 'value': 92014}, {'name': '54', 'value': 92134}, {'name': '55', 'value': 67028}, {'name': '56', 'value': 92134}, {'name': '57', 'value': 92250}, {'name': '58', 'value': 92225}, {'name': '59', 'value': 92134}, {'name': '60', 'value': 92134}, {'name': '61', 'value': 92012}, {'name': '62', 'value': 92134}, {'name': '63', 'value': 92134}, {'name': '64', 'value': 92014}, {'name': '65', 'value': 92134}, {'name': '66', 'value': 92014}, {'name': '67', 'value': 92014}, {'name': '68', 'value': 92014}, {'name': '69', 'value': 92014}, {'name': '70', 'value': 92134}, {'name': '71', 'value': 99204}, {'name': '72', 'value': 92134}, {'name': '73', 'value': 99204}, {'name': '74', 'value': 92014}, {'name': '75', 'value': 92134}, {'name': '76', 'value': 92014}, {'name': '77', 'value': 92134}, {'name': '78', 'value': 99204}, {'name': '79', 'value': 92014}, {'name': '80', 'value': 92014}, {'name': '81', 'value': 92134}, {'name': '82', 'value': 92134}, {'name': '83', 'value': 92226}, {'name': '84', 'value': 92134}, {'name': '85', 'value': 92014}, {'name': '86', 'value': 92014}, {'name': '87', 'value': 92134}, {'name': '88', 'value': 92134}, {'name': '89', 'value': 92012}, {'name': '90', 'value': 92014}, {'name': '91', 'value': 92134}, {'name': '92', 'value': 92004}, {'name': '93', 'value': 99204}, {'name': '94', 'value': 92134}, {'name': '95', 'value': 92134}, {'name': '96', 'value': 76512}, {'name': '97', 'value': 92014}, {'name': '98', 'value': 92134}, {'name': '99', 'value': 92004}, {'name': '100', 'value': 92134}, {'name': '101', 'value': 92014}, {'name': '102', 'value': 92250}, {'name': '103', 'value': 92014}, {'name': '104', 'value': 92014}, {'name': '105', 'value': 92014}, {'name': '106', 'value': 92134}, {'name': '107', 'value': 92014}, {'name': '108', 'value': 92014}, {'name': '109', 'value': 92134}, {'name': '110', 'value': 92014}, {'name': '111', 'value': 67036}, {'name': '112', 'value': 92250}, {'name': '113', 'value': 92134}, {'name': '114', 'value': 92134}, {'name': '115', 'value': 92250}, {'name': '116', 'value': 92134}, {'name': '117', 'value': 92250}, {'name': '118', 'value': 92014}, {'name': '119', 'value': 92014}, {'name': '120', 'value': 92134}, {'name': '121', 'value': 92134}, {'name': '122', 'value': 92225}, {'name': '123', 'value': 92014}, {'name': '124', 'value': 92134}, {'name': '125', 'value': 92250}, {'name': '126', 'value': 0}, {'name': '127', 'value': 67028}, {'name': '128', 'value': 92226}, {'name': '129', 'value': 92134}, {'name': '130', 'value': 92134}, {'name': '131', 'value': 92250}, {'name': '132', 'value': 67028}, {'name': '133', 'value': 92014}, {'name': '134', 'value': 92225}, {'name': '135', 'value': 92134}, {'name': '136', 'value': 92014}, {'name': '137', 'value': 92134}, {'name': '138', 'value': 92134}, {'name': '139', 'value': 92014}, {'name': '140', 'value': 92014}, {'name': '141', 'value': 92226}, {'name': '142', 'value': 92134}, {'name': '143', 'value': 92134}, {'name': '144', 'value': 92134}, {'name': '145', 'value': 92134}, {'name': '146', 'value': 92134}, {'name': '147', 'value': 92235}, {'name': '148', 'value': 92014}, {'name': '149', 'value': 92014}, {'name': '150', 'value': 92134}, {'name': '151', 'value': 92134}, {'name': '152', 'value': 92134}, {'name': '153', 'value': 92134}, {'name': '154', 'value': 92134}, {'name': '155', 'value': 92134}, {'name': '156', 'value': 92014}, {'name': '157', 'value': 0}, {'name': '158', 'value': 92134}, {'name': '159', 'value': 92014}, {'name': '160', 'value': 92134}, {'name': '161', 'value': 92134}, {'name': '162', 'value': 92014}, {'name': '163', 'value': 92134}, {'name': '164', 'value': 92226}, {'name': '165', 'value': 92134}, {'name': '166', 'value': 92134}, {'name': '167', 'value': 92134}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 92014}, {'name': '172', 'value': 92134}, {'name': '173', 'value': 92134}, {'name': '174', 'value': 92250}, {'name': '175', 'value': 92014}, {'name': '176', 'value': 92134}, {'name': '177', 'value': 92134}, {'name': '178', 'value': 92250}, {'name': '179', 'value': 92134}, {'name': '180', 'value': 92134}, {'name': '181', 'value': 99204}, {'name': '182', 'value': 92014}, {'name': '183', 'value': 92235}, {'name': '184', 'value': 99204}, {'name': '185', 'value': 0}, {'name': '186', 'value': 92134}, {'name': '187', 'value': 92014}, {'name': '188', 'value': 92014}, {'name': '189', 'value': 92134}, {'name': '190', 'value': 92014}, {'name': '191', 'value': 92134}, {'name': '192', 'value': 92014}, {'name': '193', 'value': 92134}, {'name': '194', 'value': 92014}, {'name': '195', 'value': 92134}, {'name': '196', 'value': 92014}, {'name': '197', 'value': 92014}, {'name': '198', 'value': 92014}, {'name': '199', 'value': 99204}, {'name': '200', 'value': 92134}, {'name': '201', 'value': 92014}, {'name': '202', 'value': 92134}, {'name': '203', 'value': 92014}, {'name': '204', 'value': 67028}, {'name': '205', 'value': 92250}, {'name': '206', 'value': 92134}, {'name': '207', 'value': 99213}, {'name': '208', 'value': 92134}, {'name': '209', 'value': 92250}, {'name': '210', 'value': 92134}, {'name': '211', 'value': 92014}, {'name': '212', 'value': 92134}, {'name': '213', 'value': 92134}, {'name': '214', 'value': 92134}, {'name': '215', 'value': 92014}, {'name': '216', 'value': 92250}, {'name': '217', 'value': 92134}, {'name': '218', 'value': 92134}, {'name': '219', 'value': 92134}, {'name': '220', 'value': 92134}, {'name': '221', 'value': 92235}, {'name': '222', 'value': 99204}, {'name': '223', 'value': 92134}, {'name': '224', 'value': 92134}, {'name': '225', 'value': 99203}, {'name': '226', 'value': 92014}, {'name': '227', 'value': 92014}, {'name': '228', 'value': 92014}, {'name': '229', 'value': 0}, {'name': '230', 'value': 92014}, {'name': '231', 'value': 92134}, {'name': '232', 'value': 99204}, {'name': '233', 'value': 99204}, {'name': '234', 'value': 67113}, {'name': '235', 'value': 92014}, {'name': '236', 'value': 92134}, {'name': '237', 'value': 92250}, {'name': '238', 'value': 92235}, {'name': '239', 'value': 92250}, {'name': '240', 'value': 92134}, {'name': '241', 'value': 92014}, {'name': '242', 'value': 92014}, {'name': '243', 'value': 92134}, {'name': '244', 'value': 92134}, {'name': '245', 'value': 92250}, {'name': '246', 'value': 92250}, {'name': '247', 'value': 92134}, {'name': '248', 'value': 92134}, {'name': '249', 'value': 92014}, {'name': '250', 'value': 99204}, {'name': '251', 'value': 92014}, {'name': '252', 'value': 92014}, {'name': '253', 'value': 92014}, {'name': '254', 'value': 92225}, {'name': '255', 'value': 99204}, {'name': '256', 'value': 92134}, {'name': '257', 'value': 92134}, {'name': '258', 'value': 92134}, {'name': '259', 'value': 92235}, {'name': '260', 'value': 92014}, {'name': '261', 'value': 92134}, {'name': '262', 'value': 92014}, {'name': '263', 'value': 92014}, {'name': '264', 'value': 92250}, {'name': '265', 'value': 67028}, {'name': '266', 'value': 92014}, {'name': '267', 'value': 92014}, {'name': '268', 'value': 92014}, {'name': '269', 'value': 92014}, {'name': '270', 'value': 92250}, {'name': '271', 'value': 92134}, {'name': '272', 'value': 92134}, {'name': '273', 'value': 92134}, {'name': '274', 'value': 92250}, {'name': '275', 'value': 92235}, {'name': '276', 'value': 92014}, {'name': '277', 'value': 92134}, {'name': '278', 'value': 92014}, {'name': '279', 'value': 92134}, {'name': '280', 'value': 92134}, {'name': '281', 'value': 92134}, {'name': '282', 'value': 92014}, {'name': '283', 'value': 92014}, {'name': '284', 'value': 92014}, {'name': '285', 'value': 92134}, {'name': '286', 'value': 92134}, {'name': '287', 'value': 92014}, {'name': '288', 'value': 92014}, {'name': '289', 'value': 92250}, {'name': '290', 'value': 0}, {'name': '291', 'value': 92134}, {'name': '292', 'value': 92134}, {'name': '293', 'value': 92134}, {'name': '294', 'value': 92012}, {'name': '295', 'value': 92134}, {'name': '296', 'value': 92134}, {'name': '297', 'value': 92235}, {'name': '298', 'value': 99204}, {'name': '299', 'value': 92225}, {'name': '300', 'value': 92014}, {'name': '301', 'value': 92134}, {'name': '302', 'value': 92134}, {'name': '303', 'value': 92134}, {'name': '304', 'value': 67028}, {'name': '305', 'value': 92014}, {'name': '306', 'value': 67028}, {'name': '307', 'value': 92014}, {'name': '308', 'value': 92134}, {'name': '309', 'value': 92134}, {'name': '310', 'value': 92226}, {'name': '311', 'value': 92014}, {'name': '312', 'value': 92134}, {'name': '313', 'value': 92014}, {'name': '314', 'value': 92014}, {'name': '315', 'value': 99204}, {'name': '316', 'value': 92014}, {'name': '317', 'value': 92134}, {'name': '318', 'value': 92014}, {'name': '319', 'value': 67028}, {'name': '320', 'value': 92014}, {'name': '321', 'value': 67028}, {'name': '322', 'value': 99204}, {'name': '323', 'value': 99204}, {'name': '324', 'value': 92014}, {'name': '325', 'value': 0}, {'name': '326', 'value': 92014}, {'name': '327', 'value': 92134}, {'name': '328', 'value': 92134}, {'name': '329', 'value': 92012}, {'name': '330', 'value': 92014}, {'name': '331', 'value': 92250}, {'name': '332', 'value': 92014}, {'name': '333', 'value': 92134}, {'name': '334', 'value': 92134}, {'name': '335', 'value': 92014}, {'name': '336', 'value': 92250}, {'name': '337', 'value': 92134}, {'name': '338', 'value': 92134}, {'name': '339', 'value': 92014}, {'name': '340', 'value': 92134}, {'name': '341', 'value': 92134}, {'name': '342', 'value': 92014}, {'name': '343', 'value': 92134}, {'name': '344', 'value': 92134}, {'name': '345', 'value': 92014}, {'name': '346', 'value': 67028}, {'name': '347', 'value': 92250}, {'name': '348', 'value': 92134}, {'name': '349', 'value': 92014}, {'name': '350', 'value': 92014}, {'name': '351', 'value': 92012}, {'name': '352', 'value': 92134}, {'name': '353', 'value': 92014}, {'name': '354', 'value': 67028}, {'name': '355', 'value': 99215}, {'name': '356', 'value': 92134}, {'name': '357', 'value': 92134}, {'name': '358', 'value': 92235}, {'name': '359', 'value': 92134}, {'name': '360', 'value': 92014}, {'name': '361', 'value': 92014}, {'name': '362', 'value': 92014}, {'name': '363', 'value': 92134}, {'name': '364', 'value': 92004}, {'name': '365', 'value': 92014}, {'name': '366', 'value': 67028}, {'name': '367', 'value': 92134}, {'name': '368', 'value': 92018}, {'name': '369', 'value': 67036}, {'name': '370', 'value': 92250}, {'name': '371', 'value': 92004}, {'name': '372', 'value': 0}, {'name': '373', 'value': 92014}, {'name': '374', 'value': 0}, {'name': '375', 'value': 92134}, {'name': '376', 'value': 92134}, {'name': '377', 'value': 92014}, {'name': '378', 'value': 99204}, {'name': '379', 'value': 92134}, {'name': '380', 'value': 92014}, {'name': '381', 'value': 92004}, {'name': '382', 'value': 92014}, {'name': '383', 'value': 92134}, {'name': '384', 'value': 76512}, {'name': '385', 'value': 92235}, {'name': '386', 'value': 99204}, {'name': '387', 'value': 92014}, {'name': '388', 'value': 92134}, {'name': '389', 'value': 0}, {'name': '390', 'value': 92134}, {'name': '391', 'value': 92014}, {'name': '392', 'value': 67028}, {'name': '393', 'value': 92134}, {'name': '394', 'value': 92014}, {'name': '395', 'value': 92014}, {'name': '396', 'value': 92134}, {'name': '397', 'value': 92134}, {'name': '398', 'value': 92014}, {'name': '399', 'value': 92134}, {'name': '400', 'value': 92014}, {'name': '401', 'value': 92134}, {'name': '402', 'value': 92014}, {'name': '403', 'value': 92014}, {'name': '404', 'value': 92014}, {'name': '405', 'value': 92014}, {'name': '406', 'value': 0}, {'name': '407', 'value': 99204}, {'name': '408', 'value': 92014}, {'name': '409', 'value': 92014}, {'name': '410', 'value': 92134}, {'name': '411', 'value': 92014}, {'name': '412', 'value': 99215}, {'name': '413', 'value': 99205}, {'name': '414', 'value': 92134}, {'name': '415', 'value': 92014}, {'name': '416', 'value': 92014}, {'name': '417', 'value': 92014}, {'name': '418', 'value': 92134}, {'name': '419', 'value': 92014}, {'name': '420', 'value': 92014}, {'name': '421', 'value': 92012}, {'name': '422', 'value': 92014}, {'name': '423', 'value': 92134}, {'name': '424', 'value': 0}, {'name': '425', 'value': 92134}, {'name': '426', 'value': 92134}, {'name': '427', 'value': 67028}, {'name': '428', 'value': 92134}, {'name': '429', 'value': 92134}, {'name': '430', 'value': 92014}, {'name': '431', 'value': 92012}, {'name': '432', 'value': 92134}, {'name': '433', 'value': 92134}, {'name': '434', 'value': 92012}, {'name': '435', 'value': 92134}, {'name': '436', 'value': 92134}, {'name': '437', 'value': 76512}, {'name': '438', 'value': 92134}, {'name': '439', 'value': 92012}, {'name': '440', 'value': 0}, {'name': '441', 'value': 92134}, {'name': '442', 'value': 92014}, {'name': '443', 'value': 92134}, {'name': '444', 'value': 92014}, {'name': '445', 'value': 99204}, {'name': '446', 'value': 92250}, {'name': '447', 'value': 92014}, {'name': '448', 'value': 92014}, {'name': '449', 'value': 67028}, {'name': '450', 'value': 99204}, {'name': '451', 'value': 92014}, {'name': '452', 'value': 92250}, {'name': '453', 'value': 92134}, {'name': '454', 'value': 92134}, {'name': '455', 'value': 92014}, {'name': '456', 'value': 92225}, {'name': '457', 'value': 99204}, {'name': '458', 'value': 92004}, {'name': '459', 'value': 92014}, {'name': '460', 'value': 92250}, {'name': '461', 'value': 92014}, {'name': '462', 'value': 92134}, {'name': '463', 'value': 67028}, {'name': '464', 'value': 92250}, {'name': '465', 'value': 92134}, {'name': '466', 'value': 92134}, {'name': '467', 'value': 92014}, {'name': '468', 'value': 92134}, {'name': '469', 'value': 99204}, {'name': '470', 'value': 92014}, {'name': '471', 'value': 67028}, {'name': '472', 'value': 92134}, {'name': '473', 'value': 92250}, {'name': '474', 'value': 92225}, {'name': '475', 'value': 92134}, {'name': '476', 'value': 92014}, {'name': '477', 'value': 92226}, {'name': '478', 'value': 92014}, {'name': '479', 'value': 67028}, {'name': '480', 'value': 92014}, {'name': '481', 'value': 92014}, {'name': '482', 'value': 92134}, {'name': '483', 'value': 92014}, {'name': '484', 'value': 92012}, {'name': '485', 'value': 99204}, {'name': '486', 'value': 92014}, {'name': '487', 'value': 0}, {'name': '488', 'value': 92134}, {'name': '489', 'value': 92235}, {'name': '490', 'value': 92134}, {'name': '491', 'value': 92014}, {'name': '492', 'value': 92134}, {'name': '493', 'value': 92134}, {'name': '494', 'value': 92134}, {'name': '495', 'value': 92134}, {'name': '496', 'value': 99204}, {'name': '497', 'value': 99354}, {'name': '498', 'value': 0}, {'name': '499', 'value': 92014}, {'name': '500', 'value': 99204}]}, 'CptModifier': {'name': 'CptModifier', 'series': [{'name': '1', 'value': 25}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 25}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 25}, {'name': '29', 'value': 0}, {'name': '30', 'value': 25}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 25}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 25}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 50}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 25}, {'name': '67', 'value': 0}, {'name': '68', 'value': 25}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 25}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 25}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 25}, {'name': '91', 'value': 0}, {'name': '92', 'value': 25}, {'name': '93', 'value': 25}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 25}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 25}, {'name': '105', 'value': 25}, {'name': '106', 'value': 0}, {'name': '107', 'value': 25}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 25}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 50}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 50}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 25}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 25}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 25}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 25}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 26}, {'name': '175', 'value': 25}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 25}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 25}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 25}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 25}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 50}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 25}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 25}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 25}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 25}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 25}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 50}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 25}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 25}, {'name': '277', 'value': 0}, {'name': '278', 'value': 25}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 25}, {'name': '299', 'value': 0}, {'name': '300', 'value': 25}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 50}, {'name': '305', 'value': 25}, {'name': '306', 'value': 50}, {'name': '307', 'value': 25}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 25}, {'name': '312', 'value': 0}, {'name': '313', 'value': 25}, {'name': '314', 'value': 0}, {'name': '315', 'value': 25}, {'name': '316', 'value': 25}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 50}, {'name': '320', 'value': 25}, {'name': '321', 'value': 0}, {'name': '322', 'value': 57}, {'name': '323', 'value': 25}, {'name': '324', 'value': 25}, {'name': '325', 'value': 0}, {'name': '326', 'value': 25}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 25}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 25}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 50}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 25}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 57}, {'name': '365', 'value': 0}, {'name': '366', 'value': 50}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 25}, {'name': '372', 'value': 0}, {'name': '373', 'value': 25}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 25}, {'name': '379', 'value': 0}, {'name': '380', 'value': 25}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 25}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 25}, {'name': '413', 'value': 25}, {'name': '414', 'value': 0}, {'name': '415', 'value': 25}, {'name': '416', 'value': 25}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 25}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 25}, {'name': '431', 'value': 25}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 25}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 25}, {'name': '448', 'value': 0}, {'name': '449', 'value': 50}, {'name': '450', 'value': 0}, {'name': '451', 'value': 25}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 25}, {'name': '456', 'value': 0}, {'name': '457', 'value': 57}, {'name': '458', 'value': 0}, {'name': '459', 'value': 25}, {'name': '460', 'value': 0}, {'name': '461', 'value': 25}, {'name': '462', 'value': 0}, {'name': '463', 'value': 50}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 25}, {'name': '471', 'value': 50}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 25}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'pdiagnosis1': {'name': 'pdiagnosis1', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'pdiagnosis2': {'name': 'pdiagnosis2', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'IsBillable': {'name': 'IsBillable', 'series': [{'name': '1', 'value': 1}, {'name': '2', 'value': 1}, {'name': '3', 'value': 1}, {'name': '4', 'value': 1}, {'name': '5', 'value': 1}, {'name': '6', 'value': 1}, {'name': '7', 'value': 1}, {'name': '8', 'value': 1}, {'name': '9', 'value': 1}, {'name': '10', 'value': 1}, {'name': '11', 'value': 1}, {'name': '12', 'value': 1}, {'name': '13', 'value': 1}, {'name': '14', 'value': 1}, {'name': '15', 'value': 1}, {'name': '16', 'value': 1}, {'name': '17', 'value': 1}, {'name': '18', 'value': 1}, {'name': '19', 'value': 1}, {'name': '20', 'value': 1}, {'name': '21', 'value': 1}, {'name': '22', 'value': 1}, {'name': '23', 'value': 1}, {'name': '24', 'value': 1}, {'name': '25', 'value': 1}, {'name': '26', 'value': 1}, {'name': '27', 'value': 1}, {'name': '28', 'value': 1}, {'name': '29', 'value': 1}, {'name': '30', 'value': 1}, {'name': '31', 'value': 1}, {'name': '32', 'value': 1}, {'name': '33', 'value': 1}, {'name': '34', 'value': 1}, {'name': '35', 'value': 1}, {'name': '36', 'value': 1}, {'name': '37', 'value': 1}, {'name': '38', 'value': 1}, {'name': '39', 'value': 1}, {'name': '40', 'value': 1}, {'name': '41', 'value': 1}, {'name': '42', 'value': 1}, {'name': '43', 'value': 1}, {'name': '44', 'value': 1}, {'name': '45', 'value': 1}, {'name': '46', 'value': 1}, {'name': '47', 'value': 1}, {'name': '48', 'value': 1}, {'name': '49', 'value': 1}, {'name': '50', 'value': 1}, {'name': '51', 'value': 1}, {'name': '52', 'value': 1}, {'name': '53', 'value': 1}, {'name': '54', 'value': 1}, {'name': '55', 'value': 1}, {'name': '56', 'value': 1}, {'name': '57', 'value': 1}, {'name': '58', 'value': 1}, {'name': '59', 'value': 1}, {'name': '60', 'value': 1}, {'name': '61', 'value': 1}, {'name': '62', 'value': 1}, {'name': '63', 'value': 1}, {'name': '64', 'value': 1}, {'name': '65', 'value': 1}, {'name': '66', 'value': 1}, {'name': '67', 'value': 1}, {'name': '68', 'value': 1}, {'name': '69', 'value': 1}, {'name': '70', 'value': 1}, {'name': '71', 'value': 1}, {'name': '72', 'value': 1}, {'name': '73', 'value': 1}, {'name': '74', 'value': 1}, {'name': '75', 'value': 1}, {'name': '76', 'value': 1}, {'name': '77', 'value': 1}, {'name': '78', 'value': 1}, {'name': '79', 'value': 1}, {'name': '80', 'value': 1}, {'name': '81', 'value': 1}, {'name': '82', 'value': 1}, {'name': '83', 'value': 1}, {'name': '84', 'value': 1}, {'name': '85', 'value': 1}, {'name': '86', 'value': 1}, {'name': '87', 'value': 1}, {'name': '88', 'value': 1}, {'name': '89', 'value': 1}, {'name': '90', 'value': 1}, {'name': '91', 'value': 1}, {'name': '92', 'value': 1}, {'name': '93', 'value': 1}, {'name': '94', 'value': 1}, {'name': '95', 'value': 1}, {'name': '96', 'value': 1}, {'name': '97', 'value': 1}, {'name': '98', 'value': 1}, {'name': '99', 'value': 1}, {'name': '100', 'value': 1}, {'name': '101', 'value': 1}, {'name': '102', 'value': 1}, {'name': '103', 'value': 1}, {'name': '104', 'value': 1}, {'name': '105', 'value': 1}, {'name': '106', 'value': 1}, {'name': '107', 'value': 1}, {'name': '108', 'value': 1}, {'name': '109', 'value': 1}, {'name': '110', 'value': 1}, {'name': '111', 'value': 1}, {'name': '112', 'value': 1}, {'name': '113', 'value': 1}, {'name': '114', 'value': 1}, {'name': '115', 'value': 1}, {'name': '116', 'value': 1}, {'name': '117', 'value': 1}, {'name': '118', 'value': 1}, {'name': '119', 'value': 1}, {'name': '120', 'value': 1}, {'name': '121', 'value': 1}, {'name': '122', 'value': 1}, {'name': '123', 'value': 1}, {'name': '124', 'value': 1}, {'name': '125', 'value': 1}, {'name': '126', 'value': 1}, {'name': '127', 'value': 1}, {'name': '128', 'value': 1}, {'name': '129', 'value': 1}, {'name': '130', 'value': 1}, {'name': '131', 'value': 1}, {'name': '132', 'value': 1}, {'name': '133', 'value': 1}, {'name': '134', 'value': 1}, {'name': '135', 'value': 1}, {'name': '136', 'value': 1}, {'name': '137', 'value': 1}, {'name': '138', 'value': 1}, {'name': '139', 'value': 1}, {'name': '140', 'value': 1}, {'name': '141', 'value': 1}, {'name': '142', 'value': 1}, {'name': '143', 'value': 1}, {'name': '144', 'value': 1}, {'name': '145', 'value': 1}, {'name': '146', 'value': 1}, {'name': '147', 'value': 1}, {'name': '148', 'value': 1}, {'name': '149', 'value': 1}, {'name': '150', 'value': 1}, {'name': '151', 'value': 1}, {'name': '152', 'value': 1}, {'name': '153', 'value': 1}, {'name': '154', 'value': 1}, {'name': '155', 'value': 1}, {'name': '156', 'value': 1}, {'name': '157', 'value': 1}, {'name': '158', 'value': 1}, {'name': '159', 'value': 1}, {'name': '160', 'value': 1}, {'name': '161', 'value': 1}, {'name': '162', 'value': 1}, {'name': '163', 'value': 1}, {'name': '164', 'value': 1}, {'name': '165', 'value': 1}, {'name': '166', 'value': 1}, {'name': '167', 'value': 1}, {'name': '168', 'value': 1}, {'name': '169', 'value': 1}, {'name': '170', 'value': 1}, {'name': '171', 'value': 1}, {'name': '172', 'value': 1}, {'name': '173', 'value': 1}, {'name': '174', 'value': 1}, {'name': '175', 'value': 1}, {'name': '176', 'value': 1}, {'name': '177', 'value': 1}, {'name': '178', 'value': 1}, {'name': '179', 'value': 1}, {'name': '180', 'value': 1}, {'name': '181', 'value': 1}, {'name': '182', 'value': 1}, {'name': '183', 'value': 1}, {'name': '184', 'value': 1}, {'name': '185', 'value': 1}, {'name': '186', 'value': 1}, {'name': '187', 'value': 1}, {'name': '188', 'value': 1}, {'name': '189', 'value': 1}, {'name': '190', 'value': 1}, {'name': '191', 'value': 1}, {'name': '192', 'value': 1}, {'name': '193', 'value': 1}, {'name': '194', 'value': 1}, {'name': '195', 'value': 1}, {'name': '196', 'value': 1}, {'name': '197', 'value': 1}, {'name': '198', 'value': 1}, {'name': '199', 'value': 1}, {'name': '200', 'value': 1}, {'name': '201', 'value': 1}, {'name': '202', 'value': 1}, {'name': '203', 'value': 1}, {'name': '204', 'value': 1}, {'name': '205', 'value': 1}, {'name': '206', 'value': 1}, {'name': '207', 'value': 1}, {'name': '208', 'value': 1}, {'name': '209', 'value': 1}, {'name': '210', 'value': 1}, {'name': '211', 'value': 1}, {'name': '212', 'value': 1}, {'name': '213', 'value': 1}, {'name': '214', 'value': 1}, {'name': '215', 'value': 1}, {'name': '216', 'value': 1}, {'name': '217', 'value': 1}, {'name': '218', 'value': 1}, {'name': '219', 'value': 1}, {'name': '220', 'value': 1}, {'name': '221', 'value': 1}, {'name': '222', 'value': 1}, {'name': '223', 'value': 1}, {'name': '224', 'value': 1}, {'name': '225', 'value': 1}, {'name': '226', 'value': 1}, {'name': '227', 'value': 1}, {'name': '228', 'value': 1}, {'name': '229', 'value': 1}, {'name': '230', 'value': 1}, {'name': '231', 'value': 1}, {'name': '232', 'value': 1}, {'name': '233', 'value': 1}, {'name': '234', 'value': 1}, {'name': '235', 'value': 1}, {'name': '236', 'value': 1}, {'name': '237', 'value': 1}, {'name': '238', 'value': 1}, {'name': '239', 'value': 1}, {'name': '240', 'value': 1}, {'name': '241', 'value': 1}, {'name': '242', 'value': 1}, {'name': '243', 'value': 1}, {'name': '244', 'value': 1}, {'name': '245', 'value': 1}, {'name': '246', 'value': 1}, {'name': '247', 'value': 1}, {'name': '248', 'value': 1}, {'name': '249', 'value': 1}, {'name': '250', 'value': 1}, {'name': '251', 'value': 1}, {'name': '252', 'value': 1}, {'name': '253', 'value': 1}, {'name': '254', 'value': 1}, {'name': '255', 'value': 1}, {'name': '256', 'value': 1}, {'name': '257', 'value': 1}, {'name': '258', 'value': 1}, {'name': '259', 'value': 1}, {'name': '260', 'value': 1}, {'name': '261', 'value': 1}, {'name': '262', 'value': 1}, {'name': '263', 'value': 1}, {'name': '264', 'value': 1}, {'name': '265', 'value': 1}, {'name': '266', 'value': 1}, {'name': '267', 'value': 1}, {'name': '268', 'value': 1}, {'name': '269', 'value': 1}, {'name': '270', 'value': 1}, {'name': '271', 'value': 1}, {'name': '272', 'value': 1}, {'name': '273', 'value': 1}, {'name': '274', 'value': 1}, {'name': '275', 'value': 1}, {'name': '276', 'value': 1}, {'name': '277', 'value': 1}, {'name': '278', 'value': 1}, {'name': '279', 'value': 1}, {'name': '280', 'value': 1}, {'name': '281', 'value': 1}, {'name': '282', 'value': 1}, {'name': '283', 'value': 1}, {'name': '284', 'value': 1}, {'name': '285', 'value': 1}, {'name': '286', 'value': 1}, {'name': '287', 'value': 1}, {'name': '288', 'value': 1}, {'name': '289', 'value': 1}, {'name': '290', 'value': 1}, {'name': '291', 'value': 1}, {'name': '292', 'value': 1}, {'name': '293', 'value': 1}, {'name': '294', 'value': 1}, {'name': '295', 'value': 1}, {'name': '296', 'value': 1}, {'name': '297', 'value': 1}, {'name': '298', 'value': 1}, {'name': '299', 'value': 1}, {'name': '300', 'value': 1}, {'name': '301', 'value': 1}, {'name': '302', 'value': 1}, {'name': '303', 'value': 1}, {'name': '304', 'value': 1}, {'name': '305', 'value': 1}, {'name': '306', 'value': 1}, {'name': '307', 'value': 1}, {'name': '308', 'value': 1}, {'name': '309', 'value': 1}, {'name': '310', 'value': 1}, {'name': '311', 'value': 1}, {'name': '312', 'value': 1}, {'name': '313', 'value': 1}, {'name': '314', 'value': 1}, {'name': '315', 'value': 1}, {'name': '316', 'value': 1}, {'name': '317', 'value': 1}, {'name': '318', 'value': 1}, {'name': '319', 'value': 1}, {'name': '320', 'value': 1}, {'name': '321', 'value': 1}, {'name': '322', 'value': 1}, {'name': '323', 'value': 1}, {'name': '324', 'value': 1}, {'name': '325', 'value': 1}, {'name': '326', 'value': 1}, {'name': '327', 'value': 1}, {'name': '328', 'value': 1}, {'name': '329', 'value': 1}, {'name': '330', 'value': 1}, {'name': '331', 'value': 1}, {'name': '332', 'value': 1}, {'name': '333', 'value': 1}, {'name': '334', 'value': 1}, {'name': '335', 'value': 1}, {'name': '336', 'value': 1}, {'name': '337', 'value': 1}, {'name': '338', 'value': 1}, {'name': '339', 'value': 1}, {'name': '340', 'value': 1}, {'name': '341', 'value': 1}, {'name': '342', 'value': 1}, {'name': '343', 'value': 1}, {'name': '344', 'value': 1}, {'name': '345', 'value': 1}, {'name': '346', 'value': 1}, {'name': '347', 'value': 1}, {'name': '348', 'value': 1}, {'name': '349', 'value': 1}, {'name': '350', 'value': 1}, {'name': '351', 'value': 1}, {'name': '352', 'value': 1}, {'name': '353', 'value': 1}, {'name': '354', 'value': 1}, {'name': '355', 'value': 1}, {'name': '356', 'value': 1}, {'name': '357', 'value': 1}, {'name': '358', 'value': 1}, {'name': '359', 'value': 1}, {'name': '360', 'value': 1}, {'name': '361', 'value': 1}, {'name': '362', 'value': 1}, {'name': '363', 'value': 1}, {'name': '364', 'value': 1}, {'name': '365', 'value': 1}, {'name': '366', 'value': 1}, {'name': '367', 'value': 1}, {'name': '368', 'value': 1}, {'name': '369', 'value': 1}, {'name': '370', 'value': 1}, {'name': '371', 'value': 1}, {'name': '372', 'value': 1}, {'name': '373', 'value': 1}, {'name': '374', 'value': 1}, {'name': '375', 'value': 1}, {'name': '376', 'value': 1}, {'name': '377', 'value': 1}, {'name': '378', 'value': 1}, {'name': '379', 'value': 1}, {'name': '380', 'value': 1}, {'name': '381', 'value': 1}, {'name': '382', 'value': 1}, {'name': '383', 'value': 1}, {'name': '384', 'value': 1}, {'name': '385', 'value': 1}, {'name': '386', 'value': 1}, {'name': '387', 'value': 1}, {'name': '388', 'value': 1}, {'name': '389', 'value': 1}, {'name': '390', 'value': 1}, {'name': '391', 'value': 1}, {'name': '392', 'value': 1}, {'name': '393', 'value': 1}, {'name': '394', 'value': 1}, {'name': '395', 'value': 1}, {'name': '396', 'value': 1}, {'name': '397', 'value': 1}, {'name': '398', 'value': 1}, {'name': '399', 'value': 1}, {'name': '400', 'value': 1}, {'name': '401', 'value': 1}, {'name': '402', 'value': 1}, {'name': '403', 'value': 1}, {'name': '404', 'value': 1}, {'name': '405', 'value': 1}, {'name': '406', 'value': 1}, {'name': '407', 'value': 1}, {'name': '408', 'value': 1}, {'name': '409', 'value': 1}, {'name': '410', 'value': 1}, {'name': '411', 'value': 1}, {'name': '412', 'value': 1}, {'name': '413', 'value': 1}, {'name': '414', 'value': 1}, {'name': '415', 'value': 1}, {'name': '416', 'value': 1}, {'name': '417', 'value': 1}, {'name': '418', 'value': 1}, {'name': '419', 'value': 1}, {'name': '420', 'value': 1}, {'name': '421', 'value': 1}, {'name': '422', 'value': 1}, {'name': '423', 'value': 1}, {'name': '424', 'value': 1}, {'name': '425', 'value': 1}, {'name': '426', 'value': 1}, {'name': '427', 'value': 1}, {'name': '428', 'value': 1}, {'name': '429', 'value': 1}, {'name': '430', 'value': 1}, {'name': '431', 'value': 1}, {'name': '432', 'value': 1}, {'name': '433', 'value': 1}, {'name': '434', 'value': 1}, {'name': '435', 'value': 1}, {'name': '436', 'value': 1}, {'name': '437', 'value': 1}, {'name': '438', 'value': 1}, {'name': '439', 'value': 1}, {'name': '440', 'value': 1}, {'name': '441', 'value': 1}, {'name': '442', 'value': 1}, {'name': '443', 'value': 1}, {'name': '444', 'value': 1}, {'name': '445', 'value': 1}, {'name': '446', 'value': 1}, {'name': '447', 'value': 1}, {'name': '448', 'value': 1}, {'name': '449', 'value': 1}, {'name': '450', 'value': 1}, {'name': '451', 'value': 1}, {'name': '452', 'value': 1}, {'name': '453', 'value': 1}, {'name': '454', 'value': 1}, {'name': '455', 'value': 1}, {'name': '456', 'value': 1}, {'name': '457', 'value': 1}, {'name': '458', 'value': 1}, {'name': '459', 'value': 1}, {'name': '460', 'value': 1}, {'name': '461', 'value': 1}, {'name': '462', 'value': 1}, {'name': '463', 'value': 1}, {'name': '464', 'value': 1}, {'name': '465', 'value': 1}, {'name': '466', 'value': 1}, {'name': '467', 'value': 1}, {'name': '468', 'value': 1}, {'name': '469', 'value': 1}, {'name': '470', 'value': 1}, {'name': '471', 'value': 1}, {'name': '472', 'value': 1}, {'name': '473', 'value': 1}, {'name': '474', 'value': 1}, {'name': '475', 'value': 1}, {'name': '476', 'value': 1}, {'name': '477', 'value': 1}, {'name': '478', 'value': 1}, {'name': '479', 'value': 1}, {'name': '480', 'value': 1}, {'name': '481', 'value': 1}, {'name': '482', 'value': 1}, {'name': '483', 'value': 1}, {'name': '484', 'value': 1}, {'name': '485', 'value': 1}, {'name': '486', 'value': 1}, {'name': '487', 'value': 1}, {'name': '488', 'value': 1}, {'name': '489', 'value': 1}, {'name': '490', 'value': 1}, {'name': '491', 'value': 1}, {'name': '492', 'value': 1}, {'name': '493', 'value': 1}, {'name': '494', 'value': 1}, {'name': '495', 'value': 1}, {'name': '496', 'value': 1}, {'name': '497', 'value': 1}, {'name': '498', 'value': 1}, {'name': '499', 'value': 1}, {'name': '500', 'value': 1}]}, 'Isauthorized': {'name': 'Isauthorized', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'PredeterminationIndicator': {'name': 'PredeterminationIndicator', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'MedNecessityRequiredIndicator': {'name': 'MedNecessityRequiredIndicator', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'Age': {'name': 'Age', 'series': [{'name': '1', 'value': 89}, {'name': '2', 'value': 64}, {'name': '3', 'value': 39}, {'name': '4', 'value': 74}, {'name': '5', 'value': 73}, {'name': '6', 'value': 73}, {'name': '7', 'value': 53}, {'name': '8', 'value': 57}, {'name': '9', 'value': 80}, {'name': '10', 'value': 68}, {'name': '11', 'value': 77}, {'name': '12', 'value': 50}, {'name': '13', 'value': 64}, {'name': '14', 'value': 48}, {'name': '15', 'value': 74}, {'name': '16', 'value': 88}, {'name': '17', 'value': 48}, {'name': '18', 'value': 63}, {'name': '19', 'value': 81}, {'name': '20', 'value': 54}, {'name': '21', 'value': 58}, {'name': '22', 'value': 48}, {'name': '23', 'value': 63}, {'name': '24', 'value': 62}, {'name': '25', 'value': 53}, {'name': '26', 'value': 59}, {'name': '27', 'value': 58}, {'name': '28', 'value': 61}, {'name': '29', 'value': 64}, {'name': '30', 'value': 69}, {'name': '31', 'value': 77}, {'name': '32', 'value': 67}, {'name': '33', 'value': 69}, {'name': '34', 'value': 71}, {'name': '35', 'value': 31}, {'name': '36', 'value': 74}, {'name': '37', 'value': 89}, {'name': '38', 'value': 56}, {'name': '39', 'value': 78}, {'name': '40', 'value': 79}, {'name': '41', 'value': 50}, {'name': '42', 'value': 43}, {'name': '43', 'value': 72}, {'name': '44', 'value': 70}, {'name': '45', 'value': 72}, {'name': '46', 'value': 83}, {'name': '47', 'value': 67}, {'name': '48', 'value': 92}, {'name': '49', 'value': 75}, {'name': '50', 'value': 39}, {'name': '51', 'value': 78}, {'name': '52', 'value': 56}, {'name': '53', 'value': 76}, {'name': '54', 'value': 83}, {'name': '55', 'value': 44}, {'name': '56', 'value': 90}, {'name': '57', 'value': 81}, {'name': '58', 'value': 31}, {'name': '59', 'value': 88}, {'name': '60', 'value': 76}, {'name': '61', 'value': 32}, {'name': '62', 'value': 96}, {'name': '63', 'value': 83}, {'name': '64', 'value': 64}, {'name': '65', 'value': 67}, {'name': '66', 'value': 64}, {'name': '67', 'value': 67}, {'name': '68', 'value': 83}, {'name': '69', 'value': 73}, {'name': '70', 'value': 74}, {'name': '71', 'value': 48}, {'name': '72', 'value': 89}, {'name': '73', 'value': 62}, {'name': '74', 'value': 77}, {'name': '75', 'value': 87}, {'name': '76', 'value': 74}, {'name': '77', 'value': 40}, {'name': '78', 'value': 59}, {'name': '79', 'value': 69}, {'name': '80', 'value': 67}, {'name': '81', 'value': 84}, {'name': '82', 'value': 83}, {'name': '83', 'value': 42}, {'name': '84', 'value': 87}, {'name': '85', 'value': 26}, {'name': '86', 'value': 62}, {'name': '87', 'value': 91}, {'name': '88', 'value': 72}, {'name': '89', 'value': 81}, {'name': '90', 'value': 75}, {'name': '91', 'value': 95}, {'name': '92', 'value': 74}, {'name': '93', 'value': 91}, {'name': '94', 'value': 59}, {'name': '95', 'value': 64}, {'name': '96', 'value': 63}, {'name': '97', 'value': 72}, {'name': '98', 'value': 54}, {'name': '99', 'value': 82}, {'name': '100', 'value': 74}, {'name': '101', 'value': 89}, {'name': '102', 'value': 48}, {'name': '103', 'value': 69}, {'name': '104', 'value': 91}, {'name': '105', 'value': 87}, {'name': '106', 'value': 80}, {'name': '107', 'value': 71}, {'name': '108', 'value': 41}, {'name': '109', 'value': 68}, {'name': '110', 'value': 66}, {'name': '111', 'value': 14}, {'name': '112', 'value': 69}, {'name': '113', 'value': 71}, {'name': '114', 'value': 97}, {'name': '115', 'value': 46}, {'name': '116', 'value': 84}, {'name': '117', 'value': 64}, {'name': '118', 'value': 64}, {'name': '119', 'value': 36}, {'name': '120', 'value': 54}, {'name': '121', 'value': 84}, {'name': '122', 'value': 34}, {'name': '123', 'value': 41}, {'name': '124', 'value': 88}, {'name': '125', 'value': 39}, {'name': '126', 'value': 82}, {'name': '127', 'value': 53}, {'name': '128', 'value': 77}, {'name': '129', 'value': 68}, {'name': '130', 'value': 32}, {'name': '131', 'value': 71}, {'name': '132', 'value': 61}, {'name': '133', 'value': 86}, {'name': '134', 'value': 70}, {'name': '135', 'value': 67}, {'name': '136', 'value': 77}, {'name': '137', 'value': 73}, {'name': '138', 'value': 89}, {'name': '139', 'value': 60}, {'name': '140', 'value': 53}, {'name': '141', 'value': 74}, {'name': '142', 'value': 102}, {'name': '143', 'value': 78}, {'name': '144', 'value': 92}, {'name': '145', 'value': 46}, {'name': '146', 'value': 61}, {'name': '147', 'value': 63}, {'name': '148', 'value': 72}, {'name': '149', 'value': 74}, {'name': '150', 'value': 62}, {'name': '151', 'value': 62}, {'name': '152', 'value': 78}, {'name': '153', 'value': 69}, {'name': '154', 'value': 76}, {'name': '155', 'value': 74}, {'name': '156', 'value': 71}, {'name': '157', 'value': 70}, {'name': '158', 'value': 90}, {'name': '159', 'value': 68}, {'name': '160', 'value': 71}, {'name': '161', 'value': 89}, {'name': '162', 'value': 76}, {'name': '163', 'value': 65}, {'name': '164', 'value': 43}, {'name': '165', 'value': 68}, {'name': '166', 'value': 70}, {'name': '167', 'value': 77}, {'name': '168', 'value': 70}, {'name': '169', 'value': 51}, {'name': '170', 'value': 74}, {'name': '171', 'value': 80}, {'name': '172', 'value': 73}, {'name': '173', 'value': 66}, {'name': '174', 'value': 7}, {'name': '175', 'value': 69}, {'name': '176', 'value': 80}, {'name': '177', 'value': 78}, {'name': '178', 'value': 63}, {'name': '179', 'value': 64}, {'name': '180', 'value': 63}, {'name': '181', 'value': 63}, {'name': '182', 'value': 81}, {'name': '183', 'value': 94}, {'name': '184', 'value': 67}, {'name': '185', 'value': 59}, {'name': '186', 'value': 57}, {'name': '187', 'value': 83}, {'name': '188', 'value': 83}, {'name': '189', 'value': 80}, {'name': '190', 'value': 72}, {'name': '191', 'value': 84}, {'name': '192', 'value': 65}, {'name': '193', 'value': 61}, {'name': '194', 'value': 84}, {'name': '195', 'value': 46}, {'name': '196', 'value': 78}, {'name': '197', 'value': 66}, {'name': '198', 'value': 91}, {'name': '199', 'value': 23}, {'name': '200', 'value': 46}, {'name': '201', 'value': 75}, {'name': '202', 'value': 73}, {'name': '203', 'value': 69}, {'name': '204', 'value': 61}, {'name': '205', 'value': 79}, {'name': '206', 'value': 72}, {'name': '207', 'value': 66}, {'name': '208', 'value': 84}, {'name': '209', 'value': 67}, {'name': '210', 'value': 66}, {'name': '211', 'value': 48}, {'name': '212', 'value': 72}, {'name': '213', 'value': 70}, {'name': '214', 'value': 88}, {'name': '215', 'value': 68}, {'name': '216', 'value': 52}, {'name': '217', 'value': 97}, {'name': '218', 'value': 74}, {'name': '219', 'value': 91}, {'name': '220', 'value': 69}, {'name': '221', 'value': 78}, {'name': '222', 'value': 50}, {'name': '223', 'value': 68}, {'name': '224', 'value': 71}, {'name': '225', 'value': 41}, {'name': '226', 'value': 97}, {'name': '227', 'value': 67}, {'name': '228', 'value': 69}, {'name': '229', 'value': 72}, {'name': '230', 'value': 73}, {'name': '231', 'value': 72}, {'name': '232', 'value': 65}, {'name': '233', 'value': 53}, {'name': '234', 'value': 64}, {'name': '235', 'value': 76}, {'name': '236', 'value': 75}, {'name': '237', 'value': 53}, {'name': '238', 'value': 79}, {'name': '239', 'value': 47}, {'name': '240', 'value': 76}, {'name': '241', 'value': 52}, {'name': '242', 'value': 74}, {'name': '243', 'value': 73}, {'name': '244', 'value': 75}, {'name': '245', 'value': 48}, {'name': '246', 'value': 68}, {'name': '247', 'value': 71}, {'name': '248', 'value': 79}, {'name': '249', 'value': 63}, {'name': '250', 'value': 47}, {'name': '251', 'value': 61}, {'name': '252', 'value': 70}, {'name': '253', 'value': 69}, {'name': '254', 'value': 73}, {'name': '255', 'value': 66}, {'name': '256', 'value': 44}, {'name': '257', 'value': 89}, {'name': '258', 'value': 47}, {'name': '259', 'value': 90}, {'name': '260', 'value': 55}, {'name': '261', 'value': 69}, {'name': '262', 'value': 74}, {'name': '263', 'value': 91}, {'name': '264', 'value': 47}, {'name': '265', 'value': 48}, {'name': '266', 'value': 83}, {'name': '267', 'value': 62}, {'name': '268', 'value': 81}, {'name': '269', 'value': 86}, {'name': '270', 'value': 38}, {'name': '271', 'value': 64}, {'name': '272', 'value': 77}, {'name': '273', 'value': 78}, {'name': '274', 'value': 72}, {'name': '275', 'value': 79}, {'name': '276', 'value': 91}, {'name': '277', 'value': 77}, {'name': '278', 'value': 57}, {'name': '279', 'value': 74}, {'name': '280', 'value': 75}, {'name': '281', 'value': 86}, {'name': '282', 'value': 93}, {'name': '283', 'value': 62}, {'name': '284', 'value': 56}, {'name': '285', 'value': 66}, {'name': '286', 'value': 67}, {'name': '287', 'value': 89}, {'name': '288', 'value': 88}, {'name': '289', 'value': 81}, {'name': '290', 'value': 82}, {'name': '291', 'value': 66}, {'name': '292', 'value': 65}, {'name': '293', 'value': 78}, {'name': '294', 'value': 64}, {'name': '295', 'value': 72}, {'name': '296', 'value': 65}, {'name': '297', 'value': 65}, {'name': '298', 'value': 74}, {'name': '299', 'value': 47}, {'name': '300', 'value': 89}, {'name': '301', 'value': 71}, {'name': '302', 'value': 59}, {'name': '303', 'value': 86}, {'name': '304', 'value': 42}, {'name': '305', 'value': 69}, {'name': '306', 'value': 75}, {'name': '307', 'value': 82}, {'name': '308', 'value': 41}, {'name': '309', 'value': 88}, {'name': '310', 'value': 76}, {'name': '311', 'value': 70}, {'name': '312', 'value': 83}, {'name': '313', 'value': 58}, {'name': '314', 'value': 64}, {'name': '315', 'value': 81}, {'name': '316', 'value': 96}, {'name': '317', 'value': 56}, {'name': '318', 'value': 65}, {'name': '319', 'value': 78}, {'name': '320', 'value': 91}, {'name': '321', 'value': 74}, {'name': '322', 'value': 50}, {'name': '323', 'value': 85}, {'name': '324', 'value': 62}, {'name': '325', 'value': 83}, {'name': '326', 'value': 89}, {'name': '327', 'value': 81}, {'name': '328', 'value': 87}, {'name': '329', 'value': 83}, {'name': '330', 'value': 69}, {'name': '331', 'value': 34}, {'name': '332', 'value': 15}, {'name': '333', 'value': 74}, {'name': '334', 'value': 38}, {'name': '335', 'value': 68}, {'name': '336', 'value': 63}, {'name': '337', 'value': 71}, {'name': '338', 'value': 58}, {'name': '339', 'value': 81}, {'name': '340', 'value': 60}, {'name': '341', 'value': 72}, {'name': '342', 'value': 68}, {'name': '343', 'value': 71}, {'name': '344', 'value': 91}, {'name': '345', 'value': 79}, {'name': '346', 'value': 65}, {'name': '347', 'value': 63}, {'name': '348', 'value': 56}, {'name': '349', 'value': 65}, {'name': '350', 'value': 46}, {'name': '351', 'value': 93}, {'name': '352', 'value': 62}, {'name': '353', 'value': 51}, {'name': '354', 'value': 78}, {'name': '355', 'value': 60}, {'name': '356', 'value': 55}, {'name': '357', 'value': 73}, {'name': '358', 'value': 57}, {'name': '359', 'value': 77}, {'name': '360', 'value': 87}, {'name': '361', 'value': 46}, {'name': '362', 'value': 69}, {'name': '363', 'value': 90}, {'name': '364', 'value': 32}, {'name': '365', 'value': 55}, {'name': '366', 'value': 54}, {'name': '367', 'value': 67}, {'name': '368', 'value': 5}, {'name': '369', 'value': 71}, {'name': '370', 'value': 66}, {'name': '371', 'value': 47}, {'name': '372', 'value': 61}, {'name': '373', 'value': 95}, {'name': '374', 'value': 73}, {'name': '375', 'value': 68}, {'name': '376', 'value': 76}, {'name': '377', 'value': 61}, {'name': '378', 'value': 59}, {'name': '379', 'value': 83}, {'name': '380', 'value': 78}, {'name': '381', 'value': 25}, {'name': '382', 'value': 70}, {'name': '383', 'value': 78}, {'name': '384', 'value': 65}, {'name': '385', 'value': 43}, {'name': '386', 'value': 71}, {'name': '387', 'value': 65}, {'name': '388', 'value': 85}, {'name': '389', 'value': 63}, {'name': '390', 'value': 63}, {'name': '391', 'value': 49}, {'name': '392', 'value': 77}, {'name': '393', 'value': 76}, {'name': '394', 'value': 65}, {'name': '395', 'value': 85}, {'name': '396', 'value': 81}, {'name': '397', 'value': 84}, {'name': '398', 'value': 79}, {'name': '399', 'value': 78}, {'name': '400', 'value': 76}, {'name': '401', 'value': 90}, {'name': '402', 'value': 56}, {'name': '403', 'value': 47}, {'name': '404', 'value': 33}, {'name': '405', 'value': 18}, {'name': '406', 'value': 72}, {'name': '407', 'value': 44}, {'name': '408', 'value': 69}, {'name': '409', 'value': 50}, {'name': '410', 'value': 82}, {'name': '411', 'value': 66}, {'name': '412', 'value': 97}, {'name': '413', 'value': 71}, {'name': '414', 'value': 62}, {'name': '415', 'value': 88}, {'name': '416', 'value': 78}, {'name': '417', 'value': 72}, {'name': '418', 'value': 54}, {'name': '419', 'value': 79}, {'name': '420', 'value': 76}, {'name': '421', 'value': 97}, {'name': '422', 'value': 65}, {'name': '423', 'value': 84}, {'name': '424', 'value': 70}, {'name': '425', 'value': 65}, {'name': '426', 'value': 67}, {'name': '427', 'value': 78}, {'name': '428', 'value': 68}, {'name': '429', 'value': 54}, {'name': '430', 'value': 65}, {'name': '431', 'value': 75}, {'name': '432', 'value': 53}, {'name': '433', 'value': 85}, {'name': '434', 'value': 40}, {'name': '435', 'value': 81}, {'name': '436', 'value': 61}, {'name': '437', 'value': 24}, {'name': '438', 'value': 76}, {'name': '439', 'value': 69}, {'name': '440', 'value': 43}, {'name': '441', 'value': 92}, {'name': '442', 'value': 59}, {'name': '443', 'value': 68}, {'name': '444', 'value': 59}, {'name': '445', 'value': 54}, {'name': '446', 'value': 65}, {'name': '447', 'value': 70}, {'name': '448', 'value': 54}, {'name': '449', 'value': 85}, {'name': '450', 'value': 23}, {'name': '451', 'value': 67}, {'name': '452', 'value': 7}, {'name': '453', 'value': 78}, {'name': '454', 'value': 63}, {'name': '455', 'value': 37}, {'name': '456', 'value': 48}, {'name': '457', 'value': 61}, {'name': '458', 'value': 73}, {'name': '459', 'value': 52}, {'name': '460', 'value': 82}, {'name': '461', 'value': 79}, {'name': '462', 'value': 69}, {'name': '463', 'value': 64}, {'name': '464', 'value': 58}, {'name': '465', 'value': 76}, {'name': '466', 'value': 72}, {'name': '467', 'value': 101}, {'name': '468', 'value': 57}, {'name': '469', 'value': 67}, {'name': '470', 'value': 68}, {'name': '471', 'value': 79}, {'name': '472', 'value': 90}, {'name': '473', 'value': 20}, {'name': '474', 'value': 72}, {'name': '475', 'value': 90}, {'name': '476', 'value': 63}, {'name': '477', 'value': 28}, {'name': '478', 'value': 72}, {'name': '479', 'value': 71}, {'name': '480', 'value': 61}, {'name': '481', 'value': 73}, {'name': '482', 'value': 79}, {'name': '483', 'value': 81}, {'name': '484', 'value': 74}, {'name': '485', 'value': 76}, {'name': '486', 'value': 88}, {'name': '487', 'value': 74}, {'name': '488', 'value': 76}, {'name': '489', 'value': 56}, {'name': '490', 'value': 67}, {'name': '491', 'value': 76}, {'name': '492', 'value': 78}, {'name': '493', 'value': 91}, {'name': '494', 'value': 69}, {'name': '495', 'value': 66}, {'name': '496', 'value': 36}, {'name': '497', 'value': 79}, {'name': '498', 'value': 67}, {'name': '499', 'value': 62}, {'name': '500', 'value': 46}]}, 'Gender': {'name': 'Gender', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'DepartmentCategory': {'name': 'DepartmentCategory', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'SpecialityCategory': {'name': 'SpecialityCategory', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'label': {'name': 'label', 'series': [{'name': '1', 'value': 1}, {'name': '2', 'value': 1}, {'name': '3', 'value': 1}, {'name': '4', 'value': 1}, {'name': '5', 'value': 0}, {'name': '6', 'value': 1}, {'name': '7', 'value': 1}, {'name': '8', 'value': 1}, {'name': '9', 'value': 1}, {'name': '10', 'value': 1}, {'name': '11', 'value': 1}, {'name': '12', 'value': 1}, {'name': '13', 'value': 1}, {'name': '14', 'value': 1}, {'name': '15', 'value': 1}, {'name': '16', 'value': 1}, {'name': '17', 'value': 1}, {'name': '18', 'value': 1}, {'name': '19', 'value': 1}, {'name': '20', 'value': 1}, {'name': '21', 'value': 1}, {'name': '22', 'value': 1}, {'name': '23', 'value': 1}, {'name': '24', 'value': 1}, {'name': '25', 'value': 1}, {'name': '26', 'value': 1}, {'name': '27', 'value': 1}, {'name': '28', 'value': 1}, {'name': '29', 'value': 0}, {'name': '30', 'value': 1}, {'name': '31', 'value': 1}, {'name': '32', 'value': 1}, {'name': '33', 'value': 1}, {'name': '34', 'value': 0}, {'name': '35', 'value': 1}, {'name': '36', 'value': 1}, {'name': '37', 'value': 1}, {'name': '38', 'value': 1}, {'name': '39', 'value': 1}, {'name': '40', 'value': 1}, {'name': '41', 'value': 1}, {'name': '42', 'value': 1}, {'name': '43', 'value': 1}, {'name': '44', 'value': 1}, {'name': '45', 'value': 1}, {'name': '46', 'value': 1}, {'name': '47', 'value': 1}, {'name': '48', 'value': 1}, {'name': '49', 'value': 1}, {'name': '50', 'value': 1}, {'name': '51', 'value': 1}, {'name': '52', 'value': 1}, {'name': '53', 'value': 1}, {'name': '54', 'value': 1}, {'name': '55', 'value': 1}, {'name': '56', 'value': 1}, {'name': '57', 'value': 1}, {'name': '58', 'value': 1}, {'name': '59', 'value': 1}, {'name': '60', 'value': 1}, {'name': '61', 'value': 1}, {'name': '62', 'value': 1}, {'name': '63', 'value': 1}, {'name': '64', 'value': 1}, {'name': '65', 'value': 1}, {'name': '66', 'value': 1}, {'name': '67', 'value': 1}, {'name': '68', 'value': 1}, {'name': '69', 'value': 1}, {'name': '70', 'value': 1}, {'name': '71', 'value': 1}, {'name': '72', 'value': 1}, {'name': '73', 'value': 1}, {'name': '74', 'value': 1}, {'name': '75', 'value': 1}, {'name': '76', 'value': 1}, {'name': '77', 'value': 1}, {'name': '78', 'value': 1}, {'name': '79', 'value': 1}, {'name': '80', 'value': 1}, {'name': '81', 'value': 1}, {'name': '82', 'value': 1}, {'name': '83', 'value': 1}, {'name': '84', 'value': 1}, {'name': '85', 'value': 1}, {'name': '86', 'value': 0}, {'name': '87', 'value': 1}, {'name': '88', 'value': 1}, {'name': '89', 'value': 1}, {'name': '90', 'value': 1}, {'name': '91', 'value': 1}, {'name': '92', 'value': 1}, {'name': '93', 'value': 1}, {'name': '94', 'value': 0}, {'name': '95', 'value': 1}, {'name': '96', 'value': 1}, {'name': '97', 'value': 1}, {'name': '98', 'value': 0}, {'name': '99', 'value': 1}, {'name': '100', 'value': 1}, {'name': '101', 'value': 1}, {'name': '102', 'value': 1}, {'name': '103', 'value': 1}, {'name': '104', 'value': 1}, {'name': '105', 'value': 1}, {'name': '106', 'value': 1}, {'name': '107', 'value': 1}, {'name': '108', 'value': 1}, {'name': '109', 'value': 1}, {'name': '110', 'value': 1}, {'name': '111', 'value': 1}, {'name': '112', 'value': 1}, {'name': '113', 'value': 1}, {'name': '114', 'value': 1}, {'name': '115', 'value': 1}, {'name': '116', 'value': 1}, {'name': '117', 'value': 1}, {'name': '118', 'value': 1}, {'name': '119', 'value': 1}, {'name': '120', 'value': 1}, {'name': '121', 'value': 1}, {'name': '122', 'value': 1}, {'name': '123', 'value': 1}, {'name': '124', 'value': 1}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 1}, {'name': '128', 'value': 1}, {'name': '129', 'value': 1}, {'name': '130', 'value': 1}, {'name': '131', 'value': 1}, {'name': '132', 'value': 1}, {'name': '133', 'value': 1}, {'name': '134', 'value': 1}, {'name': '135', 'value': 1}, {'name': '136', 'value': 1}, {'name': '137', 'value': 1}, {'name': '138', 'value': 1}, {'name': '139', 'value': 1}, {'name': '140', 'value': 1}, {'name': '141', 'value': 1}, {'name': '142', 'value': 1}, {'name': '143', 'value': 1}, {'name': '144', 'value': 1}, {'name': '145', 'value': 0}, {'name': '146', 'value': 1}, {'name': '147', 'value': 1}, {'name': '148', 'value': 1}, {'name': '149', 'value': 0}, {'name': '150', 'value': 1}, {'name': '151', 'value': 1}, {'name': '152', 'value': 1}, {'name': '153', 'value': 1}, {'name': '154', 'value': 1}, {'name': '155', 'value': 1}, {'name': '156', 'value': 1}, {'name': '157', 'value': 1}, {'name': '158', 'value': 1}, {'name': '159', 'value': 1}, {'name': '160', 'value': 1}, {'name': '161', 'value': 1}, {'name': '162', 'value': 1}, {'name': '163', 'value': 1}, {'name': '164', 'value': 1}, {'name': '165', 'value': 1}, {'name': '166', 'value': 1}, {'name': '167', 'value': 1}, {'name': '168', 'value': 1}, {'name': '169', 'value': 1}, {'name': '170', 'value': 1}, {'name': '171', 'value': 1}, {'name': '172', 'value': 1}, {'name': '173', 'value': 1}, {'name': '174', 'value': 0}, {'name': '175', 'value': 1}, {'name': '176', 'value': 0}, {'name': '177', 'value': 1}, {'name': '178', 'value': 1}, {'name': '179', 'value': 1}, {'name': '180', 'value': 1}, {'name': '181', 'value': 1}, {'name': '182', 'value': 1}, {'name': '183', 'value': 1}, {'name': '184', 'value': 0}, {'name': '185', 'value': 1}, {'name': '186', 'value': 1}, {'name': '187', 'value': 1}, {'name': '188', 'value': 1}, {'name': '189', 'value': 1}, {'name': '190', 'value': 1}, {'name': '191', 'value': 1}, {'name': '192', 'value': 1}, {'name': '193', 'value': 1}, {'name': '194', 'value': 1}, {'name': '195', 'value': 1}, {'name': '196', 'value': 1}, {'name': '197', 'value': 1}, {'name': '198', 'value': 1}, {'name': '199', 'value': 1}, {'name': '200', 'value': 1}, {'name': '201', 'value': 1}, {'name': '202', 'value': 1}, {'name': '203', 'value': 1}, {'name': '204', 'value': 1}, {'name': '205', 'value': 0}, {'name': '206', 'value': 1}, {'name': '207', 'value': 1}, {'name': '208', 'value': 1}, {'name': '209', 'value': 1}, {'name': '210', 'value': 1}, {'name': '211', 'value': 1}, {'name': '212', 'value': 1}, {'name': '213', 'value': 1}, {'name': '214', 'value': 1}, {'name': '215', 'value': 1}, {'name': '216', 'value': 1}, {'name': '217', 'value': 1}, {'name': '218', 'value': 0}, {'name': '219', 'value': 1}, {'name': '220', 'value': 1}, {'name': '221', 'value': 1}, {'name': '222', 'value': 1}, {'name': '223', 'value': 1}, {'name': '224', 'value': 1}, {'name': '225', 'value': 1}, {'name': '226', 'value': 1}, {'name': '227', 'value': 1}, {'name': '228', 'value': 0}, {'name': '229', 'value': 1}, {'name': '230', 'value': 1}, {'name': '231', 'value': 1}, {'name': '232', 'value': 1}, {'name': '233', 'value': 1}, {'name': '234', 'value': 1}, {'name': '235', 'value': 1}, {'name': '236', 'value': 1}, {'name': '237', 'value': 1}, {'name': '238', 'value': 1}, {'name': '239', 'value': 1}, {'name': '240', 'value': 1}, {'name': '241', 'value': 1}, {'name': '242', 'value': 1}, {'name': '243', 'value': 1}, {'name': '244', 'value': 1}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 1}, {'name': '248', 'value': 1}, {'name': '249', 'value': 1}, {'name': '250', 'value': 1}, {'name': '251', 'value': 1}, {'name': '252', 'value': 1}, {'name': '253', 'value': 1}, {'name': '254', 'value': 0}, {'name': '255', 'value': 1}, {'name': '256', 'value': 0}, {'name': '257', 'value': 1}, {'name': '258', 'value': 1}, {'name': '259', 'value': 1}, {'name': '260', 'value': 1}, {'name': '261', 'value': 1}, {'name': '262', 'value': 1}, {'name': '263', 'value': 1}, {'name': '264', 'value': 1}, {'name': '265', 'value': 1}, {'name': '266', 'value': 1}, {'name': '267', 'value': 1}, {'name': '268', 'value': 1}, {'name': '269', 'value': 1}, {'name': '270', 'value': 1}, {'name': '271', 'value': 1}, {'name': '272', 'value': 1}, {'name': '273', 'value': 1}, {'name': '274', 'value': 1}, {'name': '275', 'value': 0}, {'name': '276', 'value': 1}, {'name': '277', 'value': 1}, {'name': '278', 'value': 0}, {'name': '279', 'value': 1}, {'name': '280', 'value': 1}, {'name': '281', 'value': 1}, {'name': '282', 'value': 1}, {'name': '283', 'value': 0}, {'name': '284', 'value': 1}, {'name': '285', 'value': 1}, {'name': '286', 'value': 1}, {'name': '287', 'value': 1}, {'name': '288', 'value': 1}, {'name': '289', 'value': 0}, {'name': '290', 'value': 1}, {'name': '291', 'value': 1}, {'name': '292', 'value': 1}, {'name': '293', 'value': 1}, {'name': '294', 'value': 1}, {'name': '295', 'value': 1}, {'name': '296', 'value': 1}, {'name': '297', 'value': 1}, {'name': '298', 'value': 1}, {'name': '299', 'value': 1}, {'name': '300', 'value': 1}, {'name': '301', 'value': 1}, {'name': '302', 'value': 1}, {'name': '303', 'value': 1}, {'name': '304', 'value': 1}, {'name': '305', 'value': 1}, {'name': '306', 'value': 1}, {'name': '307', 'value': 1}, {'name': '308', 'value': 1}, {'name': '309', 'value': 1}, {'name': '310', 'value': 1}, {'name': '311', 'value': 1}, {'name': '312', 'value': 1}, {'name': '313', 'value': 1}, {'name': '314', 'value': 1}, {'name': '315', 'value': 0}, {'name': '316', 'value': 1}, {'name': '317', 'value': 1}, {'name': '318', 'value': 1}, {'name': '319', 'value': 1}, {'name': '320', 'value': 1}, {'name': '321', 'value': 1}, {'name': '322', 'value': 1}, {'name': '323', 'value': 1}, {'name': '324', 'value': 1}, {'name': '325', 'value': 1}, {'name': '326', 'value': 1}, {'name': '327', 'value': 1}, {'name': '328', 'value': 1}, {'name': '329', 'value': 1}, {'name': '330', 'value': 0}, {'name': '331', 'value': 1}, {'name': '332', 'value': 1}, {'name': '333', 'value': 1}, {'name': '334', 'value': 1}, {'name': '335', 'value': 1}, {'name': '336', 'value': 1}, {'name': '337', 'value': 1}, {'name': '338', 'value': 1}, {'name': '339', 'value': 1}, {'name': '340', 'value': 1}, {'name': '341', 'value': 1}, {'name': '342', 'value': 1}, {'name': '343', 'value': 1}, {'name': '344', 'value': 1}, {'name': '345', 'value': 1}, {'name': '346', 'value': 1}, {'name': '347', 'value': 1}, {'name': '348', 'value': 1}, {'name': '349', 'value': 0}, {'name': '350', 'value': 1}, {'name': '351', 'value': 0}, {'name': '352', 'value': 1}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 1}, {'name': '356', 'value': 1}, {'name': '357', 'value': 1}, {'name': '358', 'value': 1}, {'name': '359', 'value': 1}, {'name': '360', 'value': 0}, {'name': '361', 'value': 1}, {'name': '362', 'value': 1}, {'name': '363', 'value': 1}, {'name': '364', 'value': 1}, {'name': '365', 'value': 1}, {'name': '366', 'value': 1}, {'name': '367', 'value': 1}, {'name': '368', 'value': 1}, {'name': '369', 'value': 1}, {'name': '370', 'value': 1}, {'name': '371', 'value': 1}, {'name': '372', 'value': 1}, {'name': '373', 'value': 1}, {'name': '374', 'value': 1}, {'name': '375', 'value': 1}, {'name': '376', 'value': 0}, {'name': '377', 'value': 1}, {'name': '378', 'value': 1}, {'name': '379', 'value': 1}, {'name': '380', 'value': 1}, {'name': '381', 'value': 1}, {'name': '382', 'value': 1}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 1}, {'name': '386', 'value': 1}, {'name': '387', 'value': 1}, {'name': '388', 'value': 1}, {'name': '389', 'value': 1}, {'name': '390', 'value': 1}, {'name': '391', 'value': 1}, {'name': '392', 'value': 1}, {'name': '393', 'value': 1}, {'name': '394', 'value': 1}, {'name': '395', 'value': 0}, {'name': '396', 'value': 1}, {'name': '397', 'value': 1}, {'name': '398', 'value': 1}, {'name': '399', 'value': 1}, {'name': '400', 'value': 0}, {'name': '401', 'value': 1}, {'name': '402', 'value': 1}, {'name': '403', 'value': 1}, {'name': '404', 'value': 1}, {'name': '405', 'value': 1}, {'name': '406', 'value': 1}, {'name': '407', 'value': 1}, {'name': '408', 'value': 1}, {'name': '409', 'value': 1}, {'name': '410', 'value': 1}, {'name': '411', 'value': 1}, {'name': '412', 'value': 1}, {'name': '413', 'value': 1}, {'name': '414', 'value': 1}, {'name': '415', 'value': 0}, {'name': '416', 'value': 1}, {'name': '417', 'value': 1}, {'name': '418', 'value': 1}, {'name': '419', 'value': 1}, {'name': '420', 'value': 1}, {'name': '421', 'value': 1}, {'name': '422', 'value': 1}, {'name': '423', 'value': 1}, {'name': '424', 'value': 1}, {'name': '425', 'value': 0}, {'name': '426', 'value': 1}, {'name': '427', 'value': 1}, {'name': '428', 'value': 1}, {'name': '429', 'value': 1}, {'name': '430', 'value': 1}, {'name': '431', 'value': 1}, {'name': '432', 'value': 1}, {'name': '433', 'value': 1}, {'name': '434', 'value': 1}, {'name': '435', 'value': 0}, {'name': '436', 'value': 1}, {'name': '437', 'value': 1}, {'name': '438', 'value': 1}, {'name': '439', 'value': 1}, {'name': '440', 'value': 1}, {'name': '441', 'value': 1}, {'name': '442', 'value': 1}, {'name': '443', 'value': 1}, {'name': '444', 'value': 1}, {'name': '445', 'value': 1}, {'name': '446', 'value': 1}, {'name': '447', 'value': 1}, {'name': '448', 'value': 1}, {'name': '449', 'value': 0}, {'name': '450', 'value': 1}, {'name': '451', 'value': 1}, {'name': '452', 'value': 1}, {'name': '453', 'value': 0}, {'name': '454', 'value': 1}, {'name': '455', 'value': 0}, {'name': '456', 'value': 1}, {'name': '457', 'value': 1}, {'name': '458', 'value': 1}, {'name': '459', 'value': 1}, {'name': '460', 'value': 1}, {'name': '461', 'value': 1}, {'name': '462', 'value': 1}, {'name': '463', 'value': 1}, {'name': '464', 'value': 1}, {'name': '465', 'value': 1}, {'name': '466', 'value': 1}, {'name': '467', 'value': 1}, {'name': '468', 'value': 1}, {'name': '469', 'value': 1}, {'name': '470', 'value': 1}, {'name': '471', 'value': 1}, {'name': '472', 'value': 1}, {'name': '473', 'value': 1}, {'name': '474', 'value': 1}, {'name': '475', 'value': 1}, {'name': '476', 'value': 1}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 1}, {'name': '480', 'value': 1}, {'name': '481', 'value': 1}, {'name': '482', 'value': 1}, {'name': '483', 'value': 1}, {'name': '484', 'value': 1}, {'name': '485', 'value': 1}, {'name': '486', 'value': 1}, {'name': '487', 'value': 1}, {'name': '488', 'value': 1}, {'name': '489', 'value': 1}, {'name': '490', 'value': 1}, {'name': '491', 'value': 1}, {'name': '492', 'value': 1}, {'name': '493', 'value': 1}, {'name': '494', 'value': 1}, {'name': '495', 'value': 1}, {'name': '496', 'value': 1}, {'name': '497', 'value': 1}, {'name': '498', 'value': 1}, {'name': '499', 'value': 0}, {'name': '500', 'value': 1}]}, 'PayerId': {'name': 'PayerId', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'Gender_Recoded': {'name': 'Gender_Recoded', 'series': [{'name': '1', 'value': 2}, {'name': '2', 'value': 2}, {'name': '3', 'value': 2}, {'name': '4', 'value': 1}, {'name': '5', 'value': 1}, {'name': '6', 'value': 1}, {'name': '7', 'value': 2}, {'name': '8', 'value': 2}, {'name': '9', 'value': 1}, {'name': '10', 'value': 1}, {'name': '11', 'value': 2}, {'name': '12', 'value': 2}, {'name': '13', 'value': 2}, {'name': '14', 'value': 1}, {'name': '15', 'value': 2}, {'name': '16', 'value': 1}, {'name': '17', 'value': 1}, {'name': '18', 'value': 2}, {'name': '19', 'value': 1}, {'name': '20', 'value': 2}, {'name': '21', 'value': 2}, {'name': '22', 'value': 2}, {'name': '23', 'value': 2}, {'name': '24', 'value': 2}, {'name': '25', 'value': 2}, {'name': '26', 'value': 1}, {'name': '27', 'value': 2}, {'name': '28', 'value': 1}, {'name': '29', 'value': 2}, {'name': '30', 'value': 1}, {'name': '31', 'value': 1}, {'name': '32', 'value': 2}, {'name': '33', 'value': 2}, {'name': '34', 'value': 1}, {'name': '35', 'value': 1}, {'name': '36', 'value': 1}, {'name': '37', 'value': 2}, {'name': '38', 'value': 2}, {'name': '39', 'value': 2}, {'name': '40', 'value': 1}, {'name': '41', 'value': 1}, {'name': '42', 'value': 1}, {'name': '43', 'value': 1}, {'name': '44', 'value': 1}, {'name': '45', 'value': 1}, {'name': '46', 'value': 1}, {'name': '47', 'value': 1}, {'name': '48', 'value': 2}, {'name': '49', 'value': 1}, {'name': '50', 'value': 1}, {'name': '51', 'value': 2}, {'name': '52', 'value': 2}, {'name': '53', 'value': 2}, {'name': '54', 'value': 1}, {'name': '55', 'value': 2}, {'name': '56', 'value': 1}, {'name': '57', 'value': 1}, {'name': '58', 'value': 2}, {'name': '59', 'value': 1}, {'name': '60', 'value': 1}, {'name': '61', 'value': 2}, {'name': '62', 'value': 1}, {'name': '63', 'value': 1}, {'name': '64', 'value': 2}, {'name': '65', 'value': 2}, {'name': '66', 'value': 2}, {'name': '67', 'value': 1}, {'name': '68', 'value': 1}, {'name': '69', 'value': 2}, {'name': '70', 'value': 2}, {'name': '71', 'value': 1}, {'name': '72', 'value': 2}, {'name': '73', 'value': 1}, {'name': '74', 'value': 2}, {'name': '75', 'value': 2}, {'name': '76', 'value': 1}, {'name': '77', 'value': 1}, {'name': '78', 'value': 2}, {'name': '79', 'value': 1}, {'name': '80', 'value': 1}, {'name': '81', 'value': 1}, {'name': '82', 'value': 1}, {'name': '83', 'value': 1}, {'name': '84', 'value': 1}, {'name': '85', 'value': 1}, {'name': '86', 'value': 1}, {'name': '87', 'value': 2}, {'name': '88', 'value': 2}, {'name': '89', 'value': 1}, {'name': '90', 'value': 1}, {'name': '91', 'value': 1}, {'name': '92', 'value': 1}, {'name': '93', 'value': 2}, {'name': '94', 'value': 1}, {'name': '95', 'value': 1}, {'name': '96', 'value': 2}, {'name': '97', 'value': 1}, {'name': '98', 'value': 2}, {'name': '99', 'value': 1}, {'name': '100', 'value': 2}, {'name': '101', 'value': 1}, {'name': '102', 'value': 2}, {'name': '103', 'value': 2}, {'name': '104', 'value': 1}, {'name': '105', 'value': 2}, {'name': '106', 'value': 2}, {'name': '107', 'value': 2}, {'name': '108', 'value': 2}, {'name': '109', 'value': 2}, {'name': '110', 'value': 2}, {'name': '111', 'value': 1}, {'name': '112', 'value': 2}, {'name': '113', 'value': 2}, {'name': '114', 'value': 1}, {'name': '115', 'value': 1}, {'name': '116', 'value': 1}, {'name': '117', 'value': 1}, {'name': '118', 'value': 1}, {'name': '119', 'value': 1}, {'name': '120', 'value': 1}, {'name': '121', 'value': 1}, {'name': '122', 'value': 1}, {'name': '123', 'value': 1}, {'name': '124', 'value': 1}, {'name': '125', 'value': 1}, {'name': '126', 'value': 2}, {'name': '127', 'value': 2}, {'name': '128', 'value': 1}, {'name': '129', 'value': 1}, {'name': '130', 'value': 1}, {'name': '131', 'value': 1}, {'name': '132', 'value': 2}, {'name': '133', 'value': 1}, {'name': '134', 'value': 2}, {'name': '135', 'value': 1}, {'name': '136', 'value': 1}, {'name': '137', 'value': 2}, {'name': '138', 'value': 1}, {'name': '139', 'value': 1}, {'name': '140', 'value': 2}, {'name': '141', 'value': 1}, {'name': '142', 'value': 1}, {'name': '143', 'value': 2}, {'name': '144', 'value': 2}, {'name': '145', 'value': 1}, {'name': '146', 'value': 1}, {'name': '147', 'value': 2}, {'name': '148', 'value': 2}, {'name': '149', 'value': 1}, {'name': '150', 'value': 1}, {'name': '151', 'value': 1}, {'name': '152', 'value': 2}, {'name': '153', 'value': 2}, {'name': '154', 'value': 2}, {'name': '155', 'value': 1}, {'name': '156', 'value': 1}, {'name': '157', 'value': 1}, {'name': '158', 'value': 1}, {'name': '159', 'value': 1}, {'name': '160', 'value': 1}, {'name': '161', 'value': 1}, {'name': '162', 'value': 2}, {'name': '163', 'value': 1}, {'name': '164', 'value': 1}, {'name': '165', 'value': 2}, {'name': '166', 'value': 1}, {'name': '167', 'value': 1}, {'name': '168', 'value': 1}, {'name': '169', 'value': 1}, {'name': '170', 'value': 1}, {'name': '171', 'value': 2}, {'name': '172', 'value': 1}, {'name': '173', 'value': 1}, {'name': '174', 'value': 2}, {'name': '175', 'value': 1}, {'name': '176', 'value': 2}, {'name': '177', 'value': 1}, {'name': '178', 'value': 2}, {'name': '179', 'value': 1}, {'name': '180', 'value': 1}, {'name': '181', 'value': 1}, {'name': '182', 'value': 1}, {'name': '183', 'value': 1}, {'name': '184', 'value': 1}, {'name': '185', 'value': 1}, {'name': '186', 'value': 1}, {'name': '187', 'value': 1}, {'name': '188', 'value': 2}, {'name': '189', 'value': 1}, {'name': '190', 'value': 1}, {'name': '191', 'value': 2}, {'name': '192', 'value': 1}, {'name': '193', 'value': 1}, {'name': '194', 'value': 2}, {'name': '195', 'value': 1}, {'name': '196', 'value': 1}, {'name': '197', 'value': 1}, {'name': '198', 'value': 2}, {'name': '199', 'value': 1}, {'name': '200', 'value': 1}, {'name': '201', 'value': 1}, {'name': '202', 'value': 1}, {'name': '203', 'value': 1}, {'name': '204', 'value': 2}, {'name': '205', 'value': 1}, {'name': '206', 'value': 2}, {'name': '207', 'value': 2}, {'name': '208', 'value': 1}, {'name': '209', 'value': 1}, {'name': '210', 'value': 1}, {'name': '211', 'value': 2}, {'name': '212', 'value': 1}, {'name': '213', 'value': 2}, {'name': '214', 'value': 2}, {'name': '215', 'value': 2}, {'name': '216', 'value': 2}, {'name': '217', 'value': 1}, {'name': '218', 'value': 2}, {'name': '219', 'value': 1}, {'name': '220', 'value': 1}, {'name': '221', 'value': 2}, {'name': '222', 'value': 1}, {'name': '223', 'value': 2}, {'name': '224', 'value': 1}, {'name': '225', 'value': 2}, {'name': '226', 'value': 2}, {'name': '227', 'value': 1}, {'name': '228', 'value': 2}, {'name': '229', 'value': 2}, {'name': '230', 'value': 1}, {'name': '231', 'value': 2}, {'name': '232', 'value': 1}, {'name': '233', 'value': 1}, {'name': '234', 'value': 2}, {'name': '235', 'value': 2}, {'name': '236', 'value': 1}, {'name': '237', 'value': 2}, {'name': '238', 'value': 2}, {'name': '239', 'value': 2}, {'name': '240', 'value': 1}, {'name': '241', 'value': 1}, {'name': '242', 'value': 1}, {'name': '243', 'value': 1}, {'name': '244', 'value': 1}, {'name': '245', 'value': 1}, {'name': '246', 'value': 1}, {'name': '247', 'value': 1}, {'name': '248', 'value': 1}, {'name': '249', 'value': 1}, {'name': '250', 'value': 1}, {'name': '251', 'value': 1}, {'name': '252', 'value': 1}, {'name': '253', 'value': 1}, {'name': '254', 'value': 1}, {'name': '255', 'value': 1}, {'name': '256', 'value': 2}, {'name': '257', 'value': 2}, {'name': '258', 'value': 2}, {'name': '259', 'value': 2}, {'name': '260', 'value': 2}, {'name': '261', 'value': 2}, {'name': '262', 'value': 2}, {'name': '263', 'value': 2}, {'name': '264', 'value': 2}, {'name': '265', 'value': 1}, {'name': '266', 'value': 1}, {'name': '267', 'value': 2}, {'name': '268', 'value': 1}, {'name': '269', 'value': 1}, {'name': '270', 'value': 2}, {'name': '271', 'value': 1}, {'name': '272', 'value': 2}, {'name': '273', 'value': 2}, {'name': '274', 'value': 1}, {'name': '275', 'value': 1}, {'name': '276', 'value': 1}, {'name': '277', 'value': 1}, {'name': '278', 'value': 1}, {'name': '279', 'value': 1}, {'name': '280', 'value': 2}, {'name': '281', 'value': 2}, {'name': '282', 'value': 1}, {'name': '283', 'value': 1}, {'name': '284', 'value': 2}, {'name': '285', 'value': 2}, {'name': '286', 'value': 2}, {'name': '287', 'value': 1}, {'name': '288', 'value': 2}, {'name': '289', 'value': 2}, {'name': '290', 'value': 1}, {'name': '291', 'value': 2}, {'name': '292', 'value': 2}, {'name': '293', 'value': 1}, {'name': '294', 'value': 2}, {'name': '295', 'value': 1}, {'name': '296', 'value': 1}, {'name': '297', 'value': 1}, {'name': '298', 'value': 1}, {'name': '299', 'value': 2}, {'name': '300', 'value': 2}, {'name': '301', 'value': 1}, {'name': '302', 'value': 2}, {'name': '303', 'value': 2}, {'name': '304', 'value': 2}, {'name': '305', 'value': 1}, {'name': '306', 'value': 1}, {'name': '307', 'value': 2}, {'name': '308', 'value': 2}, {'name': '309', 'value': 2}, {'name': '310', 'value': 2}, {'name': '311', 'value': 1}, {'name': '312', 'value': 1}, {'name': '313', 'value': 1}, {'name': '314', 'value': 2}, {'name': '315', 'value': 1}, {'name': '316', 'value': 1}, {'name': '317', 'value': 2}, {'name': '318', 'value': 1}, {'name': '319', 'value': 1}, {'name': '320', 'value': 1}, {'name': '321', 'value': 2}, {'name': '322', 'value': 2}, {'name': '323', 'value': 2}, {'name': '324', 'value': 1}, {'name': '325', 'value': 2}, {'name': '326', 'value': 1}, {'name': '327', 'value': 2}, {'name': '328', 'value': 1}, {'name': '329', 'value': 1}, {'name': '330', 'value': 2}, {'name': '331', 'value': 1}, {'name': '332', 'value': 2}, {'name': '333', 'value': 1}, {'name': '334', 'value': 1}, {'name': '335', 'value': 1}, {'name': '336', 'value': 1}, {'name': '337', 'value': 1}, {'name': '338', 'value': 2}, {'name': '339', 'value': 1}, {'name': '340', 'value': 1}, {'name': '341', 'value': 1}, {'name': '342', 'value': 2}, {'name': '343', 'value': 2}, {'name': '344', 'value': 2}, {'name': '345', 'value': 1}, {'name': '346', 'value': 2}, {'name': '347', 'value': 1}, {'name': '348', 'value': 1}, {'name': '349', 'value': 1}, {'name': '350', 'value': 1}, {'name': '351', 'value': 1}, {'name': '352', 'value': 1}, {'name': '353', 'value': 1}, {'name': '354', 'value': 2}, {'name': '355', 'value': 2}, {'name': '356', 'value': 2}, {'name': '357', 'value': 1}, {'name': '358', 'value': 1}, {'name': '359', 'value': 1}, {'name': '360', 'value': 1}, {'name': '361', 'value': 2}, {'name': '362', 'value': 2}, {'name': '363', 'value': 1}, {'name': '364', 'value': 2}, {'name': '365', 'value': 2}, {'name': '366', 'value': 2}, {'name': '367', 'value': 2}, {'name': '368', 'value': 1}, {'name': '369', 'value': 2}, {'name': '370', 'value': 2}, {'name': '371', 'value': 1}, {'name': '372', 'value': 2}, {'name': '373', 'value': 1}, {'name': '374', 'value': 1}, {'name': '375', 'value': 2}, {'name': '376', 'value': 2}, {'name': '377', 'value': 1}, {'name': '378', 'value': 2}, {'name': '379', 'value': 1}, {'name': '380', 'value': 1}, {'name': '381', 'value': 2}, {'name': '382', 'value': 2}, {'name': '383', 'value': 2}, {'name': '384', 'value': 1}, {'name': '385', 'value': 1}, {'name': '386', 'value': 2}, {'name': '387', 'value': 1}, {'name': '388', 'value': 1}, {'name': '389', 'value': 2}, {'name': '390', 'value': 2}, {'name': '391', 'value': 2}, {'name': '392', 'value': 1}, {'name': '393', 'value': 1}, {'name': '394', 'value': 2}, {'name': '395', 'value': 1}, {'name': '396', 'value': 1}, {'name': '397', 'value': 1}, {'name': '398', 'value': 1}, {'name': '399', 'value': 2}, {'name': '400', 'value': 2}, {'name': '401', 'value': 1}, {'name': '402', 'value': 1}, {'name': '403', 'value': 2}, {'name': '404', 'value': 2}, {'name': '405', 'value': 2}, {'name': '406', 'value': 1}, {'name': '407', 'value': 2}, {'name': '408', 'value': 2}, {'name': '409', 'value': 1}, {'name': '410', 'value': 1}, {'name': '411', 'value': 2}, {'name': '412', 'value': 1}, {'name': '413', 'value': 2}, {'name': '414', 'value': 2}, {'name': '415', 'value': 1}, {'name': '416', 'value': 2}, {'name': '417', 'value': 2}, {'name': '418', 'value': 2}, {'name': '419', 'value': 1}, {'name': '420', 'value': 1}, {'name': '421', 'value': 1}, {'name': '422', 'value': 1}, {'name': '423', 'value': 2}, {'name': '424', 'value': 1}, {'name': '425', 'value': 1}, {'name': '426', 'value': 2}, {'name': '427', 'value': 1}, {'name': '428', 'value': 2}, {'name': '429', 'value': 1}, {'name': '430', 'value': 2}, {'name': '431', 'value': 1}, {'name': '432', 'value': 1}, {'name': '433', 'value': 1}, {'name': '434', 'value': 2}, {'name': '435', 'value': 2}, {'name': '436', 'value': 2}, {'name': '437', 'value': 2}, {'name': '438', 'value': 2}, {'name': '439', 'value': 1}, {'name': '440', 'value': 1}, {'name': '441', 'value': 1}, {'name': '442', 'value': 1}, {'name': '443', 'value': 2}, {'name': '444', 'value': 1}, {'name': '445', 'value': 1}, {'name': '446', 'value': 2}, {'name': '447', 'value': 2}, {'name': '448', 'value': 2}, {'name': '449', 'value': 1}, {'name': '450', 'value': 2}, {'name': '451', 'value': 2}, {'name': '452', 'value': 2}, {'name': '453', 'value': 2}, {'name': '454', 'value': 1}, {'name': '455', 'value': 1}, {'name': '456', 'value': 2}, {'name': '457', 'value': 1}, {'name': '458', 'value': 1}, {'name': '459', 'value': 1}, {'name': '460', 'value': 2}, {'name': '461', 'value': 2}, {'name': '462', 'value': 2}, {'name': '463', 'value': 1}, {'name': '464', 'value': 2}, {'name': '465', 'value': 2}, {'name': '466', 'value': 2}, {'name': '467', 'value': 2}, {'name': '468', 'value': 2}, {'name': '469', 'value': 2}, {'name': '470', 'value': 2}, {'name': '471', 'value': 1}, {'name': '472', 'value': 1}, {'name': '473', 'value': 1}, {'name': '474', 'value': 2}, {'name': '475', 'value': 2}, {'name': '476', 'value': 1}, {'name': '477', 'value': 2}, {'name': '478', 'value': 2}, {'name': '479', 'value': 2}, {'name': '480', 'value': 1}, {'name': '481', 'value': 1}, {'name': '482', 'value': 2}, {'name': '483', 'value': 2}, {'name': '484', 'value': 1}, {'name': '485', 'value': 2}, {'name': '486', 'value': 1}, {'name': '487', 'value': 2}, {'name': '488', 'value': 1}, {'name': '489', 'value': 2}, {'name': '490', 'value': 1}, {'name': '491', 'value': 1}, {'name': '492', 'value': 1}, {'name': '493', 'value': 1}, {'name': '494', 'value': 2}, {'name': '495', 'value': 2}, {'name': '496', 'value': 1}, {'name': '497', 'value': 2}, {'name': '498', 'value': 2}, {'name': '499', 'value': 2}, {'name': '500', 'value': 2}]}, 'ChargeDetailID_stringindexer_transform': {'name': 'ChargeDetailID_stringindexer_transform', 'series': [{'name': '1', 'value': 407}, {'name': '2', 'value': 420}, {'name': '3', 'value': 287}, {'name': '4', 'value': 467}, {'name': '5', 'value': 463}, {'name': '6', 'value': 41}, {'name': '7', 'value': 42}, {'name': '8', 'value': 151}, {'name': '9', 'value': 244}, {'name': '10', 'value': 262}, {'name': '11', 'value': 281}, {'name': '12', 'value': 445}, {'name': '13', 'value': 166}, {'name': '14', 'value': 284}, {'name': '15', 'value': 21}, {'name': '16', 'value': 469}, {'name': '17', 'value': 384}, {'name': '18', 'value': 336}, {'name': '19', 'value': 219}, {'name': '20', 'value': 326}, {'name': '21', 'value': 242}, {'name': '22', 'value': 471}, {'name': '23', 'value': 62}, {'name': '24', 'value': 310}, {'name': '25', 'value': 325}, {'name': '26', 'value': 414}, {'name': '27', 'value': 364}, {'name': '28', 'value': 159}, {'name': '29', 'value': 171}, {'name': '30', 'value': 240}, {'name': '31', 'value': 167}, {'name': '32', 'value': 296}, {'name': '33', 'value': 371}, {'name': '34', 'value': 3}, {'name': '35', 'value': 395}, {'name': '36', 'value': 453}, {'name': '37', 'value': 152}, {'name': '38', 'value': 169}, {'name': '39', 'value': 96}, {'name': '40', 'value': 389}, {'name': '41', 'value': 129}, {'name': '42', 'value': 308}, {'name': '43', 'value': 71}, {'name': '44', 'value': 462}, {'name': '45', 'value': 444}, {'name': '46', 'value': 142}, {'name': '47', 'value': 480}, {'name': '48', 'value': 232}, {'name': '49', 'value': 373}, {'name': '50', 'value': 337}, {'name': '51', 'value': 304}, {'name': '52', 'value': 379}, {'name': '53', 'value': 33}, {'name': '54', 'value': 264}, {'name': '55', 'value': 440}, {'name': '56', 'value': 417}, {'name': '57', 'value': 234}, {'name': '58', 'value': 305}, {'name': '59', 'value': 123}, {'name': '60', 'value': 298}, {'name': '61', 'value': 66}, {'name': '62', 'value': 339}, {'name': '63', 'value': 478}, {'name': '64', 'value': 224}, {'name': '65', 'value': 431}, {'name': '66', 'value': 272}, {'name': '67', 'value': 438}, {'name': '68', 'value': 195}, {'name': '69', 'value': 93}, {'name': '70', 'value': 290}, {'name': '71', 'value': 342}, {'name': '72', 'value': 59}, {'name': '73', 'value': 18}, {'name': '74', 'value': 190}, {'name': '75', 'value': 340}, {'name': '76', 'value': 13}, {'name': '77', 'value': 163}, {'name': '78', 'value': 449}, {'name': '79', 'value': 133}, {'name': '80', 'value': 168}, {'name': '81', 'value': 346}, {'name': '82', 'value': 419}, {'name': '83', 'value': 35}, {'name': '84', 'value': 497}, {'name': '85', 'value': 341}, {'name': '86', 'value': 19}, {'name': '87', 'value': 226}, {'name': '88', 'value': 112}, {'name': '89', 'value': 72}, {'name': '90', 'value': 292}, {'name': '91', 'value': 362}, {'name': '92', 'value': 125}, {'name': '93', 'value': 439}, {'name': '94', 'value': 76}, {'name': '95', 'value': 250}, {'name': '96', 'value': 198}, {'name': '97', 'value': 187}, {'name': '98', 'value': 10}, {'name': '99', 'value': 294}, {'name': '100', 'value': 413}, {'name': '101', 'value': 381}, {'name': '102', 'value': 51}, {'name': '103', 'value': 180}, {'name': '104', 'value': 205}, {'name': '105', 'value': 8}, {'name': '106', 'value': 101}, {'name': '107', 'value': 89}, {'name': '108', 'value': 136}, {'name': '109', 'value': 318}, {'name': '110', 'value': 488}, {'name': '111', 'value': 46}, {'name': '112', 'value': 412}, {'name': '113', 'value': 283}, {'name': '114', 'value': 374}, {'name': '115', 'value': 221}, {'name': '116', 'value': 68}, {'name': '117', 'value': 468}, {'name': '118', 'value': 303}, {'name': '119', 'value': 81}, {'name': '120', 'value': 476}, {'name': '121', 'value': 433}, {'name': '122', 'value': 282}, {'name': '123', 'value': 494}, {'name': '124', 'value': 127}, {'name': '125', 'value': 85}, {'name': '126', 'value': 79}, {'name': '127', 'value': 70}, {'name': '128', 'value': 58}, {'name': '129', 'value': 377}, {'name': '130', 'value': 474}, {'name': '131', 'value': 322}, {'name': '132', 'value': 53}, {'name': '133', 'value': 410}, {'name': '134', 'value': 165}, {'name': '135', 'value': 295}, {'name': '136', 'value': 238}, {'name': '137', 'value': 330}, {'name': '138', 'value': 436}, {'name': '139', 'value': 110}, {'name': '140', 'value': 177}, {'name': '141', 'value': 435}, {'name': '142', 'value': 210}, {'name': '143', 'value': 153}, {'name': '144', 'value': 191}, {'name': '145', 'value': 208}, {'name': '146', 'value': 63}, {'name': '147', 'value': 124}, {'name': '148', 'value': 197}, {'name': '149', 'value': 55}, {'name': '150', 'value': 94}, {'name': '151', 'value': 285}, {'name': '152', 'value': 2}, {'name': '153', 'value': 87}, {'name': '154', 'value': 451}, {'name': '155', 'value': 316}, {'name': '156', 'value': 100}, {'name': '157', 'value': 82}, {'name': '158', 'value': 401}, {'name': '159', 'value': 286}, {'name': '160', 'value': 91}, {'name': '161', 'value': 256}, {'name': '162', 'value': 309}, {'name': '163', 'value': 365}, {'name': '164', 'value': 229}, {'name': '165', 'value': 202}, {'name': '166', 'value': 15}, {'name': '167', 'value': 69}, {'name': '168', 'value': 23}, {'name': '169', 'value': 64}, {'name': '170', 'value': 60}, {'name': '171', 'value': 479}, {'name': '172', 'value': 275}, {'name': '173', 'value': 427}, {'name': '174', 'value': 162}, {'name': '175', 'value': 425}, {'name': '176', 'value': 291}, {'name': '177', 'value': 121}, {'name': '178', 'value': 320}, {'name': '179', 'value': 113}, {'name': '180', 'value': 184}, {'name': '181', 'value': 175}, {'name': '182', 'value': 74}, {'name': '183', 'value': 442}, {'name': '184', 'value': 355}, {'name': '185', 'value': 12}, {'name': '186', 'value': 254}, {'name': '187', 'value': 7}, {'name': '188', 'value': 98}, {'name': '189', 'value': 428}, {'name': '190', 'value': 253}, {'name': '191', 'value': 387}, {'name': '192', 'value': 299}, {'name': '193', 'value': 255}, {'name': '194', 'value': 17}, {'name': '195', 'value': 349}, {'name': '196', 'value': 11}, {'name': '197', 'value': 32}, {'name': '198', 'value': 378}, {'name': '199', 'value': 323}, {'name': '200', 'value': 366}, {'name': '201', 'value': 196}, {'name': '202', 'value': 28}, {'name': '203', 'value': 271}, {'name': '204', 'value': 472}, {'name': '205', 'value': 77}, {'name': '206', 'value': 149}, {'name': '207', 'value': 122}, {'name': '208', 'value': 249}, {'name': '209', 'value': 144}, {'name': '210', 'value': 416}, {'name': '211', 'value': 434}, {'name': '212', 'value': 329}, {'name': '213', 'value': 223}, {'name': '214', 'value': 321}, {'name': '215', 'value': 464}, {'name': '216', 'value': 183}, {'name': '217', 'value': 301}, {'name': '218', 'value': 57}, {'name': '219', 'value': 344}, {'name': '220', 'value': 45}, {'name': '221', 'value': 128}, {'name': '222', 'value': 132}, {'name': '223', 'value': 257}, {'name': '224', 'value': 289}, {'name': '225', 'value': 333}, {'name': '226', 'value': 370}, {'name': '227', 'value': 458}, {'name': '228', 'value': 49}, {'name': '229', 'value': 4}, {'name': '230', 'value': 359}, {'name': '231', 'value': 225}, {'name': '232', 'value': 241}, {'name': '233', 'value': 111}, {'name': '234', 'value': 88}, {'name': '235', 'value': 481}, {'name': '236', 'value': 176}, {'name': '237', 'value': 459}, {'name': '238', 'value': 29}, {'name': '239', 'value': 327}, {'name': '240', 'value': 84}, {'name': '241', 'value': 313}, {'name': '242', 'value': 411}, {'name': '243', 'value': 392}, {'name': '244', 'value': 324}, {'name': '245', 'value': 269}, {'name': '246', 'value': 148}, {'name': '247', 'value': 147}, {'name': '248', 'value': 314}, {'name': '249', 'value': 252}, {'name': '250', 'value': 421}, {'name': '251', 'value': 404}, {'name': '252', 'value': 345}, {'name': '253', 'value': 135}, {'name': '254', 'value': 361}, {'name': '255', 'value': 105}, {'name': '256', 'value': 357}, {'name': '257', 'value': 172}, {'name': '258', 'value': 118}, {'name': '259', 'value': 193}, {'name': '260', 'value': 338}, {'name': '261', 'value': 477}, {'name': '262', 'value': 447}, {'name': '263', 'value': 243}, {'name': '264', 'value': 83}, {'name': '265', 'value': 181}, {'name': '266', 'value': 403}, {'name': '267', 'value': 270}, {'name': '268', 'value': 155}, {'name': '269', 'value': 397}, {'name': '270', 'value': 388}, {'name': '271', 'value': 56}, {'name': '272', 'value': 103}, {'name': '273', 'value': 369}, {'name': '274', 'value': 246}, {'name': '275', 'value': 154}, {'name': '276', 'value': 293}, {'name': '277', 'value': 178}, {'name': '278', 'value': 192}, {'name': '279', 'value': 170}, {'name': '280', 'value': 90}, {'name': '281', 'value': 24}, {'name': '282', 'value': 315}, {'name': '283', 'value': 350}, {'name': '284', 'value': 80}, {'name': '285', 'value': 441}, {'name': '286', 'value': 160}, {'name': '287', 'value': 372}, {'name': '288', 'value': 146}, {'name': '289', 'value': 400}, {'name': '290', 'value': 391}, {'name': '291', 'value': 31}, {'name': '292', 'value': 394}, {'name': '293', 'value': 406}, {'name': '294', 'value': 231}, {'name': '295', 'value': 306}, {'name': '296', 'value': 116}, {'name': '297', 'value': 390}, {'name': '298', 'value': 115}, {'name': '299', 'value': 212}, {'name': '300', 'value': 266}, {'name': '301', 'value': 233}, {'name': '302', 'value': 385}, {'name': '303', 'value': 258}, {'name': '304', 'value': 277}, {'name': '305', 'value': 328}, {'name': '306', 'value': 382}, {'name': '307', 'value': 130}, {'name': '308', 'value': 335}, {'name': '309', 'value': 22}, {'name': '310', 'value': 47}, {'name': '311', 'value': 36}, {'name': '312', 'value': 235}, {'name': '313', 'value': 297}, {'name': '314', 'value': 50}, {'name': '315', 'value': 251}, {'name': '316', 'value': 114}, {'name': '317', 'value': 263}, {'name': '318', 'value': 218}, {'name': '319', 'value': 186}, {'name': '320', 'value': 39}, {'name': '321', 'value': 185}, {'name': '322', 'value': 228}, {'name': '323', 'value': 213}, {'name': '324', 'value': 65}, {'name': '325', 'value': 443}, {'name': '326', 'value': 73}, {'name': '327', 'value': 227}, {'name': '328', 'value': 399}, {'name': '329', 'value': 179}, {'name': '330', 'value': 351}, {'name': '331', 'value': 393}, {'name': '332', 'value': 457}, {'name': '333', 'value': 380}, {'name': '334', 'value': 383}, {'name': '335', 'value': 311}, {'name': '336', 'value': 490}, {'name': '337', 'value': 54}, {'name': '338', 'value': 9}, {'name': '339', 'value': 131}, {'name': '340', 'value': 495}, {'name': '341', 'value': 261}, {'name': '342', 'value': 409}, {'name': '343', 'value': 356}, {'name': '344', 'value': 473}, {'name': '345', 'value': 174}, {'name': '346', 'value': 141}, {'name': '347', 'value': 215}, {'name': '348', 'value': 486}, {'name': '349', 'value': 119}, {'name': '350', 'value': 482}, {'name': '351', 'value': 278}, {'name': '352', 'value': 236}, {'name': '353', 'value': 158}, {'name': '354', 'value': 300}, {'name': '355', 'value': 134}, {'name': '356', 'value': 343}, {'name': '357', 'value': 6}, {'name': '358', 'value': 150}, {'name': '359', 'value': 216}, {'name': '360', 'value': 423}, {'name': '361', 'value': 475}, {'name': '362', 'value': 27}, {'name': '363', 'value': 34}, {'name': '364', 'value': 207}, {'name': '365', 'value': 14}, {'name': '366', 'value': 334}, {'name': '367', 'value': 95}, {'name': '368', 'value': 44}, {'name': '369', 'value': 405}, {'name': '370', 'value': 38}, {'name': '371', 'value': 363}, {'name': '372', 'value': 126}, {'name': '373', 'value': 0}, {'name': '374', 'value': 157}, {'name': '375', 'value': 484}, {'name': '376', 'value': 418}, {'name': '377', 'value': 437}, {'name': '378', 'value': 498}, {'name': '379', 'value': 247}, {'name': '380', 'value': 75}, {'name': '381', 'value': 145}, {'name': '382', 'value': 209}, {'name': '383', 'value': 108}, {'name': '384', 'value': 461}, {'name': '385', 'value': 206}, {'name': '386', 'value': 237}, {'name': '387', 'value': 260}, {'name': '388', 'value': 92}, {'name': '389', 'value': 273}, {'name': '390', 'value': 86}, {'name': '391', 'value': 452}, {'name': '392', 'value': 104}, {'name': '393', 'value': 16}, {'name': '394', 'value': 245}, {'name': '395', 'value': 182}, {'name': '396', 'value': 201}, {'name': '397', 'value': 466}, {'name': '398', 'value': 408}, {'name': '399', 'value': 138}, {'name': '400', 'value': 43}, {'name': '401', 'value': 217}, {'name': '402', 'value': 67}, {'name': '403', 'value': 455}, {'name': '404', 'value': 454}, {'name': '405', 'value': 99}, {'name': '406', 'value': 164}, {'name': '407', 'value': 354}, {'name': '408', 'value': 332}, {'name': '409', 'value': 276}, {'name': '410', 'value': 102}, {'name': '411', 'value': 430}, {'name': '412', 'value': 499}, {'name': '413', 'value': 259}, {'name': '414', 'value': 52}, {'name': '415', 'value': 348}, {'name': '416', 'value': 424}, {'name': '417', 'value': 173}, {'name': '418', 'value': 25}, {'name': '419', 'value': 139}, {'name': '420', 'value': 432}, {'name': '421', 'value': 37}, {'name': '422', 'value': 489}, {'name': '423', 'value': 358}, {'name': '424', 'value': 398}, {'name': '425', 'value': 5}, {'name': '426', 'value': 220}, {'name': '427', 'value': 367}, {'name': '428', 'value': 26}, {'name': '429', 'value': 188}, {'name': '430', 'value': 222}, {'name': '431', 'value': 1}, {'name': '432', 'value': 143}, {'name': '433', 'value': 30}, {'name': '434', 'value': 156}, {'name': '435', 'value': 106}, {'name': '436', 'value': 20}, {'name': '437', 'value': 248}, {'name': '438', 'value': 415}, {'name': '439', 'value': 78}, {'name': '440', 'value': 204}, {'name': '441', 'value': 267}, {'name': '442', 'value': 40}, {'name': '443', 'value': 426}, {'name': '444', 'value': 274}, {'name': '445', 'value': 117}, {'name': '446', 'value': 302}, {'name': '447', 'value': 496}, {'name': '448', 'value': 493}, {'name': '449', 'value': 107}, {'name': '450', 'value': 465}, {'name': '451', 'value': 230}, {'name': '452', 'value': 203}, {'name': '453', 'value': 61}, {'name': '454', 'value': 194}, {'name': '455', 'value': 487}, {'name': '456', 'value': 307}, {'name': '457', 'value': 211}, {'name': '458', 'value': 386}, {'name': '459', 'value': 429}, {'name': '460', 'value': 470}, {'name': '461', 'value': 456}, {'name': '462', 'value': 375}, {'name': '463', 'value': 161}, {'name': '464', 'value': 347}, {'name': '465', 'value': 312}, {'name': '466', 'value': 265}, {'name': '467', 'value': 450}, {'name': '468', 'value': 189}, {'name': '469', 'value': 360}, {'name': '470', 'value': 279}, {'name': '471', 'value': 239}, {'name': '472', 'value': 368}, {'name': '473', 'value': 109}, {'name': '474', 'value': 120}, {'name': '475', 'value': 48}, {'name': '476', 'value': 353}, {'name': '477', 'value': 97}, {'name': '478', 'value': 319}, {'name': '479', 'value': 317}, {'name': '480', 'value': 376}, {'name': '481', 'value': 485}, {'name': '482', 'value': 492}, {'name': '483', 'value': 140}, {'name': '484', 'value': 288}, {'name': '485', 'value': 491}, {'name': '486', 'value': 137}, {'name': '487', 'value': 460}, {'name': '488', 'value': 199}, {'name': '489', 'value': 446}, {'name': '490', 'value': 200}, {'name': '491', 'value': 448}, {'name': '492', 'value': 396}, {'name': '493', 'value': 331}, {'name': '494', 'value': 422}, {'name': '495', 'value': 214}, {'name': '496', 'value': 402}, {'name': '497', 'value': 268}, {'name': '498', 'value': 280}, {'name': '499', 'value': 483}, {'name': '500', 'value': 352}]}, 'DateofService_dayofmonth': {'name': 'DateofService_dayofmonth', 'series': [{'name': '1', 'value': 18}, {'name': '2', 'value': 13}, {'name': '3', 'value': 16}, {'name': '4', 'value': 13}, {'name': '5', 'value': 27}, {'name': '6', 'value': 3}, {'name': '7', 'value': 2}, {'name': '8', 'value': 3}, {'name': '9', 'value': 14}, {'name': '10', 'value': 28}, {'name': '11', 'value': 30}, {'name': '12', 'value': 30}, {'name': '13', 'value': 26}, {'name': '14', 'value': 18}, {'name': '15', 'value': 9}, {'name': '16', 'value': 19}, {'name': '17', 'value': 16}, {'name': '18', 'value': 4}, {'name': '19', 'value': 2}, {'name': '20', 'value': 15}, {'name': '21', 'value': 20}, {'name': '22', 'value': 30}, {'name': '23', 'value': 5}, {'name': '24', 'value': 6}, {'name': '25', 'value': 7}, {'name': '26', 'value': 6}, {'name': '27', 'value': 13}, {'name': '28', 'value': 11}, {'name': '29', 'value': 27}, {'name': '30', 'value': 17}, {'name': '31', 'value': 12}, {'name': '32', 'value': 20}, {'name': '33', 'value': 7}, {'name': '34', 'value': 9}, {'name': '35', 'value': 7}, {'name': '36', 'value': 3}, {'name': '37', 'value': 7}, {'name': '38', 'value': 14}, {'name': '39', 'value': 6}, {'name': '40', 'value': 11}, {'name': '41', 'value': 29}, {'name': '42', 'value': 12}, {'name': '43', 'value': 10}, {'name': '44', 'value': 30}, {'name': '45', 'value': 21}, {'name': '46', 'value': 12}, {'name': '47', 'value': 16}, {'name': '48', 'value': 6}, {'name': '49', 'value': 6}, {'name': '50', 'value': 21}, {'name': '51', 'value': 16}, {'name': '52', 'value': 24}, {'name': '53', 'value': 12}, {'name': '54', 'value': 19}, {'name': '55', 'value': 12}, {'name': '56', 'value': 11}, {'name': '57', 'value': 5}, {'name': '58', 'value': 12}, {'name': '59', 'value': 30}, {'name': '60', 'value': 19}, {'name': '61', 'value': 13}, {'name': '62', 'value': 19}, {'name': '63', 'value': 16}, {'name': '64', 'value': 14}, {'name': '65', 'value': 27}, {'name': '66', 'value': 11}, {'name': '67', 'value': 6}, {'name': '68', 'value': 13}, {'name': '69', 'value': 29}, {'name': '70', 'value': 20}, {'name': '71', 'value': 6}, {'name': '72', 'value': 7}, {'name': '73', 'value': 2}, {'name': '74', 'value': 5}, {'name': '75', 'value': 27}, {'name': '76', 'value': 13}, {'name': '77', 'value': 14}, {'name': '78', 'value': 8}, {'name': '79', 'value': 16}, {'name': '80', 'value': 13}, {'name': '81', 'value': 28}, {'name': '82', 'value': 2}, {'name': '83', 'value': 30}, {'name': '84', 'value': 15}, {'name': '85', 'value': 8}, {'name': '86', 'value': 5}, {'name': '87', 'value': 12}, {'name': '88', 'value': 12}, {'name': '89', 'value': 13}, {'name': '90', 'value': 18}, {'name': '91', 'value': 14}, {'name': '92', 'value': 6}, {'name': '93', 'value': 13}, {'name': '94', 'value': 29}, {'name': '95', 'value': 12}, {'name': '96', 'value': 13}, {'name': '97', 'value': 11}, {'name': '98', 'value': 27}, {'name': '99', 'value': 27}, {'name': '100', 'value': 28}, {'name': '101', 'value': 27}, {'name': '102', 'value': 2}, {'name': '103', 'value': 20}, {'name': '104', 'value': 1}, {'name': '105', 'value': 14}, {'name': '106', 'value': 19}, {'name': '107', 'value': 14}, {'name': '108', 'value': 10}, {'name': '109', 'value': 14}, {'name': '110', 'value': 2}, {'name': '111', 'value': 17}, {'name': '112', 'value': 6}, {'name': '113', 'value': 4}, {'name': '114', 'value': 7}, {'name': '115', 'value': 14}, {'name': '116', 'value': 30}, {'name': '117', 'value': 7}, {'name': '118', 'value': 20}, {'name': '119', 'value': 14}, {'name': '120', 'value': 14}, {'name': '121', 'value': 15}, {'name': '122', 'value': 6}, {'name': '123', 'value': 9}, {'name': '124', 'value': 19}, {'name': '125', 'value': 13}, {'name': '126', 'value': 26}, {'name': '127', 'value': 28}, {'name': '128', 'value': 21}, {'name': '129', 'value': 15}, {'name': '130', 'value': 9}, {'name': '131', 'value': 1}, {'name': '132', 'value': 14}, {'name': '133', 'value': 19}, {'name': '134', 'value': 13}, {'name': '135', 'value': 7}, {'name': '136', 'value': 10}, {'name': '137', 'value': 3}, {'name': '138', 'value': 28}, {'name': '139', 'value': 5}, {'name': '140', 'value': 13}, {'name': '141', 'value': 14}, {'name': '142', 'value': 13}, {'name': '143', 'value': 18}, {'name': '144', 'value': 2}, {'name': '145', 'value': 24}, {'name': '146', 'value': 8}, {'name': '147', 'value': 4}, {'name': '148', 'value': 3}, {'name': '149', 'value': 10}, {'name': '150', 'value': 16}, {'name': '151', 'value': 7}, {'name': '152', 'value': 30}, {'name': '153', 'value': 20}, {'name': '154', 'value': 15}, {'name': '155', 'value': 21}, {'name': '156', 'value': 3}, {'name': '157', 'value': 24}, {'name': '158', 'value': 2}, {'name': '159', 'value': 16}, {'name': '160', 'value': 19}, {'name': '161', 'value': 19}, {'name': '162', 'value': 28}, {'name': '163', 'value': 18}, {'name': '164', 'value': 3}, {'name': '165', 'value': 12}, {'name': '166', 'value': 11}, {'name': '167', 'value': 6}, {'name': '168', 'value': 13}, {'name': '169', 'value': 13}, {'name': '170', 'value': 5}, {'name': '171', 'value': 7}, {'name': '172', 'value': 7}, {'name': '173', 'value': 14}, {'name': '174', 'value': 4}, {'name': '175', 'value': 4}, {'name': '176', 'value': 12}, {'name': '177', 'value': 7}, {'name': '178', 'value': 4}, {'name': '179', 'value': 28}, {'name': '180', 'value': 12}, {'name': '181', 'value': 8}, {'name': '182', 'value': 7}, {'name': '183', 'value': 14}, {'name': '184', 'value': 26}, {'name': '185', 'value': 20}, {'name': '186', 'value': 28}, {'name': '187', 'value': 29}, {'name': '188', 'value': 28}, {'name': '189', 'value': 20}, {'name': '190', 'value': 7}, {'name': '191', 'value': 7}, {'name': '192', 'value': 7}, {'name': '193', 'value': 5}, {'name': '194', 'value': 5}, {'name': '195', 'value': 12}, {'name': '196', 'value': 19}, {'name': '197', 'value': 7}, {'name': '198', 'value': 13}, {'name': '199', 'value': 17}, {'name': '200', 'value': 13}, {'name': '201', 'value': 30}, {'name': '202', 'value': 17}, {'name': '203', 'value': 21}, {'name': '204', 'value': 16}, {'name': '205', 'value': 27}, {'name': '206', 'value': 6}, {'name': '207', 'value': 9}, {'name': '208', 'value': 14}, {'name': '209', 'value': 6}, {'name': '210', 'value': 14}, {'name': '211', 'value': 7}, {'name': '212', 'value': 7}, {'name': '213', 'value': 6}, {'name': '214', 'value': 5}, {'name': '215', 'value': 21}, {'name': '216', 'value': 1}, {'name': '217', 'value': 21}, {'name': '218', 'value': 13}, {'name': '219', 'value': 13}, {'name': '220', 'value': 8}, {'name': '221', 'value': 5}, {'name': '222', 'value': 27}, {'name': '223', 'value': 18}, {'name': '224', 'value': 13}, {'name': '225', 'value': 19}, {'name': '226', 'value': 3}, {'name': '227', 'value': 30}, {'name': '228', 'value': 27}, {'name': '229', 'value': 29}, {'name': '230', 'value': 16}, {'name': '231', 'value': 2}, {'name': '232', 'value': 11}, {'name': '233', 'value': 29}, {'name': '234', 'value': 8}, {'name': '235', 'value': 4}, {'name': '236', 'value': 16}, {'name': '237', 'value': 21}, {'name': '238', 'value': 15}, {'name': '239', 'value': 16}, {'name': '240', 'value': 12}, {'name': '241', 'value': 21}, {'name': '242', 'value': 20}, {'name': '243', 'value': 13}, {'name': '244', 'value': 8}, {'name': '245', 'value': 16}, {'name': '246', 'value': 28}, {'name': '247', 'value': 14}, {'name': '248', 'value': 16}, {'name': '249', 'value': 10}, {'name': '250', 'value': 21}, {'name': '251', 'value': 13}, {'name': '252', 'value': 6}, {'name': '253', 'value': 26}, {'name': '254', 'value': 5}, {'name': '255', 'value': 31}, {'name': '256', 'value': 20}, {'name': '257', 'value': 3}, {'name': '258', 'value': 1}, {'name': '259', 'value': 18}, {'name': '260', 'value': 9}, {'name': '261', 'value': 20}, {'name': '262', 'value': 29}, {'name': '263', 'value': 12}, {'name': '264', 'value': 5}, {'name': '265', 'value': 14}, {'name': '266', 'value': 15}, {'name': '267', 'value': 21}, {'name': '268', 'value': 30}, {'name': '269', 'value': 4}, {'name': '270', 'value': 21}, {'name': '271', 'value': 27}, {'name': '272', 'value': 26}, {'name': '273', 'value': 5}, {'name': '274', 'value': 18}, {'name': '275', 'value': 11}, {'name': '276', 'value': 16}, {'name': '277', 'value': 19}, {'name': '278', 'value': 14}, {'name': '279', 'value': 24}, {'name': '280', 'value': 3}, {'name': '281', 'value': 10}, {'name': '282', 'value': 10}, {'name': '283', 'value': 7}, {'name': '284', 'value': 20}, {'name': '285', 'value': 19}, {'name': '286', 'value': 11}, {'name': '287', 'value': 6}, {'name': '288', 'value': 1}, {'name': '289', 'value': 7}, {'name': '290', 'value': 29}, {'name': '291', 'value': 9}, {'name': '292', 'value': 6}, {'name': '293', 'value': 5}, {'name': '294', 'value': 26}, {'name': '295', 'value': 18}, {'name': '296', 'value': 20}, {'name': '297', 'value': 7}, {'name': '298', 'value': 4}, {'name': '299', 'value': 31}, {'name': '300', 'value': 5}, {'name': '301', 'value': 3}, {'name': '302', 'value': 3}, {'name': '303', 'value': 28}, {'name': '304', 'value': 4}, {'name': '305', 'value': 28}, {'name': '306', 'value': 20}, {'name': '307', 'value': 13}, {'name': '308', 'value': 21}, {'name': '309', 'value': 7}, {'name': '310', 'value': 14}, {'name': '311', 'value': 6}, {'name': '312', 'value': 6}, {'name': '313', 'value': 13}, {'name': '314', 'value': 15}, {'name': '315', 'value': 12}, {'name': '316', 'value': 12}, {'name': '317', 'value': 7}, {'name': '318', 'value': 29}, {'name': '319', 'value': 28}, {'name': '320', 'value': 16}, {'name': '321', 'value': 16}, {'name': '322', 'value': 20}, {'name': '323', 'value': 19}, {'name': '324', 'value': 16}, {'name': '325', 'value': 30}, {'name': '326', 'value': 7}, {'name': '327', 'value': 19}, {'name': '328', 'value': 18}, {'name': '329', 'value': 26}, {'name': '330', 'value': 8}, {'name': '331', 'value': 1}, {'name': '332', 'value': 3}, {'name': '333', 'value': 17}, {'name': '334', 'value': 28}, {'name': '335', 'value': 9}, {'name': '336', 'value': 5}, {'name': '337', 'value': 2}, {'name': '338', 'value': 12}, {'name': '339', 'value': 27}, {'name': '340', 'value': 14}, {'name': '341', 'value': 30}, {'name': '342', 'value': 30}, {'name': '343', 'value': 19}, {'name': '344', 'value': 13}, {'name': '345', 'value': 1}, {'name': '346', 'value': 13}, {'name': '347', 'value': 29}, {'name': '348', 'value': 20}, {'name': '349', 'value': 24}, {'name': '350', 'value': 29}, {'name': '351', 'value': 3}, {'name': '352', 'value': 14}, {'name': '353', 'value': 3}, {'name': '354', 'value': 12}, {'name': '355', 'value': 6}, {'name': '356', 'value': 14}, {'name': '357', 'value': 7}, {'name': '358', 'value': 13}, {'name': '359', 'value': 26}, {'name': '360', 'value': 12}, {'name': '361', 'value': 29}, {'name': '362', 'value': 20}, {'name': '363', 'value': 13}, {'name': '364', 'value': 13}, {'name': '365', 'value': 6}, {'name': '366', 'value': 20}, {'name': '367', 'value': 3}, {'name': '368', 'value': 20}, {'name': '369', 'value': 19}, {'name': '370', 'value': 16}, {'name': '371', 'value': 31}, {'name': '372', 'value': 21}, {'name': '373', 'value': 18}, {'name': '374', 'value': 18}, {'name': '375', 'value': 17}, {'name': '376', 'value': 9}, {'name': '377', 'value': 19}, {'name': '378', 'value': 6}, {'name': '379', 'value': 20}, {'name': '380', 'value': 21}, {'name': '381', 'value': 19}, {'name': '382', 'value': 7}, {'name': '383', 'value': 19}, {'name': '384', 'value': 28}, {'name': '385', 'value': 26}, {'name': '386', 'value': 7}, {'name': '387', 'value': 6}, {'name': '388', 'value': 16}, {'name': '389', 'value': 28}, {'name': '390', 'value': 29}, {'name': '391', 'value': 19}, {'name': '392', 'value': 19}, {'name': '393', 'value': 26}, {'name': '394', 'value': 20}, {'name': '395', 'value': 15}, {'name': '396', 'value': 14}, {'name': '397', 'value': 15}, {'name': '398', 'value': 12}, {'name': '399', 'value': 18}, {'name': '400', 'value': 8}, {'name': '401', 'value': 12}, {'name': '402', 'value': 29}, {'name': '403', 'value': 19}, {'name': '404', 'value': 19}, {'name': '405', 'value': 26}, {'name': '406', 'value': 29}, {'name': '407', 'value': 19}, {'name': '408', 'value': 14}, {'name': '409', 'value': 19}, {'name': '410', 'value': 18}, {'name': '411', 'value': 15}, {'name': '412', 'value': 9}, {'name': '413', 'value': 6}, {'name': '414', 'value': 28}, {'name': '415', 'value': 27}, {'name': '416', 'value': 7}, {'name': '417', 'value': 28}, {'name': '418', 'value': 15}, {'name': '419', 'value': 29}, {'name': '420', 'value': 4}, {'name': '421', 'value': 7}, {'name': '422', 'value': 6}, {'name': '423', 'value': 10}, {'name': '424', 'value': 7}, {'name': '425', 'value': 8}, {'name': '426', 'value': 29}, {'name': '427', 'value': 8}, {'name': '428', 'value': 11}, {'name': '429', 'value': 20}, {'name': '430', 'value': 14}, {'name': '431', 'value': 11}, {'name': '432', 'value': 11}, {'name': '433', 'value': 11}, {'name': '434', 'value': 27}, {'name': '435', 'value': 6}, {'name': '436', 'value': 2}, {'name': '437', 'value': 14}, {'name': '438', 'value': 13}, {'name': '439', 'value': 3}, {'name': '440', 'value': 2}, {'name': '441', 'value': 1}, {'name': '442', 'value': 7}, {'name': '443', 'value': 11}, {'name': '444', 'value': 20}, {'name': '445', 'value': 16}, {'name': '446', 'value': 30}, {'name': '447', 'value': 27}, {'name': '448', 'value': 18}, {'name': '449', 'value': 6}, {'name': '450', 'value': 6}, {'name': '451', 'value': 19}, {'name': '452', 'value': 21}, {'name': '453', 'value': 12}, {'name': '454', 'value': 16}, {'name': '455', 'value': 14}, {'name': '456', 'value': 5}, {'name': '457', 'value': 13}, {'name': '458', 'value': 6}, {'name': '459', 'value': 27}, {'name': '460', 'value': 15}, {'name': '461', 'value': 8}, {'name': '462', 'value': 7}, {'name': '463', 'value': 6}, {'name': '464', 'value': 21}, {'name': '465', 'value': 11}, {'name': '466', 'value': 27}, {'name': '467', 'value': 16}, {'name': '468', 'value': 14}, {'name': '469', 'value': 27}, {'name': '470', 'value': 13}, {'name': '471', 'value': 4}, {'name': '472', 'value': 21}, {'name': '473', 'value': 17}, {'name': '474', 'value': 12}, {'name': '475', 'value': 13}, {'name': '476', 'value': 15}, {'name': '477', 'value': 7}, {'name': '478', 'value': 7}, {'name': '479', 'value': 8}, {'name': '480', 'value': 3}, {'name': '481', 'value': 30}, {'name': '482', 'value': 5}, {'name': '483', 'value': 11}, {'name': '484', 'value': 20}, {'name': '485', 'value': 27}, {'name': '486', 'value': 1}, {'name': '487', 'value': 11}, {'name': '488', 'value': 14}, {'name': '489', 'value': 11}, {'name': '490', 'value': 8}, {'name': '491', 'value': 9}, {'name': '492', 'value': 12}, {'name': '493', 'value': 12}, {'name': '494', 'value': 27}, {'name': '495', 'value': 12}, {'name': '496', 'value': 8}, {'name': '497', 'value': 27}, {'name': '498', 'value': 28}, {'name': '499', 'value': 30}, {'name': '500', 'value': 3}]}, 'DateofService_month': {'name': 'DateofService_month', 'series': [{'name': '1', 'value': 12}, {'name': '2', 'value': 12}, {'name': '3', 'value': 11}, {'name': '4', 'value': 12}, {'name': '5', 'value': 11}, {'name': '6', 'value': 12}, {'name': '7', 'value': 11}, {'name': '8', 'value': 12}, {'name': '9', 'value': 12}, {'name': '10', 'value': 11}, {'name': '11', 'value': 11}, {'name': '12', 'value': 11}, {'name': '13', 'value': 11}, {'name': '14', 'value': 12}, {'name': '15', 'value': 11}, {'name': '16', 'value': 12}, {'name': '17', 'value': 11}, {'name': '18', 'value': 12}, {'name': '19', 'value': 11}, {'name': '20', 'value': 11}, {'name': '21', 'value': 11}, {'name': '22', 'value': 11}, {'name': '23', 'value': 12}, {'name': '24', 'value': 12}, {'name': '25', 'value': 12}, {'name': '26', 'value': 11}, {'name': '27', 'value': 11}, {'name': '28', 'value': 12}, {'name': '29', 'value': 11}, {'name': '30', 'value': 12}, {'name': '31', 'value': 11}, {'name': '32', 'value': 11}, {'name': '33', 'value': 12}, {'name': '34', 'value': 11}, {'name': '35', 'value': 11}, {'name': '36', 'value': 12}, {'name': '37', 'value': 11}, {'name': '38', 'value': 11}, {'name': '39', 'value': 11}, {'name': '40', 'value': 12}, {'name': '41', 'value': 11}, {'name': '42', 'value': 12}, {'name': '43', 'value': 12}, {'name': '44', 'value': 11}, {'name': '45', 'value': 12}, {'name': '46', 'value': 12}, {'name': '47', 'value': 11}, {'name': '48', 'value': 12}, {'name': '49', 'value': 11}, {'name': '50', 'value': 11}, {'name': '51', 'value': 11}, {'name': '52', 'value': 12}, {'name': '53', 'value': 11}, {'name': '54', 'value': 12}, {'name': '55', 'value': 12}, {'name': '56', 'value': 12}, {'name': '57', 'value': 11}, {'name': '58', 'value': 12}, {'name': '59', 'value': 11}, {'name': '60', 'value': 11}, {'name': '61', 'value': 11}, {'name': '62', 'value': 12}, {'name': '63', 'value': 11}, {'name': '64', 'value': 11}, {'name': '65', 'value': 11}, {'name': '66', 'value': 12}, {'name': '67', 'value': 12}, {'name': '68', 'value': 11}, {'name': '69', 'value': 11}, {'name': '70', 'value': 12}, {'name': '71', 'value': 11}, {'name': '72', 'value': 12}, {'name': '73', 'value': 11}, {'name': '74', 'value': 11}, {'name': '75', 'value': 11}, {'name': '76', 'value': 12}, {'name': '77', 'value': 11}, {'name': '78', 'value': 11}, {'name': '79', 'value': 11}, {'name': '80', 'value': 11}, {'name': '81', 'value': 11}, {'name': '82', 'value': 11}, {'name': '83', 'value': 11}, {'name': '84', 'value': 11}, {'name': '85', 'value': 11}, {'name': '86', 'value': 12}, {'name': '87', 'value': 11}, {'name': '88', 'value': 11}, {'name': '89', 'value': 11}, {'name': '90', 'value': 12}, {'name': '91', 'value': 12}, {'name': '92', 'value': 12}, {'name': '93', 'value': 12}, {'name': '94', 'value': 11}, {'name': '95', 'value': 12}, {'name': '96', 'value': 11}, {'name': '97', 'value': 12}, {'name': '98', 'value': 12}, {'name': '99', 'value': 12}, {'name': '100', 'value': 11}, {'name': '101', 'value': 12}, {'name': '102', 'value': 11}, {'name': '103', 'value': 12}, {'name': '104', 'value': 11}, {'name': '105', 'value': 12}, {'name': '106', 'value': 12}, {'name': '107', 'value': 12}, {'name': '108', 'value': 12}, {'name': '109', 'value': 11}, {'name': '110', 'value': 11}, {'name': '111', 'value': 12}, {'name': '112', 'value': 12}, {'name': '113', 'value': 12}, {'name': '114', 'value': 12}, {'name': '115', 'value': 11}, {'name': '116', 'value': 11}, {'name': '117', 'value': 11}, {'name': '118', 'value': 11}, {'name': '119', 'value': 11}, {'name': '120', 'value': 12}, {'name': '121', 'value': 11}, {'name': '122', 'value': 11}, {'name': '123', 'value': 11}, {'name': '124', 'value': 12}, {'name': '125', 'value': 12}, {'name': '126', 'value': 12}, {'name': '127', 'value': 12}, {'name': '128', 'value': 12}, {'name': '129', 'value': 11}, {'name': '130', 'value': 11}, {'name': '131', 'value': 11}, {'name': '132', 'value': 12}, {'name': '133', 'value': 11}, {'name': '134', 'value': 11}, {'name': '135', 'value': 12}, {'name': '136', 'value': 12}, {'name': '137', 'value': 12}, {'name': '138', 'value': 11}, {'name': '139', 'value': 12}, {'name': '140', 'value': 11}, {'name': '141', 'value': 12}, {'name': '142', 'value': 11}, {'name': '143', 'value': 12}, {'name': '144', 'value': 11}, {'name': '145', 'value': 12}, {'name': '146', 'value': 11}, {'name': '147', 'value': 12}, {'name': '148', 'value': 12}, {'name': '149', 'value': 12}, {'name': '150', 'value': 11}, {'name': '151', 'value': 11}, {'name': '152', 'value': 11}, {'name': '153', 'value': 12}, {'name': '154', 'value': 11}, {'name': '155', 'value': 12}, {'name': '156', 'value': 12}, {'name': '157', 'value': 12}, {'name': '158', 'value': 11}, {'name': '159', 'value': 11}, {'name': '160', 'value': 11}, {'name': '161', 'value': 11}, {'name': '162', 'value': 11}, {'name': '163', 'value': 12}, {'name': '164', 'value': 12}, {'name': '165', 'value': 11}, {'name': '166', 'value': 12}, {'name': '167', 'value': 11}, {'name': '168', 'value': 11}, {'name': '169', 'value': 12}, {'name': '170', 'value': 11}, {'name': '171', 'value': 11}, {'name': '172', 'value': 12}, {'name': '173', 'value': 12}, {'name': '174', 'value': 12}, {'name': '175', 'value': 12}, {'name': '176', 'value': 12}, {'name': '177', 'value': 11}, {'name': '178', 'value': 12}, {'name': '179', 'value': 11}, {'name': '180', 'value': 11}, {'name': '181', 'value': 11}, {'name': '182', 'value': 12}, {'name': '183', 'value': 12}, {'name': '184', 'value': 11}, {'name': '185', 'value': 12}, {'name': '186', 'value': 11}, {'name': '187', 'value': 11}, {'name': '188', 'value': 11}, {'name': '189', 'value': 12}, {'name': '190', 'value': 12}, {'name': '191', 'value': 12}, {'name': '192', 'value': 12}, {'name': '193', 'value': 12}, {'name': '194', 'value': 12}, {'name': '195', 'value': 12}, {'name': '196', 'value': 11}, {'name': '197', 'value': 11}, {'name': '198', 'value': 11}, {'name': '199', 'value': 12}, {'name': '200', 'value': 12}, {'name': '201', 'value': 11}, {'name': '202', 'value': 12}, {'name': '203', 'value': 12}, {'name': '204', 'value': 11}, {'name': '205', 'value': 11}, {'name': '206', 'value': 11}, {'name': '207', 'value': 11}, {'name': '208', 'value': 12}, {'name': '209', 'value': 11}, {'name': '210', 'value': 11}, {'name': '211', 'value': 12}, {'name': '212', 'value': 12}, {'name': '213', 'value': 11}, {'name': '214', 'value': 11}, {'name': '215', 'value': 12}, {'name': '216', 'value': 11}, {'name': '217', 'value': 12}, {'name': '218', 'value': 12}, {'name': '219', 'value': 11}, {'name': '220', 'value': 11}, {'name': '221', 'value': 11}, {'name': '222', 'value': 11}, {'name': '223', 'value': 12}, {'name': '224', 'value': 12}, {'name': '225', 'value': 11}, {'name': '226', 'value': 12}, {'name': '227', 'value': 11}, {'name': '228', 'value': 11}, {'name': '229', 'value': 11}, {'name': '230', 'value': 11}, {'name': '231', 'value': 11}, {'name': '232', 'value': 12}, {'name': '233', 'value': 11}, {'name': '234', 'value': 11}, {'name': '235', 'value': 12}, {'name': '236', 'value': 11}, {'name': '237', 'value': 12}, {'name': '238', 'value': 11}, {'name': '239', 'value': 11}, {'name': '240', 'value': 11}, {'name': '241', 'value': 12}, {'name': '242', 'value': 12}, {'name': '243', 'value': 12}, {'name': '244', 'value': 11}, {'name': '245', 'value': 11}, {'name': '246', 'value': 11}, {'name': '247', 'value': 11}, {'name': '248', 'value': 11}, {'name': '249', 'value': 12}, {'name': '250', 'value': 11}, {'name': '251', 'value': 11}, {'name': '252', 'value': 11}, {'name': '253', 'value': 11}, {'name': '254', 'value': 11}, {'name': '255', 'value': 12}, {'name': '256', 'value': 11}, {'name': '257', 'value': 12}, {'name': '258', 'value': 11}, {'name': '259', 'value': 12}, {'name': '260', 'value': 11}, {'name': '261', 'value': 12}, {'name': '262', 'value': 11}, {'name': '263', 'value': 11}, {'name': '264', 'value': 12}, {'name': '265', 'value': 12}, {'name': '266', 'value': 11}, {'name': '267', 'value': 12}, {'name': '268', 'value': 11}, {'name': '269', 'value': 12}, {'name': '270', 'value': 11}, {'name': '271', 'value': 12}, {'name': '272', 'value': 11}, {'name': '273', 'value': 12}, {'name': '274', 'value': 12}, {'name': '275', 'value': 12}, {'name': '276', 'value': 11}, {'name': '277', 'value': 11}, {'name': '278', 'value': 11}, {'name': '279', 'value': 12}, {'name': '280', 'value': 12}, {'name': '281', 'value': 12}, {'name': '282', 'value': 12}, {'name': '283', 'value': 11}, {'name': '284', 'value': 11}, {'name': '285', 'value': 12}, {'name': '286', 'value': 12}, {'name': '287', 'value': 11}, {'name': '288', 'value': 11}, {'name': '289', 'value': 11}, {'name': '290', 'value': 11}, {'name': '291', 'value': 11}, {'name': '292', 'value': 11}, {'name': '293', 'value': 12}, {'name': '294', 'value': 11}, {'name': '295', 'value': 12}, {'name': '296', 'value': 12}, {'name': '297', 'value': 12}, {'name': '298', 'value': 12}, {'name': '299', 'value': 12}, {'name': '300', 'value': 12}, {'name': '301', 'value': 12}, {'name': '302', 'value': 12}, {'name': '303', 'value': 12}, {'name': '304', 'value': 12}, {'name': '305', 'value': 12}, {'name': '306', 'value': 11}, {'name': '307', 'value': 12}, {'name': '308', 'value': 11}, {'name': '309', 'value': 12}, {'name': '310', 'value': 12}, {'name': '311', 'value': 12}, {'name': '312', 'value': 12}, {'name': '313', 'value': 11}, {'name': '314', 'value': 11}, {'name': '315', 'value': 12}, {'name': '316', 'value': 11}, {'name': '317', 'value': 12}, {'name': '318', 'value': 11}, {'name': '319', 'value': 12}, {'name': '320', 'value': 11}, {'name': '321', 'value': 11}, {'name': '322', 'value': 12}, {'name': '323', 'value': 12}, {'name': '324', 'value': 11}, {'name': '325', 'value': 11}, {'name': '326', 'value': 12}, {'name': '327', 'value': 11}, {'name': '328', 'value': 12}, {'name': '329', 'value': 11}, {'name': '330', 'value': 11}, {'name': '331', 'value': 11}, {'name': '332', 'value': 12}, {'name': '333', 'value': 12}, {'name': '334', 'value': 11}, {'name': '335', 'value': 11}, {'name': '336', 'value': 11}, {'name': '337', 'value': 11}, {'name': '338', 'value': 12}, {'name': '339', 'value': 11}, {'name': '340', 'value': 11}, {'name': '341', 'value': 11}, {'name': '342', 'value': 11}, {'name': '343', 'value': 12}, {'name': '344', 'value': 11}, {'name': '345', 'value': 11}, {'name': '346', 'value': 11}, {'name': '347', 'value': 11}, {'name': '348', 'value': 11}, {'name': '349', 'value': 12}, {'name': '350', 'value': 11}, {'name': '351', 'value': 12}, {'name': '352', 'value': 12}, {'name': '353', 'value': 12}, {'name': '354', 'value': 11}, {'name': '355', 'value': 12}, {'name': '356', 'value': 11}, {'name': '357', 'value': 11}, {'name': '358', 'value': 12}, {'name': '359', 'value': 11}, {'name': '360', 'value': 11}, {'name': '361', 'value': 11}, {'name': '362', 'value': 12}, {'name': '363', 'value': 12}, {'name': '364', 'value': 11}, {'name': '365', 'value': 12}, {'name': '366', 'value': 12}, {'name': '367', 'value': 12}, {'name': '368', 'value': 11}, {'name': '369', 'value': 11}, {'name': '370', 'value': 11}, {'name': '371', 'value': 12}, {'name': '372', 'value': 11}, {'name': '373', 'value': 12}, {'name': '374', 'value': 12}, {'name': '375', 'value': 12}, {'name': '376', 'value': 11}, {'name': '377', 'value': 11}, {'name': '378', 'value': 12}, {'name': '379', 'value': 11}, {'name': '380', 'value': 11}, {'name': '381', 'value': 11}, {'name': '382', 'value': 12}, {'name': '383', 'value': 12}, {'name': '384', 'value': 11}, {'name': '385', 'value': 11}, {'name': '386', 'value': 12}, {'name': '387', 'value': 11}, {'name': '388', 'value': 11}, {'name': '389', 'value': 11}, {'name': '390', 'value': 11}, {'name': '391', 'value': 12}, {'name': '392', 'value': 11}, {'name': '393', 'value': 11}, {'name': '394', 'value': 12}, {'name': '395', 'value': 11}, {'name': '396', 'value': 12}, {'name': '397', 'value': 11}, {'name': '398', 'value': 11}, {'name': '399', 'value': 12}, {'name': '400', 'value': 11}, {'name': '401', 'value': 11}, {'name': '402', 'value': 11}, {'name': '403', 'value': 12}, {'name': '404', 'value': 11}, {'name': '405', 'value': 11}, {'name': '406', 'value': 11}, {'name': '407', 'value': 12}, {'name': '408', 'value': 12}, {'name': '409', 'value': 12}, {'name': '410', 'value': 12}, {'name': '411', 'value': 11}, {'name': '412', 'value': 11}, {'name': '413', 'value': 11}, {'name': '414', 'value': 11}, {'name': '415', 'value': 11}, {'name': '416', 'value': 12}, {'name': '417', 'value': 11}, {'name': '418', 'value': 11}, {'name': '419', 'value': 11}, {'name': '420', 'value': 12}, {'name': '421', 'value': 11}, {'name': '422', 'value': 11}, {'name': '423', 'value': 12}, {'name': '424', 'value': 11}, {'name': '425', 'value': 11}, {'name': '426', 'value': 11}, {'name': '427', 'value': 11}, {'name': '428', 'value': 12}, {'name': '429', 'value': 11}, {'name': '430', 'value': 11}, {'name': '431', 'value': 12}, {'name': '432', 'value': 12}, {'name': '433', 'value': 12}, {'name': '434', 'value': 11}, {'name': '435', 'value': 12}, {'name': '436', 'value': 11}, {'name': '437', 'value': 11}, {'name': '438', 'value': 11}, {'name': '439', 'value': 12}, {'name': '440', 'value': 11}, {'name': '441', 'value': 11}, {'name': '442', 'value': 11}, {'name': '443', 'value': 12}, {'name': '444', 'value': 11}, {'name': '445', 'value': 11}, {'name': '446', 'value': 11}, {'name': '447', 'value': 11}, {'name': '448', 'value': 12}, {'name': '449', 'value': 11}, {'name': '450', 'value': 12}, {'name': '451', 'value': 11}, {'name': '452', 'value': 11}, {'name': '453', 'value': 11}, {'name': '454', 'value': 11}, {'name': '455', 'value': 12}, {'name': '456', 'value': 11}, {'name': '457', 'value': 12}, {'name': '458', 'value': 12}, {'name': '459', 'value': 11}, {'name': '460', 'value': 11}, {'name': '461', 'value': 11}, {'name': '462', 'value': 12}, {'name': '463', 'value': 12}, {'name': '464', 'value': 12}, {'name': '465', 'value': 12}, {'name': '466', 'value': 11}, {'name': '467', 'value': 11}, {'name': '468', 'value': 11}, {'name': '469', 'value': 11}, {'name': '470', 'value': 12}, {'name': '471', 'value': 12}, {'name': '472', 'value': 11}, {'name': '473', 'value': 12}, {'name': '474', 'value': 12}, {'name': '475', 'value': 11}, {'name': '476', 'value': 11}, {'name': '477', 'value': 11}, {'name': '478', 'value': 12}, {'name': '479', 'value': 11}, {'name': '480', 'value': 12}, {'name': '481', 'value': 11}, {'name': '482', 'value': 11}, {'name': '483', 'value': 12}, {'name': '484', 'value': 11}, {'name': '485', 'value': 11}, {'name': '486', 'value': 11}, {'name': '487', 'value': 12}, {'name': '488', 'value': 12}, {'name': '489', 'value': 12}, {'name': '490', 'value': 11}, {'name': '491', 'value': 11}, {'name': '492', 'value': 12}, {'name': '493', 'value': 11}, {'name': '494', 'value': 12}, {'name': '495', 'value': 12}, {'name': '496', 'value': 11}, {'name': '497', 'value': 11}, {'name': '498', 'value': 12}, {'name': '499', 'value': 11}, {'name': '500', 'value': 12}]}, 'DateofService_year': {'name': 'DateofService_year', 'series': [{'name': '1', 'value': 2020}, {'name': '2', 'value': 2020}, {'name': '3', 'value': 2020}, {'name': '4', 'value': 2020}, {'name': '5', 'value': 2020}, {'name': '6', 'value': 2020}, {'name': '7', 'value': 2020}, {'name': '8', 'value': 2020}, {'name': '9', 'value': 2020}, {'name': '10', 'value': 2020}, {'name': '11', 'value': 2020}, {'name': '12', 'value': 2020}, {'name': '13', 'value': 2020}, {'name': '14', 'value': 2020}, {'name': '15', 'value': 2020}, {'name': '16', 'value': 2020}, {'name': '17', 'value': 2020}, {'name': '18', 'value': 2020}, {'name': '19', 'value': 2020}, {'name': '20', 'value': 2020}, {'name': '21', 'value': 2020}, {'name': '22', 'value': 2020}, {'name': '23', 'value': 2020}, {'name': '24', 'value': 2020}, {'name': '25', 'value': 2020}, {'name': '26', 'value': 2020}, {'name': '27', 'value': 2020}, {'name': '28', 'value': 2020}, {'name': '29', 'value': 2020}, {'name': '30', 'value': 2020}, {'name': '31', 'value': 2020}, {'name': '32', 'value': 2020}, {'name': '33', 'value': 2020}, {'name': '34', 'value': 2020}, {'name': '35', 'value': 2020}, {'name': '36', 'value': 2020}, {'name': '37', 'value': 2020}, {'name': '38', 'value': 2020}, {'name': '39', 'value': 2020}, {'name': '40', 'value': 2020}, {'name': '41', 'value': 2020}, {'name': '42', 'value': 2020}, {'name': '43', 'value': 2020}, {'name': '44', 'value': 2020}, {'name': '45', 'value': 2020}, {'name': '46', 'value': 2020}, {'name': '47', 'value': 2020}, {'name': '48', 'value': 2020}, {'name': '49', 'value': 2020}, {'name': '50', 'value': 2020}, {'name': '51', 'value': 2020}, {'name': '52', 'value': 2020}, {'name': '53', 'value': 2020}, {'name': '54', 'value': 2020}, {'name': '55', 'value': 2020}, {'name': '56', 'value': 2020}, {'name': '57', 'value': 2020}, {'name': '58', 'value': 2020}, {'name': '59', 'value': 2020}, {'name': '60', 'value': 2020}, {'name': '61', 'value': 2020}, {'name': '62', 'value': 2020}, {'name': '63', 'value': 2020}, {'name': '64', 'value': 2020}, {'name': '65', 'value': 2020}, {'name': '66', 'value': 2020}, {'name': '67', 'value': 2020}, {'name': '68', 'value': 2020}, {'name': '69', 'value': 2020}, {'name': '70', 'value': 2020}, {'name': '71', 'value': 2020}, {'name': '72', 'value': 2020}, {'name': '73', 'value': 2020}, {'name': '74', 'value': 2020}, {'name': '75', 'value': 2020}, {'name': '76', 'value': 2020}, {'name': '77', 'value': 2020}, {'name': '78', 'value': 2020}, {'name': '79', 'value': 2020}, {'name': '80', 'value': 2020}, {'name': '81', 'value': 2020}, {'name': '82', 'value': 2020}, {'name': '83', 'value': 2020}, {'name': '84', 'value': 2020}, {'name': '85', 'value': 2020}, {'name': '86', 'value': 2020}, {'name': '87', 'value': 2020}, {'name': '88', 'value': 2020}, {'name': '89', 'value': 2020}, {'name': '90', 'value': 2020}, {'name': '91', 'value': 2020}, {'name': '92', 'value': 2020}, {'name': '93', 'value': 2020}, {'name': '94', 'value': 2020}, {'name': '95', 'value': 2020}, {'name': '96', 'value': 2020}, {'name': '97', 'value': 2020}, {'name': '98', 'value': 2020}, {'name': '99', 'value': 2020}, {'name': '100', 'value': 2020}, {'name': '101', 'value': 2020}, {'name': '102', 'value': 2020}, {'name': '103', 'value': 2020}, {'name': '104', 'value': 2020}, {'name': '105', 'value': 2020}, {'name': '106', 'value': 2020}, {'name': '107', 'value': 2020}, {'name': '108', 'value': 2020}, {'name': '109', 'value': 2020}, {'name': '110', 'value': 2020}, {'name': '111', 'value': 2020}, {'name': '112', 'value': 2020}, {'name': '113', 'value': 2020}, {'name': '114', 'value': 2020}, {'name': '115', 'value': 2020}, {'name': '116', 'value': 2020}, {'name': '117', 'value': 2020}, {'name': '118', 'value': 2020}, {'name': '119', 'value': 2020}, {'name': '120', 'value': 2020}, {'name': '121', 'value': 2020}, {'name': '122', 'value': 2020}, {'name': '123', 'value': 2020}, {'name': '124', 'value': 2020}, {'name': '125', 'value': 2020}, {'name': '126', 'value': 2020}, {'name': '127', 'value': 2020}, {'name': '128', 'value': 2020}, {'name': '129', 'value': 2020}, {'name': '130', 'value': 2020}, {'name': '131', 'value': 2020}, {'name': '132', 'value': 2020}, {'name': '133', 'value': 2020}, {'name': '134', 'value': 2020}, {'name': '135', 'value': 2020}, {'name': '136', 'value': 2020}, {'name': '137', 'value': 2020}, {'name': '138', 'value': 2020}, {'name': '139', 'value': 2020}, {'name': '140', 'value': 2020}, {'name': '141', 'value': 2020}, {'name': '142', 'value': 2020}, {'name': '143', 'value': 2020}, {'name': '144', 'value': 2020}, {'name': '145', 'value': 2020}, {'name': '146', 'value': 2020}, {'name': '147', 'value': 2020}, {'name': '148', 'value': 2020}, {'name': '149', 'value': 2020}, {'name': '150', 'value': 2020}, {'name': '151', 'value': 2020}, {'name': '152', 'value': 2020}, {'name': '153', 'value': 2020}, {'name': '154', 'value': 2020}, {'name': '155', 'value': 2020}, {'name': '156', 'value': 2020}, {'name': '157', 'value': 2020}, {'name': '158', 'value': 2020}, {'name': '159', 'value': 2020}, {'name': '160', 'value': 2020}, {'name': '161', 'value': 2020}, {'name': '162', 'value': 2020}, {'name': '163', 'value': 2020}, {'name': '164', 'value': 2020}, {'name': '165', 'value': 2020}, {'name': '166', 'value': 2020}, {'name': '167', 'value': 2020}, {'name': '168', 'value': 2020}, {'name': '169', 'value': 2020}, {'name': '170', 'value': 2020}, {'name': '171', 'value': 2020}, {'name': '172', 'value': 2020}, {'name': '173', 'value': 2020}, {'name': '174', 'value': 2020}, {'name': '175', 'value': 2020}, {'name': '176', 'value': 2020}, {'name': '177', 'value': 2020}, {'name': '178', 'value': 2020}, {'name': '179', 'value': 2020}, {'name': '180', 'value': 2020}, {'name': '181', 'value': 2020}, {'name': '182', 'value': 2020}, {'name': '183', 'value': 2020}, {'name': '184', 'value': 2020}, {'name': '185', 'value': 2020}, {'name': '186', 'value': 2020}, {'name': '187', 'value': 2020}, {'name': '188', 'value': 2020}, {'name': '189', 'value': 2020}, {'name': '190', 'value': 2020}, {'name': '191', 'value': 2020}, {'name': '192', 'value': 2020}, {'name': '193', 'value': 2020}, {'name': '194', 'value': 2020}, {'name': '195', 'value': 2020}, {'name': '196', 'value': 2020}, {'name': '197', 'value': 2020}, {'name': '198', 'value': 2020}, {'name': '199', 'value': 2020}, {'name': '200', 'value': 2020}, {'name': '201', 'value': 2020}, {'name': '202', 'value': 2020}, {'name': '203', 'value': 2020}, {'name': '204', 'value': 2020}, {'name': '205', 'value': 2020}, {'name': '206', 'value': 2020}, {'name': '207', 'value': 2020}, {'name': '208', 'value': 2020}, {'name': '209', 'value': 2020}, {'name': '210', 'value': 2020}, {'name': '211', 'value': 2020}, {'name': '212', 'value': 2020}, {'name': '213', 'value': 2020}, {'name': '214', 'value': 2020}, {'name': '215', 'value': 2020}, {'name': '216', 'value': 2020}, {'name': '217', 'value': 2020}, {'name': '218', 'value': 2020}, {'name': '219', 'value': 2020}, {'name': '220', 'value': 2020}, {'name': '221', 'value': 2020}, {'name': '222', 'value': 2020}, {'name': '223', 'value': 2020}, {'name': '224', 'value': 2020}, {'name': '225', 'value': 2020}, {'name': '226', 'value': 2020}, {'name': '227', 'value': 2020}, {'name': '228', 'value': 2020}, {'name': '229', 'value': 2020}, {'name': '230', 'value': 2020}, {'name': '231', 'value': 2020}, {'name': '232', 'value': 2020}, {'name': '233', 'value': 2020}, {'name': '234', 'value': 2020}, {'name': '235', 'value': 2020}, {'name': '236', 'value': 2020}, {'name': '237', 'value': 2020}, {'name': '238', 'value': 2020}, {'name': '239', 'value': 2020}, {'name': '240', 'value': 2020}, {'name': '241', 'value': 2020}, {'name': '242', 'value': 2020}, {'name': '243', 'value': 2020}, {'name': '244', 'value': 2020}, {'name': '245', 'value': 2020}, {'name': '246', 'value': 2020}, {'name': '247', 'value': 2020}, {'name': '248', 'value': 2020}, {'name': '249', 'value': 2020}, {'name': '250', 'value': 2020}, {'name': '251', 'value': 2020}, {'name': '252', 'value': 2020}, {'name': '253', 'value': 2020}, {'name': '254', 'value': 2020}, {'name': '255', 'value': 2020}, {'name': '256', 'value': 2020}, {'name': '257', 'value': 2020}, {'name': '258', 'value': 2020}, {'name': '259', 'value': 2020}, {'name': '260', 'value': 2020}, {'name': '261', 'value': 2020}, {'name': '262', 'value': 2020}, {'name': '263', 'value': 2020}, {'name': '264', 'value': 2020}, {'name': '265', 'value': 2020}, {'name': '266', 'value': 2020}, {'name': '267', 'value': 2020}, {'name': '268', 'value': 2020}, {'name': '269', 'value': 2020}, {'name': '270', 'value': 2020}, {'name': '271', 'value': 2020}, {'name': '272', 'value': 2020}, {'name': '273', 'value': 2020}, {'name': '274', 'value': 2020}, {'name': '275', 'value': 2020}, {'name': '276', 'value': 2020}, {'name': '277', 'value': 2020}, {'name': '278', 'value': 2020}, {'name': '279', 'value': 2020}, {'name': '280', 'value': 2020}, {'name': '281', 'value': 2020}, {'name': '282', 'value': 2020}, {'name': '283', 'value': 2020}, {'name': '284', 'value': 2020}, {'name': '285', 'value': 2020}, {'name': '286', 'value': 2020}, {'name': '287', 'value': 2020}, {'name': '288', 'value': 2020}, {'name': '289', 'value': 2020}, {'name': '290', 'value': 2020}, {'name': '291', 'value': 2020}, {'name': '292', 'value': 2020}, {'name': '293', 'value': 2020}, {'name': '294', 'value': 2020}, {'name': '295', 'value': 2020}, {'name': '296', 'value': 2020}, {'name': '297', 'value': 2020}, {'name': '298', 'value': 2020}, {'name': '299', 'value': 2020}, {'name': '300', 'value': 2020}, {'name': '301', 'value': 2020}, {'name': '302', 'value': 2020}, {'name': '303', 'value': 2020}, {'name': '304', 'value': 2020}, {'name': '305', 'value': 2020}, {'name': '306', 'value': 2020}, {'name': '307', 'value': 2020}, {'name': '308', 'value': 2020}, {'name': '309', 'value': 2020}, {'name': '310', 'value': 2020}, {'name': '311', 'value': 2020}, {'name': '312', 'value': 2020}, {'name': '313', 'value': 2020}, {'name': '314', 'value': 2020}, {'name': '315', 'value': 2020}, {'name': '316', 'value': 2020}, {'name': '317', 'value': 2020}, {'name': '318', 'value': 2020}, {'name': '319', 'value': 2020}, {'name': '320', 'value': 2020}, {'name': '321', 'value': 2020}, {'name': '322', 'value': 2020}, {'name': '323', 'value': 2020}, {'name': '324', 'value': 2020}, {'name': '325', 'value': 2020}, {'name': '326', 'value': 2020}, {'name': '327', 'value': 2020}, {'name': '328', 'value': 2020}, {'name': '329', 'value': 2020}, {'name': '330', 'value': 2020}, {'name': '331', 'value': 2020}, {'name': '332', 'value': 2020}, {'name': '333', 'value': 2020}, {'name': '334', 'value': 2020}, {'name': '335', 'value': 2020}, {'name': '336', 'value': 2020}, {'name': '337', 'value': 2020}, {'name': '338', 'value': 2020}, {'name': '339', 'value': 2020}, {'name': '340', 'value': 2020}, {'name': '341', 'value': 2020}, {'name': '342', 'value': 2020}, {'name': '343', 'value': 2020}, {'name': '344', 'value': 2020}, {'name': '345', 'value': 2020}, {'name': '346', 'value': 2020}, {'name': '347', 'value': 2020}, {'name': '348', 'value': 2020}, {'name': '349', 'value': 2020}, {'name': '350', 'value': 2020}, {'name': '351', 'value': 2020}, {'name': '352', 'value': 2020}, {'name': '353', 'value': 2020}, {'name': '354', 'value': 2020}, {'name': '355', 'value': 2020}, {'name': '356', 'value': 2020}, {'name': '357', 'value': 2020}, {'name': '358', 'value': 2020}, {'name': '359', 'value': 2020}, {'name': '360', 'value': 2020}, {'name': '361', 'value': 2020}, {'name': '362', 'value': 2020}, {'name': '363', 'value': 2020}, {'name': '364', 'value': 2020}, {'name': '365', 'value': 2020}, {'name': '366', 'value': 2020}, {'name': '367', 'value': 2020}, {'name': '368', 'value': 2020}, {'name': '369', 'value': 2020}, {'name': '370', 'value': 2020}, {'name': '371', 'value': 2020}, {'name': '372', 'value': 2020}, {'name': '373', 'value': 2020}, {'name': '374', 'value': 2020}, {'name': '375', 'value': 2020}, {'name': '376', 'value': 2020}, {'name': '377', 'value': 2020}, {'name': '378', 'value': 2020}, {'name': '379', 'value': 2020}, {'name': '380', 'value': 2020}, {'name': '381', 'value': 2020}, {'name': '382', 'value': 2020}, {'name': '383', 'value': 2020}, {'name': '384', 'value': 2020}, {'name': '385', 'value': 2020}, {'name': '386', 'value': 2020}, {'name': '387', 'value': 2020}, {'name': '388', 'value': 2020}, {'name': '389', 'value': 2020}, {'name': '390', 'value': 2020}, {'name': '391', 'value': 2020}, {'name': '392', 'value': 2020}, {'name': '393', 'value': 2020}, {'name': '394', 'value': 2020}, {'name': '395', 'value': 2020}, {'name': '396', 'value': 2020}, {'name': '397', 'value': 2020}, {'name': '398', 'value': 2020}, {'name': '399', 'value': 2020}, {'name': '400', 'value': 2020}, {'name': '401', 'value': 2020}, {'name': '402', 'value': 2020}, {'name': '403', 'value': 2020}, {'name': '404', 'value': 2020}, {'name': '405', 'value': 2020}, {'name': '406', 'value': 2020}, {'name': '407', 'value': 2020}, {'name': '408', 'value': 2020}, {'name': '409', 'value': 2020}, {'name': '410', 'value': 2020}, {'name': '411', 'value': 2020}, {'name': '412', 'value': 2020}, {'name': '413', 'value': 2020}, {'name': '414', 'value': 2020}, {'name': '415', 'value': 2020}, {'name': '416', 'value': 2020}, {'name': '417', 'value': 2020}, {'name': '418', 'value': 2020}, {'name': '419', 'value': 2020}, {'name': '420', 'value': 2020}, {'name': '421', 'value': 2020}, {'name': '422', 'value': 2020}, {'name': '423', 'value': 2020}, {'name': '424', 'value': 2020}, {'name': '425', 'value': 2020}, {'name': '426', 'value': 2020}, {'name': '427', 'value': 2020}, {'name': '428', 'value': 2020}, {'name': '429', 'value': 2020}, {'name': '430', 'value': 2020}, {'name': '431', 'value': 2020}, {'name': '432', 'value': 2020}, {'name': '433', 'value': 2020}, {'name': '434', 'value': 2020}, {'name': '435', 'value': 2020}, {'name': '436', 'value': 2020}, {'name': '437', 'value': 2020}, {'name': '438', 'value': 2020}, {'name': '439', 'value': 2020}, {'name': '440', 'value': 2020}, {'name': '441', 'value': 2020}, {'name': '442', 'value': 2020}, {'name': '443', 'value': 2020}, {'name': '444', 'value': 2020}, {'name': '445', 'value': 2020}, {'name': '446', 'value': 2020}, {'name': '447', 'value': 2020}, {'name': '448', 'value': 2020}, {'name': '449', 'value': 2020}, {'name': '450', 'value': 2020}, {'name': '451', 'value': 2020}, {'name': '452', 'value': 2020}, {'name': '453', 'value': 2020}, {'name': '454', 'value': 2020}, {'name': '455', 'value': 2020}, {'name': '456', 'value': 2020}, {'name': '457', 'value': 2020}, {'name': '458', 'value': 2020}, {'name': '459', 'value': 2020}, {'name': '460', 'value': 2020}, {'name': '461', 'value': 2020}, {'name': '462', 'value': 2020}, {'name': '463', 'value': 2020}, {'name': '464', 'value': 2020}, {'name': '465', 'value': 2020}, {'name': '466', 'value': 2020}, {'name': '467', 'value': 2020}, {'name': '468', 'value': 2020}, {'name': '469', 'value': 2020}, {'name': '470', 'value': 2020}, {'name': '471', 'value': 2020}, {'name': '472', 'value': 2020}, {'name': '473', 'value': 2020}, {'name': '474', 'value': 2020}, {'name': '475', 'value': 2020}, {'name': '476', 'value': 2020}, {'name': '477', 'value': 2020}, {'name': '478', 'value': 2020}, {'name': '479', 'value': 2020}, {'name': '480', 'value': 2020}, {'name': '481', 'value': 2020}, {'name': '482', 'value': 2020}, {'name': '483', 'value': 2020}, {'name': '484', 'value': 2020}, {'name': '485', 'value': 2020}, {'name': '486', 'value': 2020}, {'name': '487', 'value': 2020}, {'name': '488', 'value': 2020}, {'name': '489', 'value': 2020}, {'name': '490', 'value': 2020}, {'name': '491', 'value': 2020}, {'name': '492', 'value': 2020}, {'name': '493', 'value': 2020}, {'name': '494', 'value': 2020}, {'name': '495', 'value': 2020}, {'name': '496', 'value': 2020}, {'name': '497', 'value': 2020}, {'name': '498', 'value': 2020}, {'name': '499', 'value': 2020}, {'name': '500', 'value': 2020}]}, 'CPTCode_stringindexer_transform': {'name': 'CPTCode_stringindexer_transform', 'series': [{'name': '1', 'value': 1}, {'name': '2', 'value': 1}, {'name': '3', 'value': 1}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 1}, {'name': '7', 'value': 0}, {'name': '8', 'value': 2}, {'name': '9', 'value': 1}, {'name': '10', 'value': 1}, {'name': '11', 'value': 1}, {'name': '12', 'value': 4}, {'name': '13', 'value': 2}, {'name': '14', 'value': 0}, {'name': '15', 'value': 2}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 3}, {'name': '19', 'value': 0}, {'name': '20', 'value': 4}, {'name': '21', 'value': 15}, {'name': '22', 'value': 2}, {'name': '23', 'value': 1}, {'name': '24', 'value': 4}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 2}, {'name': '28', 'value': 12}, {'name': '29', 'value': 4}, {'name': '30', 'value': 1}, {'name': '31', 'value': 1}, {'name': '32', 'value': 0}, {'name': '33', 'value': 1}, {'name': '34', 'value': 1}, {'name': '35', 'value': 5}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 9}, {'name': '39', 'value': 2}, {'name': '40', 'value': 0}, {'name': '41', 'value': 1}, {'name': '42', 'value': 1}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 3}, {'name': '46', 'value': 0}, {'name': '47', 'value': 5}, {'name': '48', 'value': 1}, {'name': '49', 'value': 0}, {'name': '50', 'value': 1}, {'name': '51', 'value': 0}, {'name': '52', 'value': 1}, {'name': '53', 'value': 1}, {'name': '54', 'value': 0}, {'name': '55', 'value': 4}, {'name': '56', 'value': 0}, {'name': '57', 'value': 2}, {'name': '58', 'value': 8}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 5}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 1}, {'name': '65', 'value': 0}, {'name': '66', 'value': 1}, {'name': '67', 'value': 1}, {'name': '68', 'value': 1}, {'name': '69', 'value': 1}, {'name': '70', 'value': 0}, {'name': '71', 'value': 3}, {'name': '72', 'value': 0}, {'name': '73', 'value': 3}, {'name': '74', 'value': 1}, {'name': '75', 'value': 0}, {'name': '76', 'value': 1}, {'name': '77', 'value': 0}, {'name': '78', 'value': 3}, {'name': '79', 'value': 1}, {'name': '80', 'value': 1}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 11}, {'name': '84', 'value': 0}, {'name': '85', 'value': 1}, {'name': '86', 'value': 1}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 5}, {'name': '90', 'value': 1}, {'name': '91', 'value': 0}, {'name': '92', 'value': 10}, {'name': '93', 'value': 3}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 13}, {'name': '97', 'value': 1}, {'name': '98', 'value': 0}, {'name': '99', 'value': 10}, {'name': '100', 'value': 0}, {'name': '101', 'value': 1}, {'name': '102', 'value': 2}, {'name': '103', 'value': 1}, {'name': '104', 'value': 1}, {'name': '105', 'value': 1}, {'name': '106', 'value': 0}, {'name': '107', 'value': 1}, {'name': '108', 'value': 1}, {'name': '109', 'value': 0}, {'name': '110', 'value': 1}, {'name': '111', 'value': 14}, {'name': '112', 'value': 2}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 2}, {'name': '116', 'value': 0}, {'name': '117', 'value': 2}, {'name': '118', 'value': 1}, {'name': '119', 'value': 1}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 8}, {'name': '123', 'value': 1}, {'name': '124', 'value': 0}, {'name': '125', 'value': 2}, {'name': '126', 'value': 9}, {'name': '127', 'value': 4}, {'name': '128', 'value': 11}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 2}, {'name': '132', 'value': 4}, {'name': '133', 'value': 1}, {'name': '134', 'value': 8}, {'name': '135', 'value': 0}, {'name': '136', 'value': 1}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 1}, {'name': '140', 'value': 1}, {'name': '141', 'value': 11}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 6}, {'name': '148', 'value': 1}, {'name': '149', 'value': 1}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 1}, {'name': '157', 'value': 9}, {'name': '158', 'value': 0}, {'name': '159', 'value': 1}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 1}, {'name': '163', 'value': 0}, {'name': '164', 'value': 11}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 9}, {'name': '169', 'value': 7}, {'name': '170', 'value': 7}, {'name': '171', 'value': 1}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 2}, {'name': '175', 'value': 1}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 2}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 3}, {'name': '182', 'value': 1}, {'name': '183', 'value': 6}, {'name': '184', 'value': 3}, {'name': '185', 'value': 7}, {'name': '186', 'value': 0}, {'name': '187', 'value': 1}, {'name': '188', 'value': 1}, {'name': '189', 'value': 0}, {'name': '190', 'value': 1}, {'name': '191', 'value': 0}, {'name': '192', 'value': 1}, {'name': '193', 'value': 0}, {'name': '194', 'value': 1}, {'name': '195', 'value': 0}, {'name': '196', 'value': 1}, {'name': '197', 'value': 1}, {'name': '198', 'value': 1}, {'name': '199', 'value': 3}, {'name': '200', 'value': 0}, {'name': '201', 'value': 1}, {'name': '202', 'value': 0}, {'name': '203', 'value': 1}, {'name': '204', 'value': 4}, {'name': '205', 'value': 2}, {'name': '206', 'value': 0}, {'name': '207', 'value': 20}, {'name': '208', 'value': 0}, {'name': '209', 'value': 2}, {'name': '210', 'value': 0}, {'name': '211', 'value': 1}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 1}, {'name': '216', 'value': 2}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 6}, {'name': '222', 'value': 3}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 22}, {'name': '226', 'value': 1}, {'name': '227', 'value': 1}, {'name': '228', 'value': 1}, {'name': '229', 'value': 7}, {'name': '230', 'value': 1}, {'name': '231', 'value': 0}, {'name': '232', 'value': 3}, {'name': '233', 'value': 3}, {'name': '234', 'value': 19}, {'name': '235', 'value': 1}, {'name': '236', 'value': 0}, {'name': '237', 'value': 2}, {'name': '238', 'value': 6}, {'name': '239', 'value': 2}, {'name': '240', 'value': 0}, {'name': '241', 'value': 1}, {'name': '242', 'value': 1}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 2}, {'name': '246', 'value': 2}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 1}, {'name': '250', 'value': 3}, {'name': '251', 'value': 1}, {'name': '252', 'value': 1}, {'name': '253', 'value': 1}, {'name': '254', 'value': 8}, {'name': '255', 'value': 3}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 6}, {'name': '260', 'value': 1}, {'name': '261', 'value': 0}, {'name': '262', 'value': 1}, {'name': '263', 'value': 1}, {'name': '264', 'value': 2}, {'name': '265', 'value': 4}, {'name': '266', 'value': 1}, {'name': '267', 'value': 1}, {'name': '268', 'value': 1}, {'name': '269', 'value': 1}, {'name': '270', 'value': 2}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 2}, {'name': '275', 'value': 6}, {'name': '276', 'value': 1}, {'name': '277', 'value': 0}, {'name': '278', 'value': 1}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 1}, {'name': '283', 'value': 1}, {'name': '284', 'value': 1}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 1}, {'name': '288', 'value': 1}, {'name': '289', 'value': 2}, {'name': '290', 'value': 7}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 5}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 6}, {'name': '298', 'value': 3}, {'name': '299', 'value': 8}, {'name': '300', 'value': 1}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 4}, {'name': '305', 'value': 1}, {'name': '306', 'value': 4}, {'name': '307', 'value': 1}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 11}, {'name': '311', 'value': 1}, {'name': '312', 'value': 0}, {'name': '313', 'value': 1}, {'name': '314', 'value': 1}, {'name': '315', 'value': 3}, {'name': '316', 'value': 1}, {'name': '317', 'value': 0}, {'name': '318', 'value': 1}, {'name': '319', 'value': 4}, {'name': '320', 'value': 1}, {'name': '321', 'value': 4}, {'name': '322', 'value': 3}, {'name': '323', 'value': 3}, {'name': '324', 'value': 1}, {'name': '325', 'value': 16}, {'name': '326', 'value': 1}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 5}, {'name': '330', 'value': 1}, {'name': '331', 'value': 2}, {'name': '332', 'value': 1}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 1}, {'name': '336', 'value': 2}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 1}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 1}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 1}, {'name': '346', 'value': 4}, {'name': '347', 'value': 2}, {'name': '348', 'value': 0}, {'name': '349', 'value': 1}, {'name': '350', 'value': 1}, {'name': '351', 'value': 5}, {'name': '352', 'value': 0}, {'name': '353', 'value': 1}, {'name': '354', 'value': 4}, {'name': '355', 'value': 12}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 6}, {'name': '359', 'value': 0}, {'name': '360', 'value': 1}, {'name': '361', 'value': 1}, {'name': '362', 'value': 1}, {'name': '363', 'value': 0}, {'name': '364', 'value': 10}, {'name': '365', 'value': 1}, {'name': '366', 'value': 4}, {'name': '367', 'value': 0}, {'name': '368', 'value': 21}, {'name': '369', 'value': 14}, {'name': '370', 'value': 2}, {'name': '371', 'value': 10}, {'name': '372', 'value': 7}, {'name': '373', 'value': 1}, {'name': '374', 'value': 18}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 1}, {'name': '378', 'value': 3}, {'name': '379', 'value': 0}, {'name': '380', 'value': 1}, {'name': '381', 'value': 10}, {'name': '382', 'value': 1}, {'name': '383', 'value': 0}, {'name': '384', 'value': 13}, {'name': '385', 'value': 6}, {'name': '386', 'value': 3}, {'name': '387', 'value': 1}, {'name': '388', 'value': 0}, {'name': '389', 'value': 7}, {'name': '390', 'value': 0}, {'name': '391', 'value': 1}, {'name': '392', 'value': 4}, {'name': '393', 'value': 0}, {'name': '394', 'value': 1}, {'name': '395', 'value': 1}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 1}, {'name': '399', 'value': 0}, {'name': '400', 'value': 1}, {'name': '401', 'value': 0}, {'name': '402', 'value': 1}, {'name': '403', 'value': 1}, {'name': '404', 'value': 1}, {'name': '405', 'value': 1}, {'name': '406', 'value': 9}, {'name': '407', 'value': 3}, {'name': '408', 'value': 1}, {'name': '409', 'value': 1}, {'name': '410', 'value': 0}, {'name': '411', 'value': 1}, {'name': '412', 'value': 12}, {'name': '413', 'value': 17}, {'name': '414', 'value': 0}, {'name': '415', 'value': 1}, {'name': '416', 'value': 1}, {'name': '417', 'value': 1}, {'name': '418', 'value': 0}, {'name': '419', 'value': 1}, {'name': '420', 'value': 1}, {'name': '421', 'value': 5}, {'name': '422', 'value': 1}, {'name': '423', 'value': 0}, {'name': '424', 'value': 9}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 4}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 1}, {'name': '431', 'value': 5}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 5}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 13}, {'name': '438', 'value': 0}, {'name': '439', 'value': 5}, {'name': '440', 'value': 9}, {'name': '441', 'value': 0}, {'name': '442', 'value': 1}, {'name': '443', 'value': 0}, {'name': '444', 'value': 1}, {'name': '445', 'value': 3}, {'name': '446', 'value': 2}, {'name': '447', 'value': 1}, {'name': '448', 'value': 1}, {'name': '449', 'value': 4}, {'name': '450', 'value': 3}, {'name': '451', 'value': 1}, {'name': '452', 'value': 2}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 1}, {'name': '456', 'value': 8}, {'name': '457', 'value': 3}, {'name': '458', 'value': 10}, {'name': '459', 'value': 1}, {'name': '460', 'value': 2}, {'name': '461', 'value': 1}, {'name': '462', 'value': 0}, {'name': '463', 'value': 4}, {'name': '464', 'value': 2}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 1}, {'name': '468', 'value': 0}, {'name': '469', 'value': 3}, {'name': '470', 'value': 1}, {'name': '471', 'value': 4}, {'name': '472', 'value': 0}, {'name': '473', 'value': 2}, {'name': '474', 'value': 8}, {'name': '475', 'value': 0}, {'name': '476', 'value': 1}, {'name': '477', 'value': 11}, {'name': '478', 'value': 1}, {'name': '479', 'value': 4}, {'name': '480', 'value': 1}, {'name': '481', 'value': 1}, {'name': '482', 'value': 0}, {'name': '483', 'value': 1}, {'name': '484', 'value': 5}, {'name': '485', 'value': 3}, {'name': '486', 'value': 1}, {'name': '487', 'value': 7}, {'name': '488', 'value': 0}, {'name': '489', 'value': 6}, {'name': '490', 'value': 0}, {'name': '491', 'value': 1}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 3}, {'name': '497', 'value': 15}, {'name': '498', 'value': 7}, {'name': '499', 'value': 1}, {'name': '500', 'value': 3}]}, 'CptModifier_stringindexer_transform': {'name': 'CptModifier_stringindexer_transform', 'series': [{'name': '1', 'value': 1}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 1}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 4}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 2}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 2}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 1}, {'name': '29', 'value': 2}, {'name': '30', 'value': 1}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 1}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 1}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 3}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 2}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 1}, {'name': '67', 'value': 0}, {'name': '68', 'value': 1}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 1}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 1}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 4}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 1}, {'name': '91', 'value': 0}, {'name': '92', 'value': 1}, {'name': '93', 'value': 1}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 2}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 1}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 1}, {'name': '105', 'value': 1}, {'name': '106', 'value': 0}, {'name': '107', 'value': 1}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 4}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 1}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 4}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 3}, {'name': '128', 'value': 2}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 3}, {'name': '133', 'value': 0}, {'name': '134', 'value': 4}, {'name': '135', 'value': 0}, {'name': '136', 'value': 1}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 4}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 1}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 1}, {'name': '163', 'value': 0}, {'name': '164', 'value': 2}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 1}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 6}, {'name': '175', 'value': 1}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 1}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 1}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 1}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 1}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 3}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 1}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 1}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 1}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 2}, {'name': '235', 'value': 1}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 1}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 4}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 3}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 1}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 1}, {'name': '277', 'value': 0}, {'name': '278', 'value': 1}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 1}, {'name': '299', 'value': 2}, {'name': '300', 'value': 1}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 3}, {'name': '305', 'value': 1}, {'name': '306', 'value': 3}, {'name': '307', 'value': 1}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 2}, {'name': '311', 'value': 1}, {'name': '312', 'value': 0}, {'name': '313', 'value': 1}, {'name': '314', 'value': 0}, {'name': '315', 'value': 1}, {'name': '316', 'value': 1}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 3}, {'name': '320', 'value': 1}, {'name': '321', 'value': 4}, {'name': '322', 'value': 5}, {'name': '323', 'value': 1}, {'name': '324', 'value': 1}, {'name': '325', 'value': 0}, {'name': '326', 'value': 1}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 1}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 1}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 3}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 2}, {'name': '355', 'value': 1}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 5}, {'name': '365', 'value': 0}, {'name': '366', 'value': 3}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 2}, {'name': '370', 'value': 0}, {'name': '371', 'value': 1}, {'name': '372', 'value': 0}, {'name': '373', 'value': 1}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 1}, {'name': '379', 'value': 0}, {'name': '380', 'value': 1}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 4}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 2}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 1}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 1}, {'name': '413', 'value': 1}, {'name': '414', 'value': 0}, {'name': '415', 'value': 1}, {'name': '416', 'value': 1}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 1}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 4}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 1}, {'name': '431', 'value': 1}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 4}, {'name': '438', 'value': 0}, {'name': '439', 'value': 1}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 1}, {'name': '448', 'value': 0}, {'name': '449', 'value': 3}, {'name': '450', 'value': 0}, {'name': '451', 'value': 1}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 1}, {'name': '456', 'value': 4}, {'name': '457', 'value': 5}, {'name': '458', 'value': 0}, {'name': '459', 'value': 1}, {'name': '460', 'value': 0}, {'name': '461', 'value': 1}, {'name': '462', 'value': 0}, {'name': '463', 'value': 3}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 1}, {'name': '471', 'value': 3}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 4}, {'name': '475', 'value': 0}, {'name': '476', 'value': 1}, {'name': '477', 'value': 2}, {'name': '478', 'value': 0}, {'name': '479', 'value': 2}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'pdiagnosis1_stringindexer_transform': {'name': 'pdiagnosis1_stringindexer_transform', 'series': [{'name': '1', 'value': 2}, {'name': '2', 'value': 45}, {'name': '3', 'value': 24}, {'name': '4', 'value': 3}, {'name': '5', 'value': 5}, {'name': '6', 'value': 54}, {'name': '7', 'value': 13}, {'name': '8', 'value': 125}, {'name': '9', 'value': 79}, {'name': '10', 'value': 117}, {'name': '11', 'value': 7}, {'name': '12', 'value': 26}, {'name': '13', 'value': 52}, {'name': '14', 'value': 0}, {'name': '15', 'value': 118}, {'name': '16', 'value': 1}, {'name': '17', 'value': 4}, {'name': '18', 'value': 63}, {'name': '19', 'value': 110}, {'name': '20', 'value': 67}, {'name': '21', 'value': 23}, {'name': '22', 'value': 15}, {'name': '23', 'value': 8}, {'name': '24', 'value': 6}, {'name': '25', 'value': 67}, {'name': '26', 'value': 3}, {'name': '27', 'value': 55}, {'name': '28', 'value': 94}, {'name': '29', 'value': 31}, {'name': '30', 'value': 48}, {'name': '31', 'value': 44}, {'name': '32', 'value': 19}, {'name': '33', 'value': 0}, {'name': '34', 'value': 21}, {'name': '35', 'value': 135}, {'name': '36', 'value': 7}, {'name': '37', 'value': 43}, {'name': '38', 'value': 0}, {'name': '39', 'value': 16}, {'name': '40', 'value': 3}, {'name': '41', 'value': 14}, {'name': '42', 'value': 138}, {'name': '43', 'value': 98}, {'name': '44', 'value': 25}, {'name': '45', 'value': 112}, {'name': '46', 'value': 25}, {'name': '47', 'value': 17}, {'name': '48', 'value': 56}, {'name': '49', 'value': 56}, {'name': '50', 'value': 19}, {'name': '51', 'value': 3}, {'name': '52', 'value': 5}, {'name': '53', 'value': 38}, {'name': '54', 'value': 1}, {'name': '55', 'value': 0}, {'name': '56', 'value': 39}, {'name': '57', 'value': 29}, {'name': '58', 'value': 74}, {'name': '59', 'value': 1}, {'name': '60', 'value': 2}, {'name': '61', 'value': 134}, {'name': '62', 'value': 1}, {'name': '63', 'value': 7}, {'name': '64', 'value': 20}, {'name': '65', 'value': 9}, {'name': '66', 'value': 11}, {'name': '67', 'value': 76}, {'name': '68', 'value': 12}, {'name': '69', 'value': 8}, {'name': '70', 'value': 14}, {'name': '71', 'value': 72}, {'name': '72', 'value': 2}, {'name': '73', 'value': 19}, {'name': '74', 'value': 9}, {'name': '75', 'value': 1}, {'name': '76', 'value': 9}, {'name': '77', 'value': 147}, {'name': '78', 'value': 86}, {'name': '79', 'value': 4}, {'name': '80', 'value': 26}, {'name': '81', 'value': 7}, {'name': '82', 'value': 103}, {'name': '83', 'value': 101}, {'name': '84', 'value': 1}, {'name': '85', 'value': 129}, {'name': '86', 'value': 24}, {'name': '87', 'value': 12}, {'name': '88', 'value': 0}, {'name': '89', 'value': 29}, {'name': '90', 'value': 17}, {'name': '91', 'value': 3}, {'name': '92', 'value': 64}, {'name': '93', 'value': 43}, {'name': '94', 'value': 6}, {'name': '95', 'value': 90}, {'name': '96', 'value': 24}, {'name': '97', 'value': 9}, {'name': '98', 'value': 0}, {'name': '99', 'value': 1}, {'name': '100', 'value': 13}, {'name': '101', 'value': 7}, {'name': '102', 'value': 15}, {'name': '103', 'value': 28}, {'name': '104', 'value': 76}, {'name': '105', 'value': 2}, {'name': '106', 'value': 9}, {'name': '107', 'value': 14}, {'name': '108', 'value': 75}, {'name': '109', 'value': 4}, {'name': '110', 'value': 17}, {'name': '111', 'value': 75}, {'name': '112', 'value': 9}, {'name': '113', 'value': 3}, {'name': '114', 'value': 12}, {'name': '115', 'value': 146}, {'name': '116', 'value': 2}, {'name': '117', 'value': 13}, {'name': '118', 'value': 14}, {'name': '119', 'value': 72}, {'name': '120', 'value': 5}, {'name': '121', 'value': 7}, {'name': '122', 'value': 8}, {'name': '123', 'value': 60}, {'name': '124', 'value': 1}, {'name': '125', 'value': 105}, {'name': '126', 'value': 32}, {'name': '127', 'value': 0}, {'name': '128', 'value': 11}, {'name': '129', 'value': 38}, {'name': '130', 'value': 0}, {'name': '131', 'value': 7}, {'name': '132', 'value': 0}, {'name': '133', 'value': 136}, {'name': '134', 'value': 8}, {'name': '135', 'value': 8}, {'name': '136', 'value': 4}, {'name': '137', 'value': 99}, {'name': '138', 'value': 7}, {'name': '139', 'value': 127}, {'name': '140', 'value': 24}, {'name': '141', 'value': 16}, {'name': '142', 'value': 3}, {'name': '143', 'value': 14}, {'name': '144', 'value': 16}, {'name': '145', 'value': 25}, {'name': '146', 'value': 19}, {'name': '147', 'value': 18}, {'name': '148', 'value': 69}, {'name': '149', 'value': 70}, {'name': '150', 'value': 0}, {'name': '151', 'value': 2}, {'name': '152', 'value': 2}, {'name': '153', 'value': 10}, {'name': '154', 'value': 6}, {'name': '155', 'value': 0}, {'name': '156', 'value': 57}, {'name': '157', 'value': 6}, {'name': '158', 'value': 1}, {'name': '159', 'value': 95}, {'name': '160', 'value': 41}, {'name': '161', 'value': 2}, {'name': '162', 'value': 91}, {'name': '163', 'value': 0}, {'name': '164', 'value': 11}, {'name': '165', 'value': 14}, {'name': '166', 'value': 1}, {'name': '167', 'value': 8}, {'name': '168', 'value': 5}, {'name': '169', 'value': 31}, {'name': '170', 'value': 26}, {'name': '171', 'value': 28}, {'name': '172', 'value': 2}, {'name': '173', 'value': 2}, {'name': '174', 'value': 120}, {'name': '175', 'value': 33}, {'name': '176', 'value': 1}, {'name': '177', 'value': 16}, {'name': '178', 'value': 17}, {'name': '179', 'value': 3}, {'name': '180', 'value': 1}, {'name': '181', 'value': 7}, {'name': '182', 'value': 8}, {'name': '183', 'value': 9}, {'name': '184', 'value': 7}, {'name': '185', 'value': 32}, {'name': '186', 'value': 141}, {'name': '187', 'value': 47}, {'name': '188', 'value': 42}, {'name': '189', 'value': 4}, {'name': '190', 'value': 21}, {'name': '191', 'value': 6}, {'name': '192', 'value': 49}, {'name': '193', 'value': 18}, {'name': '194', 'value': 15}, {'name': '195', 'value': 13}, {'name': '196', 'value': 35}, {'name': '197', 'value': 38}, {'name': '198', 'value': 1}, {'name': '199', 'value': 20}, {'name': '200', 'value': 53}, {'name': '201', 'value': 5}, {'name': '202', 'value': 35}, {'name': '203', 'value': 35}, {'name': '204', 'value': 6}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 73}, {'name': '208', 'value': 10}, {'name': '209', 'value': 0}, {'name': '210', 'value': 42}, {'name': '211', 'value': 11}, {'name': '212', 'value': 2}, {'name': '213', 'value': 12}, {'name': '214', 'value': 2}, {'name': '215', 'value': 8}, {'name': '216', 'value': 124}, {'name': '217', 'value': 61}, {'name': '218', 'value': 1}, {'name': '219', 'value': 4}, {'name': '220', 'value': 10}, {'name': '221', 'value': 46}, {'name': '222', 'value': 15}, {'name': '223', 'value': 11}, {'name': '224', 'value': 2}, {'name': '225', 'value': 121}, {'name': '226', 'value': 57}, {'name': '227', 'value': 62}, {'name': '228', 'value': 6}, {'name': '229', 'value': 5}, {'name': '230', 'value': 8}, {'name': '231', 'value': 4}, {'name': '232', 'value': 33}, {'name': '233', 'value': 40}, {'name': '234', 'value': 17}, {'name': '235', 'value': 82}, {'name': '236', 'value': 0}, {'name': '237', 'value': 23}, {'name': '238', 'value': 4}, {'name': '239', 'value': 68}, {'name': '240', 'value': 13}, {'name': '241', 'value': 60}, {'name': '242', 'value': 128}, {'name': '243', 'value': 22}, {'name': '244', 'value': 3}, {'name': '245', 'value': 0}, {'name': '246', 'value': 37}, {'name': '247', 'value': 18}, {'name': '248', 'value': 62}, {'name': '249', 'value': 71}, {'name': '250', 'value': 0}, {'name': '251', 'value': 4}, {'name': '252', 'value': 83}, {'name': '253', 'value': 33}, {'name': '254', 'value': 36}, {'name': '255', 'value': 139}, {'name': '256', 'value': 34}, {'name': '257', 'value': 3}, {'name': '258', 'value': 22}, {'name': '259', 'value': 2}, {'name': '260', 'value': 14}, {'name': '261', 'value': 39}, {'name': '262', 'value': 52}, {'name': '263', 'value': 12}, {'name': '264', 'value': 74}, {'name': '265', 'value': 0}, {'name': '266', 'value': 12}, {'name': '267', 'value': 122}, {'name': '268', 'value': 107}, {'name': '269', 'value': 18}, {'name': '270', 'value': 143}, {'name': '271', 'value': 3}, {'name': '272', 'value': 50}, {'name': '273', 'value': 3}, {'name': '274', 'value': 13}, {'name': '275', 'value': 10}, {'name': '276', 'value': 28}, {'name': '277', 'value': 2}, {'name': '278', 'value': 140}, {'name': '279', 'value': 2}, {'name': '280', 'value': 16}, {'name': '281', 'value': 1}, {'name': '282', 'value': 27}, {'name': '283', 'value': 68}, {'name': '284', 'value': 111}, {'name': '285', 'value': 0}, {'name': '286', 'value': 15}, {'name': '287', 'value': 10}, {'name': '288', 'value': 41}, {'name': '289', 'value': 18}, {'name': '290', 'value': 22}, {'name': '291', 'value': 30}, {'name': '292', 'value': 0}, {'name': '293', 'value': 50}, {'name': '294', 'value': 14}, {'name': '295', 'value': 47}, {'name': '296', 'value': 32}, {'name': '297', 'value': 0}, {'name': '298', 'value': 6}, {'name': '299', 'value': 126}, {'name': '300', 'value': 46}, {'name': '301', 'value': 1}, {'name': '302', 'value': 26}, {'name': '303', 'value': 2}, {'name': '304', 'value': 0}, {'name': '305', 'value': 4}, {'name': '306', 'value': 5}, {'name': '307', 'value': 21}, {'name': '308', 'value': 0}, {'name': '309', 'value': 3}, {'name': '310', 'value': 2}, {'name': '311', 'value': 10}, {'name': '312', 'value': 43}, {'name': '313', 'value': 142}, {'name': '314', 'value': 49}, {'name': '315', 'value': 5}, {'name': '316', 'value': 21}, {'name': '317', 'value': 15}, {'name': '318', 'value': 63}, {'name': '319', 'value': 0}, {'name': '320', 'value': 1}, {'name': '321', 'value': 27}, {'name': '322', 'value': 51}, {'name': '323', 'value': 53}, {'name': '324', 'value': 4}, {'name': '325', 'value': 26}, {'name': '326', 'value': 88}, {'name': '327', 'value': 10}, {'name': '328', 'value': 2}, {'name': '329', 'value': 130}, {'name': '330', 'value': 58}, {'name': '331', 'value': 81}, {'name': '332', 'value': 20}, {'name': '333', 'value': 7}, {'name': '334', 'value': 6}, {'name': '335', 'value': 5}, {'name': '336', 'value': 70}, {'name': '337', 'value': 1}, {'name': '338', 'value': 0}, {'name': '339', 'value': 46}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 85}, {'name': '343', 'value': 8}, {'name': '344', 'value': 61}, {'name': '345', 'value': 65}, {'name': '346', 'value': 0}, {'name': '347', 'value': 119}, {'name': '348', 'value': 11}, {'name': '349', 'value': 20}, {'name': '350', 'value': 133}, {'name': '351', 'value': 1}, {'name': '352', 'value': 48}, {'name': '353', 'value': 66}, {'name': '354', 'value': 6}, {'name': '355', 'value': 23}, {'name': '356', 'value': 71}, {'name': '357', 'value': 10}, {'name': '358', 'value': 0}, {'name': '359', 'value': 80}, {'name': '360', 'value': 21}, {'name': '361', 'value': 8}, {'name': '362', 'value': 5}, {'name': '363', 'value': 1}, {'name': '364', 'value': 45}, {'name': '365', 'value': 5}, {'name': '366', 'value': 6}, {'name': '367', 'value': 4}, {'name': '368', 'value': 59}, {'name': '369', 'value': 144}, {'name': '370', 'value': 9}, {'name': '371', 'value': 64}, {'name': '372', 'value': 0}, {'name': '373', 'value': 30}, {'name': '374', 'value': 32}, {'name': '375', 'value': 0}, {'name': '376', 'value': 34}, {'name': '377', 'value': 7}, {'name': '378', 'value': 48}, {'name': '379', 'value': 1}, {'name': '380', 'value': 102}, {'name': '381', 'value': 108}, {'name': '382', 'value': 9}, {'name': '383', 'value': 2}, {'name': '384', 'value': 145}, {'name': '385', 'value': 115}, {'name': '386', 'value': 7}, {'name': '387', 'value': 17}, {'name': '388', 'value': 16}, {'name': '389', 'value': 22}, {'name': '390', 'value': 10}, {'name': '391', 'value': 0}, {'name': '392', 'value': 6}, {'name': '393', 'value': 1}, {'name': '394', 'value': 55}, {'name': '395', 'value': 3}, {'name': '396', 'value': 41}, {'name': '397', 'value': 3}, {'name': '398', 'value': 16}, {'name': '399', 'value': 3}, {'name': '400', 'value': 84}, {'name': '401', 'value': 2}, {'name': '402', 'value': 69}, {'name': '403', 'value': 78}, {'name': '404', 'value': 106}, {'name': '405', 'value': 47}, {'name': '406', 'value': 31}, {'name': '407', 'value': 148}, {'name': '408', 'value': 40}, {'name': '409', 'value': 51}, {'name': '410', 'value': 19}, {'name': '411', 'value': 73}, {'name': '412', 'value': 19}, {'name': '413', 'value': 123}, {'name': '414', 'value': 5}, {'name': '415', 'value': 27}, {'name': '416', 'value': 1}, {'name': '417', 'value': 49}, {'name': '418', 'value': 24}, {'name': '419', 'value': 9}, {'name': '420', 'value': 44}, {'name': '421', 'value': 92}, {'name': '422', 'value': 10}, {'name': '423', 'value': 5}, {'name': '424', 'value': 31}, {'name': '425', 'value': 1}, {'name': '426', 'value': 22}, {'name': '427', 'value': 27}, {'name': '428', 'value': 11}, {'name': '429', 'value': 23}, {'name': '430', 'value': 28}, {'name': '431', 'value': 132}, {'name': '432', 'value': 0}, {'name': '433', 'value': 12}, {'name': '434', 'value': 29}, {'name': '435', 'value': 1}, {'name': '436', 'value': 15}, {'name': '437', 'value': 89}, {'name': '438', 'value': 1}, {'name': '439', 'value': 58}, {'name': '440', 'value': 0}, {'name': '441', 'value': 1}, {'name': '442', 'value': 65}, {'name': '443', 'value': 37}, {'name': '444', 'value': 40}, {'name': '445', 'value': 37}, {'name': '446', 'value': 4}, {'name': '447', 'value': 20}, {'name': '448', 'value': 116}, {'name': '449', 'value': 1}, {'name': '450', 'value': 96}, {'name': '451', 'value': 50}, {'name': '452', 'value': 59}, {'name': '453', 'value': 0}, {'name': '454', 'value': 109}, {'name': '455', 'value': 131}, {'name': '456', 'value': 36}, {'name': '457', 'value': 54}, {'name': '458', 'value': 66}, {'name': '459', 'value': 100}, {'name': '460', 'value': 30}, {'name': '461', 'value': 97}, {'name': '462', 'value': 39}, {'name': '463', 'value': 5}, {'name': '464', 'value': 114}, {'name': '465', 'value': 1}, {'name': '466', 'value': 29}, {'name': '467', 'value': 25}, {'name': '468', 'value': 13}, {'name': '469', 'value': 4}, {'name': '470', 'value': 34}, {'name': '471', 'value': 42}, {'name': '472', 'value': 3}, {'name': '473', 'value': 36}, {'name': '474', 'value': 33}, {'name': '475', 'value': 3}, {'name': '476', 'value': 113}, {'name': '477', 'value': 137}, {'name': '478', 'value': 0}, {'name': '479', 'value': 6}, {'name': '480', 'value': 87}, {'name': '481', 'value': 25}, {'name': '482', 'value': 30}, {'name': '483', 'value': 18}, {'name': '484', 'value': 3}, {'name': '485', 'value': 11}, {'name': '486', 'value': 9}, {'name': '487', 'value': 13}, {'name': '488', 'value': 104}, {'name': '489', 'value': 4}, {'name': '490', 'value': 45}, {'name': '491', 'value': 44}, {'name': '492', 'value': 12}, {'name': '493', 'value': 1}, {'name': '494', 'value': 3}, {'name': '495', 'value': 11}, {'name': '496', 'value': 93}, {'name': '497', 'value': 23}, {'name': '498', 'value': 6}, {'name': '499', 'value': 0}, {'name': '500', 'value': 77}]}, 'pdiagnosis2_stringindexer_transform': {'name': 'pdiagnosis2_stringindexer_transform', 'series': [{'name': '1', 'value': 4}, {'name': '2', 'value': 72}, {'name': '3', 'value': 45}, {'name': '4', 'value': 10}, {'name': '5', 'value': 5}, {'name': '6', 'value': 16}, {'name': '7', 'value': 0}, {'name': '8', 'value': 15}, {'name': '9', 'value': 3}, {'name': '10', 'value': 13}, {'name': '11', 'value': 16}, {'name': '12', 'value': 0}, {'name': '13', 'value': 32}, {'name': '14', 'value': 17}, {'name': '15', 'value': 18}, {'name': '16', 'value': 22}, {'name': '17', 'value': 9}, {'name': '18', 'value': 73}, {'name': '19', 'value': 2}, {'name': '20', 'value': 0}, {'name': '21', 'value': 31}, {'name': '22', 'value': 75}, {'name': '23', 'value': 24}, {'name': '24', 'value': 0}, {'name': '25', 'value': 12}, {'name': '26', 'value': 42}, {'name': '27', 'value': 23}, {'name': '28', 'value': 31}, {'name': '29', 'value': 0}, {'name': '30', 'value': 5}, {'name': '31', 'value': 7}, {'name': '32', 'value': 12}, {'name': '33', 'value': 29}, {'name': '34', 'value': 4}, {'name': '35', 'value': 86}, {'name': '36', 'value': 2}, {'name': '37', 'value': 6}, {'name': '38', 'value': 0}, {'name': '39', 'value': 76}, {'name': '40', 'value': 10}, {'name': '41', 'value': 145}, {'name': '42', 'value': 48}, {'name': '43', 'value': 16}, {'name': '44', 'value': 1}, {'name': '45', 'value': 59}, {'name': '46', 'value': 1}, {'name': '47', 'value': 71}, {'name': '48', 'value': 1}, {'name': '49', 'value': 2}, {'name': '50', 'value': 58}, {'name': '51', 'value': 10}, {'name': '52', 'value': 0}, {'name': '53', 'value': 83}, {'name': '54', 'value': 6}, {'name': '55', 'value': 0}, {'name': '56', 'value': 3}, {'name': '57', 'value': 65}, {'name': '58', 'value': 74}, {'name': '59', 'value': 13}, {'name': '60', 'value': 24}, {'name': '61', 'value': 105}, {'name': '62', 'value': 45}, {'name': '63', 'value': 9}, {'name': '64', 'value': 120}, {'name': '65', 'value': 3}, {'name': '66', 'value': 110}, {'name': '67', 'value': 3}, {'name': '68', 'value': 46}, {'name': '69', 'value': 11}, {'name': '70', 'value': 12}, {'name': '71', 'value': 50}, {'name': '72', 'value': 2}, {'name': '73', 'value': 1}, {'name': '74', 'value': 43}, {'name': '75', 'value': 22}, {'name': '76', 'value': 17}, {'name': '77', 'value': 12}, {'name': '78', 'value': 106}, {'name': '79', 'value': 24}, {'name': '80', 'value': 148}, {'name': '81', 'value': 3}, {'name': '82', 'value': 13}, {'name': '83', 'value': 9}, {'name': '84', 'value': 93}, {'name': '85', 'value': 119}, {'name': '86', 'value': 5}, {'name': '87', 'value': 38}, {'name': '88', 'value': 1}, {'name': '89', 'value': 4}, {'name': '90', 'value': 46}, {'name': '91', 'value': 38}, {'name': '92', 'value': 117}, {'name': '93', 'value': 102}, {'name': '94', 'value': 20}, {'name': '95', 'value': 69}, {'name': '96', 'value': 19}, {'name': '97', 'value': 8}, {'name': '98', 'value': 17}, {'name': '99', 'value': 40}, {'name': '100', 'value': 139}, {'name': '101', 'value': 5}, {'name': '102', 'value': 1}, {'name': '103', 'value': 107}, {'name': '104', 'value': 35}, {'name': '105', 'value': 85}, {'name': '106', 'value': 8}, {'name': '107', 'value': 19}, {'name': '108', 'value': 8}, {'name': '109', 'value': 7}, {'name': '110', 'value': 51}, {'name': '111', 'value': 21}, {'name': '112', 'value': 49}, {'name': '113', 'value': 26}, {'name': '114', 'value': 2}, {'name': '115', 'value': 25}, {'name': '116', 'value': 2}, {'name': '117', 'value': 1}, {'name': '118', 'value': 3}, {'name': '119', 'value': 36}, {'name': '120', 'value': 0}, {'name': '121', 'value': 43}, {'name': '122', 'value': 153}, {'name': '123', 'value': 142}, {'name': '124', 'value': 22}, {'name': '125', 'value': 66}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 13}, {'name': '129', 'value': 6}, {'name': '130', 'value': 84}, {'name': '131', 'value': 46}, {'name': '132', 'value': 0}, {'name': '133', 'value': 134}, {'name': '134', 'value': 6}, {'name': '135', 'value': 3}, {'name': '136', 'value': 81}, {'name': '137', 'value': 1}, {'name': '138', 'value': 9}, {'name': '139', 'value': 99}, {'name': '140', 'value': 76}, {'name': '141', 'value': 11}, {'name': '142', 'value': 37}, {'name': '143', 'value': 6}, {'name': '144', 'value': 5}, {'name': '145', 'value': 1}, {'name': '146', 'value': 69}, {'name': '147', 'value': 39}, {'name': '148', 'value': 90}, {'name': '149', 'value': 57}, {'name': '150', 'value': 0}, {'name': '151', 'value': 2}, {'name': '152', 'value': 1}, {'name': '153', 'value': 6}, {'name': '154', 'value': 44}, {'name': '155', 'value': 4}, {'name': '156', 'value': 11}, {'name': '157', 'value': 20}, {'name': '158', 'value': 4}, {'name': '159', 'value': 47}, {'name': '160', 'value': 65}, {'name': '161', 'value': 11}, {'name': '162', 'value': 5}, {'name': '163', 'value': 0}, {'name': '164', 'value': 72}, {'name': '165', 'value': 4}, {'name': '166', 'value': 8}, {'name': '167', 'value': 146}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 12}, {'name': '172', 'value': 2}, {'name': '173', 'value': 100}, {'name': '174', 'value': 108}, {'name': '175', 'value': 113}, {'name': '176', 'value': 3}, {'name': '177', 'value': 7}, {'name': '178', 'value': 36}, {'name': '179', 'value': 68}, {'name': '180', 'value': 45}, {'name': '181', 'value': 60}, {'name': '182', 'value': 33}, {'name': '183', 'value': 3}, {'name': '184', 'value': 5}, {'name': '185', 'value': 0}, {'name': '186', 'value': 2}, {'name': '187', 'value': 127}, {'name': '188', 'value': 3}, {'name': '189', 'value': 9}, {'name': '190', 'value': 80}, {'name': '191', 'value': 44}, {'name': '192', 'value': 91}, {'name': '193', 'value': 41}, {'name': '194', 'value': 40}, {'name': '195', 'value': 1}, {'name': '196', 'value': 3}, {'name': '197', 'value': 12}, {'name': '198', 'value': 79}, {'name': '199', 'value': 14}, {'name': '200', 'value': 11}, {'name': '201', 'value': 104}, {'name': '202', 'value': 61}, {'name': '203', 'value': 1}, {'name': '204', 'value': 20}, {'name': '205', 'value': 25}, {'name': '206', 'value': 1}, {'name': '207', 'value': 48}, {'name': '208', 'value': 1}, {'name': '209', 'value': 67}, {'name': '210', 'value': 8}, {'name': '211', 'value': 13}, {'name': '212', 'value': 34}, {'name': '213', 'value': 2}, {'name': '214', 'value': 2}, {'name': '215', 'value': 1}, {'name': '216', 'value': 64}, {'name': '217', 'value': 2}, {'name': '218', 'value': 22}, {'name': '219', 'value': 9}, {'name': '220', 'value': 5}, {'name': '221', 'value': 2}, {'name': '222', 'value': 1}, {'name': '223', 'value': 56}, {'name': '224', 'value': 2}, {'name': '225', 'value': 56}, {'name': '226', 'value': 62}, {'name': '227', 'value': 87}, {'name': '228', 'value': 27}, {'name': '229', 'value': 0}, {'name': '230', 'value': 11}, {'name': '231', 'value': 26}, {'name': '232', 'value': 24}, {'name': '233', 'value': 23}, {'name': '234', 'value': 28}, {'name': '235', 'value': 59}, {'name': '236', 'value': 4}, {'name': '237', 'value': 49}, {'name': '238', 'value': 135}, {'name': '239', 'value': 28}, {'name': '240', 'value': 12}, {'name': '241', 'value': 132}, {'name': '242', 'value': 4}, {'name': '243', 'value': 57}, {'name': '244', 'value': 34}, {'name': '245', 'value': 0}, {'name': '246', 'value': 1}, {'name': '247', 'value': 32}, {'name': '248', 'value': 0}, {'name': '249', 'value': 3}, {'name': '250', 'value': 96}, {'name': '251', 'value': 17}, {'name': '252', 'value': 1}, {'name': '253', 'value': 51}, {'name': '254', 'value': 17}, {'name': '255', 'value': 16}, {'name': '256', 'value': 141}, {'name': '257', 'value': 37}, {'name': '258', 'value': 18}, {'name': '259', 'value': 2}, {'name': '260', 'value': 41}, {'name': '261', 'value': 5}, {'name': '262', 'value': 94}, {'name': '263', 'value': 38}, {'name': '264', 'value': 7}, {'name': '265', 'value': 0}, {'name': '266', 'value': 34}, {'name': '267', 'value': 6}, {'name': '268', 'value': 89}, {'name': '269', 'value': 128}, {'name': '270', 'value': 150}, {'name': '271', 'value': 68}, {'name': '272', 'value': 4}, {'name': '273', 'value': 37}, {'name': '274', 'value': 0}, {'name': '275', 'value': 98}, {'name': '276', 'value': 62}, {'name': '277', 'value': 122}, {'name': '278', 'value': 144}, {'name': '279', 'value': 54}, {'name': '280', 'value': 54}, {'name': '281', 'value': 35}, {'name': '282', 'value': 64}, {'name': '283', 'value': 66}, {'name': '284', 'value': 97}, {'name': '285', 'value': 0}, {'name': '286', 'value': 9}, {'name': '287', 'value': 55}, {'name': '288', 'value': 88}, {'name': '289', 'value': 4}, {'name': '290', 'value': 0}, {'name': '291', 'value': 11}, {'name': '292', 'value': 27}, {'name': '293', 'value': 82}, {'name': '294', 'value': 121}, {'name': '295', 'value': 23}, {'name': '296', 'value': 20}, {'name': '297', 'value': 0}, {'name': '298', 'value': 18}, {'name': '299', 'value': 74}, {'name': '300', 'value': 3}, {'name': '301', 'value': 3}, {'name': '302', 'value': 4}, {'name': '303', 'value': 2}, {'name': '304', 'value': 0}, {'name': '305', 'value': 3}, {'name': '306', 'value': 0}, {'name': '307', 'value': 103}, {'name': '308', 'value': 147}, {'name': '309', 'value': 10}, {'name': '310', 'value': 123}, {'name': '311', 'value': 15}, {'name': '312', 'value': 136}, {'name': '313', 'value': 4}, {'name': '314', 'value': 7}, {'name': '315', 'value': 30}, {'name': '316', 'value': 3}, {'name': '317', 'value': 101}, {'name': '318', 'value': 5}, {'name': '319', 'value': 0}, {'name': '320', 'value': 137}, {'name': '321', 'value': 0}, {'name': '322', 'value': 14}, {'name': '323', 'value': 112}, {'name': '324', 'value': 50}, {'name': '325', 'value': 0}, {'name': '326', 'value': 19}, {'name': '327', 'value': 11}, {'name': '328', 'value': 2}, {'name': '329', 'value': 151}, {'name': '330', 'value': 71}, {'name': '331', 'value': 14}, {'name': '332', 'value': 53}, {'name': '333', 'value': 61}, {'name': '334', 'value': 44}, {'name': '335', 'value': 8}, {'name': '336', 'value': 114}, {'name': '337', 'value': 60}, {'name': '338', 'value': 25}, {'name': '339', 'value': 33}, {'name': '340', 'value': 0}, {'name': '341', 'value': 25}, {'name': '342', 'value': 129}, {'name': '343', 'value': 5}, {'name': '344', 'value': 2}, {'name': '345', 'value': 4}, {'name': '346', 'value': 0}, {'name': '347', 'value': 39}, {'name': '348', 'value': 5}, {'name': '349', 'value': 7}, {'name': '350', 'value': 75}, {'name': '351', 'value': 6}, {'name': '352', 'value': 155}, {'name': '353', 'value': 6}, {'name': '354', 'value': 0}, {'name': '355', 'value': 31}, {'name': '356', 'value': 41}, {'name': '357', 'value': 8}, {'name': '358', 'value': 0}, {'name': '359', 'value': 2}, {'name': '360', 'value': 3}, {'name': '361', 'value': 33}, {'name': '362', 'value': 1}, {'name': '363', 'value': 55}, {'name': '364', 'value': 73}, {'name': '365', 'value': 30}, {'name': '366', 'value': 18}, {'name': '367', 'value': 47}, {'name': '368', 'value': 63}, {'name': '369', 'value': 140}, {'name': '370', 'value': 16}, {'name': '371', 'value': 30}, {'name': '372', 'value': 0}, {'name': '373', 'value': 4}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 3}, {'name': '377', 'value': 109}, {'name': '378', 'value': 49}, {'name': '379', 'value': 12}, {'name': '380', 'value': 7}, {'name': '381', 'value': 124}, {'name': '382', 'value': 36}, {'name': '383', 'value': 2}, {'name': '384', 'value': 131}, {'name': '385', 'value': 21}, {'name': '386', 'value': 3}, {'name': '387', 'value': 6}, {'name': '388', 'value': 7}, {'name': '389', 'value': 0}, {'name': '390', 'value': 5}, {'name': '391', 'value': 52}, {'name': '392', 'value': 0}, {'name': '393', 'value': 17}, {'name': '394', 'value': 28}, {'name': '395', 'value': 13}, {'name': '396', 'value': 9}, {'name': '397', 'value': 77}, {'name': '398', 'value': 19}, {'name': '399', 'value': 149}, {'name': '400', 'value': 4}, {'name': '401', 'value': 2}, {'name': '402', 'value': 126}, {'name': '403', 'value': 15}, {'name': '404', 'value': 78}, {'name': '405', 'value': 115}, {'name': '406', 'value': 70}, {'name': '407', 'value': 32}, {'name': '408', 'value': 52}, {'name': '409', 'value': 14}, {'name': '410', 'value': 138}, {'name': '411', 'value': 48}, {'name': '412', 'value': 19}, {'name': '413', 'value': 130}, {'name': '414', 'value': 4}, {'name': '415', 'value': 4}, {'name': '416', 'value': 8}, {'name': '417', 'value': 4}, {'name': '418', 'value': 7}, {'name': '419', 'value': 14}, {'name': '420', 'value': 15}, {'name': '421', 'value': 35}, {'name': '422', 'value': 30}, {'name': '423', 'value': 21}, {'name': '424', 'value': 70}, {'name': '425', 'value': 32}, {'name': '426', 'value': 26}, {'name': '427', 'value': 0}, {'name': '428', 'value': 154}, {'name': '429', 'value': 116}, {'name': '430', 'value': 14}, {'name': '431', 'value': 10}, {'name': '432', 'value': 0}, {'name': '433', 'value': 43}, {'name': '434', 'value': 29}, {'name': '435', 'value': 1}, {'name': '436', 'value': 15}, {'name': '437', 'value': 50}, {'name': '438', 'value': 5}, {'name': '439', 'value': 14}, {'name': '440', 'value': 0}, {'name': '441', 'value': 8}, {'name': '442', 'value': 23}, {'name': '443', 'value': 26}, {'name': '444', 'value': 51}, {'name': '445', 'value': 1}, {'name': '446', 'value': 16}, {'name': '447', 'value': 1}, {'name': '448', 'value': 29}, {'name': '449', 'value': 2}, {'name': '450', 'value': 58}, {'name': '451', 'value': 118}, {'name': '452', 'value': 63}, {'name': '453', 'value': 4}, {'name': '454', 'value': 15}, {'name': '455', 'value': 3}, {'name': '456', 'value': 53}, {'name': '457', 'value': 3}, {'name': '458', 'value': 28}, {'name': '459', 'value': 27}, {'name': '460', 'value': 4}, {'name': '461', 'value': 125}, {'name': '462', 'value': 6}, {'name': '463', 'value': 0}, {'name': '464', 'value': 13}, {'name': '465', 'value': 1}, {'name': '466', 'value': 7}, {'name': '467', 'value': 42}, {'name': '468', 'value': 15}, {'name': '469', 'value': 133}, {'name': '470', 'value': 6}, {'name': '471', 'value': 0}, {'name': '472', 'value': 10}, {'name': '473', 'value': 92}, {'name': '474', 'value': 21}, {'name': '475', 'value': 77}, {'name': '476', 'value': 1}, {'name': '477', 'value': 27}, {'name': '478', 'value': 111}, {'name': '479', 'value': 0}, {'name': '480', 'value': 143}, {'name': '481', 'value': 42}, {'name': '482', 'value': 10}, {'name': '483', 'value': 47}, {'name': '484', 'value': 33}, {'name': '485', 'value': 39}, {'name': '486', 'value': 8}, {'name': '487', 'value': 0}, {'name': '488', 'value': 2}, {'name': '489', 'value': 9}, {'name': '490', 'value': 40}, {'name': '491', 'value': 29}, {'name': '492', 'value': 10}, {'name': '493', 'value': 3}, {'name': '494', 'value': 10}, {'name': '495', 'value': 1}, {'name': '496', 'value': 95}, {'name': '497', 'value': 31}, {'name': '498', 'value': 18}, {'name': '499', 'value': 67}, {'name': '500', 'value': 152}]}, 'Gender_stringindexer_transform': {'name': 'Gender_stringindexer_transform', 'series': [{'name': '1', 'value': 1}, {'name': '2', 'value': 1}, {'name': '3', 'value': 1}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 1}, {'name': '8', 'value': 1}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 1}, {'name': '12', 'value': 1}, {'name': '13', 'value': 1}, {'name': '14', 'value': 0}, {'name': '15', 'value': 1}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 1}, {'name': '19', 'value': 0}, {'name': '20', 'value': 1}, {'name': '21', 'value': 1}, {'name': '22', 'value': 1}, {'name': '23', 'value': 1}, {'name': '24', 'value': 1}, {'name': '25', 'value': 1}, {'name': '26', 'value': 0}, {'name': '27', 'value': 1}, {'name': '28', 'value': 0}, {'name': '29', 'value': 1}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 1}, {'name': '33', 'value': 1}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 1}, {'name': '38', 'value': 1}, {'name': '39', 'value': 1}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 1}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 1}, {'name': '52', 'value': 1}, {'name': '53', 'value': 1}, {'name': '54', 'value': 0}, {'name': '55', 'value': 1}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 1}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 1}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 1}, {'name': '65', 'value': 1}, {'name': '66', 'value': 1}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 1}, {'name': '70', 'value': 1}, {'name': '71', 'value': 0}, {'name': '72', 'value': 1}, {'name': '73', 'value': 0}, {'name': '74', 'value': 1}, {'name': '75', 'value': 1}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 1}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 1}, {'name': '88', 'value': 1}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 1}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 1}, {'name': '97', 'value': 0}, {'name': '98', 'value': 1}, {'name': '99', 'value': 0}, {'name': '100', 'value': 1}, {'name': '101', 'value': 0}, {'name': '102', 'value': 1}, {'name': '103', 'value': 1}, {'name': '104', 'value': 0}, {'name': '105', 'value': 1}, {'name': '106', 'value': 1}, {'name': '107', 'value': 1}, {'name': '108', 'value': 1}, {'name': '109', 'value': 1}, {'name': '110', 'value': 1}, {'name': '111', 'value': 0}, {'name': '112', 'value': 1}, {'name': '113', 'value': 1}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 1}, {'name': '127', 'value': 1}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 1}, {'name': '133', 'value': 0}, {'name': '134', 'value': 1}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 1}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 1}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 1}, {'name': '144', 'value': 1}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 1}, {'name': '148', 'value': 1}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 1}, {'name': '153', 'value': 1}, {'name': '154', 'value': 1}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 1}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 1}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 1}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 1}, {'name': '175', 'value': 0}, {'name': '176', 'value': 1}, {'name': '177', 'value': 0}, {'name': '178', 'value': 1}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 1}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 1}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 1}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 1}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 1}, {'name': '205', 'value': 0}, {'name': '206', 'value': 1}, {'name': '207', 'value': 1}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 1}, {'name': '212', 'value': 0}, {'name': '213', 'value': 1}, {'name': '214', 'value': 1}, {'name': '215', 'value': 1}, {'name': '216', 'value': 1}, {'name': '217', 'value': 0}, {'name': '218', 'value': 1}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 1}, {'name': '222', 'value': 0}, {'name': '223', 'value': 1}, {'name': '224', 'value': 0}, {'name': '225', 'value': 1}, {'name': '226', 'value': 1}, {'name': '227', 'value': 0}, {'name': '228', 'value': 1}, {'name': '229', 'value': 1}, {'name': '230', 'value': 0}, {'name': '231', 'value': 1}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 1}, {'name': '235', 'value': 1}, {'name': '236', 'value': 0}, {'name': '237', 'value': 1}, {'name': '238', 'value': 1}, {'name': '239', 'value': 1}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 1}, {'name': '257', 'value': 1}, {'name': '258', 'value': 1}, {'name': '259', 'value': 1}, {'name': '260', 'value': 1}, {'name': '261', 'value': 1}, {'name': '262', 'value': 1}, {'name': '263', 'value': 1}, {'name': '264', 'value': 1}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 1}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 1}, {'name': '271', 'value': 0}, {'name': '272', 'value': 1}, {'name': '273', 'value': 1}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 1}, {'name': '281', 'value': 1}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 1}, {'name': '285', 'value': 1}, {'name': '286', 'value': 1}, {'name': '287', 'value': 0}, {'name': '288', 'value': 1}, {'name': '289', 'value': 1}, {'name': '290', 'value': 0}, {'name': '291', 'value': 1}, {'name': '292', 'value': 1}, {'name': '293', 'value': 0}, {'name': '294', 'value': 1}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 1}, {'name': '300', 'value': 1}, {'name': '301', 'value': 0}, {'name': '302', 'value': 1}, {'name': '303', 'value': 1}, {'name': '304', 'value': 1}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 1}, {'name': '308', 'value': 1}, {'name': '309', 'value': 1}, {'name': '310', 'value': 1}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 1}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 1}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 1}, {'name': '322', 'value': 1}, {'name': '323', 'value': 1}, {'name': '324', 'value': 0}, {'name': '325', 'value': 1}, {'name': '326', 'value': 0}, {'name': '327', 'value': 1}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 1}, {'name': '331', 'value': 0}, {'name': '332', 'value': 1}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 1}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 1}, {'name': '343', 'value': 1}, {'name': '344', 'value': 1}, {'name': '345', 'value': 0}, {'name': '346', 'value': 1}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 1}, {'name': '355', 'value': 1}, {'name': '356', 'value': 1}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 1}, {'name': '362', 'value': 1}, {'name': '363', 'value': 0}, {'name': '364', 'value': 1}, {'name': '365', 'value': 1}, {'name': '366', 'value': 1}, {'name': '367', 'value': 1}, {'name': '368', 'value': 0}, {'name': '369', 'value': 1}, {'name': '370', 'value': 1}, {'name': '371', 'value': 0}, {'name': '372', 'value': 1}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 1}, {'name': '376', 'value': 1}, {'name': '377', 'value': 0}, {'name': '378', 'value': 1}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 1}, {'name': '382', 'value': 1}, {'name': '383', 'value': 1}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 1}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 1}, {'name': '390', 'value': 1}, {'name': '391', 'value': 1}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 1}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 1}, {'name': '400', 'value': 1}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 1}, {'name': '404', 'value': 1}, {'name': '405', 'value': 1}, {'name': '406', 'value': 0}, {'name': '407', 'value': 1}, {'name': '408', 'value': 1}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 1}, {'name': '412', 'value': 0}, {'name': '413', 'value': 1}, {'name': '414', 'value': 1}, {'name': '415', 'value': 0}, {'name': '416', 'value': 1}, {'name': '417', 'value': 1}, {'name': '418', 'value': 1}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 1}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 1}, {'name': '427', 'value': 0}, {'name': '428', 'value': 1}, {'name': '429', 'value': 0}, {'name': '430', 'value': 1}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 1}, {'name': '435', 'value': 1}, {'name': '436', 'value': 1}, {'name': '437', 'value': 1}, {'name': '438', 'value': 1}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 1}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 1}, {'name': '447', 'value': 1}, {'name': '448', 'value': 1}, {'name': '449', 'value': 0}, {'name': '450', 'value': 1}, {'name': '451', 'value': 1}, {'name': '452', 'value': 1}, {'name': '453', 'value': 1}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 1}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 1}, {'name': '461', 'value': 1}, {'name': '462', 'value': 1}, {'name': '463', 'value': 0}, {'name': '464', 'value': 1}, {'name': '465', 'value': 1}, {'name': '466', 'value': 1}, {'name': '467', 'value': 1}, {'name': '468', 'value': 1}, {'name': '469', 'value': 1}, {'name': '470', 'value': 1}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 1}, {'name': '475', 'value': 1}, {'name': '476', 'value': 0}, {'name': '477', 'value': 1}, {'name': '478', 'value': 1}, {'name': '479', 'value': 1}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 1}, {'name': '483', 'value': 1}, {'name': '484', 'value': 0}, {'name': '485', 'value': 1}, {'name': '486', 'value': 0}, {'name': '487', 'value': 1}, {'name': '488', 'value': 0}, {'name': '489', 'value': 1}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 1}, {'name': '495', 'value': 1}, {'name': '496', 'value': 0}, {'name': '497', 'value': 1}, {'name': '498', 'value': 1}, {'name': '499', 'value': 1}, {'name': '500', 'value': 1}]}, 'DepartmentCategory_stringindexer_transform': {'name': 'DepartmentCategory_stringindexer_transform', 'series': [{'name': '1', 'value': 1}, {'name': '2', 'value': 1}, {'name': '3', 'value': 1}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 1}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 1}, {'name': '10', 'value': 1}, {'name': '11', 'value': 1}, {'name': '12', 'value': 2}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 3}, {'name': '19', 'value': 0}, {'name': '20', 'value': 2}, {'name': '21', 'value': 6}, {'name': '22', 'value': 0}, {'name': '23', 'value': 1}, {'name': '24', 'value': 2}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 1}, {'name': '29', 'value': 2}, {'name': '30', 'value': 1}, {'name': '31', 'value': 1}, {'name': '32', 'value': 0}, {'name': '33', 'value': 1}, {'name': '34', 'value': 1}, {'name': '35', 'value': 1}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 5}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 1}, {'name': '42', 'value': 1}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 3}, {'name': '46', 'value': 0}, {'name': '47', 'value': 1}, {'name': '48', 'value': 1}, {'name': '49', 'value': 0}, {'name': '50', 'value': 1}, {'name': '51', 'value': 0}, {'name': '52', 'value': 1}, {'name': '53', 'value': 1}, {'name': '54', 'value': 0}, {'name': '55', 'value': 2}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 2}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 1}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 1}, {'name': '65', 'value': 0}, {'name': '66', 'value': 1}, {'name': '67', 'value': 1}, {'name': '68', 'value': 1}, {'name': '69', 'value': 1}, {'name': '70', 'value': 0}, {'name': '71', 'value': 3}, {'name': '72', 'value': 0}, {'name': '73', 'value': 3}, {'name': '74', 'value': 1}, {'name': '75', 'value': 0}, {'name': '76', 'value': 1}, {'name': '77', 'value': 0}, {'name': '78', 'value': 3}, {'name': '79', 'value': 1}, {'name': '80', 'value': 1}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 2}, {'name': '84', 'value': 0}, {'name': '85', 'value': 1}, {'name': '86', 'value': 1}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 1}, {'name': '90', 'value': 1}, {'name': '91', 'value': 0}, {'name': '92', 'value': 3}, {'name': '93', 'value': 3}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 1}, {'name': '98', 'value': 0}, {'name': '99', 'value': 3}, {'name': '100', 'value': 0}, {'name': '101', 'value': 1}, {'name': '102', 'value': 0}, {'name': '103', 'value': 1}, {'name': '104', 'value': 1}, {'name': '105', 'value': 1}, {'name': '106', 'value': 0}, {'name': '107', 'value': 1}, {'name': '108', 'value': 1}, {'name': '109', 'value': 0}, {'name': '110', 'value': 1}, {'name': '111', 'value': 2}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 1}, {'name': '119', 'value': 1}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 2}, {'name': '123', 'value': 1}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 5}, {'name': '127', 'value': 2}, {'name': '128', 'value': 2}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 2}, {'name': '133', 'value': 1}, {'name': '134', 'value': 2}, {'name': '135', 'value': 0}, {'name': '136', 'value': 1}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 1}, {'name': '140', 'value': 1}, {'name': '141', 'value': 2}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 1}, {'name': '149', 'value': 1}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 1}, {'name': '157', 'value': 5}, {'name': '158', 'value': 0}, {'name': '159', 'value': 1}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 1}, {'name': '163', 'value': 0}, {'name': '164', 'value': 2}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 5}, {'name': '169', 'value': 4}, {'name': '170', 'value': 4}, {'name': '171', 'value': 1}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 1}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 3}, {'name': '182', 'value': 1}, {'name': '183', 'value': 0}, {'name': '184', 'value': 3}, {'name': '185', 'value': 4}, {'name': '186', 'value': 0}, {'name': '187', 'value': 1}, {'name': '188', 'value': 1}, {'name': '189', 'value': 0}, {'name': '190', 'value': 1}, {'name': '191', 'value': 0}, {'name': '192', 'value': 1}, {'name': '193', 'value': 0}, {'name': '194', 'value': 1}, {'name': '195', 'value': 0}, {'name': '196', 'value': 1}, {'name': '197', 'value': 1}, {'name': '198', 'value': 1}, {'name': '199', 'value': 3}, {'name': '200', 'value': 0}, {'name': '201', 'value': 1}, {'name': '202', 'value': 0}, {'name': '203', 'value': 1}, {'name': '204', 'value': 2}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 1}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 1}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 1}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 3}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 3}, {'name': '226', 'value': 1}, {'name': '227', 'value': 1}, {'name': '228', 'value': 1}, {'name': '229', 'value': 4}, {'name': '230', 'value': 1}, {'name': '231', 'value': 0}, {'name': '232', 'value': 3}, {'name': '233', 'value': 3}, {'name': '234', 'value': 2}, {'name': '235', 'value': 1}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 1}, {'name': '242', 'value': 1}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 1}, {'name': '250', 'value': 3}, {'name': '251', 'value': 1}, {'name': '252', 'value': 1}, {'name': '253', 'value': 1}, {'name': '254', 'value': 2}, {'name': '255', 'value': 3}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 1}, {'name': '261', 'value': 0}, {'name': '262', 'value': 1}, {'name': '263', 'value': 1}, {'name': '264', 'value': 0}, {'name': '265', 'value': 2}, {'name': '266', 'value': 1}, {'name': '267', 'value': 1}, {'name': '268', 'value': 1}, {'name': '269', 'value': 1}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 1}, {'name': '277', 'value': 0}, {'name': '278', 'value': 1}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 1}, {'name': '283', 'value': 1}, {'name': '284', 'value': 1}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 1}, {'name': '288', 'value': 1}, {'name': '289', 'value': 0}, {'name': '290', 'value': 4}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 1}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 3}, {'name': '299', 'value': 2}, {'name': '300', 'value': 1}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 2}, {'name': '305', 'value': 1}, {'name': '306', 'value': 2}, {'name': '307', 'value': 1}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 2}, {'name': '311', 'value': 1}, {'name': '312', 'value': 0}, {'name': '313', 'value': 1}, {'name': '314', 'value': 1}, {'name': '315', 'value': 3}, {'name': '316', 'value': 1}, {'name': '317', 'value': 0}, {'name': '318', 'value': 1}, {'name': '319', 'value': 2}, {'name': '320', 'value': 1}, {'name': '321', 'value': 2}, {'name': '322', 'value': 3}, {'name': '323', 'value': 3}, {'name': '324', 'value': 1}, {'name': '325', 'value': 7}, {'name': '326', 'value': 1}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 1}, {'name': '330', 'value': 1}, {'name': '331', 'value': 0}, {'name': '332', 'value': 1}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 1}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 1}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 1}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 1}, {'name': '346', 'value': 2}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 1}, {'name': '350', 'value': 1}, {'name': '351', 'value': 1}, {'name': '352', 'value': 0}, {'name': '353', 'value': 1}, {'name': '354', 'value': 2}, {'name': '355', 'value': 1}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 1}, {'name': '361', 'value': 1}, {'name': '362', 'value': 1}, {'name': '363', 'value': 0}, {'name': '364', 'value': 3}, {'name': '365', 'value': 1}, {'name': '366', 'value': 2}, {'name': '367', 'value': 0}, {'name': '368', 'value': 2}, {'name': '369', 'value': 2}, {'name': '370', 'value': 0}, {'name': '371', 'value': 3}, {'name': '372', 'value': 4}, {'name': '373', 'value': 1}, {'name': '374', 'value': 6}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 1}, {'name': '378', 'value': 3}, {'name': '379', 'value': 0}, {'name': '380', 'value': 1}, {'name': '381', 'value': 3}, {'name': '382', 'value': 1}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 3}, {'name': '387', 'value': 1}, {'name': '388', 'value': 0}, {'name': '389', 'value': 4}, {'name': '390', 'value': 0}, {'name': '391', 'value': 1}, {'name': '392', 'value': 2}, {'name': '393', 'value': 0}, {'name': '394', 'value': 1}, {'name': '395', 'value': 1}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 1}, {'name': '399', 'value': 0}, {'name': '400', 'value': 1}, {'name': '401', 'value': 0}, {'name': '402', 'value': 1}, {'name': '403', 'value': 1}, {'name': '404', 'value': 1}, {'name': '405', 'value': 1}, {'name': '406', 'value': 5}, {'name': '407', 'value': 3}, {'name': '408', 'value': 1}, {'name': '409', 'value': 1}, {'name': '410', 'value': 0}, {'name': '411', 'value': 1}, {'name': '412', 'value': 1}, {'name': '413', 'value': 3}, {'name': '414', 'value': 0}, {'name': '415', 'value': 1}, {'name': '416', 'value': 1}, {'name': '417', 'value': 1}, {'name': '418', 'value': 0}, {'name': '419', 'value': 1}, {'name': '420', 'value': 1}, {'name': '421', 'value': 1}, {'name': '422', 'value': 1}, {'name': '423', 'value': 0}, {'name': '424', 'value': 5}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 2}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 1}, {'name': '431', 'value': 1}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 1}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 1}, {'name': '440', 'value': 5}, {'name': '441', 'value': 0}, {'name': '442', 'value': 1}, {'name': '443', 'value': 0}, {'name': '444', 'value': 1}, {'name': '445', 'value': 3}, {'name': '446', 'value': 0}, {'name': '447', 'value': 1}, {'name': '448', 'value': 1}, {'name': '449', 'value': 2}, {'name': '450', 'value': 3}, {'name': '451', 'value': 1}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 1}, {'name': '456', 'value': 2}, {'name': '457', 'value': 3}, {'name': '458', 'value': 3}, {'name': '459', 'value': 1}, {'name': '460', 'value': 0}, {'name': '461', 'value': 1}, {'name': '462', 'value': 0}, {'name': '463', 'value': 2}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 1}, {'name': '468', 'value': 0}, {'name': '469', 'value': 3}, {'name': '470', 'value': 1}, {'name': '471', 'value': 2}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 2}, {'name': '475', 'value': 0}, {'name': '476', 'value': 1}, {'name': '477', 'value': 2}, {'name': '478', 'value': 1}, {'name': '479', 'value': 2}, {'name': '480', 'value': 1}, {'name': '481', 'value': 1}, {'name': '482', 'value': 0}, {'name': '483', 'value': 1}, {'name': '484', 'value': 1}, {'name': '485', 'value': 3}, {'name': '486', 'value': 1}, {'name': '487', 'value': 4}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 1}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 3}, {'name': '497', 'value': 6}, {'name': '498', 'value': 4}, {'name': '499', 'value': 1}, {'name': '500', 'value': 3}]}, 'SpecialityCategory_stringindexer_transform': {'name': 'SpecialityCategory_stringindexer_transform', 'series': [{'name': '1', 'value': 0}, {'name': '2', 'value': 0}, {'name': '3', 'value': 0}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 0}, {'name': '8', 'value': 0}, {'name': '9', 'value': 0}, {'name': '10', 'value': 0}, {'name': '11', 'value': 0}, {'name': '12', 'value': 0}, {'name': '13', 'value': 0}, {'name': '14', 'value': 0}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 0}, {'name': '18', 'value': 0}, {'name': '19', 'value': 0}, {'name': '20', 'value': 0}, {'name': '21', 'value': 0}, {'name': '22', 'value': 0}, {'name': '23', 'value': 0}, {'name': '24', 'value': 0}, {'name': '25', 'value': 0}, {'name': '26', 'value': 0}, {'name': '27', 'value': 0}, {'name': '28', 'value': 0}, {'name': '29', 'value': 0}, {'name': '30', 'value': 0}, {'name': '31', 'value': 0}, {'name': '32', 'value': 0}, {'name': '33', 'value': 0}, {'name': '34', 'value': 0}, {'name': '35', 'value': 0}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 0}, {'name': '40', 'value': 0}, {'name': '41', 'value': 0}, {'name': '42', 'value': 0}, {'name': '43', 'value': 0}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 0}, {'name': '47', 'value': 0}, {'name': '48', 'value': 0}, {'name': '49', 'value': 0}, {'name': '50', 'value': 0}, {'name': '51', 'value': 0}, {'name': '52', 'value': 0}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 0}, {'name': '56', 'value': 0}, {'name': '57', 'value': 0}, {'name': '58', 'value': 0}, {'name': '59', 'value': 0}, {'name': '60', 'value': 0}, {'name': '61', 'value': 0}, {'name': '62', 'value': 0}, {'name': '63', 'value': 0}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 0}, {'name': '67', 'value': 0}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 0}, {'name': '72', 'value': 0}, {'name': '73', 'value': 0}, {'name': '74', 'value': 0}, {'name': '75', 'value': 0}, {'name': '76', 'value': 0}, {'name': '77', 'value': 0}, {'name': '78', 'value': 0}, {'name': '79', 'value': 0}, {'name': '80', 'value': 0}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 0}, {'name': '84', 'value': 0}, {'name': '85', 'value': 0}, {'name': '86', 'value': 0}, {'name': '87', 'value': 0}, {'name': '88', 'value': 0}, {'name': '89', 'value': 0}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 0}, {'name': '97', 'value': 0}, {'name': '98', 'value': 0}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 0}, {'name': '102', 'value': 0}, {'name': '103', 'value': 0}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 0}, {'name': '109', 'value': 0}, {'name': '110', 'value': 0}, {'name': '111', 'value': 0}, {'name': '112', 'value': 0}, {'name': '113', 'value': 0}, {'name': '114', 'value': 0}, {'name': '115', 'value': 0}, {'name': '116', 'value': 0}, {'name': '117', 'value': 0}, {'name': '118', 'value': 0}, {'name': '119', 'value': 0}, {'name': '120', 'value': 0}, {'name': '121', 'value': 0}, {'name': '122', 'value': 0}, {'name': '123', 'value': 0}, {'name': '124', 'value': 0}, {'name': '125', 'value': 0}, {'name': '126', 'value': 0}, {'name': '127', 'value': 0}, {'name': '128', 'value': 0}, {'name': '129', 'value': 0}, {'name': '130', 'value': 0}, {'name': '131', 'value': 0}, {'name': '132', 'value': 0}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 0}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 0}, {'name': '140', 'value': 0}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 0}, {'name': '145', 'value': 0}, {'name': '146', 'value': 0}, {'name': '147', 'value': 0}, {'name': '148', 'value': 0}, {'name': '149', 'value': 0}, {'name': '150', 'value': 0}, {'name': '151', 'value': 0}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 0}, {'name': '156', 'value': 0}, {'name': '157', 'value': 0}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 0}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 0}, {'name': '164', 'value': 0}, {'name': '165', 'value': 0}, {'name': '166', 'value': 0}, {'name': '167', 'value': 0}, {'name': '168', 'value': 0}, {'name': '169', 'value': 0}, {'name': '170', 'value': 0}, {'name': '171', 'value': 0}, {'name': '172', 'value': 0}, {'name': '173', 'value': 0}, {'name': '174', 'value': 0}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 0}, {'name': '179', 'value': 0}, {'name': '180', 'value': 0}, {'name': '181', 'value': 0}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 0}, {'name': '186', 'value': 0}, {'name': '187', 'value': 0}, {'name': '188', 'value': 0}, {'name': '189', 'value': 0}, {'name': '190', 'value': 0}, {'name': '191', 'value': 0}, {'name': '192', 'value': 0}, {'name': '193', 'value': 0}, {'name': '194', 'value': 0}, {'name': '195', 'value': 0}, {'name': '196', 'value': 0}, {'name': '197', 'value': 0}, {'name': '198', 'value': 0}, {'name': '199', 'value': 0}, {'name': '200', 'value': 0}, {'name': '201', 'value': 0}, {'name': '202', 'value': 0}, {'name': '203', 'value': 0}, {'name': '204', 'value': 0}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 0}, {'name': '208', 'value': 0}, {'name': '209', 'value': 0}, {'name': '210', 'value': 0}, {'name': '211', 'value': 0}, {'name': '212', 'value': 0}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 0}, {'name': '216', 'value': 0}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 0}, {'name': '221', 'value': 0}, {'name': '222', 'value': 0}, {'name': '223', 'value': 0}, {'name': '224', 'value': 0}, {'name': '225', 'value': 0}, {'name': '226', 'value': 0}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 0}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 0}, {'name': '233', 'value': 0}, {'name': '234', 'value': 0}, {'name': '235', 'value': 0}, {'name': '236', 'value': 0}, {'name': '237', 'value': 0}, {'name': '238', 'value': 0}, {'name': '239', 'value': 0}, {'name': '240', 'value': 0}, {'name': '241', 'value': 0}, {'name': '242', 'value': 0}, {'name': '243', 'value': 0}, {'name': '244', 'value': 0}, {'name': '245', 'value': 0}, {'name': '246', 'value': 0}, {'name': '247', 'value': 0}, {'name': '248', 'value': 0}, {'name': '249', 'value': 0}, {'name': '250', 'value': 0}, {'name': '251', 'value': 0}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 0}, {'name': '256', 'value': 0}, {'name': '257', 'value': 0}, {'name': '258', 'value': 0}, {'name': '259', 'value': 0}, {'name': '260', 'value': 0}, {'name': '261', 'value': 0}, {'name': '262', 'value': 0}, {'name': '263', 'value': 0}, {'name': '264', 'value': 0}, {'name': '265', 'value': 0}, {'name': '266', 'value': 0}, {'name': '267', 'value': 0}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 0}, {'name': '271', 'value': 0}, {'name': '272', 'value': 0}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 0}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 0}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 0}, {'name': '285', 'value': 0}, {'name': '286', 'value': 0}, {'name': '287', 'value': 0}, {'name': '288', 'value': 0}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 0}, {'name': '292', 'value': 0}, {'name': '293', 'value': 0}, {'name': '294', 'value': 0}, {'name': '295', 'value': 0}, {'name': '296', 'value': 0}, {'name': '297', 'value': 0}, {'name': '298', 'value': 0}, {'name': '299', 'value': 0}, {'name': '300', 'value': 0}, {'name': '301', 'value': 0}, {'name': '302', 'value': 0}, {'name': '303', 'value': 0}, {'name': '304', 'value': 0}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 0}, {'name': '309', 'value': 0}, {'name': '310', 'value': 0}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 0}, {'name': '314', 'value': 0}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 0}, {'name': '318', 'value': 0}, {'name': '319', 'value': 0}, {'name': '320', 'value': 0}, {'name': '321', 'value': 0}, {'name': '322', 'value': 0}, {'name': '323', 'value': 0}, {'name': '324', 'value': 0}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 0}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 0}, {'name': '332', 'value': 0}, {'name': '333', 'value': 0}, {'name': '334', 'value': 0}, {'name': '335', 'value': 0}, {'name': '336', 'value': 0}, {'name': '337', 'value': 0}, {'name': '338', 'value': 0}, {'name': '339', 'value': 0}, {'name': '340', 'value': 0}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 0}, {'name': '345', 'value': 0}, {'name': '346', 'value': 0}, {'name': '347', 'value': 0}, {'name': '348', 'value': 0}, {'name': '349', 'value': 0}, {'name': '350', 'value': 0}, {'name': '351', 'value': 0}, {'name': '352', 'value': 0}, {'name': '353', 'value': 0}, {'name': '354', 'value': 0}, {'name': '355', 'value': 0}, {'name': '356', 'value': 0}, {'name': '357', 'value': 0}, {'name': '358', 'value': 0}, {'name': '359', 'value': 0}, {'name': '360', 'value': 0}, {'name': '361', 'value': 0}, {'name': '362', 'value': 0}, {'name': '363', 'value': 0}, {'name': '364', 'value': 0}, {'name': '365', 'value': 0}, {'name': '366', 'value': 0}, {'name': '367', 'value': 0}, {'name': '368', 'value': 0}, {'name': '369', 'value': 0}, {'name': '370', 'value': 0}, {'name': '371', 'value': 0}, {'name': '372', 'value': 0}, {'name': '373', 'value': 0}, {'name': '374', 'value': 0}, {'name': '375', 'value': 0}, {'name': '376', 'value': 0}, {'name': '377', 'value': 0}, {'name': '378', 'value': 0}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 0}, {'name': '382', 'value': 0}, {'name': '383', 'value': 0}, {'name': '384', 'value': 0}, {'name': '385', 'value': 0}, {'name': '386', 'value': 0}, {'name': '387', 'value': 0}, {'name': '388', 'value': 0}, {'name': '389', 'value': 0}, {'name': '390', 'value': 0}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 0}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 0}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 0}, {'name': '403', 'value': 0}, {'name': '404', 'value': 0}, {'name': '405', 'value': 0}, {'name': '406', 'value': 0}, {'name': '407', 'value': 0}, {'name': '408', 'value': 0}, {'name': '409', 'value': 0}, {'name': '410', 'value': 0}, {'name': '411', 'value': 0}, {'name': '412', 'value': 0}, {'name': '413', 'value': 0}, {'name': '414', 'value': 0}, {'name': '415', 'value': 0}, {'name': '416', 'value': 0}, {'name': '417', 'value': 0}, {'name': '418', 'value': 0}, {'name': '419', 'value': 0}, {'name': '420', 'value': 0}, {'name': '421', 'value': 0}, {'name': '422', 'value': 0}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 0}, {'name': '426', 'value': 0}, {'name': '427', 'value': 0}, {'name': '428', 'value': 0}, {'name': '429', 'value': 0}, {'name': '430', 'value': 0}, {'name': '431', 'value': 0}, {'name': '432', 'value': 0}, {'name': '433', 'value': 0}, {'name': '434', 'value': 0}, {'name': '435', 'value': 0}, {'name': '436', 'value': 0}, {'name': '437', 'value': 0}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 0}, {'name': '443', 'value': 0}, {'name': '444', 'value': 0}, {'name': '445', 'value': 0}, {'name': '446', 'value': 0}, {'name': '447', 'value': 0}, {'name': '448', 'value': 0}, {'name': '449', 'value': 0}, {'name': '450', 'value': 0}, {'name': '451', 'value': 0}, {'name': '452', 'value': 0}, {'name': '453', 'value': 0}, {'name': '454', 'value': 0}, {'name': '455', 'value': 0}, {'name': '456', 'value': 0}, {'name': '457', 'value': 0}, {'name': '458', 'value': 0}, {'name': '459', 'value': 0}, {'name': '460', 'value': 0}, {'name': '461', 'value': 0}, {'name': '462', 'value': 0}, {'name': '463', 'value': 0}, {'name': '464', 'value': 0}, {'name': '465', 'value': 0}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 0}, {'name': '469', 'value': 0}, {'name': '470', 'value': 0}, {'name': '471', 'value': 0}, {'name': '472', 'value': 0}, {'name': '473', 'value': 0}, {'name': '474', 'value': 0}, {'name': '475', 'value': 0}, {'name': '476', 'value': 0}, {'name': '477', 'value': 0}, {'name': '478', 'value': 0}, {'name': '479', 'value': 0}, {'name': '480', 'value': 0}, {'name': '481', 'value': 0}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 0}, {'name': '485', 'value': 0}, {'name': '486', 'value': 0}, {'name': '487', 'value': 0}, {'name': '488', 'value': 0}, {'name': '489', 'value': 0}, {'name': '490', 'value': 0}, {'name': '491', 'value': 0}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 0}, {'name': '495', 'value': 0}, {'name': '496', 'value': 0}, {'name': '497', 'value': 0}, {'name': '498', 'value': 0}, {'name': '499', 'value': 0}, {'name': '500', 'value': 0}]}, 'PayerId_stringindexer_transform': {'name': 'PayerId_stringindexer_transform', 'series': [{'name': '1', 'value': 6}, {'name': '2', 'value': 49}, {'name': '3', 'value': 3}, {'name': '4', 'value': 0}, {'name': '5', 'value': 0}, {'name': '6', 'value': 0}, {'name': '7', 'value': 4}, {'name': '8', 'value': 1}, {'name': '9', 'value': 26}, {'name': '10', 'value': 19}, {'name': '11', 'value': 0}, {'name': '12', 'value': 5}, {'name': '13', 'value': 8}, {'name': '14', 'value': 1}, {'name': '15', 'value': 0}, {'name': '16', 'value': 0}, {'name': '17', 'value': 1}, {'name': '18', 'value': 4}, {'name': '19', 'value': 2}, {'name': '20', 'value': 1}, {'name': '21', 'value': 0}, {'name': '22', 'value': 3}, {'name': '23', 'value': 1}, {'name': '24', 'value': 0}, {'name': '25', 'value': 14}, {'name': '26', 'value': 1}, {'name': '27', 'value': 1}, {'name': '28', 'value': 5}, {'name': '29', 'value': 35}, {'name': '30', 'value': 2}, {'name': '31', 'value': 0}, {'name': '32', 'value': 1}, {'name': '33', 'value': 29}, {'name': '34', 'value': 46}, {'name': '35', 'value': 3}, {'name': '36', 'value': 0}, {'name': '37', 'value': 0}, {'name': '38', 'value': 0}, {'name': '39', 'value': 9}, {'name': '40', 'value': 0}, {'name': '41', 'value': 1}, {'name': '42', 'value': 1}, {'name': '43', 'value': 13}, {'name': '44', 'value': 0}, {'name': '45', 'value': 0}, {'name': '46', 'value': 1}, {'name': '47', 'value': 5}, {'name': '48', 'value': 2}, {'name': '49', 'value': 0}, {'name': '50', 'value': 5}, {'name': '51', 'value': 40}, {'name': '52', 'value': 1}, {'name': '53', 'value': 0}, {'name': '54', 'value': 0}, {'name': '55', 'value': 1}, {'name': '56', 'value': 0}, {'name': '57', 'value': 1}, {'name': '58', 'value': 5}, {'name': '59', 'value': 0}, {'name': '60', 'value': 17}, {'name': '61', 'value': 1}, {'name': '62', 'value': 0}, {'name': '63', 'value': 11}, {'name': '64', 'value': 0}, {'name': '65', 'value': 0}, {'name': '66', 'value': 7}, {'name': '67', 'value': 1}, {'name': '68', 'value': 0}, {'name': '69', 'value': 0}, {'name': '70', 'value': 0}, {'name': '71', 'value': 3}, {'name': '72', 'value': 0}, {'name': '73', 'value': 1}, {'name': '74', 'value': 0}, {'name': '75', 'value': 6}, {'name': '76', 'value': 16}, {'name': '77', 'value': 15}, {'name': '78', 'value': 1}, {'name': '79', 'value': 0}, {'name': '80', 'value': 27}, {'name': '81', 'value': 0}, {'name': '82', 'value': 0}, {'name': '83', 'value': 5}, {'name': '84', 'value': 0}, {'name': '85', 'value': 4}, {'name': '86', 'value': 0}, {'name': '87', 'value': 2}, {'name': '88', 'value': 0}, {'name': '89', 'value': 1}, {'name': '90', 'value': 0}, {'name': '91', 'value': 0}, {'name': '92', 'value': 0}, {'name': '93', 'value': 0}, {'name': '94', 'value': 0}, {'name': '95', 'value': 0}, {'name': '96', 'value': 1}, {'name': '97', 'value': 0}, {'name': '98', 'value': 22}, {'name': '99', 'value': 0}, {'name': '100', 'value': 0}, {'name': '101', 'value': 12}, {'name': '102', 'value': 1}, {'name': '103', 'value': 6}, {'name': '104', 'value': 0}, {'name': '105', 'value': 0}, {'name': '106', 'value': 0}, {'name': '107', 'value': 0}, {'name': '108', 'value': 4}, {'name': '109', 'value': 0}, {'name': '110', 'value': 20}, {'name': '111', 'value': 1}, {'name': '112', 'value': 0}, {'name': '113', 'value': 10}, {'name': '114', 'value': 0}, {'name': '115', 'value': 21}, {'name': '116', 'value': 0}, {'name': '117', 'value': 2}, {'name': '118', 'value': 0}, {'name': '119', 'value': 1}, {'name': '120', 'value': 1}, {'name': '121', 'value': 0}, {'name': '122', 'value': 4}, {'name': '123', 'value': 9}, {'name': '124', 'value': 0}, {'name': '125', 'value': 22}, {'name': '126', 'value': 0}, {'name': '127', 'value': 3}, {'name': '128', 'value': 0}, {'name': '129', 'value': 12}, {'name': '130', 'value': 1}, {'name': '131', 'value': 2}, {'name': '132', 'value': 3}, {'name': '133', 'value': 0}, {'name': '134', 'value': 0}, {'name': '135', 'value': 2}, {'name': '136', 'value': 0}, {'name': '137', 'value': 0}, {'name': '138', 'value': 0}, {'name': '139', 'value': 30}, {'name': '140', 'value': 1}, {'name': '141', 'value': 0}, {'name': '142', 'value': 0}, {'name': '143', 'value': 0}, {'name': '144', 'value': 2}, {'name': '145', 'value': 8}, {'name': '146', 'value': 0}, {'name': '147', 'value': 3}, {'name': '148', 'value': 0}, {'name': '149', 'value': 10}, {'name': '150', 'value': 1}, {'name': '151', 'value': 5}, {'name': '152', 'value': 0}, {'name': '153', 'value': 0}, {'name': '154', 'value': 0}, {'name': '155', 'value': 7}, {'name': '156', 'value': 2}, {'name': '157', 'value': 10}, {'name': '158', 'value': 0}, {'name': '159', 'value': 0}, {'name': '160', 'value': 2}, {'name': '161', 'value': 0}, {'name': '162', 'value': 0}, {'name': '163', 'value': 1}, {'name': '164', 'value': 36}, {'name': '165', 'value': 0}, {'name': '166', 'value': 2}, {'name': '167', 'value': 0}, {'name': '168', 'value': 14}, {'name': '169', 'value': 7}, {'name': '170', 'value': 1}, {'name': '171', 'value': 0}, {'name': '172', 'value': 48}, {'name': '173', 'value': 1}, {'name': '174', 'value': 24}, {'name': '175', 'value': 0}, {'name': '176', 'value': 0}, {'name': '177', 'value': 0}, {'name': '178', 'value': 7}, {'name': '179', 'value': 9}, {'name': '180', 'value': 1}, {'name': '181', 'value': 4}, {'name': '182', 'value': 0}, {'name': '183', 'value': 0}, {'name': '184', 'value': 0}, {'name': '185', 'value': 4}, {'name': '186', 'value': 7}, {'name': '187', 'value': 0}, {'name': '188', 'value': 2}, {'name': '189', 'value': 0}, {'name': '190', 'value': 32}, {'name': '191', 'value': 0}, {'name': '192', 'value': 7}, {'name': '193', 'value': 4}, {'name': '194', 'value': 0}, {'name': '195', 'value': 4}, {'name': '196', 'value': 2}, {'name': '197', 'value': 11}, {'name': '198', 'value': 6}, {'name': '199', 'value': 1}, {'name': '200', 'value': 4}, {'name': '201', 'value': 6}, {'name': '202', 'value': 0}, {'name': '203', 'value': 2}, {'name': '204', 'value': 3}, {'name': '205', 'value': 0}, {'name': '206', 'value': 0}, {'name': '207', 'value': 4}, {'name': '208', 'value': 0}, {'name': '209', 'value': 1}, {'name': '210', 'value': 1}, {'name': '211', 'value': 4}, {'name': '212', 'value': 16}, {'name': '213', 'value': 0}, {'name': '214', 'value': 0}, {'name': '215', 'value': 6}, {'name': '216', 'value': 1}, {'name': '217', 'value': 0}, {'name': '218', 'value': 0}, {'name': '219', 'value': 0}, {'name': '220', 'value': 1}, {'name': '221', 'value': 1}, {'name': '222', 'value': 4}, {'name': '223', 'value': 10}, {'name': '224', 'value': 3}, {'name': '225', 'value': 7}, {'name': '226', 'value': 13}, {'name': '227', 'value': 0}, {'name': '228', 'value': 0}, {'name': '229', 'value': 11}, {'name': '230', 'value': 0}, {'name': '231', 'value': 0}, {'name': '232', 'value': 3}, {'name': '233', 'value': 4}, {'name': '234', 'value': 4}, {'name': '235', 'value': 0}, {'name': '236', 'value': 6}, {'name': '237', 'value': 3}, {'name': '238', 'value': 2}, {'name': '239', 'value': 5}, {'name': '240', 'value': 0}, {'name': '241', 'value': 18}, {'name': '242', 'value': 0}, {'name': '243', 'value': 2}, {'name': '244', 'value': 0}, {'name': '245', 'value': 5}, {'name': '246', 'value': 5}, {'name': '247', 'value': 2}, {'name': '248', 'value': 10}, {'name': '249', 'value': 0}, {'name': '250', 'value': 1}, {'name': '251', 'value': 3}, {'name': '252', 'value': 0}, {'name': '253', 'value': 0}, {'name': '254', 'value': 0}, {'name': '255', 'value': 3}, {'name': '256', 'value': 1}, {'name': '257', 'value': 0}, {'name': '258', 'value': 4}, {'name': '259', 'value': 26}, {'name': '260', 'value': 15}, {'name': '261', 'value': 6}, {'name': '262', 'value': 2}, {'name': '263', 'value': 2}, {'name': '264', 'value': 5}, {'name': '265', 'value': 5}, {'name': '266', 'value': 0}, {'name': '267', 'value': 25}, {'name': '268', 'value': 0}, {'name': '269', 'value': 0}, {'name': '270', 'value': 1}, {'name': '271', 'value': 9}, {'name': '272', 'value': 2}, {'name': '273', 'value': 0}, {'name': '274', 'value': 0}, {'name': '275', 'value': 0}, {'name': '276', 'value': 0}, {'name': '277', 'value': 0}, {'name': '278', 'value': 19}, {'name': '279', 'value': 0}, {'name': '280', 'value': 0}, {'name': '281', 'value': 23}, {'name': '282', 'value': 0}, {'name': '283', 'value': 0}, {'name': '284', 'value': 3}, {'name': '285', 'value': 27}, {'name': '286', 'value': 4}, {'name': '287', 'value': 0}, {'name': '288', 'value': 6}, {'name': '289', 'value': 0}, {'name': '290', 'value': 0}, {'name': '291', 'value': 28}, {'name': '292', 'value': 4}, {'name': '293', 'value': 0}, {'name': '294', 'value': 3}, {'name': '295', 'value': 6}, {'name': '296', 'value': 3}, {'name': '297', 'value': 8}, {'name': '298', 'value': 0}, {'name': '299', 'value': 1}, {'name': '300', 'value': 11}, {'name': '301', 'value': 0}, {'name': '302', 'value': 8}, {'name': '303', 'value': 0}, {'name': '304', 'value': 7}, {'name': '305', 'value': 0}, {'name': '306', 'value': 0}, {'name': '307', 'value': 0}, {'name': '308', 'value': 1}, {'name': '309', 'value': 0}, {'name': '310', 'value': 3}, {'name': '311', 'value': 0}, {'name': '312', 'value': 0}, {'name': '313', 'value': 38}, {'name': '314', 'value': 7}, {'name': '315', 'value': 0}, {'name': '316', 'value': 0}, {'name': '317', 'value': 4}, {'name': '318', 'value': 1}, {'name': '319', 'value': 2}, {'name': '320', 'value': 13}, {'name': '321', 'value': 10}, {'name': '322', 'value': 1}, {'name': '323', 'value': 31}, {'name': '324', 'value': 1}, {'name': '325', 'value': 0}, {'name': '326', 'value': 0}, {'name': '327', 'value': 0}, {'name': '328', 'value': 6}, {'name': '329', 'value': 0}, {'name': '330', 'value': 0}, {'name': '331', 'value': 1}, {'name': '332', 'value': 1}, {'name': '333', 'value': 0}, {'name': '334', 'value': 3}, {'name': '335', 'value': 44}, {'name': '336', 'value': 3}, {'name': '337', 'value': 1}, {'name': '338', 'value': 5}, {'name': '339', 'value': 0}, {'name': '340', 'value': 1}, {'name': '341', 'value': 0}, {'name': '342', 'value': 0}, {'name': '343', 'value': 0}, {'name': '344', 'value': 23}, {'name': '345', 'value': 0}, {'name': '346', 'value': 2}, {'name': '347', 'value': 1}, {'name': '348', 'value': 15}, {'name': '349', 'value': 3}, {'name': '350', 'value': 1}, {'name': '351', 'value': 0}, {'name': '352', 'value': 1}, {'name': '353', 'value': 43}, {'name': '354', 'value': 0}, {'name': '355', 'value': 1}, {'name': '356', 'value': 3}, {'name': '357', 'value': 0}, {'name': '358', 'value': 2}, {'name': '359', 'value': 0}, {'name': '360', 'value': 2}, {'name': '361', 'value': 1}, {'name': '362', 'value': 2}, {'name': '363', 'value': 0}, {'name': '364', 'value': 4}, {'name': '365', 'value': 4}, {'name': '366', 'value': 20}, {'name': '367', 'value': 1}, {'name': '368', 'value': 41}, {'name': '369', 'value': 0}, {'name': '370', 'value': 3}, {'name': '371', 'value': 3}, {'name': '372', 'value': 1}, {'name': '373', 'value': 0}, {'name': '374', 'value': 2}, {'name': '375', 'value': 0}, {'name': '376', 'value': 14}, {'name': '377', 'value': 3}, {'name': '378', 'value': 5}, {'name': '379', 'value': 0}, {'name': '380', 'value': 0}, {'name': '381', 'value': 1}, {'name': '382', 'value': 1}, {'name': '383', 'value': 0}, {'name': '384', 'value': 33}, {'name': '385', 'value': 5}, {'name': '386', 'value': 0}, {'name': '387', 'value': 8}, {'name': '388', 'value': 0}, {'name': '389', 'value': 1}, {'name': '390', 'value': 1}, {'name': '391', 'value': 0}, {'name': '392', 'value': 0}, {'name': '393', 'value': 0}, {'name': '394', 'value': 9}, {'name': '395', 'value': 0}, {'name': '396', 'value': 0}, {'name': '397', 'value': 2}, {'name': '398', 'value': 0}, {'name': '399', 'value': 0}, {'name': '400', 'value': 0}, {'name': '401', 'value': 0}, {'name': '402', 'value': 1}, {'name': '403', 'value': 7}, {'name': '404', 'value': 3}, {'name': '405', 'value': 28}, {'name': '406', 'value': 0}, {'name': '407', 'value': 1}, {'name': '408', 'value': 0}, {'name': '409', 'value': 5}, {'name': '410', 'value': 0}, {'name': '411', 'value': 4}, {'name': '412', 'value': 0}, {'name': '413', 'value': 50}, {'name': '414', 'value': 1}, {'name': '415', 'value': 2}, {'name': '416', 'value': 17}, {'name': '417', 'value': 3}, {'name': '418', 'value': 1}, {'name': '419', 'value': 0}, {'name': '420', 'value': 2}, {'name': '421', 'value': 0}, {'name': '422', 'value': 25}, {'name': '423', 'value': 0}, {'name': '424', 'value': 0}, {'name': '425', 'value': 1}, {'name': '426', 'value': 4}, {'name': '427', 'value': 2}, {'name': '428', 'value': 0}, {'name': '429', 'value': 1}, {'name': '430', 'value': 42}, {'name': '431', 'value': 0}, {'name': '432', 'value': 1}, {'name': '433', 'value': 47}, {'name': '434', 'value': 3}, {'name': '435', 'value': 0}, {'name': '436', 'value': 1}, {'name': '437', 'value': 18}, {'name': '438', 'value': 0}, {'name': '439', 'value': 0}, {'name': '440', 'value': 0}, {'name': '441', 'value': 0}, {'name': '442', 'value': 34}, {'name': '443', 'value': 16}, {'name': '444', 'value': 1}, {'name': '445', 'value': 1}, {'name': '446', 'value': 3}, {'name': '447', 'value': 0}, {'name': '448', 'value': 9}, {'name': '449', 'value': 0}, {'name': '450', 'value': 45}, {'name': '451', 'value': 6}, {'name': '452', 'value': 24}, {'name': '453', 'value': 0}, {'name': '454', 'value': 3}, {'name': '455', 'value': 37}, {'name': '456', 'value': 39}, {'name': '457', 'value': 1}, {'name': '458', 'value': 0}, {'name': '459', 'value': 4}, {'name': '460', 'value': 2}, {'name': '461', 'value': 0}, {'name': '462', 'value': 1}, {'name': '463', 'value': 3}, {'name': '464', 'value': 1}, {'name': '465', 'value': 2}, {'name': '466', 'value': 0}, {'name': '467', 'value': 0}, {'name': '468', 'value': 1}, {'name': '469', 'value': 21}, {'name': '470', 'value': 1}, {'name': '471', 'value': 2}, {'name': '472', 'value': 0}, {'name': '473', 'value': 1}, {'name': '474', 'value': 2}, {'name': '475', 'value': 12}, {'name': '476', 'value': 0}, {'name': '477', 'value': 1}, {'name': '478', 'value': 0}, {'name': '479', 'value': 11}, {'name': '480', 'value': 4}, {'name': '481', 'value': 6}, {'name': '482', 'value': 0}, {'name': '483', 'value': 0}, {'name': '484', 'value': 2}, {'name': '485', 'value': 12}, {'name': '486', 'value': 2}, {'name': '487', 'value': 2}, {'name': '488', 'value': 0}, {'name': '489', 'value': 1}, {'name': '490', 'value': 8}, {'name': '491', 'value': 6}, {'name': '492', 'value': 0}, {'name': '493', 'value': 0}, {'name': '494', 'value': 1}, {'name': '495', 'value': 7}, {'name': '496', 'value': 7}, {'name': '497', 'value': 6}, {'name': '498', 'value': 1}, {'name': '499', 'value': 0}, {'name': '500', 'value': 1}]}}}, 'ManualFE': {'FE': [{'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'max': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'transformationsData': [{'feature_label': 'DateofService', 'transformation_label': 'Extract Date'}], 'feature': 'DateofService', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False, 'updatedLabel': 'DateofService'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Amount', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '357.38', 'stddev': '823.57', 'min': '45.0', 'max': '6688.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'Amount'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PatientBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '-1.54', 'stddev': '18.4', 'min': '-250.1', 'max': '123.41', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientBal'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'InsuranceBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.27', 'stddev': '3.94', 'min': '0.0', 'max': '76.68', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'InsuranceBal'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PrimaryInsuranceBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.15', 'stddev': '3.43', 'min': '0.0', 'max': '76.68', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryInsuranceBal'}, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '91232.81', 'stddev': '6100.5', 'min': '67028', 'max': 'J9035', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '5.5', 'stddev': '12.4', 'min': '0000', 'max': 'RT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'B58.01', 'max': 'T85.29XA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'C69.22', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'IsBillable', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '1.0', 'stddev': '0.0', 'min': '1.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'IsBillable'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Isauthorized', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'Isauthorized'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PredeterminationIndicator', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'PredeterminationIndicator'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'MedNecessityRequiredIndicator', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'MedNecessityRequiredIndic...'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Age', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '67.76', 'stddev': '16.27', 'min': '5.0', 'max': '102.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'Age'}, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'label', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.91', 'stddev': '0.29', 'min': '0.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'label'}, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Gender_Recoded', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '500', 'mean': '1.45', 'stddev': '0.5', 'min': '1', 'max': '2', 'missing': '0'}, 'updatedLabel': 'Gender_Recoded'}, {'feature': 'ChargeDetailID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '249.5', 'stddev': '144.48', 'min': '0.0', 'max': '499.0', 'missing': '0'}, 'updatedLabel': 'ChargeDetailID_stringinde...'}, {'feature': 'DateofService_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '14.58', 'stddev': '8.27', 'min': '1', 'max': '31', 'missing': '0'}, 'updatedLabel': 'DateofService_dayofmonth'}, {'feature': 'DateofService_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '11.44', 'stddev': '0.5', 'min': '11', 'max': '12', 'missing': '0'}, 'updatedLabel': 'DateofService_month'}, {'feature': 'DateofService_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '2020.0', 'stddev': '0.0', 'min': '2020', 'max': '2020', 'missing': '0'}, 'updatedLabel': 'DateofService_year'}, {'feature': 'CPTCode_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '2.18', 'stddev': '3.55', 'min': '0.0', 'max': '22.0', 'missing': '0'}, 'updatedLabel': 'CPTCode_stringindexer_tra...'}, {'feature': 'CptModifier_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '0.42', 'stddev': '0.98', 'min': '0.0', 'max': '6.0', 'missing': '0'}, 'updatedLabel': 'CptModifier_stringindexer...'}, {'feature': 'pdiagnosis1_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '33.73', 'stddev': '38.52', 'min': '0.0', 'max': '148.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis1_stringindexer...'}, {'feature': 'pdiagnosis2_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '35.52', 'stddev': '41.1', 'min': '0.0', 'max': '155.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis2_stringindexer...'}, {'feature': 'Gender_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '500', 'mean': '0.45', 'stddev': '0.5', 'min': '0', 'max': '1', 'missing': '0'}, 'updatedLabel': 'Gender_stringindexer_tran...'}, {'feature': 'DepartmentCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '0.91', 'stddev': '1.2', 'min': '0.0', 'max': '7.0', 'missing': '0'}, 'updatedLabel': 'DepartmentCategory_string...'}, {'feature': 'SpecialityCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0'}, 'updatedLabel': 'SpecialityCategory_string...'}, {'feature': 'PayerId_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '4.89', 'stddev': '9.31', 'min': '0.0', 'max': '50.0', 'missing': '0'}, 'updatedLabel': 'PayerId_stringindexer_tra...'}], 'dataPercentage': '10', 'samplecount': 500, 'originalcount': 18968, 'FEGraph': [{'values': {'label': ['174DA322-870E-429F-A9A9-BE22340E9836', 'A1575911-0E83-4479-B182-D26EDAD439F6', 'A87EF1E9-E636-4742-8E2B-414477926782', '1FFF3E45-B9F4-4CFB-8580-C4B05D4822DD', '20BC5C22-2C34-494C-B75F-55BDD8B54263', '210DAE19-9257-4180-9057-E5817F555A88', '3FA21D4D-05B5-4C43-A33E-A01D20B45A99', 'DC65D7C3-0822-4E54-86EF-DC7639D38C62', '11327DC4-1599-46B2-A000-C05D135E1F91', '853B5BAB-0A5F-4B44-85E0-FEE0AA663796', '92F7294B-4F6C-41B5-A9BD-65CE72B27FC0', '97BB8C89-8E47-400B-B546-B662E9EDCA00', '9C7B2481-4469-4FB8-AC5D-18EB2D29F213', 'BCE331FC-BC9E-43B2-B9C3-1ADE95CF4081', '0391FBA9-7748-4F50-8CA2-FC4EB5116F6A', '88D923C8-5217-46AF-887E-CB88A2E6C3E3', '8AAE78A7-7725-4C1C-B55C-82B41A364A2B', 'B5C5BD20-64AE-4166-8115-14DA13624764', '343273F8-E598-4815-A700-0683D31F3291', '50F7ADC5-2166-4461-B01E-8D8FAA92304B', 'A9EC659A-ED07-4B6C-84CA-D4E45411C3C6', 'DCBDA7E0-A5F8-4D0F-836D-66187FEB5B6F', 'E907166F-CE0F-49D2-A04B-40734F1A6F0C', '6518F89E-CD31-4903-89E2-2F31BEB28508', '7B41BFB4-B1D7-47A9-9659-6E8319429423', '7E4D4049-B1BC-4135-8020-D69318B66B18', '94FEF155-5ED7-4DF9-836F-78557484B8E4', 'A6B1DE4F-2FBD-4AB9-B09E-77B3E8C44771', 'B9F344F0-CEE0-4EA2-A55C-274FAB39E9DB', 'BBE4E59F-B282-490E-8FCD-6C9F5E8069EB', 'EC206AC2-403A-4047-B82E-82C383E29CCE', '01867531-8532-4FD4-BE17-232F39CFA32D', '2ECD88B7-05A0-4959-8736-397F8CF74784', '524D7B98-2D7C-458D-8877-35CDE8F3D981', '54E3991B-1609-470D-8095-C14FF5A9444D', '8F8BDCC6-5B74-4EBA-A154-8576DE0FFC13', 'A28023B3-9D64-44D3-BD13-2A22A1350A24', '1B71D2B0-C473-4A8D-9FA0-76A0692C5102', '6CF3754A-BC46-430C-A116-9329959CD556', 'A6CED81D-7B1D-441A-8EFC-544E8D31FD26', 'A82270CF-0B59-4C62-877D-9FAF720B6E5D', 'D9A3771E-E97A-4EBE-BFA9-54577F445F21', '1AA261AC-C877-4F0D-8372-F268A4D5F90B', '2B385821-C105-42CA-84F0-CEF8BFD1681A', '9D5CF424-8DA6-4A2B-8716-FDB93B808F3C', 'A17DF3A2-8077-49B4-A114-11F77F67E7C9', '0E47C422-450E-4A04-9B90-680F296C9686', '46CB6A8C-362F-4345-941E-842F429B48B5', '8CD490B4-CEBE-4DFE-BA0A-C805ACDF0D34', '9F54BD25-7C67-454A-B100-CD9974B62C7D', 'A78BD078-2165-4C68-B398-9AB6E94C0FB6', 'C72EBDAD-1D1D-40D0-BA4D-A9CDD08430A6', 'DBFEC39B-37B2-4B37-9D95-A444C27A619C', 'E1C8F5F6-2DDF-4370-B40C-D86322919657', 'EB7C11AD-0A01-4359-B1AB-289373C669DC', '16062E35-A49E-4227-A3E6-E8DDBFEBA5D2', '20416F30-920B-4925-8642-C83F38ECFCEE', '642D42E4-275F-4E86-8378-21E2CD291224', 'FBB301BB-1560-415C-91B5-A6E8B0B26B6C', '4011D505-BE33-4A24-AD40-240BFDDB2766', '74FC2B0E-AEC7-4263-951F-14E72EF044DA', 'A5881100-AA89-4D5D-80AC-AE2419271C5D', 'D0667457-EF74-4FA4-91D9-7169C64A53DB', 'E16D36F3-C4D2-4C4B-A004-E06C823C7960', '20463C7A-45B7-4BD4-A3EA-53ACC03E93F1', '6E834EA0-8A83-4326-8FAE-80C6DEE3B045', '8607EC2B-26F1-4A40-9646-5990D37EC622', 'A050F893-D9BB-4FB6-9B4B-B7AA83355D60', 'C2F471A9-22FA-4A9A-A14C-49A5F292662D', 'D6F6C707-D42D-4485-A571-82EEF5002E64', 'E164830A-C381-4AEC-81EE-3B116E56B496', '2701DB53-B3A0-466C-BFCB-6684D7059DC4', '89D7AAB0-BF88-493E-A7AB-7D6D793273DB', '8A07AC76-56FD-4E13-BFEA-A724E4C2D603', '8E5FAFBE-87F5-4C26-88EA-002384A50B1D', 'E67C3D6F-A4D2-4EA9-B41A-E5BEB54E0209', 'AC68E17B-9EBB-45A3-BD22-60367A203515', 'D0BA312C-C42D-4695-A6E4-71096BE6854B', '50C6C060-08F2-4F75-91BE-EF1D34F6EDA9', 'C99C5CC6-DF6E-4024-88A6-04D326F6D6B2', '1A70E6C3-0B4F-4338-A15C-903F552EC30C', '279BE041-8831-43CF-A0EB-219F709A986F', '3E0A6D69-9B07-4BFC-B389-24E0C884701E', '9017B6B5-6C3F-42FC-ABD4-67A54E07D67F', 'F16597BA-3D84-47E3-AF6C-072ECBD33467', '2AA7F9ED-3694-4ADD-91A8-3EFB8F8054FB', '45F8D74C-DE28-4F0D-B781-AB81CCD5D5CD', 'D7B8BD7C-A3F9-422B-B491-59F236BADE4D', 'E9DB7A1D-67B8-4936-B7C7-FB071EA72907', '2B6E92D7-EA15-4F53-B133-2A1F9B1E958C', '86E8386B-2651-4928-8CD1-631CE24D0432', '97D21C13-7AFC-4865-888F-5E92CC51F2ED', 'A40F161A-A931-41D9-A65B-597347BC9D6A', 'C43925F7-AA9E-419B-B0DB-0A044E67C4B5', 'D05F93BC-037E-4BD7-8E65-FDA46391791F', '14167C30-8123-4DCF-B3C5-05D3F16B6391', '2022006D-F4D6-4F8A-B76D-1617C97083CA', '25EF819E-5F4C-43DA-BEBB-C4FAD71B0EF5', '8648E212-02DB-4D77-BB02-88B67C56A9DE', 'EDDC8ACA-6742-4C6C-A200-FA42DD89285D', '03E0DCDC-A071-4987-A7CE-268D9C3D017B', '161B0D6C-5A50-4EEE-9B2A-E4DE8D9EBC1C', '1BD4EAF6-F3F8-4369-93B3-4E6144A69CAD', '20AC5B30-ACB1-4714-9794-5454B92C1A23', '3FE9456A-15CB-4840-90CB-05F7DF34CACE', '42F5D00A-6F59-4F63-ACCA-9EFDA0DD9194', '6E80EF7D-E761-4720-A1BD-9E2CA7FDD21F', '734CA534-2CD7-4A77-9D03-CF2390E8B0D6', 'B0501331-13EF-4126-B052-44D67EDFE7F9', 'C0B29F0F-BBE1-47C7-B625-0989FDCC5288', '136261A4-52B0-43B5-B099-1EF929506A93', '252D8E17-A9E8-48F5-84D3-6DA9B452EE75', '5A8270A6-53BD-4916-8805-B762FF67BE20', '67841660-18E4-45DE-B386-285C53582762', '9C60FA6D-FAAF-46A9-A149-1F941986D089', 'D6FE7A69-6B2B-495F-A36A-789DF588F6D5', 'D9C0CCE1-6D5D-4DB7-8376-D8562C150145', '42B818D0-170D-4838-BC4A-7D22039C0F53', '7DC8A130-8030-4915-8BF5-54015B27DAB6', 'A3555B0C-5A99-4619-B6E8-9831D99F0210', 'AECE54B4-2AE0-44A1-9409-2BFB52D07BD2', 'BE7612D0-7548-49C7-800E-173BAF9DF290', '0EB85789-CE25-4CCC-850E-44D2C3496272', '45F8ABD8-1D29-495F-B63D-866175B861A0', 'A69FF747-5E5E-44A3-A2AE-806C56D75743', 'B1C24F17-5A6A-4D76-A42E-A273CF94898F', '41D99DEA-233E-47A6-98C0-8B6F522D1B70', '53BD3672-31BA-4CE7-B2F3-2AFFC29C5F6F', '55F86AF5-F804-4A84-90ED-12E4125E1A8D', '8C94AF61-5EC3-44D7-B4B3-BBEB64AD2A23', 'B52B5682-9815-4EC1-A594-912A9724EB74', '079F8DAC-D8C1-48FA-9743-BC2AA08E287C', '2D60073D-3E0E-4347-B0A6-7E0CE91E92FA', '99772A77-B764-48FF-9242-5700C9572B65', 'E45AB682-6C1F-4B84-A1AC-3D1CD79AC3A9', '51526EAE-3FB7-4C7E-9A33-26990B70EFFE', '540B2A5F-11A9-4F0F-8038-B5A1ECE5E583', '91D48DF5-F63A-466A-AFB9-2E65C4F2D44D', 'ADF5D273-DB58-4F7F-8DF0-9D9D9647DA16', 'F672AF9B-E361-4572-A9C0-8B0731F9CD3E', '34803FD1-6C40-4323-8657-F1FCC3E34990', '5327CFC0-5C48-435B-98EB-A76AAE858941', 'D2DF16C4-D3AB-4E5C-9350-8F6FDE2EDA8D', 'E2DBCD71-B901-41F3-97B1-ED71030C7F2F', '049BA1F1-F362-48BE-9393-C05CCEA8694B', '13002A76-350E-4525-BDA1-9F181E056F74', '167E1195-CC7A-4F8C-8D91-9577BCBEE8FB', '1F264D61-D5D7-4505-9AC2-25ACF275E3BD', '23BCDE27-9F80-48E4-B29D-348C871829B3', 'A71A1D2F-3F1A-4CA6-AA98-E5D6A855C250', 'B62DA993-E320-4CEE-9D23-B3238CA5759A', 'B6375BF2-EFA5-44C9-A05C-F71DACD5F8A8', 'C924A1C9-2E9A-4BF3-8BC7-25BFAEB02CE1', 'D2300410-689E-453C-895D-FC7FDA559759', 'F12A6DBF-FBB4-4C62-8EE0-0F677D58EC3B', 'FA9EEE47-7F76-4264-9A82-9B370BCF9534', 'FBBC8CD5-C35A-42E5-AEBA-C5F61050B726', '045700E8-F572-417C-A8DD-41E76C5C159A', '1A088AE8-D8BA-404D-97E5-9DB1DFB972F8', '1C109ACE-6610-4098-BE1A-C64F7805EBCA', '25251236-8157-4EE2-89CD-C5A600E278BE', 'A1818C97-82BE-4115-8C3C-3532CB8DE838', 'AC9C3FBE-B694-4A6F-9A6F-6CC4563C5C73', 'B5E35D7B-A0F8-4A47-B72F-1D372236FC63', 'C1119267-721A-4606-9D06-09FF240505EC', 'C3339BC7-4E2E-4E0E-822E-7EE8E489451C', 'CDDFE064-998C-4CDD-97A2-04647A33EE7A', 'D4930936-02DA-4DBB-A3B2-5391AC5D7153', 'FD8029C5-4ACA-4F97-8CE6-144AE49C8BCB', '4C3D48F4-860F-49EF-812D-75B08DC1359C', '58207A31-753A-42B1-86D8-E750AEA10079', 'EAF0CF6B-9569-41B6-8B77-6335F34B3D7E', '693B32F4-ECA0-4844-80DF-6D9C94DD57BA', '7E2F8EA5-4A1D-4490-A208-49CE684BB427', '83A30AD9-1329-4675-99E9-AF5ACBE98CFA', '84F4A744-F946-4B1E-99DF-20E0538C5F4F', '90C2C4BC-4F1D-4730-8078-6CF025EB493F', 'AE214CAC-A747-4549-B4E0-957075E94F38', 'BD337B93-0D12-4AE2-9566-35705B577186', 'CE11AF94-7EB4-4B98-BC57-51593FD36B72', '1F6C5DD9-4889-43B7-8768-BA1D2873322F', '322A11BB-2B77-480A-BABA-2FD89CDD0305', 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', '3342F086-2D91-4081-BEE0-C18AEAB0C981', 'C4FFF996-0FB6-427D-9266-BE10E9F5DAA9', '510C2305-6CB3-460D-84C7-9EB6281F8E9D', 'FA7AC95E-1961-4A7B-AC57-3FD3212BAF20', '04CCB591-8B59-4CCB-9EB0-5A4AECB3C705', '0D66C132-CD95-442B-800E-E19F2651E018', '0ECEF203-5A40-428B-A19E-BFA699806763', '2A1A46A7-1E50-41C1-ADD1-EBC0F82168F9', '6C6C6230-52AE-4590-AFA7-42A14991BD69', '71B8A2A8-14EC-4B19-A4DA-6DE30E8A6E30', '2B3AACB7-A045-4E46-A2DB-8B841AD87F65', '2FCDBCD0-49FC-4AA6-B249-041E476D766A', '3DF7C0B1-EA0D-4014-916E-E76984AFDE2A', '8A1DE8B3-2FDE-4581-AC94-4EA74A997CB0', 'F49D6604-F705-480F-BCB9-E6F16B76DACF', '31C70405-0F52-4BD8-A785-13AFD15809A6', '323DACC7-ED74-42A3-A1E6-21BC104AAD51', '35841CCE-C3B1-4652-B011-C44F79BB442D', '8875D93D-866D-4897-81C3-67DB4570B102', 'A7A12FBC-72F0-4216-8360-126AF826214C', 'BDAADB25-9498-4D4F-809E-4590BAEE53B5', '0438621D-9EC2-4530-B3E8-F0C68FD8295F', '55E3B0D0-8E40-48F3-854B-A8E3C893E803', '793B38EA-690C-456B-9597-5444E6D1499F', '899B4439-339E-4CC1-AC1E-4C1F4BA312EB', 'A4CD837B-6B10-47C0-B9DC-E8D6ADF73B88', 'CC817F76-EE46-4AD5-9911-86A53FB1DBAE', 'F1C5E648-7E44-43B6-A884-48A861BA52BB', '3D2ECD44-776B-437D-9DD6-936DD5B7A4BF', 'AE3A61BE-5820-45D3-BCD1-9982D0BA93EF', 'C94A98D6-223E-47E8-9589-5631EBCB55E9', '05BBEB22-65F8-4773-89A4-452C6FD9DDE9', '0C7AF11E-16DB-461A-A412-B711F0810E4F', '8745504F-A5BD-470D-ABAD-CDA68F032BE0', '8B63DA7A-BB50-450B-BA2A-E68DFFF3F4F6', '0BC312A3-FBB5-4F2E-9A5C-6BAC5566A285', '1BA29057-DE25-4EF0-A06C-94A526C557A3', '8E5DABE9-6BE2-4D98-BF87-D87C6A70C35B', '9AE67B15-8F75-4CED-8129-80E003318733', 'D84605C6-4883-46E5-8BE3-4C82081FC5E0', 'EF5E604D-077C-4903-95D7-5568576B7F96', '14710135-067E-4E9D-91B1-E52FD6B72796', 'ADC1E05E-8451-4C66-976B-3DBBB7103A27', '1BDC9A8F-39D7-43D8-B3A8-5E9457AEB7CE', '45BD84A3-1EE3-4B60-8352-D372DFEB677C', '6F66C6AD-3D4F-4EC2-9C71-D7580A768CF5', '74E7B927-7E80-423E-8EBE-94FC0B7E514F', '82948A9C-6345-49ED-AB80-DD62B36CE11F', 'BA941CAA-6F1A-4BBD-9D4A-42C905C8C41A', 'ED3A209D-7398-4255-A9FB-3712B16F9DC5', 'FBCA9AD8-4FC2-48A9-B059-0347A0EDA4E1', '142DD0C0-133D-4FCB-934A-C6C50D564E61', '803ED239-BF70-4FCD-B02C-B5977404BA98', '967ABAFC-DBB1-4878-89E6-2B32E0E8387C', 'B3EE5A6F-6BC4-4F1D-B196-1127D5A7FED1', 'B77643EB-41F3-4EF0-818C-B4617862C124', 'CF7B7502-BFFD-479E-87EF-8C30089033C8', '04A3E774-1DF2-461B-8DCA-90EAB47932A0', '1054BD81-AB8A-4FDC-9A87-F41814188945', '202111D3-0B41-42EF-8113-161C06700D53', '23863534-0389-4F1E-BDDC-96C82F56F4C0', '906E67BB-7D18-4CBE-8802-172E4D09F677', 'A930D4B2-2DE6-4134-A58E-36E88012A437', '043D822F-B7D6-433B-B8F7-3A0F58BFFC22', '37409C70-626D-4D18-BFD4-749375A6D10F', 'C1DDC816-520D-4175-8613-E726EAF36CBA', 'CCC6C19C-C3C3-4F0B-8D42-083ABB99F0E9', 'FF11B590-D529-4372-A722-252F47C1AA76', 'AF3AFE4B-9BD1-42B1-9D03-5031CDB276F7', 'D45A198D-C1A5-4A54-8BCE-CD4AD0B2FBCF', 'E7C8D3FA-DCE9-4622-96F0-2047722D8564', '399470D8-4590-411F-8652-380D7FB48A6F', 'A8670196-A390-42F7-B247-84D40DCE973F', 'F39EE67E-0589-4138-9AEF-261777DFC2B1', 'FA571DDD-0422-48A6-BE3F-1DFAABE49A5D', '010ED4A3-EB64-4AE2-99CE-B458B59B538C', '1EA4A9FD-2C9B-4C54-9296-841FA5F9F23D', '3DAA771F-294A-4395-9B6B-30B898BC01E7', '411875F3-AAAD-48AC-92E6-112579D42F27', '7A0B900D-FB81-4652-AC59-4D27FB1BF2DA', 'D1CA4C65-B76E-4F88-991D-E39CDBCB9A25', 'D501A556-9400-4BBE-876D-4E2269A69E50', 'E44DA795-311C-464F-A98C-AF783E9A8687', '8800B6CD-48B2-414A-AA85-2FB6E2F94F62', '0C738DFF-CDE2-446B-8605-885F7FFEA814', '387FB6B8-8FE5-4483-A401-B595EF6BE611', '9E48B685-ED16-456A-9BDA-73F1CDE529D5', 'FF73DA08-83EA-4F38-8153-B686DFBD480C', '8E785A6C-A77D-46F3-B74F-701B18C72B2B', 'B881C1B6-9321-4EB8-901E-843FB577ECDE', '6C01513E-DC0D-48F4-85F7-5DE34C582CA7', '88E69525-5461-45A9-879A-21A99C906710', '980CDD7C-B8FF-4C24-9EB8-E66EF9502A25', '29761757-EC8E-4BC8-9BD4-764810C0D435', '2E9CCC26-EE83-4CE0-BBE6-78198DE26939', '3596B626-33E4-4438-A2D7-C7B7F2884A61', '722ED05A-EAA6-41C2-AB96-ED5E231D9CA5', '8D12EF16-4731-4FDD-802D-73518A27A384', 'BB4D3CAB-1412-4339-A438-A8AFB1ECCA82', '113F2B98-6796-4055-8712-8147E996D2FB', '19635F03-1921-4E20-A610-C32DCD0A86E0', '1EDBDDB5-5D98-4516-87A4-135EB82D051F', '28A393A5-A7B5-475F-8710-5DFB463F120E', '36009550-373F-4179-BD6D-42DC2B91652B', '8EEEA997-D2B5-4EC9-A50E-B2CBD0094405', 'C295E50E-9D1F-4967-B8F4-CE7ACB0DD858', 'F3A2BB7A-DCFC-4740-AB2C-51B4DE18A84F', '077A557E-846D-453A-AE7D-9B3003FFA94F', '0EDBE7EE-67FD-48BA-86E9-F70C9C504642', 'B06CBEAD-36DB-426E-84CE-89B56FD46159', '1DEC7009-90C7-4C0B-8617-D9E805BD39DC', '65AAE408-BFF7-45D0-98E7-78DFDAEC64EC', '823ACF06-0F8B-4DC7-AB68-4DFE535C2F92', 'B86E4CA3-AF97-467C-80C1-A05484D3A866', 'D525418D-529A-470D-B010-7B90CD096743', '15695896-D50A-4650-8607-86092DE66F86', '724A277E-2530-4A79-AC3F-6E4467BB2688', '7C334D9D-8773-4B52-871C-7F4CE8FED784', 'A15234F5-28B1-425F-8338-F05A4BF34B5A', '0CD457A8-5460-4F98-8059-107464922EFC', '1CE42B39-5F00-4C37-A276-6DFEE0CB161F', '2AD5B7D3-9824-461C-8E4C-F5088C5A5061', '6295FB65-95AE-465C-A7BE-93CDEA6FCD41', '9E709F1E-38BF-4B10-B126-7C344A21DD11', '9EB1990C-6484-48F0-9B12-F05AA4B6815E', 'E2272BDD-653A-47F9-A8BA-EEEDBC803C1B', 'FB109A88-81CE-4832-B4AC-56DD97251649', '53B02D9D-4160-42AA-936D-FCBB76F49B95', '99D51AF1-7AD3-4DD6-96F0-25AA1E59972F', '07A0559D-6D61-4D02-9F83-0730224B6C93', '4D06240B-F581-412B-B7B2-5F3A1060CB47', 'D9501B03-2C0C-4096-A3F1-A4EBEC12D7AE', 'EC536097-D4FA-4221-9C08-0C6168F80D46', '11077BFE-4169-4F6F-A104-75B18FD0FF87', '2EB8DC5F-2F79-46F0-A78F-A59491C148F6', '36EF7301-7E02-412A-9E23-47987397A484', '620495B1-AE1E-4BEA-881B-AA8645DDAAB3', '73A89B2A-DEAD-456A-95A2-EA941FF2DBFB', '83B3ABC6-1E23-4471-9F1E-78EA18B3CCE0', '9231D20B-208F-4B7A-BEEA-28718633282D', '9FE9533D-C993-42CC-A477-DA0FD4E5FF72', 'DA09EFC9-6D5E-4472-8CF3-C9017C22A30E', 'F9A0096A-9F38-42FA-B2FF-20B3F570FEB8', '116B1537-FBEA-479D-8725-F3F98CE8696A', '496D2DE6-BB81-4484-AC58-3BB0D13B4E8E', '8D6F6E45-269B-4E19-AB45-130E4C2F8EE5', 'BF7CB069-0FD9-44DA-A277-EDEB8C31E102', '02705462-64DD-4C6E-AB7A-2CDDB023FE7A', '3882B031-F286-4F77-8AAE-3ECC1E13433E', '9C0A75B2-5E85-479D-B614-3C97418C4570', 'F5C916EC-F2CA-4136-9312-5A71933D7C4C', '846A3024-1033-46EB-9F06-0F3E17DAD4AA', 'CBA90F0E-ADB0-4F8F-A849-CD75DB7DD2A1', 'D7A3A337-59AB-4474-8D56-FF4B8402E128', '15527B5F-A9E1-4797-A410-9E324DD82868', '5649E152-6876-444A-97D3-66E0C38D7661', 'BB2FFBE2-24D7-4618-B3FE-DC8B3ECA14AC', 'E760489F-6EE4-4B15-9B6D-5B12F6A5E3DC', 'FAD56DDC-6281-4433-94D8-23FCB65FD823', '74492123-CD52-45B6-AAFB-CDD634C4C9BC', '79956D3A-C616-4A7A-9F06-BA0DC2806695', '9C582B3F-8717-4E03-84C9-F6EAAC3F4948', '394DE176-F74B-4877-A640-EF3C34905AE0', '3F563146-EE95-44B0-B130-41E8E1DE3030', '48C82871-65CB-40BD-98D3-6EB4CB2E0F61', '49B6D9F9-0BB5-43E9-B453-A0D4616A3689', '5C1B73F4-3EDD-41BE-9FEC-7BAC25366092', '9D0125FC-58A9-4CD0-A823-57B6464A308F', '9E726301-D189-4224-945B-2FBA8FA26A03', 'A5EE1C1B-278D-478B-AC1F-980E200CB843', 'C8D69246-8334-4EBF-954E-5AAFA7C1F65F', 'CAABED88-1591-4E9D-82C4-7926EA41DD52', 'DD23176B-512C-49D2-AC85-CFAA4256C72D', 'FAD99EF5-F102-4FC6-BCCC-ECDEF99456BB', '2653D41E-11AE-432B-993D-8CC4D4DAA1FA', '71E5F463-049A-42A7-A510-EACEC14E8301', '86D2CB7E-E54C-40E4-BCDD-C6D0DE7A6D7C', 'AB9BAF44-7980-4636-B844-488341E49BA9', 'F66CDB59-6144-4776-B129-5FEED24970C3', 'F829CBA3-AD5D-4CF8-BE1F-3FDC44044AF7', '244D35E3-5D74-4F5C-BB72-5860E1D4561C', '2C3E981A-853E-45E7-87BD-799325176251', '3B1A2F21-FFBF-4454-B0FB-BCBF1CEA4FE3', '49FF4EFE-6A51-44EA-BDDA-27F6A4663C04', '6BD754C1-07F9-4C5D-A1B8-B77B284AFC7F', '71A70101-0BDF-43F3-B580-4DB046CB1EBD', '8076FF62-D8A1-4309-9EAD-F4B83E85DB1D', 'DB05D823-A5CC-4AD0-87B0-6FBF69B7493E', 'DC1BE341-E3CB-4EBF-B968-F5764D826DC7', 'DCF767F0-7167-41FC-B981-296FBE63588B', '46445DC3-6A22-4664-8F2F-C73C4F9B7010', '9F250BA0-F703-4F1D-8AD6-82F2F4E58B70', 'BCCDC778-B59A-44F9-B1B6-9B67462C4B0A', 'CF8CDC47-5AD3-442E-9F3C-07CF84EC35A5', 'EADDE82D-5C40-4BF2-8C07-A91EEAAF8EC5', 'F32643CC-08F3-4108-B5D1-4A881B590328', 'C9139FC7-C65B-4F97-B7CC-56F810FD45DF', 'ECA40D88-F3E1-4286-8064-FD546E3C844E', 'FDBE8690-4E2C-4BD4-991A-74F5F3CC2872', '04A27415-9D35-49E4-A5F5-305DFD917D5F', '04EA280C-12AB-4842-9913-E97DFDDF508F', '0CB92950-EE3F-424C-AA0A-445F2D738518', '29737DE0-22D6-4D41-870C-DA59BE4A4E75', 'AA77712E-61BA-45A8-8E10-28057191D585', 'D271CCF9-CBA9-4A48-8202-5ED6FE135BDA', 'F488446B-42F0-4C47-B313-AE5AD0AD4C08', '0DC89466-2E6B-4DC7-B913-E5BBE5B168A0', '27DEFE26-1A3A-4654-A273-799CEA02BB33', '3D48B549-0C2E-4EC1-AA68-CD7DE250D65A', '790CB21B-A420-4C48-8EBC-67AAC5413D69', '9DD3CB15-B27B-4F87-A3C8-A9FC8D1F56B5', '2F71902A-768E-4A58-BE77-2AB583872514', '305A3131-EFD4-420C-88B8-361D358D9857', '65920EF9-C672-4D37-9A6B-795B0F656EDD', '736AECC6-FFE6-48E7-9F0D-4AF0CDC06DED', '9DEA15FC-C28D-42CB-8283-AC5B84B7FCEF', '0B1BA7BE-21C6-4B0E-A25F-C198051FD597', '0E9EF90A-1AEA-403E-9058-D6F60A449238', '235B02A5-8AC3-47BD-86BB-CAA1A63C317B', '48A25AFB-573A-4213-9B1C-E3654825F634', '49C499CF-DD82-4195-B4F3-F889BE7E0BC4', '691933B0-0D3D-4355-B453-79E791D6DC36', '83B52DD3-F13E-450B-84A7-CAA79A556718', '842AC936-21A9-4F15-8BF5-ACF6D4756EB0', 'A3CC38B5-605E-476B-A987-2AC63C81F539', 'C3A37CEA-E1D8-4DB2-837C-D1041863EC7F', 'CF82712C-91A7-46D2-9F6D-19A1BC8A1A0D', '485494F4-D20A-4303-89BB-F09C5646A9EA', '4FEA26C5-DD7F-4A65-B859-D5FECFF66684', 'B48620D1-DD13-471B-936F-AE9970B850B6', 'BBF82897-F57E-484D-8159-C4E14BBECB5C', 'F0B25A0F-D965-460E-AA37-EA676B8BE27F', '279D822D-CECC-407C-8F7D-DA683C10C7B1', '27E0E710-3B33-4DEC-BED0-FDB55F95EEE1', '8793C097-7DC2-47B0-8639-CBD108B34F5F', '1D2CA217-3B28-4C61-89C2-7C55BAB37BDD', '3A4437B5-45EE-4734-BF9C-C7BA808C123B', '13872B75-E219-4B25-9F7E-24F021CF0B48', '377FADE6-202C-4841-BDD4-82367D36097C', '6A29D415-E299-4812-B8CD-8CD471F28115', '84A66A4F-CB48-4BA8-B11C-F2EA34C4A1A4', 'D7B0792D-0B5F-4CAA-9787-237DFECCA98B', '6D22E643-C022-4637-A81F-22A0DB410A33', '7C771536-6EAB-4FEA-B848-E077662307D7', 'DD0C3E18-53F4-4001-9334-BEC408540A3E', 'EA11E355-7A8A-4BAD-9066-1FC3E824CBC5', '267E6CA7-6B3A-4582-B01B-9DD2C4AC8DEE', '43B4ED70-4EAA-4C2A-AB60-F4A3848BF5E9', '7D59A1A0-3C61-4F7A-AD1E-0EF39B392E83', 'FF80B585-D64F-45DC-9CE9-59C51F3001D2', '324408CC-B86D-49E1-A0DE-99A7EE03276E', '3E505BF0-6673-4C9F-996F-64E07E72E60C', '5FFAD31C-B0EF-4A88-B7EA-2F02D7B336D5', '8CC0B44A-2E34-417F-A084-3E2F067F6966', 'B29EEE24-30E4-4B06-9FA3-777B1E99463D', 'C84DCC7E-7D84-433A-8D23-74F8BB79EE40', 'E776DC1F-10CF-4EC1-A151-05EC29E6C03C', '40F1314D-EBF6-49BC-B1A8-674F07F92039', '66E28365-DDB9-4F59-916F-2B171C5D93AB', '88539DEA-C521-4D06-A758-7FEBC99D5C5E', '8CCEE88C-C36B-4DE3-88D7-00D78C1D9B0A', 'C0AC5031-0382-4350-82C5-5B80634DEEAC', 'E5775C4C-F86F-49D2-8B58-22ACA6668352', '08683FBF-CD1F-42D9-A442-EF696A6D7C83', '289AA716-A1D3-48FF-B830-4BF9DD4A4FB7', '28D97A0D-03AE-4405-BFD3-66E00EDAE377', '55F62337-F077-4BA9-87D8-4C7AC0736913', '56A0F3CD-F2AA-4492-A3F3-D8694240FF34', 'B16FA6E7-E985-411C-AA06-A1F7E6BBCEFB', '0BB8A5C4-7444-4DEE-A3AC-2E0CFF7A3477', '1DB8D73F-61E5-4065-9342-592BE88E5981', '4C323EC0-5AB8-4CAF-9381-0011A0141AD7', '07BDF3AA-3B6E-4650-BDFC-596FE687C33F', '3F664203-58B4-411D-835A-667F32E1AF1E', '94F9C8EF-40F1-4CB4-B8F1-71EA8B1482D9', 'A56274A7-1DCD-46CB-84FC-83FD3C26472D', 'BDB2ADB4-356A-40AF-9E57-C6ECA2EB1235', 'F28706C5-DDB5-4FC3-9F7D-4896DB86AF18', '3DA05017-5FD5-4CBE-9CC5-956876EECBE6', '6AD7EB61-BE8E-49A3-929C-CE2CA6C95779', '8B79B6F2-FE78-4775-8668-2A35ECA963E5', '98AEC5FB-BE93-4E59-A4A9-9533419B73B6', 'A24C0F04-7684-4CC9-BFBF-CD7E01137187', 'C22535F6-D4C8-4B58-B0C4-BA24CE26B9E7', 'D9115251-3465-4EF5-B9B3-54B0EAF82045', '24253A09-E06B-4C51-A3AD-0709BD0DA4B5', '449D29CA-9C67-4DA9-95FE-C1531DA5DE4C', '77C723D8-92FE-487C-825B-93C8605DC74E', '8BAEB725-2E28-4DE9-B911-836846DF6F11', 'D89C117A-4B7C-471A-94A0-1F6F022193BB', 'E0D39756-5A76-4FA1-BA20-C957C3486DFC', '0A6DE37A-0F94-43B3-84E3-7590769F820E', '0F43FFB3-7DF2-4156-BD39-A6C0667C7392', '08AD40B0-B651-4069-AA92-9A64FC246B17', '0EC59014-45A0-45B7-AB3D-12367ECD4174', '13E519C2-73BD-451A-A651-55B67924A9B6', '26C382E6-F716-4F7F-A72C-A6C7150C2066', '989C36A9-F0D6-48B5-8D26-F5A56E610792', 'C4ADE72E-AD16-4711-886D-AB997CCB5E71', 'DE90AF3B-500B-42BC-B1F8-D4E5971DD853', '03A51C92-3977-42BE-B158-53B124029BCC', '176F9200-4582-49AB-80EB-558534EC297C', '2861F7ED-8D48-436D-9AD0-FEF67A864566', '7157C3A4-F8EE-42C2-864E-C9E7FD5C62C6', 'FB5D5A33-BA7C-4140-AB6B-9AC8E94FABA6', '3AAEBB1A-A489-4699-A7B4-F237F130B7EE', 'A3646A59-090D-4D1F-AE88-16B127AF81ED', 'A90B0781-610E-4FE4-80ED-BD9F6ABDE174', 'AF591C35-31A5-45DD-9399-292D82D794AF', 'D06DE332-BDE9-4D63-8959-BF68C5A40297', '23AEEFFE-19BA-487C-A1D5-A88964FF3BC1', '341B166A-3402-4A75-AD95-D57FDDA27DC9', '45D50939-3539-478D-AABE-C57C6791CF1F', '4A1D1603-893E-46B7-92C1-8C3069A4B63D', '502690F0-D587-4D27-B3A7-5B9679F7E217', '97E40160-D786-4460-B2D1-FCFFE2DB599F', 'A372D150-FF7E-4BA6-9349-DDD6451A1084'], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'ChargeDetailID', 'count': 500, 'min': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'max': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192'}, {'values': {'label': ['2020-12-18 00:00:00', '2020-12-13 00:00:00', '2020-11-16 00:00:00', '2020-11-27 00:00:00', '2020-12-03 00:00:00', '2020-11-02 00:00:00', '2020-12-14 00:00:00', '2020-11-28 00:00:00', '2020-11-30 00:00:00', '2020-11-26 00:00:00', '2020-11-09 00:00:00', '2020-12-19 00:00:00', '2020-12-04 00:00:00', '2020-11-15 00:00:00', '2020-11-20 00:00:00', '2020-12-05 00:00:00', '2020-12-06 00:00:00', '2020-12-07 00:00:00', '2020-11-06 00:00:00', '2020-11-13 00:00:00', '2020-12-11 00:00:00', '2020-12-17 00:00:00', '2020-11-12 00:00:00', '2020-11-07 00:00:00', '2020-11-14 00:00:00', '2020-11-29 00:00:00', '2020-12-12 00:00:00', '2020-12-10 00:00:00', '2020-12-21 00:00:00', '2020-11-21 00:00:00', '2020-12-24 00:00:00', '2020-11-05 00:00:00', '2020-11-19 00:00:00', '2020-12-20 00:00:00', '2020-11-08 00:00:00', '2020-12-27 00:00:00', '2020-11-01 00:00:00', '2020-12-26 00:00:00', '2020-12-28 00:00:00', '2020-12-31 00:00:00'], 'data': [15, 15, 23, 16, 19, 12, 17, 14, 17, 10, 11, 13, 12, 14, 12, 9, 14, 21, 16, 19, 18, 7, 15, 14, 14, 17, 12, 8, 10, 8, 5, 9, 14, 12, 15, 5, 9, 1, 5, 3]}, 'label': 'DateofService', 'count': 40, 'min': '', 'max': ''}, {'values': {'label': ['45.0 - 709.3', '709.3 - 1373.6', '1373.6 - 2037.9', '2037.9 - 2702.2', '2702.2 - 3366.5', '3366.5 - 4030.8', '4030.8 - 4695.1', '4695.1 - 5359.4', '5359.4 - 6023.7', '6023.7 - 6688.0'], 'data': [467, 13, 5, 0, 3, 5, 0, 0, 6, 1]}, 'label': 'Amount', 'count': 10, 'min': 45, 'max': 6688}, {'values': {'label': ['-250.0 - -212.7', '-212.7 - -175.4', '-175.4 - -138.1', '-138.1 - -100.8', '-100.8 - -63.5', '-63.5 - -26.2', '-26.2 - 11.1', '11.1 - 48.4', '48.4 - 85.7', '85.7 - 123.0'], 'data': [1, 0, 2, 0, 2, 3, 489, 0, 2, 1]}, 'label': 'PatientBal', 'count': 10, 'min': -250, 'max': 123}, {'values': {'label': ['0.0', '24.0', '76.0', '36.0'], 'data': [497, 1, 1, 1]}, 'label': 'InsuranceBal', 'count': 4, 'min': '0.0', 'max': '76.68'}, {'values': {'label': ['0.0', '76.0'], 'data': [499, 1]}, 'label': 'PrimaryInsuranceBal', 'count': 2, 'min': '0.0', 'max': '76.68'}, {'values': {'label': ['92014', '92134', '92250', '67028', '99204', '99354', '99215', '92012', 'J0178', '92225', '92226', '92004', '76512', '67036', '92235', 'J2778PF3', '99213', '99203', '67113', 'J7312', '92018', 'J9035', '99205'], 'data': [156, 185, 35, 22, 28, 2, 3, 12, 7, 7, 6, 6, 3, 2, 10, 9, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'CPTCode', 'count': 23, 'min': '67028', 'max': 'J9035'}, {'values': {'label': ['25', '0000', 'LT', 'RT', '50', '26', '57'], 'data': [70, 385, 13, 15, 13, 1, 3]}, 'label': 'CptModifier', 'count': 7, 'min': '0000', 'max': 'RT'}, {'values': {'label': ['H35.3112', 'H33.312', 'H35.711', 'H35.3211', 'E11.3313', 'H33.311', 'E11.3413', 'H33.101', 'H35.3121', 'H30.021', 'H35.3132', 'E11.3512', 'H30.91', 'E11.3513', 'E11.3391', 'H35.3231', 'H35.033', 'H35.413', 'H35.61', 'E10.3511', 'C69.32', 'H34.8110', 'H35.372', 'E11.3511', 'H33.301', 'C69.31', 'E11.3411', 'D31.31', 'H35.352', 'H43.813', 'H35.3134', 'H20.013', 'H35.3221', 'H35.3122', 'H35.351', 'B58.01', 'H34.231', 'H34.8310', 'H25.813', 'H33.001', 'H35.3113', 'H43.821', 'H35.3130', 'H34.8120', 'H35.411', 'H44.2A2', 'H33.002', 'H35.373', 'H35.371', 'H35.3222', 'H35.3212', 'H25.13', 'E10.3513', 'H35.051', 'H35.362', 'D31.22', 'H44.113', 'H43.11', 'H35.343', 'H43.811', 'H30.23', 'H33.313', 'H53.9', 'H30.133', 'E11.3311', 'H47.012', 'H34.8330', 'H47.10', 'H35.353', 'H20.9', 'H30.93', 'H35.3131', 'H35.3124', 'H31.113', 'H35.341', 'E10.3292', 'H33.22', 'H43.812', 'H35.053', 'H40.051', 'E11.3213', 'H35.342', 'H35.3133', 'H35.052', 'H20.12', 'H43.10', 'H35.3213', 'H35.3114', 'H53.8', 'E11.3212', 'H33.321', 'C79.32', 'H35.89', 'H30.22', 'E11.3211', 'H34.8312', 'H35.721', 'H35.433', 'H35.412', 'H40.033', 'H34.8320', 'H43.13', 'H47.211', 'H35.461', 'H43.823', 'H57.11', 'E11.3312', 'H15.121', 'H35.3111', 'H31.091', 'H44.2D2', 'H33.322', 'E11.3292', 'H44.111', 'H43.12', 'H33.103', 'H21.01', 'H04.123', 'H31.011', 'H35.54', 'H43.392', 'H35.3110', 'C69.21', 'H59.021', 'H43.22', 'H11.31', 'H20.012', 'H34.8130', 'H35.073', 'H35.023', 'E10.3391', 'H35.713', 'H31.8', 'H35.63', 'D31.32', 'H33.042', 'T85.29XA', 'H33.323', 'H40.053', 'H44.23', 'H10.45', 'E11.9', 'T66.XXXA', 'H43.391', 'H35.101', 'H30.013', 'H35.50', 'Q14.1', 'H35.712'], 'data': [21, 3, 5, 20, 14, 2, 8, 1, 1, 1, 13, 5, 2, 38, 1, 30, 15, 2, 1, 2, 5, 7, 11, 14, 2, 1, 4, 3, 3, 6, 5, 1, 3, 7, 8, 1, 1, 5, 1, 6, 2, 3, 3, 4, 2, 1, 5, 11, 9, 2, 8, 2, 1, 1, 1, 1, 1, 2, 1, 4, 2, 1, 2, 1, 4, 1, 1, 1, 6, 2, 2, 10, 2, 1, 3, 1, 1, 4, 1, 3, 3, 3, 3, 2, 2, 1, 2, 3, 1, 2, 3, 1, 2, 1, 5, 3, 2, 1, 3, 1, 3, 1, 1, 1, 3, 1, 4, 1, 4, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'pdiagnosis1', 'count': 149, 'min': 'B58.01', 'max': 'T85.29XA'}, {'values': {'label': ['H35.373', 'H35.411', 'H35.721', 'H35.3122', 'H43.811', 'E11.9', 'Z79.4', 'H43.823', 'H43.813', 'H35.351', 'H35.70', 'H43.822', 'E11.3312', 'H35.723', 'H34.8320', 'H52.13', 'H35.3221', 'Z71.89', 'H35.432', 'H43.821', 'H35.371', 'H35.61', 'H35.412', 'H35.372', 'H20.9', 'H21.543', 'H43.812', 'H35.3211', 'H30.101', 'H43.89', 'H35.033', 'H04.123', 'H25.811', 'H35.463', 'H35.3212', 'H35.3132', 'H35.461', 'H44.22', 'H35.032', 'H40.021', 'H35.341', 'H35.352', 'Z98.890', 'H34.8322', 'H35.711', 'E11.3311', 'H35.3133', 'H30.113', 'H35.3124', 'S01.111D', 'H47.211', 'E11.3412', 'Z79.899', 'D31.31', 'H35.353', 'Z96.1', 'H40.052', 'H35.3130', 'H35.3134', 'H40.013', 'H33.312', 'H43.12', 'D31.32', 'H34.8120', 'H43.11', 'H33.311', 'H31.092', 'H33.331', 'H30.93', 'H33.42', 'H47.10', 'H40.1110', 'H21.542', 'H35.3223', 'E11.3513', 'H59.022', 'E11.3292', 'E11.3592', 'H35.3131', 'H47.011', 'H35.3220', 'H35.022', 'H25.813', 'H35.3120', 'E11.3293', 'H35.342', 'H35.3112', 'H35.52', 'H25.11', 'H43.392', 'H43.23', 'H35.413', 'H40.053', 'H35.722', 'H43.13', 'H35.3121', 'E11.3511', 'H43.393', 'H40.1130', 'E11.3291', 'H33.41', 'H33.321', 'E11.3213', 'G45.9', 'E11.3532', 'H35.22', 'I10', 'H31.012', 'H31.011', 'H30.92', 'H47.091', 'H11.32', 'H33.43', 'H35.433', 'K50.918', 'H35.40', 'H40.043', 'H35.052', 'T85.398A', 'H11.31', 'H35.81', 'E11.3391', 'H44.111', 'H25.13', 'E11.3212', 'H01.003', 'T66.XXXS', 'H40.051', 'H33.322', 'H43.391', 'G89.11', 'H35.713', 'C69.22', 'H27.131', 'H35.89', 'S05.11XA', 'H26.222', 'H33.002', 'H35.3222', 'E11.3392', 'H40.42X0', 'E10.3312', 'H44.701', 'E11.3512', 'H44.23', 'H25.9', 'H33.003', 'T66.XXXA', 'H11.001', 'H35.3231', 'H31.091', 'H33.001', 'H31.422', 'H44.2A3', 'H47.233', 'H31.002'], 'data': [20, 2, 3, 9, 14, 6, 47, 7, 22, 7, 4, 6, 5, 4, 9, 2, 23, 4, 2, 4, 8, 3, 4, 10, 4, 1, 12, 2, 1, 3, 25, 2, 2, 2, 1, 2, 2, 1, 1, 1, 3, 8, 3, 3, 1, 1, 1, 1, 3, 1, 1, 4, 2, 5, 10, 3, 1, 1, 3, 1, 3, 4, 3, 4, 4, 3, 1, 1, 2, 1, 1, 1, 1, 3, 3, 1, 2, 3, 3, 1, 1, 1, 1, 2, 2, 4, 1, 1, 1, 3, 1, 7, 1, 2, 2, 3, 2, 2, 2, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'pdiagnosis2', 'count': 156, 'min': 'C69.22', 'max': 'Z98.890'}, {'values': {'label': ['1.0'], 'data': [500]}, 'label': 'IsBillable', 'count': 1, 'min': '1.0', 'max': '1.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'Isauthorized', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'PredeterminationIndicator', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'MedNecessityRequiredIndicator', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['5.0 - 14.7', '14.7 - 24.4', '24.4 - 34.1', '34.1 - 43.8', '43.8 - 53.5', '53.5 - 63.2', '63.2 - 72.9', '72.9 - 82.6', '82.6 - 92.3', '92.3 - 102.0'], 'data': [4, 6, 11, 20, 44, 75, 134, 120, 72, 14]}, 'label': 'Age', 'count': 10, 'min': 5, 'max': 102}, {'values': {'label': ['M', 'F'], 'data': [225, 275]}, 'label': 'Gender', 'count': 2, 'min': 'F', 'max': 'M'}, {'values': {'label': ['Pulmonary', 'Emergency', 'Cardiology', 'Orthopedic', 'Pediatrics', 'Physical Therapy', 'Ophtamology', 'Gastroenterology'], 'data': [172, 233, 39, 36, 3, 7, 9, 1]}, 'label': 'DepartmentCategory', 'count': 8, 'min': 'Cardiology', 'max': 'Pulmonary'}, {'values': {'label': ['Ophtamology'], 'data': [500]}, 'label': 'SpecialityCategory', 'count': 1, 'min': 'Ophtamology', 'max': 'Ophtamology'}, {'values': {'label': ['1.0', '0.0'], 'data': [455, 45]}, 'label': 'label', 'count': 2, 'min': '0.0', 'max': '1.0'}, {'values': {'label': ['B03B74CE-C223-4B1E-802B-5F5F76BFE64C', '105C1090-6ED0-4189-B07E-4C22940D948F', 'AD96772F-54C7-4750-86B1-1BD565B55244', '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', '4502FB31-5B01-4EEF-BB9E-2D396D96A6FC', '32E7EC13-C493-40B1-A999-6AC94E7FA3B9', '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', '0A471D4C-3977-4BF5-8039-5064519FDCD4', '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'BCA515C2-1B14-4060-9E24-B80BF9260648', '2CD6534B-F8F6-4CF9-8DEB-CCEF9EF87F9A', 'A7CD19DF-074A-47D3-958D-2B5E00671A3D', '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', '84222BD3-605E-43EA-9A84-7C0169BE0B0D', '4CED3627-B1D6-4530-87D7-9609525FB29C', '041FE834-D4EA-40F1-A495-4B94A97ACD4D', 'F3FB074E-423D-4F7F-A817-90C0551007A4', '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', '8351B681-269A-4AE6-936A-FF10AFAFD629', 'E459B9E9-8650-4DA2-91F6-00B77900FC50', '2EAF4989-97B8-4F1B-9259-03E37314202F', 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'F473817F-7515-46B7-97F9-348C88D7C348', '0BC878EE-8F48-415E-9233-25A7C28264DB', '789050C0-D1C3-40AD-8942-E67329E2F890', '193BA5E7-F797-4FD1-A9AB-27590E630FCB', '5EA62C5D-D9C0-4AFE-ACAF-4AD36F6B1057', '72F5084C-9DF9-4F0A-8BA1-A9B40DFCC9C6', '330E2DD7-352E-4E5C-9669-AE796ABFE5DC', 'E0FE1EC8-51D4-430D-8B8B-E2F9966875F8', 'A4D682CE-4D10-4765-AE88-0D5D9E1862E7', '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'C4A6C186-B867-46C5-A163-5AF23FE61DED', 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'BCCB5D33-476A-4FAC-BB0C-5C7479803EB4', '3BEA3D9A-B5EF-4B67-A4AB-868B7BE2C0A1', '0779204D-FC67-444E-97E3-83CB62A9D0A2', 'EDD37436-0E9D-45B5-8C46-A22CA763970A', '8CA1878F-1BD4-4595-9AFF-3FF6967CD5CB', '315C773B-4EF1-4164-AB43-6817D993329F', 'AAB72015-9704-40ED-A2E7-87468E793D48', '20C2761F-16B5-42B7-8C93-82A112735F02', '835BC6C3-FEDE-4AF9-8B88-91628A98389C', 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', 'D1C4AE8C-46B5-464F-91E1-A39C5C76CC25'], 'data': [15, 1, 30, 199, 25, 81, 2, 2, 16, 6, 37, 3, 1, 1, 1, 6, 3, 1, 2, 5, 12, 3, 3, 2, 2, 4, 2, 6, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'PayerId', 'count': 51, 'min': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52'}, {'values': {'label': ['2.0', '1.0'], 'data': [225, 275]}, 'label': 'Gender_Recoded', 'count': 2, 'min': '1', 'max': '2'}, {'values': {'label': ['0.0 - 49.9', '49.9 - 99.8', '99.8 - 149.7', '149.7 - 199.6', '199.6 - 249.5', '249.5 - 299.4', '299.4 - 349.3', '349.3 - 399.2', '399.2 - 449.1', '449.1 - 499.0'], 'data': [50, 50, 50, 50, 50, 50, 50, 50, 50, 50]}, 'label': 'ChargeDetailID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 499}, {'values': {'label': ['1 - 4', '4 - 7', '7 - 10', '10 - 13', '13 - 16', '16 - 19', '19 - 22', '22 - 25', '25 - 28', '28 - 31'], 'data': [40, 60, 61, 53, 79, 45, 69, 5, 32, 56]}, 'label': 'DateofService_dayofmonth', 'count': 10, 'min': 1, 'max': 31}, {'values': {'label': ['12', '11'], 'data': [221, 279]}, 'label': 'DateofService_month', 'count': 2, 'min': '11', 'max': '12'}, {'values': {'label': ['2020'], 'data': [500]}, 'label': 'DateofService_year', 'count': 1, 'min': '2020', 'max': '2020'}, {'values': {'label': ['0.0 - 2.2', '2.2 - 4.4', '4.4 - 6.6', '6.6 - 8.8', '8.8 - 11.0', '11.0 - 13.2', '13.2 - 15.4', '15.4 - 17.6', '17.6 - 19.8', '19.8 - 22.0'], 'data': [376, 50, 22, 16, 13, 12, 4, 2, 2, 3]}, 'label': 'CPTCode_stringindexer_transform', 'count': 10, 'min': 0, 'max': 22}, {'values': {'label': ['1.0', '0.0', '4.0', '2.0', '3.0', '6.0', '5.0'], 'data': [70, 385, 13, 15, 13, 1, 3]}, 'label': 'CptModifier_stringindexer_transform', 'count': 7, 'min': '0.0', 'max': '6.0'}, {'values': {'label': ['0.0 - 14.8', '14.8 - 29.6', '29.6 - 44.4', '44.4 - 59.2', '59.2 - 74.0', '74.0 - 88.8', '88.8 - 103.6', '103.6 - 118.4', '118.4 - 133.2', '133.2 - 148.0'], 'data': [230, 79, 49, 36, 28, 18, 15, 15, 15, 15]}, 'label': 'pdiagnosis1_stringindexer_transform', 'count': 10, 'min': 0, 'max': 148}, {'values': {'label': ['0.0 - 15.5', '15.5 - 31.0', '31.0 - 46.5', '46.5 - 62.0', '62.0 - 77.5', '77.5 - 93.0', '93.0 - 108.5', '108.5 - 124.0', '124.0 - 139.5', '139.5 - 155.0'], 'data': [238, 66, 51, 35, 32, 15, 16, 15, 16, 16]}, 'label': 'pdiagnosis2_stringindexer_transform', 'count': 10, 'min': 0, 'max': 155}, {'values': {'label': ['1', '0'], 'data': [225, 275]}, 'label': 'Gender_stringindexer_transform', 'count': 2, 'min': '0', 'max': '1'}, {'values': {'label': ['1.0', '0.0', '2.0', '3.0', '6.0', '5.0', '4.0', '7.0'], 'data': [172, 233, 39, 36, 3, 7, 9, 1]}, 'label': 'DepartmentCategory_stringindexer_transform', 'count': 8, 'min': '0.0', 'max': '7.0'}, {'values': {'label': ['0'], 'data': [500]}, 'label': 'SpecialityCategory_stringindexer_transform', 'count': 1, 'min': '0', 'max': '0'}, {'values': {'label': ['0.0 - 5.0', '5.0 - 10.0', '10.0 - 15.0', '15.0 - 20.0', '20.0 - 25.0', '25.0 - 30.0', '30.0 - 35.0', '35.0 - 40.0', '40.0 - 45.0', '45.0 - 50.0'], 'data': [372, 55, 21, 12, 10, 9, 5, 5, 5, 6]}, 'label': 'PayerId_stringindexer_transform', 'count': 10, 'min': 0, 'max': 50}], 'FESamplePath': '/FileStore/platform/sampleData/60891770a187c956b337cf50_ManualFE/part-00000-tid-7570618811626959673-94990ba1-2b39-4e6c-ac98-d7d939c2d4f4-172005-1-c000.csv', 'manualfesamplepath': '/FileStore/platform/sampleData/60891770a187c956b337cf50_ManualFE/part-00000-tid-7570618811626959673-94990ba1-2b39-4e6c-ac98-d7d939c2d4f4-172005-1-c000.csv', 'stageId': '60891770a187c956b337cf50', 'stage_id': '60891770a187c956b337cf50', 'status': 1, 'autoFE_flag': True, 'userid': '60406edac5c640163a69c2d6', 'message': 'Successfully executed', 'process_type': 'Auto FE', 'sampledata': [{'ChargeDetailID': '174DA322-870E-429F-A9A9-BE22340E9836', 'DateofService': 1608249600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 89, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 407, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': 'A1575911-0E83-4479-B182-D26EDAD439F6', 'DateofService': 1607817600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.312', 'pdiagnosis2': 'H35.411', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '105C1090-6ED0-4189-B07E-4C22940D948F', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 420, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 45, 'pdiagnosis2_stringindexer_transform': 72, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 49}, {'ChargeDetailID': 'A87EF1E9-E636-4742-8E2B-414477926782', 'DateofService': 1605484800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.711', 'pdiagnosis2': 'H35.721', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 39, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 287, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 24, 'pdiagnosis2_stringindexer_transform': 45, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '1FFF3E45-B9F4-4CFB-8580-C4B05D4822DD', 'DateofService': 1607817600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 467, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 10, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '20BC5C22-2C34-494C-B75F-55BDD8B54263', 'DateofService': 1606435200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 463, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '210DAE19-9257-4180-9057-E5817F555A88', 'DateofService': 1606953600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.311', 'pdiagnosis2': 'E11.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 41, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 54, 'pdiagnosis2_stringindexer_transform': 16, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '3FA21D4D-05B5-4C43-A33E-A01D20B45A99', 'DateofService': 1604275200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 53, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 42, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 13, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': 'DC65D7C3-0822-4E54-86EF-DC7639D38C62', 'DateofService': 1606953600000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H33.101', 'pdiagnosis2': 'H43.823', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 57, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 151, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 125, 'pdiagnosis2_stringindexer_transform': 15, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '11327DC4-1599-46B2-A000-C05D135E1F91', 'DateofService': 1607904000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3121', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 80, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '4502FB31-5B01-4EEF-BB9E-2D396D96A6FC', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 244, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 79, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 26}, {'ChargeDetailID': '853B5BAB-0A5F-4B44-85E0-FEE0AA663796', 'DateofService': 1606521600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H30.021', 'pdiagnosis2': 'H35.351', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '32E7EC13-C493-40B1-A999-6AC94E7FA3B9', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 262, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 117, 'pdiagnosis2_stringindexer_transform': 13, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 19}, {'ChargeDetailID': '92F7294B-4F6C-41B5-A9BD-65CE72B27FC0', 'DateofService': 1606694400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'E11.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 77, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 281, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 16, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '97BB8C89-8E47-400B-B546-B662E9EDCA00', 'DateofService': 1606694400000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': 'LT', 'pdiagnosis1': 'E11.3512', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 50, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 445, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 26, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': '9C7B2481-4469-4FB8-AC5D-18EB2D29F213', 'DateofService': 1606348800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H30.91', 'pdiagnosis2': 'H35.70', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 166, 'DateofService_dayofmonth': 26, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 52, 'pdiagnosis2_stringindexer_transform': 32, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 8}, {'ChargeDetailID': 'BCE331FC-BC9E-43B2-B9C3-1ADE95CF4081', 'DateofService': 1608249600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.822', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 48, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 284, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 17, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '0391FBA9-7748-4F50-8CA2-FC4EB5116F6A', 'DateofService': 1604880000000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3391', 'pdiagnosis2': 'E11.3312', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 21, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 118, 'pdiagnosis2_stringindexer_transform': 18, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '88D923C8-5217-46AF-887E-CB88A2E6C3E3', 'DateofService': 1608336000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.723', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 88, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 469, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 22, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '8AAE78A7-7725-4C1C-B55C-82B41A364A2B', 'DateofService': 1605484800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H34.8320', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 48, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 384, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 9, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'B5C5BD20-64AE-4166-8115-14DA13624764', 'DateofService': 1607040000000.0, 'Amount': 300, 'PatientBal': -172, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.413', 'pdiagnosis2': 'H52.13', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 336, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 63, 'pdiagnosis2_stringindexer_transform': 73, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '343273F8-E598-4815-A700-0683D31F3291', 'DateofService': 1604275200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.61', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 219, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 110, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '50F7ADC5-2166-4461-B01E-8D8FAA92304B', 'DateofService': 1605398400000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E10.3511', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 54, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 326, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 67, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'A9EC659A-ED07-4B6C-84CA-D4E45411C3C6', 'DateofService': 1605830400000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99354', 'CptModifier': '0000', 'pdiagnosis1': 'C69.32', 'pdiagnosis2': 'Z71.89', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 58, 'Gender': 'M', 'DepartmentCategory': 'Pediatrics', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 242, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 15, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 23, 'pdiagnosis2_stringindexer_transform': 31, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 6, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'DCBDA7E0-A5F8-4D0F-836D-66187FEB5B6F', 'DateofService': 1606694400000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H35.432', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 48, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 471, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 15, 'pdiagnosis2_stringindexer_transform': 75, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'E907166F-CE0F-49D2-A04B-40734F1A6F0C', 'DateofService': 1607126400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H43.821', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 62, 'DateofService_dayofmonth': 5, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 24, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '6518F89E-CD31-4903-89E2-2F31BEB28508', 'DateofService': 1607212800000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 310, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '7B41BFB4-B1D7-47A9-9659-6E8319429423', 'DateofService': 1607299200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E10.3511', 'pdiagnosis2': 'H35.371', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 53, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'BCA515C2-1B14-4060-9E24-B80BF9260648', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 325, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 67, 'pdiagnosis2_stringindexer_transform': 12, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 14}, {'ChargeDetailID': '7E4D4049-B1BC-4135-8020-D69318B66B18', 'DateofService': 1604620800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.61', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 59, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 414, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 42, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '94FEF155-5ED7-4DF9-836F-78557484B8E4', 'DateofService': 1605225600000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H33.301', 'pdiagnosis2': 'H35.412', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 58, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 364, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 55, 'pdiagnosis2_stringindexer_transform': 23, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'A6B1DE4F-2FBD-4AB9-B09E-77B3E8C44771', 'DateofService': 1607644800000.0, 'Amount': 250, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99215', 'CptModifier': '25', 'pdiagnosis1': 'C69.31', 'pdiagnosis2': 'Z71.89', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 159, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 12, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 94, 'pdiagnosis2_stringindexer_transform': 31, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': 'B9F344F0-CEE0-4EA2-A55C-274FAB39E9DB', 'DateofService': 1606435200000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E11.3411', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '2CD6534B-F8F6-4CF9-8DEB-CCEF9EF87F9A', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 171, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 31, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 35}, {'ChargeDetailID': 'BBE4E59F-B282-490E-8FCD-6C9F5E8069EB', 'DateofService': 1608163200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'D31.31', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 240, 'DateofService_dayofmonth': 17, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 48, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'EC206AC2-403A-4047-B82E-82C383E29CCE', 'DateofService': 1605139200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.352', 'pdiagnosis2': 'H35.372', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 77, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 167, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 44, 'pdiagnosis2_stringindexer_transform': 7, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '01867531-8532-4FD4-BE17-232F39CFA32D', 'DateofService': 1605830400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.371', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 296, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 19, 'pdiagnosis2_stringindexer_transform': 12, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '2ECD88B7-05A0-4959-8736-397F8CF74784', 'DateofService': 1607299200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H20.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7CD19DF-074A-47D3-958D-2B5E00671A3D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 371, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 29, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 29}, {'ChargeDetailID': '524D7B98-2D7C-458D-8877-35CDE8F3D981', 'DateofService': 1604880000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3134', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 3, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 21, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 46}, {'ChargeDetailID': '54E3991B-1609-470D-8095-C14FF5A9444D', 'DateofService': 1604707200000.0, 'Amount': 118, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H20.013', 'pdiagnosis2': 'H21.543', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 31, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 395, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 135, 'pdiagnosis2_stringindexer_transform': 86, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '8F8BDCC6-5B74-4EBA-A154-8576DE0FFC13', 'DateofService': 1606953600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 453, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A28023B3-9D64-44D3-BD13-2A22A1350A24', 'DateofService': 1604707200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3221', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 89, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 152, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 43, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '1B71D2B0-C473-4A8D-9FA0-76A0692C5102', 'DateofService': 1605312000000.0, 'Amount': 6000, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 56, 'Gender': 'M', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 169, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 9, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 5, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '6CF3754A-BC46-430C-A116-9329959CD556', 'DateofService': 1604620800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.3211', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 96, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 16, 'pdiagnosis2_stringindexer_transform': 76, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 9}, {'ChargeDetailID': 'A6CED81D-7B1D-441A-8EFC-544E8D31FD26', 'DateofService': 1607644800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 389, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 10, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A82270CF-0B59-4C62-877D-9FAF720B6E5D', 'DateofService': 1606608000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H30.101', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 50, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 129, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 14, 'pdiagnosis2_stringindexer_transform': 145, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'D9A3771E-E97A-4EBE-BFA9-54577F445F21', 'DateofService': 1607731200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'B58.01', 'pdiagnosis2': 'H43.89', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 43, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 308, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 138, 'pdiagnosis2_stringindexer_transform': 48, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '1AA261AC-C877-4F0D-8372-F268A4D5F90B', 'DateofService': 1607558400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.231', 'pdiagnosis2': 'E11.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '4CED3627-B1D6-4530-87D7-9609525FB29C', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 71, 'DateofService_dayofmonth': 10, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 98, 'pdiagnosis2_stringindexer_transform': 16, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 13}, {'ChargeDetailID': '2B385821-C105-42CA-84F0-CEF8BFD1681A', 'DateofService': 1606694400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 462, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 25, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '9D5CF424-8DA6-4A2B-8716-FDB93B808F3C', 'DateofService': 1608508800000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H25.813', 'pdiagnosis2': 'H04.123', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 444, 'DateofService_dayofmonth': 21, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 112, 'pdiagnosis2_stringindexer_transform': 59, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A17DF3A2-8077-49B4-A114-11F77F67E7C9', 'DateofService': 1607731200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 142, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 25, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '0E47C422-450E-4A04-9B90-680F296C9686', 'DateofService': 1605484800000.0, 'Amount': 118, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H25.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 480, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 17, 'pdiagnosis2_stringindexer_transform': 71, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': '46CB6A8C-362F-4345-941E-842F429B48B5', 'DateofService': 1607212800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3113', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 92, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 232, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 56, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '8CD490B4-CEBE-4DFE-BA0A-C805ACDF0D34', 'DateofService': 1604620800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3113', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 75, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 373, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 56, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '9F54BD25-7C67-454A-B100-CD9974B62C7D', 'DateofService': 1605916800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.463', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 39, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 337, 'DateofService_dayofmonth': 21, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 19, 'pdiagnosis2_stringindexer_transform': 58, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': 'A78BD078-2165-4C68-B398-9AB6E94C0FB6', 'DateofService': 1605484800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '041FE834-D4EA-40F1-A495-4B94A97ACD4D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 304, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 10, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 40}, {'ChargeDetailID': 'C72EBDAD-1D1D-40D0-BA4D-A9CDD08430A6', 'DateofService': 1608768000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 56, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 379, 'DateofService_dayofmonth': 24, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'DBFEC39B-37B2-4B37-9D95-A444C27A619C', 'DateofService': 1605139200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.821', 'pdiagnosis2': 'H35.3212', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 33, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 38, 'pdiagnosis2_stringindexer_transform': 83, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'E1C8F5F6-2DDF-4370-B40C-D86322919657', 'DateofService': 1608336000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 264, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'EB7C11AD-0A01-4359-B1AB-289373C669DC', 'DateofService': 1607731200000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 44, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 440, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '16062E35-A49E-4227-A3E6-E8DDBFEBA5D2', 'DateofService': 1607644800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3130', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 90, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 417, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 39, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '20416F30-920B-4925-8642-C83F38ECFCEE', 'DateofService': 1604534400000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8120', 'pdiagnosis2': 'H35.3132', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 234, 'DateofService_dayofmonth': 5, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 29, 'pdiagnosis2_stringindexer_transform': 65, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '642D42E4-275F-4E86-8378-21E2CD291224', 'DateofService': 1607731200000.0, 'Amount': 50, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92225', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.411', 'pdiagnosis2': 'H35.461', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 31, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 305, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 8, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 74, 'pdiagnosis2_stringindexer_transform': 74, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': 'FBB301BB-1560-415C-91B5-A6E8B0B26B6C', 'DateofService': 1606694400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.351', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 88, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 123, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 13, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '4011D505-BE33-4A24-AD40-240BFDDB2766', 'DateofService': 1605744000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H43.821', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 298, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 24, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 17}, {'ChargeDetailID': '74FC2B0E-AEC7-4263-951F-14E72EF044DA', 'DateofService': 1605225600000.0, 'Amount': 118, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H44.2A2', 'pdiagnosis2': 'H44.22', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 32, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 66, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 134, 'pdiagnosis2_stringindexer_transform': 105, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'A5881100-AA89-4D5D-80AC-AE2419271C5D', 'DateofService': 1608336000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.721', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 96, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 339, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 45, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D0667457-EF74-4FA4-91D9-7169C64A53DB', 'DateofService': 1605484800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H34.8320', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 478, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 9, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 11}, {'ChargeDetailID': 'E16D36F3-C4D2-4C4B-A004-E06C823C7960', 'DateofService': 1605312000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H35.032', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 224, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 20, 'pdiagnosis2_stringindexer_transform': 120, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '20463C7A-45B7-4BD4-A3EA-53ACC03E93F1', 'DateofService': 1606435200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 431, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '6E834EA0-8A83-4326-8FAE-80C6DEE3B045', 'DateofService': 1607644800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H40.021', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 272, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 11, 'pdiagnosis2_stringindexer_transform': 110, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': '8607EC2B-26F1-4A40-9646-5990D37EC622', 'DateofService': 1607212800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3222', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 438, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 76, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'A050F893-D9BB-4FB6-9B4B-B7AA83355D60', 'DateofService': 1605225600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.341', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 195, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 12, 'pdiagnosis2_stringindexer_transform': 46, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C2F471A9-22FA-4A9A-A14C-49A5F292662D', 'DateofService': 1606608000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H35.352', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 93, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 11, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D6F6C707-D42D-4485-A571-82EEF5002E64', 'DateofService': 1608422400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H35.371', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 290, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 14, 'pdiagnosis2_stringindexer_transform': 12, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'E164830A-C381-4AEC-81EE-3B116E56B496', 'DateofService': 1604620800000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H25.13', 'pdiagnosis2': 'Z98.890', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 48, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 342, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 72, 'pdiagnosis2_stringindexer_transform': 50, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '2701DB53-B3A0-466C-BFCB-6684D7059DC4', 'DateofService': 1607299200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 89, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 59, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '89D7AAB0-BF88-493E-A7AB-7D6D793273DB', 'DateofService': 1604275200000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 18, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 19, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '8A07AC76-56FD-4E13-BFEA-A724E4C2D603', 'DateofService': 1604534400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H34.8322', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 77, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 190, 'DateofService_dayofmonth': 5, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 43, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '8E5FAFBE-87F5-4C26-88EA-002384A50B1D', 'DateofService': 1606435200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.723', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 87, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 340, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 22, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': 'E67C3D6F-A4D2-4EA9-B41A-E5BEB54E0209', 'DateofService': 1607817600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H43.822', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 13, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 17, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 16}, {'ChargeDetailID': 'AC68E17B-9EBB-45A3-BD22-60367A203515', 'DateofService': 1605312000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E10.3513', 'pdiagnosis2': 'H35.371', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 40, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '8351B681-269A-4AE6-936A-FF10AFAFD629', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 163, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 147, 'pdiagnosis2_stringindexer_transform': 12, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 15}, {'ChargeDetailID': 'D0BA312C-C42D-4695-A6E4-71096BE6854B', 'DateofService': 1604793600000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.051', 'pdiagnosis2': 'H35.711', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 59, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 449, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 86, 'pdiagnosis2_stringindexer_transform': 106, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '50C6C060-08F2-4F75-91BE-EF1D34F6EDA9', 'DateofService': 1605484800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H43.821', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 133, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 24, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C99C5CC6-DF6E-4024-88A6-04D326F6D6B2', 'DateofService': 1605225600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3512', 'pdiagnosis2': 'E11.3311', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'E459B9E9-8650-4DA2-91F6-00B77900FC50', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 168, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 26, 'pdiagnosis2_stringindexer_transform': 148, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 27}, {'ChargeDetailID': '1A70E6C3-0B4F-4338-A15C-903F552EC30C', 'DateofService': 1606521600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 84, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 346, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '279BE041-8831-43CF-A0EB-219F709A986F', 'DateofService': 1604275200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.362', 'pdiagnosis2': 'H35.351', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 419, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 103, 'pdiagnosis2_stringindexer_transform': 13, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '3E0A6D69-9B07-4BFC-B389-24E0C884701E', 'DateofService': 1606694400000.0, 'Amount': 45, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92226', 'CptModifier': 'LT', 'pdiagnosis1': 'D31.22', 'pdiagnosis2': 'H34.8320', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 42, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 35, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 11, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 101, 'pdiagnosis2_stringindexer_transform': 9, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': '9017B6B5-6C3F-42FC-ABD4-67A54E07D67F', 'DateofService': 1605398400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.3133', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 87, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 497, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 93, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'F16597BA-3D84-47E3-AF6C-072ECBD33467', 'DateofService': 1604793600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H44.113', 'pdiagnosis2': 'H30.113', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 26, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 341, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 129, 'pdiagnosis2_stringindexer_transform': 119, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '2AA7F9ED-3694-4ADD-91A8-3EFB8F8054FB', 'DateofService': 1607126400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 24, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.711', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 19, 'DateofService_dayofmonth': 5, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 24, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '45F8D74C-DE28-4F0D-B781-AB81CCD5D5CD', 'DateofService': 1605139200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3124', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 91, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 226, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 12, 'pdiagnosis2_stringindexer_transform': 38, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'D7B8BD7C-A3F9-422B-B491-59F236BADE4D', 'DateofService': 1605139200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 112, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'E9DB7A1D-67B8-4936-B7C7-FB071EA72907', 'DateofService': 1605225600000.0, 'Amount': 118, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8120', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 72, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 29, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '2B6E92D7-EA15-4F53-B133-2A1F9B1E958C', 'DateofService': 1608249600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H35.341', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 75, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 292, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 17, 'pdiagnosis2_stringindexer_transform': 46, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '86E8386B-2651-4928-8CD1-631CE24D0432', 'DateofService': 1607904000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3124', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 95, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 362, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 38, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '97D21C13-7AFC-4865-888F-5E92CC51F2ED', 'DateofService': 1607212800000.0, 'Amount': 246, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92004', 'CptModifier': '25', 'pdiagnosis1': 'H43.11', 'pdiagnosis2': 'S01.111D', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 125, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 10, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 64, 'pdiagnosis2_stringindexer_transform': 117, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A40F161A-A931-41D9-A65B-597347BC9D6A', 'DateofService': 1607817600000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '25', 'pdiagnosis1': 'H35.3221', 'pdiagnosis2': 'H47.211', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 91, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 439, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 43, 'pdiagnosis2_stringindexer_transform': 102, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C43925F7-AA9E-419B-B0DB-0A044E67C4B5', 'DateofService': 1606608000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3412', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 59, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 76, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 20, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D05F93BC-037E-4BD7-8E65-FDA46391791F', 'DateofService': 1607731200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.343', 'pdiagnosis2': 'Z79.899', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 250, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 90, 'pdiagnosis2_stringindexer_transform': 69, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '14167C30-8123-4DCF-B3C5-05D3F16B6391', 'DateofService': 1605225600000.0, 'Amount': 383, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '76512', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.711', 'pdiagnosis2': 'D31.31', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 198, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 13, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 24, 'pdiagnosis2_stringindexer_transform': 19, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '2022006D-F4D6-4F8A-B76D-1617C97083CA', 'DateofService': 1607644800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H35.353', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 187, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 8, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '25EF819E-5F4C-43DA-BEBB-C4FAD71B0EF5', 'DateofService': 1609027200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.822', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 54, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '2EAF4989-97B8-4F1B-9259-03E37314202F', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 10, 'DateofService_dayofmonth': 27, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 17, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 22}, {'ChargeDetailID': '8648E212-02DB-4D77-BB02-88B67C56A9DE', 'DateofService': 1609027200000.0, 'Amount': 246, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92004', 'CptModifier': '25', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'Z96.1', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 82, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 294, 'DateofService_dayofmonth': 27, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 10, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 40, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'EDDC8ACA-6742-4C6C-A200-FA42DD89285D', 'DateofService': 1606521600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H40.052', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 413, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 13, 'pdiagnosis2_stringindexer_transform': 139, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '03E0DCDC-A071-4987-A7CE-268D9C3D017B', 'DateofService': 1609027200000.0, 'Amount': 212, 'PatientBal': -40, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 89, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 381, 'DateofService_dayofmonth': 27, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 12}, {'ChargeDetailID': '161B0D6C-5A50-4EEE-9B2A-E4DE8D9EBC1C', 'DateofService': 1604275200000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 48, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 51, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 15, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '1BD4EAF6-F3F8-4369-93B3-4E6144A69CAD', 'DateofService': 1608422400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.811', 'pdiagnosis2': 'H35.3130', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 180, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 28, 'pdiagnosis2_stringindexer_transform': 107, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': '20AC5B30-ACB1-4714-9794-5454B92C1A23', 'DateofService': 1604188800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3222', 'pdiagnosis2': 'H35.3134', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 91, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 205, 'DateofService_dayofmonth': 1, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 76, 'pdiagnosis2_stringindexer_transform': 35, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '3FE9456A-15CB-4840-90CB-05F7DF34CACE', 'DateofService': 1607904000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H40.013', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 87, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 8, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 85, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '42F5D00A-6F59-4F63-ACCA-9EFDA0DD9194', 'DateofService': 1608336000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H35.353', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 80, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 101, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 8, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '6E80EF7D-E761-4720-A1BD-9E2CA7FDD21F', 'DateofService': 1607904000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'D31.31', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 89, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 14, 'pdiagnosis2_stringindexer_transform': 19, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '734CA534-2CD7-4A77-9D03-CF2390E8B0D6', 'DateofService': 1607558400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H30.23', 'pdiagnosis2': 'H35.353', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 41, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 136, 'DateofService_dayofmonth': 10, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 75, 'pdiagnosis2_stringindexer_transform': 8, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': 'B0501331-13EF-4126-B052-44D67EDFE7F9', 'DateofService': 1605312000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H35.372', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 318, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 7, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C0B29F0F-BBE1-47C7-B625-0989FDCC5288', 'DateofService': 1604275200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H33.312', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 488, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 17, 'pdiagnosis2_stringindexer_transform': 51, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 20}, {'ChargeDetailID': '136261A4-52B0-43B5-B099-1EF929506A93', 'DateofService': 1608163200000.0, 'Amount': 3348, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67036', 'CptModifier': 'LT', 'pdiagnosis1': 'H30.23', 'pdiagnosis2': 'H43.12', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 14, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 46, 'DateofService_dayofmonth': 17, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 14, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 75, 'pdiagnosis2_stringindexer_transform': 21, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '252D8E17-A9E8-48F5-84D3-6DA9B452EE75', 'DateofService': 1607212800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'D31.32', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 412, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 49, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '5A8270A6-53BD-4916-8805-B762FF67BE20', 'DateofService': 1607040000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H34.8120', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 283, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 26, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 10}, {'ChargeDetailID': '67841660-18E4-45DE-B386-285C53582762', 'DateofService': 1607299200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 97, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 374, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 12, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '9C60FA6D-FAAF-46A9-A149-1F941986D089', 'DateofService': 1605312000000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H33.313', 'pdiagnosis2': 'H43.11', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 46, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'F473817F-7515-46B7-97F9-348C88D7C348', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 221, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 146, 'pdiagnosis2_stringindexer_transform': 25, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 21}, {'ChargeDetailID': 'D6FE7A69-6B2B-495F-A36A-789DF588F6D5', 'DateofService': 1606694400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 84, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 68, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D9C0CCE1-6D5D-4DB7-8376-D8562C150145', 'DateofService': 1604707200000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 468, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 13, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '42B818D0-170D-4838-BC4A-7D22039C0F53', 'DateofService': 1605830400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 303, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 14, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '7DC8A130-8030-4915-8BF5-54015B27DAB6', 'DateofService': 1605312000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H25.13', 'pdiagnosis2': 'H33.311', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 36, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 81, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 72, 'pdiagnosis2_stringindexer_transform': 36, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'A3555B0C-5A99-4619-B6E8-9831D99F0210', 'DateofService': 1607904000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 54, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 476, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'AECE54B4-2AE0-44A1-9409-2BFB52D07BD2', 'DateofService': 1605398400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H34.8322', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 84, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 433, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 43, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'BE7612D0-7548-49C7-800E-173BAF9DF290', 'DateofService': 1604620800000.0, 'Amount': 50, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92225', 'CptModifier': 'LT', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H31.092', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 34, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 282, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 8, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 153, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '0EB85789-CE25-4CCC-850E-44D2C3496272', 'DateofService': 1604880000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H53.9', 'pdiagnosis2': 'H33.331', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 41, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 494, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 60, 'pdiagnosis2_stringindexer_transform': 142, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 9}, {'ChargeDetailID': '45F8ABD8-1D29-495F-B63D-866175B861A0', 'DateofService': 1608336000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.723', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 88, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 127, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 22, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A69FF747-5E5E-44A3-A2AE-806C56D75743', 'DateofService': 1607817600000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H30.133', 'pdiagnosis2': 'H30.93', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 39, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '2EAF4989-97B8-4F1B-9259-03E37314202F', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 85, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 105, 'pdiagnosis2_stringindexer_transform': 66, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 22}, {'ChargeDetailID': 'B1C24F17-5A6A-4D76-A42E-A273CF94898F', 'DateofService': 1608940800000.0, 'Amount': 3000, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3311', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 82, 'Gender': 'M', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 79, 'DateofService_dayofmonth': 26, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 9, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 32, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 5, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '41D99DEA-233E-47A6-98C0-8B6F522D1B70', 'DateofService': 1609113600000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 53, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 70, 'DateofService_dayofmonth': 28, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '53BD3672-31BA-4CE7-B2F3-2AFFC29C5F6F', 'DateofService': 1608508800000.0, 'Amount': 45, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92226', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H35.351', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 77, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 58, 'DateofService_dayofmonth': 21, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 11, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 11, 'pdiagnosis2_stringindexer_transform': 13, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '55F86AF5-F804-4A84-90ED-12E4125E1A8D', 'DateofService': 1605398400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.821', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 377, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 38, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 12}, {'ChargeDetailID': '8C94AF61-5EC3-44D7-B4B3-BBEB64AD2A23', 'DateofService': 1604880000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H33.42', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 32, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 474, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 84, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'B52B5682-9815-4EC1-A594-912A9724EB74', 'DateofService': 1604188800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H35.341', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 322, 'DateofService_dayofmonth': 1, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 46, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '079F8DAC-D8C1-48FA-9743-BC2AA08E287C', 'DateofService': 1607904000000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 53, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '2D60073D-3E0E-4347-B0A6-7E0CE91E92FA', 'DateofService': 1605744000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H47.012', 'pdiagnosis2': 'H47.10', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 86, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 410, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 136, 'pdiagnosis2_stringindexer_transform': 134, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '99772A77-B764-48FF-9242-5700C9572B65', 'DateofService': 1605225600000.0, 'Amount': 50, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92225', 'CptModifier': 'LT', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 165, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 8, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'E45AB682-6C1F-4B84-A1AC-3D1CD79AC3A9', 'DateofService': 1607299200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 295, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '51526EAE-3FB7-4C7E-9A33-26990B70EFFE', 'DateofService': 1607558400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H40.1110', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 77, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 238, 'DateofService_dayofmonth': 10, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 81, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '540B2A5F-11A9-4F0F-8038-B5A1ECE5E583', 'DateofService': 1606953600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8330', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 330, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 99, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '91D48DF5-F63A-466A-AFB9-2E65C4F2D44D', 'DateofService': 1606521600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H34.8320', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 89, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 436, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 9, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'ADF5D273-DB58-4F7F-8DF0-9D9D9647DA16', 'DateofService': 1607126400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H47.10', 'pdiagnosis2': 'H21.542', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 60, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '0BC878EE-8F48-415E-9233-25A7C28264DB', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 110, 'DateofService_dayofmonth': 5, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 127, 'pdiagnosis2_stringindexer_transform': 99, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 30}, {'ChargeDetailID': 'F672AF9B-E361-4572-A9C0-8B0731F9CD3E', 'DateofService': 1605225600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.711', 'pdiagnosis2': 'H35.3211', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 53, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 177, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 24, 'pdiagnosis2_stringindexer_transform': 76, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '34803FD1-6C40-4323-8657-F1FCC3E34990', 'DateofService': 1607904000000.0, 'Amount': 45, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92226', 'CptModifier': 'LT', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.352', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 435, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 11, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 16, 'pdiagnosis2_stringindexer_transform': 11, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '5327CFC0-5C48-435B-98EB-A76AAE858941', 'DateofService': 1605225600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3223', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 102, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 210, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 37, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D2DF16C4-D3AB-4E5C-9350-8F6FDE2EDA8D', 'DateofService': 1608249600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 153, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 14, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'E2DBCD71-B901-41F3-97B1-ED71030C7F2F', 'DateofService': 1604275200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 92, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 191, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 16, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '049BA1F1-F362-48BE-9393-C05CCEA8694B', 'DateofService': 1608768000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 46, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 208, 'DateofService_dayofmonth': 24, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 25, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 8}, {'ChargeDetailID': '13002A76-350E-4525-BDA1-9F181E056F74', 'DateofService': 1604793600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'Z79.899', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 63, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 19, 'pdiagnosis2_stringindexer_transform': 69, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '167E1195-CC7A-4F8C-8D91-9577BCBEE8FB', 'DateofService': 1607040000000.0, 'Amount': 286, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'E11.3513', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 124, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 6, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 18, 'pdiagnosis2_stringindexer_transform': 39, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '1F264D61-D5D7-4505-9AC2-25ACF275E3BD', 'DateofService': 1606953600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H20.9', 'pdiagnosis2': 'H59.022', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 197, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 69, 'pdiagnosis2_stringindexer_transform': 90, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '23BCDE27-9F80-48E4-B29D-348C871829B3', 'DateofService': 1607558400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H30.93', 'pdiagnosis2': 'E11.3292', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 55, 'DateofService_dayofmonth': 10, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 70, 'pdiagnosis2_stringindexer_transform': 57, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 10}, {'ChargeDetailID': 'A71A1D2F-3F1A-4CA6-AA98-E5D6A855C250', 'DateofService': 1605484800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 94, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'B62DA993-E320-4CEE-9D23-B3238CA5759A', 'DateofService': 1604707200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 285, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': 'B6375BF2-EFA5-44C9-A05C-F71DACD5F8A8', 'DateofService': 1606694400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 2, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C924A1C9-2E9A-4BF3-8BC7-25BFAEB02CE1', 'DateofService': 1608422400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 87, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D2300410-689E-453C-895D-FC7FDA559759', 'DateofService': 1605398400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3592', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 451, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 44, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'F12A6DBF-FBB4-4C62-8EE0-0F677D58EC3B', 'DateofService': 1608508800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 316, 'DateofService_dayofmonth': 21, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': 'FA9EEE47-7F76-4264-9A82-9B370BCF9534', 'DateofService': 1606953600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3124', 'pdiagnosis2': 'H35.352', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 100, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 57, 'pdiagnosis2_stringindexer_transform': 11, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'FBBC8CD5-C35A-42E5-AEBA-C5F61050B726', 'DateofService': 1608768000000.0, 'Amount': 6000, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3412', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'F', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 82, 'DateofService_dayofmonth': 24, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 9, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 20, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 5, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 10}, {'ChargeDetailID': '045700E8-F572-417C-A8DD-41E76C5C159A', 'DateofService': 1604275200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 90, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 401, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '1A088AE8-D8BA-404D-97E5-9DB1DFB972F8', 'DateofService': 1605484800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H31.113', 'pdiagnosis2': 'H35.3131', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 286, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 95, 'pdiagnosis2_stringindexer_transform': 47, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '1C109ACE-6610-4098-BE1A-C64F7805EBCA', 'DateofService': 1605744000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.341', 'pdiagnosis2': 'H35.3132', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 91, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 41, 'pdiagnosis2_stringindexer_transform': 65, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '25251236-8157-4EE2-89CD-C5A600E278BE', 'DateofService': 1605744000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.352', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 89, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 256, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 11, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A1818C97-82BE-4115-8C3C-3532CB8DE838', 'DateofService': 1606521600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E10.3292', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 309, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 91, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'AC9C3FBE-B694-4A6F-9A6F-6CC4563C5C73', 'DateofService': 1608249600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 365, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'B5E35D7B-A0F8-4A47-B72F-1D372236FC63', 'DateofService': 1606953600000.0, 'Amount': 45, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92226', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H35.411', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 43, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '789050C0-D1C3-40AD-8942-E67329E2F890', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 229, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 11, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 11, 'pdiagnosis2_stringindexer_transform': 72, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 36}, {'ChargeDetailID': 'C1119267-721A-4606-9D06-09FF240505EC', 'DateofService': 1605139200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 202, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 14, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C3339BC7-4E2E-4E0E-822E-7EE8E489451C', 'DateofService': 1607644800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.353', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 15, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 8, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'CDDFE064-998C-4CDD-97A2-04647A33EE7A', 'DateofService': 1604620800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H47.011', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 77, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 69, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 146, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D4930936-02DA-4DBB-A3B2-5391AC5D7153', 'DateofService': 1605225600000.0, 'Amount': 6000, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'F', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'BCA515C2-1B14-4060-9E24-B80BF9260648', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 23, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 9, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 5, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 14}, {'ChargeDetailID': 'FD8029C5-4ACA-4F97-8CE6-144AE49C8BCB', 'DateofService': 1607817600000.0, 'Amount': 1800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3411', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 51, 'Gender': 'F', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 64, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 7, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 31, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 4, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': '4C3D48F4-860F-49EF-812D-75B08DC1359C', 'DateofService': 1604534400000.0, 'Amount': 1800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3512', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 60, 'DateofService_dayofmonth': 5, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 7, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 26, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 4, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '58207A31-753A-42B1-86D8-E750AEA10079', 'DateofService': 1604707200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.811', 'pdiagnosis2': 'H35.371', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 80, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 479, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 28, 'pdiagnosis2_stringindexer_transform': 12, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'EAF0CF6B-9569-41B6-8B77-6335F34B3D7E', 'DateofService': 1607299200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '193BA5E7-F797-4FD1-A9AB-27590E630FCB', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 275, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 48}, {'ChargeDetailID': '693B32F4-ECA0-4844-80DF-6D9C94DD57BA', 'DateofService': 1607904000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3220', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 427, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 100, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '7E2F8EA5-4A1D-4490-A208-49CE684BB427', 'DateofService': 1607040000000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '26', 'pdiagnosis1': 'H33.22', 'pdiagnosis2': 'H35.022', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 7, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '5EA62C5D-D9C0-4AFE-ACAF-4AD36F6B1057', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 162, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 6, 'pdiagnosis1_stringindexer_transform': 120, 'pdiagnosis2_stringindexer_transform': 108, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 24}, {'ChargeDetailID': '83A30AD9-1329-4675-99E9-AF5ACBE98CFA', 'DateofService': 1607040000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.812', 'pdiagnosis2': 'H25.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 425, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 33, 'pdiagnosis2_stringindexer_transform': 113, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '84F4A744-F946-4B1E-99DF-20E0538C5F4F', 'DateofService': 1607731200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 80, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 291, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '90C2C4BC-4F1D-4730-8078-6CF025EB493F', 'DateofService': 1604707200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.372', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 121, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 16, 'pdiagnosis2_stringindexer_transform': 7, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'AE214CAC-A747-4549-B4E0-957075E94F38', 'DateofService': 1607040000000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H33.311', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 320, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 17, 'pdiagnosis2_stringindexer_transform': 36, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': 'BD337B93-0D12-4AE2-9566-35705B577186', 'DateofService': 1606521600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3120', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 113, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 68, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 9}, {'ChargeDetailID': 'CE11AF94-7EB4-4B98-BC57-51593FD36B72', 'DateofService': 1605139200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.721', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 184, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 45, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '1F6C5DD9-4889-43B7-8768-BA1D2873322F', 'DateofService': 1604793600000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'E11.3293', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 175, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 60, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '322A11BB-2B77-480A-BABA-2FD89CDD0305', 'DateofService': 1607299200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H35.342', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 74, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 33, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', 'DateofService': 1607904000000.0, 'Amount': 286, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 94, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 442, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 6, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '3342F086-2D91-4081-BEE0-C18AEAB0C981', 'DateofService': 1606348800000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 355, 'DateofService_dayofmonth': 26, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C4FFF996-0FB6-427D-9266-BE10E9F5DAA9', 'DateofService': 1608422400000.0, 'Amount': 1800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3311', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 59, 'Gender': 'F', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 12, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 7, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 32, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 4, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '510C2305-6CB3-460D-84C7-9EB6281F8E9D', 'DateofService': 1606521600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.053', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 57, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 254, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 141, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': 'FA7AC95E-1961-4A7B-AC57-3FD3212BAF20', 'DateofService': 1606608000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H40.051', 'pdiagnosis2': 'H35.3112', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 7, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 47, 'pdiagnosis2_stringindexer_transform': 127, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '04CCB591-8B59-4CCB-9EB0-5A4AECB3C705', 'DateofService': 1606521600000.0, 'Amount': 212, 'PatientBal': -10, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3213', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 98, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 42, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '0D66C132-CD95-442B-800E-E19F2651E018', 'DateofService': 1608422400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H34.8320', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 80, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 428, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 9, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '0ECEF203-5A40-428B-A19E-BFA699806763', 'DateofService': 1607299200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3134', 'pdiagnosis2': 'H35.52', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '72F5084C-9DF9-4F0A-8BA1-A9B40DFCC9C6', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 253, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 21, 'pdiagnosis2_stringindexer_transform': 80, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 32}, {'ChargeDetailID': '2A1A46A7-1E50-41C1-ADD1-EBC0F82168F9', 'DateofService': 1607299200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3592', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 84, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 387, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 44, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '6C6C6230-52AE-4590-AFA7-42A14991BD69', 'DateofService': 1607299200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.342', 'pdiagnosis2': 'H25.11', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 299, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 49, 'pdiagnosis2_stringindexer_transform': 91, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': '71B8A2A8-14EC-4B19-A4DA-6DE30E8A6E30', 'DateofService': 1607126400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'H43.392', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 255, 'DateofService_dayofmonth': 5, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 18, 'pdiagnosis2_stringindexer_transform': 41, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '2B3AACB7-A045-4E46-A2DB-8B841AD87F65', 'DateofService': 1607126400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'Z96.1', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 84, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 17, 'DateofService_dayofmonth': 5, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 15, 'pdiagnosis2_stringindexer_transform': 40, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '2FCDBCD0-49FC-4AA6-B249-041E476D766A', 'DateofService': 1607731200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 46, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 349, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 13, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '3DF7C0B1-EA0D-4014-916E-E76984AFDE2A', 'DateofService': 1605744000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3133', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 11, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 35, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '8A1DE8B3-2FDE-4581-AC94-4EA74A997CB0', 'DateofService': 1604707200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.821', 'pdiagnosis2': 'H35.371', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 32, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 38, 'pdiagnosis2_stringindexer_transform': 12, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 11}, {'ChargeDetailID': 'F49D6604-F705-480F-BCB9-E6F16B76DACF', 'DateofService': 1605225600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.23', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 91, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 378, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 79, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': '31C70405-0F52-4BD8-A785-13AFD15809A6', 'DateofService': 1608163200000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H35.413', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 23, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 323, 'DateofService_dayofmonth': 17, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 20, 'pdiagnosis2_stringindexer_transform': 14, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '323DACC7-ED74-42A3-A1E6-21BC104AAD51', 'DateofService': 1607817600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.052', 'pdiagnosis2': 'H35.352', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 46, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 366, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 53, 'pdiagnosis2_stringindexer_transform': 11, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '35841CCE-C3B1-4652-B011-C44F79BB442D', 'DateofService': 1606694400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H40.053', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 75, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 196, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 104, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': '8875D93D-866D-4897-81C3-67DB4570B102', 'DateofService': 1608163200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3133', 'pdiagnosis2': 'H35.722', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 28, 'DateofService_dayofmonth': 17, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 35, 'pdiagnosis2_stringindexer_transform': 61, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A7A12FBC-72F0-4216-8360-126AF826214C', 'DateofService': 1608508800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3133', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 271, 'DateofService_dayofmonth': 21, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 35, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'BDAADB25-9498-4D4F-809E-4590BAEE53B5', 'DateofService': 1605484800000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3412', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 472, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 20, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '0438621D-9EC2-4530-B3E8-F0C68FD8295F', 'DateofService': 1606435200000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.11', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 77, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 25, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '55E3B0D0-8E40-48F3-854B-A8E3C893E803', 'DateofService': 1604620800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 149, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '793B38EA-690C-456B-9597-5444E6D1499F', 'DateofService': 1604880000000.0, 'Amount': 130, 'PatientBal': -10, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99213', 'CptModifier': '0000', 'pdiagnosis1': 'H20.12', 'pdiagnosis2': 'H43.89', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 122, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 20, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 73, 'pdiagnosis2_stringindexer_transform': 48, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '899B4439-339E-4CC1-AC1E-4C1F4BA312EB', 'DateofService': 1607904000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 84, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 249, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A4CD837B-6B10-47C0-B9DC-E8D6ADF73B88', 'DateofService': 1604620800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.13', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 144, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 67, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'CC817F76-EE46-4AD5-9911-86A53FB1DBAE', 'DateofService': 1605312000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3213', 'pdiagnosis2': 'H35.353', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 416, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 42, 'pdiagnosis2_stringindexer_transform': 8, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'F1C5E648-7E44-43B6-A884-48A861BA52BB', 'DateofService': 1607299200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H35.351', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 48, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 434, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 11, 'pdiagnosis2_stringindexer_transform': 13, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '3D2ECD44-776B-437D-9DD6-936DD5B7A4BF', 'DateofService': 1607299200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3121', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 329, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 34, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 16}, {'ChargeDetailID': 'AE3A61BE-5820-45D3-BCD1-9982D0BA93EF', 'DateofService': 1604620800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 223, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 12, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C94A98D6-223E-47E8-9589-5631EBCB55E9', 'DateofService': 1604534400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 88, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 321, 'DateofService_dayofmonth': 5, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '05BBEB22-65F8-4773-89A4-452C6FD9DDE9', 'DateofService': 1608508800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 464, 'DateofService_dayofmonth': 21, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': '0C7AF11E-16DB-461A-A412-B711F0810E4F', 'DateofService': 1604188800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H43.10', 'pdiagnosis2': 'E11.3511', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 52, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 183, 'DateofService_dayofmonth': 1, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 124, 'pdiagnosis2_stringindexer_transform': 64, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '8745504F-A5BD-470D-ABAD-CDA68F032BE0', 'DateofService': 1608508800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3213', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 97, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 301, 'DateofService_dayofmonth': 21, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 61, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '8B63DA7A-BB50-450B-BA2A-E68DFFF3F4F6', 'DateofService': 1607817600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.723', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 57, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 22, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '0BC312A3-FBB5-4F2E-9A5C-6BAC5566A285', 'DateofService': 1605225600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H34.8320', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 91, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 344, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 9, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '1BA29057-DE25-4EF0-A06C-94A526C557A3', 'DateofService': 1604793600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 45, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '8E5DABE9-6BE2-4D98-BF87-D87C6A70C35B', 'DateofService': 1604534400000.0, 'Amount': 286, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3114', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 128, 'DateofService_dayofmonth': 5, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 6, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 46, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '9AE67B15-8F75-4CED-8129-80E003318733', 'DateofService': 1606435200000.0, 'Amount': 300, 'PatientBal': -250, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 50, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 132, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 15, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': 'D84605C6-4883-46E5-8BE3-4C82081FC5E0', 'DateofService': 1608249600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H43.393', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 257, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 11, 'pdiagnosis2_stringindexer_transform': 56, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 10}, {'ChargeDetailID': 'EF5E604D-077C-4903-95D7-5568576B7F96', 'DateofService': 1607817600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 289, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '14710135-067E-4E9D-91B1-E52FD6B72796', 'DateofService': 1605744000000.0, 'Amount': 254, 'PatientBal': -99, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99203', 'CptModifier': '0000', 'pdiagnosis1': 'H53.8', 'pdiagnosis2': 'H43.393', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 41, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 333, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 22, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 121, 'pdiagnosis2_stringindexer_transform': 56, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': 'ADC1E05E-8451-4C66-976B-3DBBB7103A27', 'DateofService': 1606953600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3124', 'pdiagnosis2': 'H40.1130', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 97, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '4CED3627-B1D6-4530-87D7-9609525FB29C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 370, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 57, 'pdiagnosis2_stringindexer_transform': 62, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 13}, {'ChargeDetailID': '1BDC9A8F-39D7-43D8-B3A8-5E9457AEB7CE', 'DateofService': 1606694400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3212', 'pdiagnosis2': 'E11.3291', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 458, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 62, 'pdiagnosis2_stringindexer_transform': 87, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '45BD84A3-1EE3-4B60-8352-D372DFEB677C', 'DateofService': 1606435200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'H33.41', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 49, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 27, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '6F66C6AD-3D4F-4EC2-9C71-D7580A768CF5', 'DateofService': 1606608000000.0, 'Amount': 3600, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'M', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 4, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 7, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 4, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 11}, {'ChargeDetailID': '74E7B927-7E80-423E-8EBE-94FC0B7E514F', 'DateofService': 1605484800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H35.352', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 359, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 11, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '82948A9C-6345-49ED-AB80-DD62B36CE11F', 'DateofService': 1604275200000.0, 'Amount': 148, 'PatientBal': -14, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H34.8120', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 225, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 26, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'BA941CAA-6F1A-4BBD-9D4A-42C905C8C41A', 'DateofService': 1607644800000.0, 'Amount': 300, 'PatientBal': -93, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H43.812', 'pdiagnosis2': 'H43.821', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 241, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 33, 'pdiagnosis2_stringindexer_transform': 24, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'ED3A209D-7398-4255-A9FB-3712B16F9DC5', 'DateofService': 1606608000000.0, 'Amount': 300, 'PatientBal': -170, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H33.321', 'pdiagnosis2': 'H35.412', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 53, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 111, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 40, 'pdiagnosis2_stringindexer_transform': 23, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': 'FBCA9AD8-4FC2-48A9-B059-0347A0EDA4E1', 'DateofService': 1604793600000.0, 'Amount': 6688, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67113', 'CptModifier': 'RT', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H33.321', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 88, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 19, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 17, 'pdiagnosis2_stringindexer_transform': 28, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '142DD0C0-133D-4FCB-934A-C6C50D564E61', 'DateofService': 1607040000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'C79.32', 'pdiagnosis2': 'H04.123', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 481, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 82, 'pdiagnosis2_stringindexer_transform': 59, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '803ED239-BF70-4FCD-B02C-B5977404BA98', 'DateofService': 1605484800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 75, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 176, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': '967ABAFC-DBB1-4878-89E6-2B32E0E8387C', 'DateofService': 1608508800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'C69.32', 'pdiagnosis2': 'D31.32', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 53, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 459, 'DateofService_dayofmonth': 21, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 23, 'pdiagnosis2_stringindexer_transform': 49, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'B3EE5A6F-6BC4-4F1D-B196-1127D5A7FED1', 'DateofService': 1605398400000.0, 'Amount': 286, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'E11.3213', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 29, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 6, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 135, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'B77643EB-41F3-4EF0-818C-B4617862C124', 'DateofService': 1605484800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.89', 'pdiagnosis2': 'H33.321', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 47, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 327, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 68, 'pdiagnosis2_stringindexer_transform': 28, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': 'CF7B7502-BFFD-479E-87EF-8C30089033C8', 'DateofService': 1605139200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H35.371', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 84, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 13, 'pdiagnosis2_stringindexer_transform': 12, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '04A3E774-1DF2-461B-8DCA-90EAB47932A0', 'DateofService': 1608508800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H53.9', 'pdiagnosis2': 'G45.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 52, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '330E2DD7-352E-4E5C-9669-AE796ABFE5DC', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 313, 'DateofService_dayofmonth': 21, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 60, 'pdiagnosis2_stringindexer_transform': 132, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 18}, {'ChargeDetailID': '1054BD81-AB8A-4FDC-9A87-F41814188945', 'DateofService': 1608422400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H30.22', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 411, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 128, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '202111D3-0B41-42EF-8113-161C06700D53', 'DateofService': 1607817600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3211', 'pdiagnosis2': 'E11.3292', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 392, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 22, 'pdiagnosis2_stringindexer_transform': 57, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '23863534-0389-4F1E-BDDC-96C82F56F4C0', 'DateofService': 1604793600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3121', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 75, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 324, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 34, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '906E67BB-7D18-4CBE-8802-172E4D09F677', 'DateofService': 1605484800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 48, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 269, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': 'A930D4B2-2DE6-4134-A58E-36E88012A437', 'DateofService': 1606521600000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8312', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 148, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 37, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': '043D822F-B7D6-433B-B8F7-3A0F58BFFC22', 'DateofService': 1605312000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'H35.70', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 147, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 18, 'pdiagnosis2_stringindexer_transform': 32, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '37409C70-626D-4D18-BFD4-749375A6D10F', 'DateofService': 1605484800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3212', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 314, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 62, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 10}, {'ChargeDetailID': 'C1DDC816-520D-4175-8613-E726EAF36CBA', 'DateofService': 1607558400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.721', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 252, 'DateofService_dayofmonth': 10, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 71, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'CCC6C19C-C3C3-4F0B-8D42-083ABB99F0E9', 'DateofService': 1605916800000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'E11.3532', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 47, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 421, 'DateofService_dayofmonth': 21, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 96, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'FF11B590-D529-4372-A722-252F47C1AA76', 'DateofService': 1605225600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H43.822', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 404, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 17, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'AF3AFE4B-9BD1-42B1-9D03-5031CDB276F7', 'DateofService': 1604620800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.433', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 345, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 83, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D45A198D-C1A5-4A54-8BCE-CD4AD0B2FBCF', 'DateofService': 1606348800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.812', 'pdiagnosis2': 'H33.312', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 135, 'DateofService_dayofmonth': 26, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 33, 'pdiagnosis2_stringindexer_transform': 51, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'E7C8D3FA-DCE9-4622-96F0-2047722D8564', 'DateofService': 1604534400000.0, 'Amount': 50, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92225', 'CptModifier': 'LT', 'pdiagnosis1': 'H35.412', 'pdiagnosis2': 'H43.822', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 361, 'DateofService_dayofmonth': 5, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 8, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 36, 'pdiagnosis2_stringindexer_transform': 17, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '399470D8-4590-411F-8652-380D7FB48A6F', 'DateofService': 1609372800000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H40.033', 'pdiagnosis2': 'E11.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 105, 'DateofService_dayofmonth': 31, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 139, 'pdiagnosis2_stringindexer_transform': 16, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'A8670196-A390-42F7-B247-84D40DCE973F', 'DateofService': 1605830400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8320', 'pdiagnosis2': 'H35.22', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 44, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 357, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 34, 'pdiagnosis2_stringindexer_transform': 141, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'F39EE67E-0589-4138-9AEF-261777DFC2B1', 'DateofService': 1606953600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3223', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 89, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 172, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 37, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'FA571DDD-0422-48A6-BE3F-1DFAABE49A5D', 'DateofService': 1604188800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3211', 'pdiagnosis2': 'E11.3312', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 47, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 118, 'DateofService_dayofmonth': 1, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 22, 'pdiagnosis2_stringindexer_transform': 18, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'DateofService': 1608249600000.0, 'Amount': 286, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 90, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '4502FB31-5B01-4EEF-BB9E-2D396D96A6FC', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 193, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 6, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 26}, {'ChargeDetailID': '1EA4A9FD-2C9B-4C54-9296-841FA5F9F23D', 'DateofService': 1604880000000.0, 'Amount': 212, 'PatientBal': 50, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'H43.392', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 55, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '8351B681-269A-4AE6-936A-FF10AFAFD629', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 338, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 14, 'pdiagnosis2_stringindexer_transform': 41, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 15}, {'ChargeDetailID': '3DAA771F-294A-4395-9B6B-30B898BC01E7', 'DateofService': 1608422400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3130', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 477, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 39, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': '411875F3-AAAD-48AC-92E6-112579D42F27', 'DateofService': 1606608000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H30.91', 'pdiagnosis2': 'I10', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 447, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 52, 'pdiagnosis2_stringindexer_transform': 94, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '7A0B900D-FB81-4652-AC59-4D27FB1BF2DA', 'DateofService': 1605139200000.0, 'Amount': 212, 'PatientBal': -10, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3124', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 91, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 243, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 12, 'pdiagnosis2_stringindexer_transform': 38, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'D1CA4C65-B76E-4F88-991D-E39CDBCB9A25', 'DateofService': 1607126400000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.411', 'pdiagnosis2': 'H35.372', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 47, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 83, 'DateofService_dayofmonth': 5, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 74, 'pdiagnosis2_stringindexer_transform': 7, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': 'D501A556-9400-4BBE-876D-4E2269A69E50', 'DateofService': 1607904000000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 48, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 181, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': 'E44DA795-311C-464F-A98C-AF783E9A8687', 'DateofService': 1605398400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3121', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 403, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 12, 'pdiagnosis2_stringindexer_transform': 34, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '8800B6CD-48B2-414A-AA85-2FB6E2F94F62', 'DateofService': 1608508800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.13', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'E0FE1EC8-51D4-430D-8B8B-E2F9966875F8', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 270, 'DateofService_dayofmonth': 21, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 122, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 25}, {'ChargeDetailID': '0C738DFF-CDE2-446B-8605-885F7FFEA814', 'DateofService': 1606694400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H47.211', 'pdiagnosis2': 'H31.012', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 155, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 107, 'pdiagnosis2_stringindexer_transform': 89, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '387FB6B8-8FE5-4483-A401-B595EF6BE611', 'DateofService': 1607040000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'H31.011', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 86, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 397, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 18, 'pdiagnosis2_stringindexer_transform': 128, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '9E48B685-ED16-456A-9BDA-73F1CDE529D5', 'DateofService': 1605916800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.461', 'pdiagnosis2': 'H30.92', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 38, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 388, 'DateofService_dayofmonth': 21, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 143, 'pdiagnosis2_stringindexer_transform': 150, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'FF73DA08-83EA-4F38-8153-B686DFBD480C', 'DateofService': 1609027200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3120', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 56, 'DateofService_dayofmonth': 27, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 68, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 9}, {'ChargeDetailID': '8E785A6C-A77D-46F3-B74F-701B18C72B2B', 'DateofService': 1606348800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.823', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 77, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 103, 'DateofService_dayofmonth': 26, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 50, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'B881C1B6-9321-4EB8-901E-843FB577ECDE', 'DateofService': 1607126400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3223', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 369, 'DateofService_dayofmonth': 5, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 37, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '6C01513E-DC0D-48F4-85F7-5DE34C582CA7', 'DateofService': 1608249600000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 246, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 13, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '88E69525-5461-45A9-879A-21A99C906710', 'DateofService': 1607644800000.0, 'Amount': 286, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H47.091', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 154, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 6, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 98, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '980CDD7C-B8FF-4C24-9EB8-E66EF9502A25', 'DateofService': 1605484800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.811', 'pdiagnosis2': 'H40.1130', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 91, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 293, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 28, 'pdiagnosis2_stringindexer_transform': 62, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '29761757-EC8E-4BC8-9BD4-764810C0D435', 'DateofService': 1605744000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H11.32', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 77, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 178, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 122, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '2E9CCC26-EE83-4CE0-BBE6-78198DE26939', 'DateofService': 1605312000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H57.11', 'pdiagnosis2': 'H33.43', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 57, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '32E7EC13-C493-40B1-A999-6AC94E7FA3B9', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 192, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 140, 'pdiagnosis2_stringindexer_transform': 144, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 19}, {'ChargeDetailID': '3596B626-33E4-4438-A2D7-C7B7F2884A61', 'DateofService': 1608768000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.433', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 170, 'DateofService_dayofmonth': 24, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 54, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '722ED05A-EAA6-41C2-AB96-ED5E231D9CA5', 'DateofService': 1606953600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.433', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 75, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 90, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 16, 'pdiagnosis2_stringindexer_transform': 54, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '8D12EF16-4731-4FDD-802D-73518A27A384', 'DateofService': 1607558400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.3134', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 86, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A4D682CE-4D10-4765-AE88-0D5D9E1862E7', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 24, 'DateofService_dayofmonth': 10, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 35, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 23}, {'ChargeDetailID': 'BB4D3CAB-1412-4339-A438-A8AFB1ECCA82', 'DateofService': 1607558400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3312', 'pdiagnosis2': 'E11.3511', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 93, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 315, 'DateofService_dayofmonth': 10, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 27, 'pdiagnosis2_stringindexer_transform': 64, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '113F2B98-6796-4055-8712-8147E996D2FB', 'DateofService': 1604707200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.89', 'pdiagnosis2': 'H30.93', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 350, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 68, 'pdiagnosis2_stringindexer_transform': 66, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '19635F03-1921-4E20-A610-C32DCD0A86E0', 'DateofService': 1605830400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H15.121', 'pdiagnosis2': 'K50.918', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 56, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 80, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 111, 'pdiagnosis2_stringindexer_transform': 97, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '1EDBDDB5-5D98-4516-87A4-135EB82D051F', 'DateofService': 1608336000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'E459B9E9-8650-4DA2-91F6-00B77900FC50', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 441, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 27}, {'ChargeDetailID': '28A393A5-A7B5-475F-8710-5DFB463F120E', 'DateofService': 1607644800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H34.8320', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 160, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 15, 'pdiagnosis2_stringindexer_transform': 9, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '36009550-373F-4179-BD6D-42DC2B91652B', 'DateofService': 1604620800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H35.40', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 89, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 372, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 55, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '8EEEA997-D2B5-4EC9-A50E-B2CBD0094405', 'DateofService': 1604188800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.341', 'pdiagnosis2': 'H40.043', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 88, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 146, 'DateofService_dayofmonth': 1, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 41, 'pdiagnosis2_stringindexer_transform': 88, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': 'C295E50E-9D1F-4967-B8F4-CE7ACB0DD858', 'DateofService': 1604707200000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 400, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 18, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'F3A2BB7A-DCFC-4740-AB2C-51B4DE18A84F', 'DateofService': 1606608000000.0, 'Amount': 1800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3211', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 82, 'Gender': 'F', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 391, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 7, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 22, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 4, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '077A557E-846D-453A-AE7D-9B3003FFA94F', 'DateofService': 1604880000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3111', 'pdiagnosis2': 'H35.352', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 31, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 30, 'pdiagnosis2_stringindexer_transform': 11, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 28}, {'ChargeDetailID': '0EDBE7EE-67FD-48BA-86E9-F70C9C504642', 'DateofService': 1604620800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H33.41', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 394, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 27, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': 'B06CBEAD-36DB-426E-84CE-89B56FD46159', 'DateofService': 1607126400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.823', 'pdiagnosis2': 'H35.052', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 406, 'DateofService_dayofmonth': 5, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 50, 'pdiagnosis2_stringindexer_transform': 82, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '1DEC7009-90C7-4C0B-8617-D9E805BD39DC', 'DateofService': 1606348800000.0, 'Amount': 118, 'PatientBal': 60, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H35.351', 'pdiagnosis2': 'T85.398A', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 231, 'DateofService_dayofmonth': 26, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 14, 'pdiagnosis2_stringindexer_transform': 121, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '65AAE408-BFF7-45D0-98E7-78DFDAEC64EC', 'DateofService': 1608249600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H40.051', 'pdiagnosis2': 'H35.412', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 306, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 47, 'pdiagnosis2_stringindexer_transform': 23, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': '823ACF06-0F8B-4DC7-AB68-4DFE535C2F92', 'DateofService': 1608422400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3311', 'pdiagnosis2': 'E11.3412', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 116, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 32, 'pdiagnosis2_stringindexer_transform': 20, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'B86E4CA3-AF97-467C-80C1-A05484D3A866', 'DateofService': 1607299200000.0, 'Amount': 286, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 390, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 6, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 8}, {'ChargeDetailID': 'D525418D-529A-470D-B010-7B90CD096743', 'DateofService': 1607040000000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '25', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3312', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 115, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 18, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '15695896-D50A-4650-8607-86092DE66F86', 'DateofService': 1609372800000.0, 'Amount': 50, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92225', 'CptModifier': 'RT', 'pdiagnosis1': 'H31.091', 'pdiagnosis2': 'H35.461', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 47, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 212, 'DateofService_dayofmonth': 31, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 8, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 126, 'pdiagnosis2_stringindexer_transform': 74, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '724A277E-2530-4A79-AC3F-6E4467BB2688', 'DateofService': 1607126400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3114', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 89, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 266, 'DateofService_dayofmonth': 5, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 46, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 11}, {'ChargeDetailID': '7C334D9D-8773-4B52-871C-7F4CE8FED784', 'DateofService': 1606953600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 233, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A15234F5-28B1-425F-8338-F05A4BF34B5A', 'DateofService': 1606953600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3512', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 59, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 385, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 26, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 8}, {'ChargeDetailID': '0CD457A8-5460-4F98-8059-107464922EFC', 'DateofService': 1609113600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 86, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 258, 'DateofService_dayofmonth': 28, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '1CE42B39-5F00-4C37-A276-6DFEE0CB161F', 'DateofService': 1607040000000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 42, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 277, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': '2AD5B7D3-9824-461C-8E4C-F5088C5A5061', 'DateofService': 1609113600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 328, 'DateofService_dayofmonth': 28, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '6295FB65-95AE-465C-A7BE-93CDEA6FCD41', 'DateofService': 1605830400000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 75, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 382, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '9E709F1E-38BF-4B10-B126-7C344A21DD11', 'DateofService': 1607817600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3134', 'pdiagnosis2': 'H11.31', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 82, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 130, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 21, 'pdiagnosis2_stringindexer_transform': 103, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '9EB1990C-6484-48F0-9B12-F05AA4B6815E', 'DateofService': 1605916800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.81', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 41, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 335, 'DateofService_dayofmonth': 21, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 147, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'E2272BDD-653A-47F9-A8BA-EEEDBC803C1B', 'DateofService': 1607299200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 88, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 22, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 10, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'FB109A88-81CE-4832-B4AC-56DD97251649', 'DateofService': 1607904000000.0, 'Amount': 45, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92226', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'E11.3391', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 47, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 11, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 123, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '53B02D9D-4160-42AA-936D-FCBB76F49B95', 'DateofService': 1607212800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H43.823', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 36, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 15, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '99D51AF1-7AD3-4DD6-96F0-25AA1E59972F', 'DateofService': 1607212800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3221', 'pdiagnosis2': 'H44.111', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 235, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 43, 'pdiagnosis2_stringindexer_transform': 136, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '07A0559D-6D61-4D02-9F83-0730224B6C93', 'DateofService': 1605225600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H44.2D2', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 58, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'C4A6C186-B867-46C5-A163-5AF23FE61DED', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 297, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 142, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 38}, {'ChargeDetailID': '4D06240B-F581-412B-B7B2-5F3A1060CB47', 'DateofService': 1605398400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.342', 'pdiagnosis2': 'H35.372', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 50, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 49, 'pdiagnosis2_stringindexer_transform': 7, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': 'D9501B03-2C0C-4096-A3F1-A4EBEC12D7AE', 'DateofService': 1607731200000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '25', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H25.13', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 251, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 30, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'EC536097-D4FA-4221-9C08-0C6168F80D46', 'DateofService': 1605139200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3134', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 96, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 114, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 21, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '11077BFE-4169-4F6F-A104-75B18FD0FF87', 'DateofService': 1607299200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'E11.3212', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 56, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 263, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 15, 'pdiagnosis2_stringindexer_transform': 101, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '2EB8DC5F-2F79-46F0-A78F-A59491C148F6', 'DateofService': 1606608000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.413', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 218, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 63, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '36EF7301-7E02-412A-9E23-47987397A484', 'DateofService': 1609113600000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 186, 'DateofService_dayofmonth': 28, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '620495B1-AE1E-4BEA-881B-AA8645DDAAB3', 'DateofService': 1605484800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H01.003', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 91, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '4CED3627-B1D6-4530-87D7-9609525FB29C', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 39, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 137, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 13}, {'ChargeDetailID': '73A89B2A-DEAD-456A-95A2-EA941FF2DBFB', 'DateofService': 1605484800000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': 'LT', 'pdiagnosis1': 'E11.3312', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 185, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 27, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 10}, {'ChargeDetailID': '83B3ABC6-1E23-4471-9F1E-78EA18B3CCE0', 'DateofService': 1608422400000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '57', 'pdiagnosis1': 'H33.322', 'pdiagnosis2': 'H35.413', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 50, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 228, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 5, 'pdiagnosis1_stringindexer_transform': 51, 'pdiagnosis2_stringindexer_transform': 14, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '9231D20B-208F-4B7A-BEEA-28718633282D', 'DateofService': 1608336000000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '25', 'pdiagnosis1': 'H35.052', 'pdiagnosis2': 'T66.XXXS', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 85, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 213, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 53, 'pdiagnosis2_stringindexer_transform': 112, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 31}, {'ChargeDetailID': '9FE9533D-C993-42CC-A477-DA0FD4E5FF72', 'DateofService': 1605484800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'Z98.890', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 65, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 50, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'DA09EFC9-6D5E-4472-8CF3-C9017C22A30E', 'DateofService': 1606694400000.0, 'Amount': 3885, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J7312', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3512', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'M', 'DepartmentCategory': 'Gastroenterology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 443, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 16, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 26, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 7, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'F9A0096A-9F38-42FA-B2FF-20B3F570FEB8', 'DateofService': 1607299200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E11.3292', 'pdiagnosis2': 'D31.31', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 89, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 73, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 88, 'pdiagnosis2_stringindexer_transform': 19, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '116B1537-FBEA-479D-8725-F3F98CE8696A', 'DateofService': 1605744000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H35.352', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 227, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 11, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '496D2DE6-BB81-4484-AC58-3BB0D13B4E8E', 'DateofService': 1608249600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 87, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 399, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': '8D6F6E45-269B-4E19-AB45-130E4C2F8EE5', 'DateofService': 1606348800000.0, 'Amount': 118, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H44.111', 'pdiagnosis2': 'H40.051', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 179, 'DateofService_dayofmonth': 26, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 130, 'pdiagnosis2_stringindexer_transform': 151, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'BF7CB069-0FD9-44DA-A277-EDEB8C31E102', 'DateofService': 1604793600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.12', 'pdiagnosis2': 'H25.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 351, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 58, 'pdiagnosis2_stringindexer_transform': 71, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '02705462-64DD-4C6E-AB7A-2CDDB023FE7A', 'DateofService': 1604188800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H33.103', 'pdiagnosis2': 'H35.413', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 34, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 393, 'DateofService_dayofmonth': 1, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 81, 'pdiagnosis2_stringindexer_transform': 14, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '3882B031-F286-4F77-8AAE-3ECC1E13433E', 'DateofService': 1606953600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H33.322', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 15, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 457, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 20, 'pdiagnosis2_stringindexer_transform': 53, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '9C0A75B2-5E85-479D-B614-3C97418C4570', 'DateofService': 1608163200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H35.722', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 380, 'DateofService_dayofmonth': 17, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 61, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'F5C916EC-F2CA-4136-9312-5A71933D7C4C', 'DateofService': 1606521600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3592', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 38, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 383, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 44, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '846A3024-1033-46EB-9F06-0F3E17DAD4AA', 'DateofService': 1604880000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H35.353', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'BCCB5D33-476A-4FAC-BB0C-5C7479803EB4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 311, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 8, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 44}, {'ChargeDetailID': 'CBA90F0E-ADB0-4F8F-A849-CD75DB7DD2A1', 'DateofService': 1604534400000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H30.93', 'pdiagnosis2': 'H43.391', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 490, 'DateofService_dayofmonth': 5, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 70, 'pdiagnosis2_stringindexer_transform': 114, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'D7A3A337-59AB-4474-8D56-FF4B8402E128', 'DateofService': 1604275200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'E11.3293', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 54, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 60, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '15527B5F-A9E1-4797-A410-9E324DD82868', 'DateofService': 1607731200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.11', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 58, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 9, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 25, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': '5649E152-6876-444A-97D3-66E0C38D7661', 'DateofService': 1606435200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3114', 'pdiagnosis2': 'H35.342', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 131, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 46, 'pdiagnosis2_stringindexer_transform': 33, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'BB2FFBE2-24D7-4618-B3FE-DC8B3ECA14AC', 'DateofService': 1605312000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 60, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 495, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'E760489F-6EE4-4B15-9B6D-5B12F6A5E3DC', 'DateofService': 1606694400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.11', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 261, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 25, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'FAD56DDC-6281-4433-94D8-23FCB65FD823', 'DateofService': 1606694400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H21.01', 'pdiagnosis2': 'G89.11', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 409, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 85, 'pdiagnosis2_stringindexer_transform': 129, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '74492123-CD52-45B6-AAFB-CDD634C4C9BC', 'DateofService': 1608336000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 356, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '79956D3A-C616-4A7A-9F06-BA0DC2806695', 'DateofService': 1605225600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3213', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 91, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A4D682CE-4D10-4765-AE88-0D5D9E1862E7', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 473, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 61, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 23}, {'ChargeDetailID': '9C582B3F-8717-4E03-84C9-F6EAAC3F4948', 'DateofService': 1604188800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H04.123', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 174, 'DateofService_dayofmonth': 1, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 65, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '394DE176-F74B-4877-A640-EF3C34905AE0', 'DateofService': 1605225600000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 141, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '3F563146-EE95-44B0-B130-41E8E1DE3030', 'DateofService': 1606608000000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H31.011', 'pdiagnosis2': 'E11.3513', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 215, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 119, 'pdiagnosis2_stringindexer_transform': 39, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '48C82871-65CB-40BD-98D3-6EB4CB2E0F61', 'DateofService': 1605830400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 56, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '8351B681-269A-4AE6-936A-FF10AFAFD629', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 486, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 11, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 15}, {'ChargeDetailID': '49B6D9F9-0BB5-43E9-B453-A0D4616A3689', 'DateofService': 1608768000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H35.372', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 119, 'DateofService_dayofmonth': 24, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 20, 'pdiagnosis2_stringindexer_transform': 7, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '5C1B73F4-3EDD-41BE-9FEC-7BAC25366092', 'DateofService': 1606608000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.54', 'pdiagnosis2': 'H35.432', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 46, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 482, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 133, 'pdiagnosis2_stringindexer_transform': 75, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '9D0125FC-58A9-4CD0-A823-57B6464A308F', 'DateofService': 1606953600000.0, 'Amount': 118, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 93, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 278, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '9E726301-D189-4224-945B-2FBA8FA26A03', 'DateofService': 1607904000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'D31.31', 'pdiagnosis2': 'H35.713', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 236, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 48, 'pdiagnosis2_stringindexer_transform': 155, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'A5EE1C1B-278D-478B-AC1F-980E200CB843', 'DateofService': 1606953600000.0, 'Amount': 212, 'PatientBal': 10, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H43.392', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 51, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '3BEA3D9A-B5EF-4B67-A4AB-868B7BE2C0A1', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 158, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 66, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 43}, {'ChargeDetailID': 'C8D69246-8334-4EBF-954E-5AAFA7C1F65F', 'DateofService': 1605139200000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 300, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'CAABED88-1591-4E9D-82C4-7926EA41DD52', 'DateofService': 1607212800000.0, 'Amount': 250, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99215', 'CptModifier': '25', 'pdiagnosis1': 'C69.32', 'pdiagnosis2': 'Z71.89', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 60, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 134, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 12, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 23, 'pdiagnosis2_stringindexer_transform': 31, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'DD23176B-512C-49D2-AC85-CFAA4256C72D', 'DateofService': 1605312000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.721', 'pdiagnosis2': 'H43.392', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 55, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 343, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 71, 'pdiagnosis2_stringindexer_transform': 41, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'FAD99EF5-F102-4FC6-BCCC-ECDEF99456BB', 'DateofService': 1604707200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H35.353', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 6, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 8, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '2653D41E-11AE-432B-993D-8CC4D4DAA1FA', 'DateofService': 1607817600000.0, 'Amount': 286, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 57, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 150, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 6, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '71E5F463-049A-42A7-A510-EACEC14E8301', 'DateofService': 1606348800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3110', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 77, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 216, 'DateofService_dayofmonth': 26, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 80, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '86D2CB7E-E54C-40E4-BCDD-C6D0DE7A6D7C', 'DateofService': 1605139200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3134', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 87, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 423, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 21, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'AB9BAF44-7980-4636-B844-488341E49BA9', 'DateofService': 1606608000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.372', 'pdiagnosis2': 'H35.342', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 46, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 475, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 8, 'pdiagnosis2_stringindexer_transform': 33, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'F66CDB59-6144-4776-B129-5FEED24970C3', 'DateofService': 1608422400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 27, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'F829CBA3-AD5D-4CF8-BE1F-3FDC44044AF7', 'DateofService': 1607817600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.40', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 90, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 34, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 55, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '244D35E3-5D74-4F5C-BB72-5860E1D4561C', 'DateofService': 1605225600000.0, 'Amount': 246, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92004', 'CptModifier': '57', 'pdiagnosis1': 'H33.312', 'pdiagnosis2': 'H52.13', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 32, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 207, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 10, 'CptModifier_stringindexer_transform': 5, 'pdiagnosis1_stringindexer_transform': 45, 'pdiagnosis2_stringindexer_transform': 73, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '2C3E981A-853E-45E7-87BD-799325176251', 'DateofService': 1607212800000.0, 'Amount': 212, 'PatientBal': -2, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H25.13', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 55, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 14, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 30, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '3B1A2F21-FFBF-4454-B0FB-BCBF1CEA4FE3', 'DateofService': 1608422400000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3312', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 54, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 334, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 18, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 20}, {'ChargeDetailID': '49FF4EFE-6A51-44EA-BDDA-27F6A4663C04', 'DateofService': 1606953600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H35.3131', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 95, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 47, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '6BD754C1-07F9-4C5D-A1B8-B77B284AFC7F', 'DateofService': 1605830400000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92018', 'CptModifier': '0000', 'pdiagnosis1': 'C69.21', 'pdiagnosis2': 'C69.22', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 5, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '0779204D-FC67-444E-97E3-83CB62A9D0A2', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 44, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 21, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 59, 'pdiagnosis2_stringindexer_transform': 63, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 41}, {'ChargeDetailID': '71A70101-0BDF-43F3-B580-4DB046CB1EBD', 'DateofService': 1605744000000.0, 'Amount': 3348, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67036', 'CptModifier': 'RT', 'pdiagnosis1': 'H59.021', 'pdiagnosis2': 'H27.131', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 405, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 14, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 144, 'pdiagnosis2_stringindexer_transform': 140, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '8076FF62-D8A1-4309-9EAD-F4B83E85DB1D', 'DateofService': 1605484800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'E11.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 38, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 16, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'DB05D823-A5CC-4AD0-87B0-6FBF69B7493E', 'DateofService': 1609372800000.0, 'Amount': 246, 'PatientBal': -63, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92004', 'CptModifier': '25', 'pdiagnosis1': 'H43.11', 'pdiagnosis2': 'H25.13', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 47, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 363, 'DateofService_dayofmonth': 31, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 10, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 64, 'pdiagnosis2_stringindexer_transform': 30, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'DC1BE341-E3CB-4EBF-B968-F5764D826DC7', 'DateofService': 1605916800000.0, 'Amount': 3600, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'M', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 126, 'DateofService_dayofmonth': 21, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 7, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 4, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'DCF767F0-7167-41FC-B981-296FBE63588B', 'DateofService': 1608249600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3111', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 95, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 0, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 30, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '46445DC3-6A22-4664-8F2F-C73C4F9B7010', 'DateofService': 1608249600000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J9035', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3311', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Pediatrics', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 157, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 18, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 32, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 6, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '9F250BA0-F703-4F1D-8AD6-82F2F4E58B70', 'DateofService': 1608163200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 484, 'DateofService_dayofmonth': 17, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'BCCDC778-B59A-44F9-B1B6-9B67462C4B0A', 'DateofService': 1604880000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8320', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': 'BCA515C2-1B14-4060-9E24-B80BF9260648', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 418, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 34, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 14}, {'ChargeDetailID': 'CF8CDC47-5AD3-442E-9F3C-07CF84EC35A5', 'DateofService': 1605744000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H35.89', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 437, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 109, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': 'EADDE82D-5C40-4BF2-8C07-A91EEAAF8EC5', 'DateofService': 1607212800000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '25', 'pdiagnosis1': 'D31.31', 'pdiagnosis2': 'D31.32', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 59, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 498, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 48, 'pdiagnosis2_stringindexer_transform': 49, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': 'F32643CC-08F3-4108-B5D1-4A881B590328', 'DateofService': 1605830400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.371', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 83, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 247, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 12, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C9139FC7-C65B-4F97-B7CC-56F810FD45DF', 'DateofService': 1605916800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.22', 'pdiagnosis2': 'H35.372', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 75, 'DateofService_dayofmonth': 21, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 102, 'pdiagnosis2_stringindexer_transform': 7, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'ECA40D88-F3E1-4286-8064-FD546E3C844E', 'DateofService': 1605744000000.0, 'Amount': 246, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92004', 'CptModifier': '0000', 'pdiagnosis1': 'H11.31', 'pdiagnosis2': 'S05.11XA', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 25, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 145, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 10, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 108, 'pdiagnosis2_stringindexer_transform': 124, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'FDBE8690-4E2C-4BD4-991A-74F5F3CC2872', 'DateofService': 1607299200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H33.311', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 209, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 36, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '04A27415-9D35-49E4-A5F5-305DFD917D5F', 'DateofService': 1608336000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 108, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '04EA280C-12AB-4842-9913-E97DFDDF508F', 'DateofService': 1606521600000.0, 'Amount': 383, 'PatientBal': 0, 'InsuranceBal': 76, 'PrimaryInsuranceBal': 76, 'CPTCode': '76512', 'CptModifier': 'LT', 'pdiagnosis1': 'H20.012', 'pdiagnosis2': 'H26.222', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': 'EDD37436-0E9D-45B5-8C46-A22CA763970A', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 461, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 13, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 145, 'pdiagnosis2_stringindexer_transform': 131, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 33}, {'ChargeDetailID': '0CB92950-EE3F-424C-AA0A-445F2D738518', 'DateofService': 1606348800000.0, 'Amount': 286, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8130', 'pdiagnosis2': 'H43.12', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 43, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 206, 'DateofService_dayofmonth': 26, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 6, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 115, 'pdiagnosis2_stringindexer_transform': 21, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': '29737DE0-22D6-4D41-870C-DA59BE4A4E75', 'DateofService': 1607299200000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3132', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 237, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 7, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'AA77712E-61BA-45A8-8E10-28057191D585', 'DateofService': 1604620800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.001', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 260, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 17, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 8}, {'ChargeDetailID': 'D271CCF9-CBA9-4A48-8202-5ED6FE135BDA', 'DateofService': 1605484800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'H35.372', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 85, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 92, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 16, 'pdiagnosis2_stringindexer_transform': 7, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'F488446B-42F0-4C47-B313-AE5AD0AD4C08', 'DateofService': 1606521600000.0, 'Amount': 1800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3211', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'M', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 273, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 7, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 22, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 4, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '0DC89466-2E6B-4DC7-B913-E5BBE5B168A0', 'DateofService': 1606608000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 86, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '27DEFE26-1A3A-4654-A273-799CEA02BB33', 'DateofService': 1608336000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H33.002', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 49, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 452, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 52, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '3D48B549-0C2E-4EC1-AA68-CD7DE250D65A', 'DateofService': 1605744000000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 77, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 104, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '790CB21B-A420-4C48-8EBC-67AAC5413D69', 'DateofService': 1606348800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.822', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 16, 'DateofService_dayofmonth': 26, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 17, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '9DD3CB15-B27B-4F87-A3C8-A9FC8D1F56B5', 'DateofService': 1608422400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.301', 'pdiagnosis2': 'H33.321', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 245, 'DateofService_dayofmonth': 20, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 55, 'pdiagnosis2_stringindexer_transform': 28, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 9}, {'ChargeDetailID': '2F71902A-768E-4A58-BE77-2AB583872514', 'DateofService': 1605398400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.351', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 85, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 182, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 13, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '305A3131-EFD4-420C-88B8-361D358D9857', 'DateofService': 1607904000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.341', 'pdiagnosis2': 'H34.8320', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 201, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 41, 'pdiagnosis2_stringindexer_transform': 9, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '65920EF9-C672-4D37-9A6B-795B0F656EDD', 'DateofService': 1605398400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3222', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 84, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 466, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 77, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '736AECC6-FFE6-48E7-9F0D-4AF0CDC06DED', 'DateofService': 1605139200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3122', 'pdiagnosis2': 'D31.31', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 408, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 16, 'pdiagnosis2_stringindexer_transform': 19, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '9DEA15FC-C28D-42CB-8283-AC5B84B7FCEF', 'DateofService': 1608249600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'E11.3392', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 138, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 149, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '0B1BA7BE-21C6-4B0E-A25F-C198051FD597', 'DateofService': 1604793600000.0, 'Amount': 212, 'PatientBal': 123, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.073', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 43, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 84, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '0E9EF90A-1AEA-403E-9058-D6F60A449238', 'DateofService': 1605139200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3112', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 90, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 217, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 2, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '235B02A5-8AC3-47BD-86BB-CAA1A63C317B', 'DateofService': 1606608000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H20.9', 'pdiagnosis2': 'H40.42X0', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 56, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 67, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 69, 'pdiagnosis2_stringindexer_transform': 126, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '48A25AFB-573A-4213-9B1C-E3654825F634', 'DateofService': 1608336000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.023', 'pdiagnosis2': 'H43.823', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 47, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 455, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 78, 'pdiagnosis2_stringindexer_transform': 15, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': '49C499CF-DD82-4195-B4F3-F889BE7E0BC4', 'DateofService': 1605744000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E10.3391', 'pdiagnosis2': 'E10.3312', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 33, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 454, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 106, 'pdiagnosis2_stringindexer_transform': 78, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '691933B0-0D3D-4355-B453-79E791D6DC36', 'DateofService': 1606348800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H40.051', 'pdiagnosis2': 'H44.701', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 18, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 99, 'DateofService_dayofmonth': 26, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 47, 'pdiagnosis2_stringindexer_transform': 115, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 28}, {'ChargeDetailID': '83B52DD3-F13E-450B-84A7-CAA79A556718', 'DateofService': 1606608000000.0, 'Amount': 6000, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3411', 'pdiagnosis2': 'E11.3512', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'F', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 164, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 9, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 31, 'pdiagnosis2_stringindexer_transform': 70, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 5, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '842AC936-21A9-4F15-8BF5-ACF6D4756EB0', 'DateofService': 1608336000000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.713', 'pdiagnosis2': 'H35.70', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 44, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 354, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 148, 'pdiagnosis2_stringindexer_transform': 32, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'A3CC38B5-605E-476B-A987-2AC63C81F539', 'DateofService': 1607904000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.321', 'pdiagnosis2': 'H33.002', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 332, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 40, 'pdiagnosis2_stringindexer_transform': 52, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C3A37CEA-E1D8-4DB2-837C-D1041863EC7F', 'DateofService': 1608336000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.322', 'pdiagnosis2': 'H35.413', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 50, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 276, 'DateofService_dayofmonth': 19, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 51, 'pdiagnosis2_stringindexer_transform': 14, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 5}, {'ChargeDetailID': 'CF82712C-91A7-46D2-9F6D-19A1BC8A1A0D', 'DateofService': 1608249600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'H44.23', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 82, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 102, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 19, 'pdiagnosis2_stringindexer_transform': 138, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '485494F4-D20A-4303-89BB-F09C5646A9EA', 'DateofService': 1605398400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H20.12', 'pdiagnosis2': 'H43.89', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 430, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 73, 'pdiagnosis2_stringindexer_transform': 48, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '4FEA26C5-DD7F-4A65-B859-D5FECFF66684', 'DateofService': 1604880000000.0, 'Amount': 250, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99215', 'CptModifier': '25', 'pdiagnosis1': 'H43.813', 'pdiagnosis2': 'D31.31', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 97, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 499, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 12, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 19, 'pdiagnosis2_stringindexer_transform': 19, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'B48620D1-DD13-471B-936F-AE9970B850B6', 'DateofService': 1604620800000.0, 'Amount': 342, 'PatientBal': -15, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99205', 'CptModifier': '25', 'pdiagnosis1': 'H31.8', 'pdiagnosis2': 'H25.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '8CA1878F-1BD4-4595-9AFF-3FF6967CD5CB', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 259, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 17, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 123, 'pdiagnosis2_stringindexer_transform': 130, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 50}, {'ChargeDetailID': 'BBF82897-F57E-484D-8159-C4E14BBECB5C', 'DateofService': 1606521600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 52, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'F0B25A0F-D965-460E-AA37-EA676B8BE27F', 'DateofService': 1606435200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E11.3312', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 88, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 348, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 27, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '279D822D-CECC-407C-8F7D-DA683C10C7B1', 'DateofService': 1607299200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.353', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'F3FB074E-423D-4F7F-A817-90C0551007A4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 424, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 8, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 17}, {'ChargeDetailID': '27E0E710-3B33-4DEC-BED0-FDB55F95EEE1', 'DateofService': 1606521600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.342', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 173, 'DateofService_dayofmonth': 28, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 49, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '8793C097-7DC2-47B0-8639-CBD108B34F5F', 'DateofService': 1605398400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.711', 'pdiagnosis2': 'H35.372', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 54, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 25, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 24, 'pdiagnosis2_stringindexer_transform': 7, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '1D2CA217-3B28-4C61-89C2-7C55BAB37BDD', 'DateofService': 1606608000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H35.413', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 139, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 14, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '3A4437B5-45EE-4734-BF9C-C7BA808C123B', 'DateofService': 1607040000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.352', 'pdiagnosis2': 'H43.823', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 432, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 44, 'pdiagnosis2_stringindexer_transform': 15, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '13872B75-E219-4B25-9F7E-24F021CF0B48', 'DateofService': 1604707200000.0, 'Amount': 118, 'PatientBal': -38, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '25', 'pdiagnosis1': 'H35.63', 'pdiagnosis2': 'H35.3134', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 97, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 37, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 92, 'pdiagnosis2_stringindexer_transform': 35, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '377FADE6-202C-4841-BDD4-82367D36097C', 'DateofService': 1604620800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3131', 'pdiagnosis2': 'H25.13', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'E0FE1EC8-51D4-430D-8B8B-E2F9966875F8', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 489, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 10, 'pdiagnosis2_stringindexer_transform': 30, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 25}, {'ChargeDetailID': '6A29D415-E299-4812-B8CD-8CD471F28115', 'DateofService': 1607558400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'H43.12', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 84, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 358, 'DateofService_dayofmonth': 10, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 21, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '84A66A4F-CB48-4BA8-B11C-F2EA34C4A1A4', 'DateofService': 1604707200000.0, 'Amount': 6000, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3411', 'pdiagnosis2': 'E11.3512', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'F', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 398, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 9, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 31, 'pdiagnosis2_stringindexer_transform': 70, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 5, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D7B0792D-0B5F-4CAA-9787-237DFECCA98B', 'DateofService': 1604793600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.70', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 5, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 32, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '6D22E643-C022-4637-A81F-22A0DB410A33', 'DateofService': 1606608000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3211', 'pdiagnosis2': 'H34.8120', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 220, 'DateofService_dayofmonth': 29, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 22, 'pdiagnosis2_stringindexer_transform': 26, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '7C771536-6EAB-4FEA-B848-E077662307D7', 'DateofService': 1604793600000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': 'LT', 'pdiagnosis1': 'E11.3312', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 367, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 27, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'DD0C3E18-53F4-4001-9334-BEC408540A3E', 'DateofService': 1607644800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H33.003', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 26, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 11, 'pdiagnosis2_stringindexer_transform': 154, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'EA11E355-7A8A-4BAD-9066-1FC3E824CBC5', 'DateofService': 1605830400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'C69.32', 'pdiagnosis2': 'T66.XXXA', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 54, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 188, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 23, 'pdiagnosis2_stringindexer_transform': 116, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '267E6CA7-6B3A-4582-B01B-9DD2C4AC8DEE', 'DateofService': 1605312000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.811', 'pdiagnosis2': 'H35.413', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '315C773B-4EF1-4164-AB43-6817D993329F', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 222, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 28, 'pdiagnosis2_stringindexer_transform': 14, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 42}, {'ChargeDetailID': '43B4ED70-4EAA-4C2A-AB60-F4A3848BF5E9', 'DateofService': 1607644800000.0, 'Amount': 118, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '25', 'pdiagnosis1': 'D31.32', 'pdiagnosis2': 'H35.3122', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 75, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 1, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 132, 'pdiagnosis2_stringindexer_transform': 10, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '7D59A1A0-3C61-4F7A-AD1E-0EF39B392E83', 'DateofService': 1607644800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 53, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 143, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'FF80B585-D64F-45DC-9CE9-59C51F3001D2', 'DateofService': 1607644800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H34.8322', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 85, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AAB72015-9704-40ED-A2E7-87468E793D48', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 30, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 12, 'pdiagnosis2_stringindexer_transform': 43, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 47}, {'ChargeDetailID': '324408CC-B86D-49E1-A0DE-99A7EE03276E', 'DateofService': 1606435200000.0, 'Amount': 118, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8120', 'pdiagnosis2': 'H20.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 40, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 156, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 29, 'pdiagnosis2_stringindexer_transform': 29, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '3E505BF0-6673-4C9F-996F-64E07E72E60C', 'DateofService': 1607212800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 106, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '5FFAD31C-B0EF-4A88-B7EA-2F02D7B336D5', 'DateofService': 1604275200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8110', 'pdiagnosis2': 'H43.823', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 20, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 15, 'pdiagnosis2_stringindexer_transform': 15, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '8CC0B44A-2E34-417F-A084-3E2F067F6966', 'DateofService': 1605312000000.0, 'Amount': 383, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '76512', 'CptModifier': 'LT', 'pdiagnosis1': 'H33.042', 'pdiagnosis2': 'Z98.890', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 24, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '330E2DD7-352E-4E5C-9669-AE796ABFE5DC', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 248, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 13, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 89, 'pdiagnosis2_stringindexer_transform': 50, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 18}, {'ChargeDetailID': 'B29EEE24-30E4-4B06-9FA3-777B1E99463D', 'DateofService': 1605225600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.811', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 415, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 5, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C84DCC7E-7D84-433A-8D23-74F8BB79EE40', 'DateofService': 1606953600000.0, 'Amount': 118, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '25', 'pdiagnosis1': 'H43.12', 'pdiagnosis2': 'H35.413', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 78, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 58, 'pdiagnosis2_stringindexer_transform': 14, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'E776DC1F-10CF-4EC1-A151-05EC29E6C03C', 'DateofService': 1604275200000.0, 'Amount': 6000, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J0178', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 43, 'Gender': 'F', 'DepartmentCategory': 'Physical Therapy', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 204, 'DateofService_dayofmonth': 2, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 9, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 5, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '40F1314D-EBF6-49BC-B1A8-674F07F92039', 'DateofService': 1604188800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.353', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 92, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 267, 'DateofService_dayofmonth': 1, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 8, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '66E28365-DDB9-4F59-916F-2B171C5D93AB', 'DateofService': 1604707200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H04.123', 'pdiagnosis2': 'H35.412', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 59, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '20C2761F-16B5-42B7-8C93-82A112735F02', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 40, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 65, 'pdiagnosis2_stringindexer_transform': 23, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 34}, {'ChargeDetailID': '88539DEA-C521-4D06-A758-7FEBC99D5C5E', 'DateofService': 1607644800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8312', 'pdiagnosis2': 'H34.8120', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 426, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 37, 'pdiagnosis2_stringindexer_transform': 26, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 16}, {'ChargeDetailID': '8CCEE88C-C36B-4DE3-88D7-00D78C1D9B0A', 'DateofService': 1605830400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H33.321', 'pdiagnosis2': 'H33.312', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 59, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 274, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 40, 'pdiagnosis2_stringindexer_transform': 51, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'C0AC5031-0382-4350-82C5-5B80634DEEAC', 'DateofService': 1605484800000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8312', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 54, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 117, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 37, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'E5775C4C-F86F-49D2-8B58-22ACA6668352', 'DateofService': 1606694400000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'E11.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 65, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 302, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 16, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '08683FBF-CD1F-42D9-A442-EF696A6D7C83', 'DateofService': 1606435200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H33.002', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 70, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 496, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 20, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '289AA716-A1D3-48FF-B830-4BF9DD4A4FB7', 'DateofService': 1608249600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'T85.29XA', 'pdiagnosis2': 'H20.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 54, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '84222BD3-605E-43EA-9A84-7C0169BE0B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 493, 'DateofService_dayofmonth': 18, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 116, 'pdiagnosis2_stringindexer_transform': 29, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 9}, {'ChargeDetailID': '28D97A0D-03AE-4405-BFD3-66E00EDAE377', 'DateofService': 1604620800000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 85, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 107, 'DateofService_dayofmonth': 6, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '55F62337-F077-4BA9-87D8-4C7AC0736913', 'DateofService': 1607212800000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H33.323', 'pdiagnosis2': 'H35.463', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 23, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '835BC6C3-FEDE-4AF9-8B88-91628A98389C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 465, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 96, 'pdiagnosis2_stringindexer_transform': 58, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 45}, {'ChargeDetailID': '56A0F3CD-F2AA-4492-A3F3-D8694240FF34', 'DateofService': 1605744000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.823', 'pdiagnosis2': 'H11.001', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 230, 'DateofService_dayofmonth': 19, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 50, 'pdiagnosis2_stringindexer_transform': 118, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': 'B16FA6E7-E985-411C-AA06-A1F7E6BBCEFB', 'DateofService': 1605916800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'C69.21', 'pdiagnosis2': 'C69.22', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 7, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '5EA62C5D-D9C0-4AFE-ACAF-4AD36F6B1057', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 203, 'DateofService_dayofmonth': 21, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 59, 'pdiagnosis2_stringindexer_transform': 63, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 24}, {'ChargeDetailID': '0BB8A5C4-7444-4DEE-A3AC-2E0CFF7A3477', 'DateofService': 1605139200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 61, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '1DB8D73F-61E5-4065-9342-592BE88E5981', 'DateofService': 1605484800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H40.053', 'pdiagnosis2': 'H43.823', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 194, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 109, 'pdiagnosis2_stringindexer_transform': 15, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '4C323EC0-5AB8-4CAF-9381-0011A0141AD7', 'DateofService': 1607904000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H44.23', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 37, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 487, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 131, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 37}, {'ChargeDetailID': '07BDF3AA-3B6E-4650-BDFC-596FE687C33F', 'DateofService': 1604534400000.0, 'Amount': 50, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92225', 'CptModifier': 'LT', 'pdiagnosis1': 'H35.412', 'pdiagnosis2': 'H33.322', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 48, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D1C4AE8C-46B5-464F-91E1-A39C5C76CC25', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 307, 'DateofService_dayofmonth': 5, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 8, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 36, 'pdiagnosis2_stringindexer_transform': 53, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 39}, {'ChargeDetailID': '3F664203-58B4-411D-835A-667F32E1AF1E', 'DateofService': 1607817600000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '57', 'pdiagnosis1': 'H33.311', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 211, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 5, 'pdiagnosis1_stringindexer_transform': 54, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '94F9C8EF-40F1-4CB4-B8F1-71EA8B1482D9', 'DateofService': 1607212800000.0, 'Amount': 246, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92004', 'CptModifier': '0000', 'pdiagnosis1': 'H43.392', 'pdiagnosis2': 'H33.321', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 386, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 10, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 66, 'pdiagnosis2_stringindexer_transform': 28, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A56274A7-1DCD-46CB-84FC-83FD3C26472D', 'DateofService': 1606435200000.0, 'Amount': 212, 'PatientBal': -25, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H10.45', 'pdiagnosis2': 'H33.41', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 52, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 429, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 100, 'pdiagnosis2_stringindexer_transform': 27, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': 'BDB2ADB4-356A-40AF-9E57-C6ECA2EB1235', 'DateofService': 1605398400000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3111', 'pdiagnosis2': 'H35.373', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 82, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 470, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 30, 'pdiagnosis2_stringindexer_transform': 4, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'F28706C5-DDB5-4FC3-9F7D-4896DB86AF18', 'DateofService': 1604793600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'E11.9', 'pdiagnosis2': 'H35.3231', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 456, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 97, 'pdiagnosis2_stringindexer_transform': 125, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '3DA05017-5FD5-4CBE-9CC5-956876EECBE6', 'DateofService': 1607299200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3130', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 375, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 39, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '6AD7EB61-BE8E-49A3-929C-CE2CA6C95779', 'DateofService': 1607212800000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3313', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 64, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AD96772F-54C7-4750-86B1-1BD565B55244', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 161, 'DateofService_dayofmonth': 6, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 5, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 3}, {'ChargeDetailID': '8B79B6F2-FE78-4775-8668-2A35ECA963E5', 'DateofService': 1608508800000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'T66.XXXA', 'pdiagnosis2': 'H35.351', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 58, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 347, 'DateofService_dayofmonth': 21, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 114, 'pdiagnosis2_stringindexer_transform': 13, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '98AEC5FB-BE93-4E59-A4A9-9533419B73B6', 'DateofService': 1607644800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 312, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'A24C0F04-7684-4CC9-BFBF-CD7E01137187', 'DateofService': 1606435200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8120', 'pdiagnosis2': 'H35.372', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 265, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 29, 'pdiagnosis2_stringindexer_transform': 7, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'C22535F6-D4C8-4B58-B0C4-BA24CE26B9E7', 'DateofService': 1605484800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.61', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 101, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 450, 'DateofService_dayofmonth': 16, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 25, 'pdiagnosis2_stringindexer_transform': 42, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D9115251-3465-4EF5-B9B3-54B0EAF82045', 'DateofService': 1605312000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 36, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'H43.823', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 57, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 189, 'DateofService_dayofmonth': 14, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 13, 'pdiagnosis2_stringindexer_transform': 15, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '24253A09-E06B-4C51-A3AD-0709BD0DA4B5', 'DateofService': 1606435200000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H31.091', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'F473817F-7515-46B7-97F9-348C88D7C348', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 360, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 133, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 21}, {'ChargeDetailID': '449D29CA-9C67-4DA9-95FE-C1531DA5DE4C', 'DateofService': 1607817600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H34.8320', 'pdiagnosis2': 'H43.812', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 68, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 279, 'DateofService_dayofmonth': 13, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 34, 'pdiagnosis2_stringindexer_transform': 6, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '77C723D8-92FE-487C-825B-93C8605DC74E', 'DateofService': 1607040000000.0, 'Amount': 800, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': '50', 'pdiagnosis1': 'E11.3213', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'F', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 239, 'DateofService_dayofmonth': 4, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 3, 'pdiagnosis1_stringindexer_transform': 42, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '8BAEB725-2E28-4DE9-B911-836846DF6F11', 'DateofService': 1605916800000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 90, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 368, 'DateofService_dayofmonth': 21, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 10, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D89C117A-4B7C-471A-94A0-1F6F022193BB', 'DateofService': 1608163200000.0, 'Amount': 162, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92250', 'CptModifier': '0000', 'pdiagnosis1': 'H35.412', 'pdiagnosis2': 'H33.001', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 20, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 109, 'DateofService_dayofmonth': 17, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 2, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 36, 'pdiagnosis2_stringindexer_transform': 92, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'E0D39756-5A76-4FA1-BA20-C957C3486DFC', 'DateofService': 1607731200000.0, 'Amount': 50, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92225', 'CptModifier': 'LT', 'pdiagnosis1': 'H43.812', 'pdiagnosis2': 'H43.12', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 120, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 8, 'CptModifier_stringindexer_transform': 4, 'pdiagnosis1_stringindexer_transform': 33, 'pdiagnosis2_stringindexer_transform': 21, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '0A6DE37A-0F94-43B3-84E3-7590769F820E', 'DateofService': 1605225600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3222', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 90, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 48, 'DateofService_dayofmonth': 13, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 77, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 12}, {'ChargeDetailID': '0F43FFB3-7DF2-4156-BD39-A6C0667C7392', 'DateofService': 1605398400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '25', 'pdiagnosis1': 'H43.391', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 63, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 353, 'DateofService_dayofmonth': 15, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 1, 'pdiagnosis1_stringindexer_transform': 113, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '08AD40B0-B651-4069-AA92-9A64FC246B17', 'DateofService': 1604707200000.0, 'Amount': 45, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92226', 'CptModifier': 'RT', 'pdiagnosis1': 'H35.101', 'pdiagnosis2': 'H33.41', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 28, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 97, 'DateofService_dayofmonth': 7, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 11, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 137, 'pdiagnosis2_stringindexer_transform': 27, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '0EC59014-45A0-45B7-AB3D-12367ECD4174', 'DateofService': 1607299200000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H31.422', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 72, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 319, 'DateofService_dayofmonth': 7, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 111, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '13E519C2-73BD-451A-A651-55B67924A9B6', 'DateofService': 1604793600000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '67028', 'CptModifier': 'RT', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 71, 'Gender': 'M', 'DepartmentCategory': 'Cardiology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 317, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 4, 'CptModifier_stringindexer_transform': 2, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 2, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 11}, {'ChargeDetailID': '26C382E6-F716-4F7F-A72C-A6C7150C2066', 'DateofService': 1606953600000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H30.013', 'pdiagnosis2': 'H44.2A3', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 61, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 376, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 87, 'pdiagnosis2_stringindexer_transform': 143, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 4}, {'ChargeDetailID': '989C36A9-F0D6-48B5-8D26-F5A56E610792', 'DateofService': 1606694400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H34.8310', 'pdiagnosis2': 'H35.61', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 73, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 485, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 25, 'pdiagnosis2_stringindexer_transform': 42, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': 'C4ADE72E-AD16-4711-886D-AB997CCB5E71', 'DateofService': 1604534400000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3111', 'pdiagnosis2': 'H35.3122', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 492, 'DateofService_dayofmonth': 5, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 30, 'pdiagnosis2_stringindexer_transform': 10, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'DE90AF3B-500B-42BC-B1F8-D4E5971DD853', 'DateofService': 1607644800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.353', 'pdiagnosis2': 'H35.3131', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 81, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 140, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 18, 'pdiagnosis2_stringindexer_transform': 47, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '03A51C92-3977-42BE-B158-53B124029BCC', 'DateofService': 1605830400000.0, 'Amount': 118, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92012', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.342', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 288, 'DateofService_dayofmonth': 20, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 5, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 33, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '176F9200-4582-49AB-80EB-558534EC297C', 'DateofService': 1606435200000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'E11.3513', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 491, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 11, 'pdiagnosis2_stringindexer_transform': 39, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 12}, {'ChargeDetailID': '2861F7ED-8D48-436D-9AD0-FEF67A864566', 'DateofService': 1604188800000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.373', 'pdiagnosis2': 'H35.353', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 88, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 137, 'DateofService_dayofmonth': 1, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 9, 'pdiagnosis2_stringindexer_transform': 8, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': '7157C3A4-F8EE-42C2-864E-C9E7FD5C62C6', 'DateofService': 1607644800000.0, 'Amount': 3600, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3413', 'pdiagnosis2': 'Z79.4', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 74, 'Gender': 'M', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 460, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 7, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 13, 'pdiagnosis2_stringindexer_transform': 0, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 4, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 2}, {'ChargeDetailID': 'FB5D5A33-BA7C-4140-AB6B-9AC8E94FABA6', 'DateofService': 1607904000000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.50', 'pdiagnosis2': 'H35.3221', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 199, 'DateofService_dayofmonth': 14, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 104, 'pdiagnosis2_stringindexer_transform': 2, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '3AAEBB1A-A489-4699-A7B4-F237F130B7EE', 'DateofService': 1607644800000.0, 'Amount': 286, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92235', 'CptModifier': '0000', 'pdiagnosis1': 'H35.033', 'pdiagnosis2': 'H34.8320', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 56, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 446, 'DateofService_dayofmonth': 11, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 6, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 4, 'pdiagnosis2_stringindexer_transform': 9, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': 'A3646A59-090D-4D1F-AE88-16B127AF81ED', 'DateofService': 1604793600000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H33.312', 'pdiagnosis2': 'Z96.1', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '0A471D4C-3977-4BF5-8039-5064519FDCD4', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 200, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 45, 'pdiagnosis2_stringindexer_transform': 40, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 8}, {'ChargeDetailID': 'A90B0781-610E-4FE4-80ED-BD9F6ABDE174', 'DateofService': 1604880000000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'H35.352', 'pdiagnosis2': 'H20.9', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 76, 'Gender': 'F', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 448, 'DateofService_dayofmonth': 9, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 44, 'pdiagnosis2_stringindexer_transform': 29, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': 'AF591C35-31A5-45DD-9399-292D82D794AF', 'DateofService': 1607731200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3212', 'pdiagnosis2': 'H35.3122', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 78, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 396, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 12, 'pdiagnosis2_stringindexer_transform': 10, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'D06DE332-BDE9-4D63-8959-BF68C5A40297', 'DateofService': 1605139200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3231', 'pdiagnosis2': 'H43.813', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 91, 'Gender': 'F', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 331, 'DateofService_dayofmonth': 12, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 1, 'pdiagnosis2_stringindexer_transform': 3, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': '23AEEFFE-19BA-487C-A1D5-A88964FF3BC1', 'DateofService': 1609027200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.3211', 'pdiagnosis2': 'H35.3122', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 69, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 422, 'DateofService_dayofmonth': 27, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 3, 'pdiagnosis2_stringindexer_transform': 10, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '341B166A-3402-4A75-AD95-D57FDDA27DC9', 'DateofService': 1607731200000.0, 'Amount': 148, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92134', 'CptModifier': '0000', 'pdiagnosis1': 'H35.371', 'pdiagnosis2': 'H35.033', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 66, 'Gender': 'M', 'DepartmentCategory': 'Emergency', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 214, 'DateofService_dayofmonth': 12, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 0, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 11, 'pdiagnosis2_stringindexer_transform': 1, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 0, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': '45D50939-3539-478D-AABE-C57C6791CF1F', 'DateofService': 1604793600000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'Q14.1', 'pdiagnosis2': 'H47.233', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 36, 'Gender': 'F', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'Gender_Recoded': 1, 'ChargeDetailID_stringindexer_transform': 402, 'DateofService_dayofmonth': 8, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 93, 'pdiagnosis2_stringindexer_transform': 95, 'Gender_stringindexer_transform': 0, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 7}, {'ChargeDetailID': '4A1D1603-893E-46B7-92C1-8C3069A4B63D', 'DateofService': 1606435200000.0, 'Amount': 400, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99354', 'CptModifier': '0000', 'pdiagnosis1': 'C69.32', 'pdiagnosis2': 'Z71.89', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 79, 'Gender': 'M', 'DepartmentCategory': 'Pediatrics', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'B03B74CE-C223-4B1E-802B-5F5F76BFE64C', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 268, 'DateofService_dayofmonth': 27, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 15, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 23, 'pdiagnosis2_stringindexer_transform': 31, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 6, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 6}, {'ChargeDetailID': '502690F0-D587-4D27-B3A7-5B9679F7E217', 'DateofService': 1609113600000.0, 'Amount': 3600, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': 'J2778PF3', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3511', 'pdiagnosis2': 'E11.3312', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 67, 'Gender': 'M', 'DepartmentCategory': 'Ophtamology', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 280, 'DateofService_dayofmonth': 28, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 7, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 6, 'pdiagnosis2_stringindexer_transform': 18, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 4, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}, {'ChargeDetailID': '97E40160-D786-4460-B2D1-FCFFE2DB599F', 'DateofService': 1606694400000.0, 'Amount': 212, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '92014', 'CptModifier': '0000', 'pdiagnosis1': 'E11.3513', 'pdiagnosis2': 'H43.13', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 62, 'Gender': 'M', 'DepartmentCategory': 'Pulmonary', 'SpecialityCategory': 'Ophtamology', 'label': 0, 'PayerId': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 483, 'DateofService_dayofmonth': 30, 'DateofService_month': 11, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 1, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 0, 'pdiagnosis2_stringindexer_transform': 67, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 1, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 0}, {'ChargeDetailID': 'A372D150-FF7E-4BA6-9349-DDD6451A1084', 'DateofService': 1606953600000.0, 'Amount': 300, 'PatientBal': 0, 'InsuranceBal': 0, 'PrimaryInsuranceBal': 0, 'CPTCode': '99204', 'CptModifier': '0000', 'pdiagnosis1': 'H35.712', 'pdiagnosis2': 'H31.002', 'IsBillable': 1, 'Isauthorized': 0, 'PredeterminationIndicator': 0, 'MedNecessityRequiredIndicator': 0, 'Age': 46, 'Gender': 'M', 'DepartmentCategory': 'Orthopedic', 'SpecialityCategory': 'Ophtamology', 'label': 1, 'PayerId': 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', 'Gender_Recoded': 2, 'ChargeDetailID_stringindexer_transform': 352, 'DateofService_dayofmonth': 3, 'DateofService_month': 12, 'DateofService_year': 2020, 'CPTCode_stringindexer_transform': 3, 'CptModifier_stringindexer_transform': 0, 'pdiagnosis1_stringindexer_transform': 77, 'pdiagnosis2_stringindexer_transform': 152, 'Gender_stringindexer_transform': 1, 'DepartmentCategory_stringindexer_transform': 3, 'SpecialityCategory_stringindexer_transform': 0, 'PayerId_stringindexer_transform': 1}], 'attributesFT': [{'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'max': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'DateofService', 'transformation_label': 'Extract Date'}], 'feature': 'DateofService', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Amount', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '357.38', 'stddev': '823.57', 'min': '45.0', 'max': '6688.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PatientBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '-1.54', 'stddev': '18.4', 'min': '-250.1', 'max': '123.41', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'InsuranceBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.27', 'stddev': '3.94', 'min': '0.0', 'max': '76.68', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PrimaryInsuranceBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.15', 'stddev': '3.43', 'min': '0.0', 'max': '76.68', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '91232.81', 'stddev': '6100.5', 'min': '67028', 'max': 'J9035', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '5.5', 'stddev': '12.4', 'min': '0000', 'max': 'RT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'B58.01', 'max': 'T85.29XA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'C69.22', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'IsBillable', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '1.0', 'stddev': '0.0', 'min': '1.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Isauthorized', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PredeterminationIndicator', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'MedNecessityRequiredIndicator', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Age', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '67.76', 'stddev': '16.27', 'min': '5.0', 'max': '102.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'label', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.91', 'stddev': '0.29', 'min': '0.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}]}, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Gender_Recoded', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '500', 'mean': '1.45', 'stddev': '0.5', 'min': '1', 'max': '2', 'missing': '0'}}, {'feature': 'ChargeDetailID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '249.5', 'stddev': '144.48', 'min': '0.0', 'max': '499.0', 'missing': '0'}}, {'feature': 'DateofService_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '14.58', 'stddev': '8.27', 'min': '1', 'max': '31', 'missing': '0'}}, {'feature': 'DateofService_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '11.44', 'stddev': '0.5', 'min': '11', 'max': '12', 'missing': '0'}}, {'feature': 'DateofService_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '2020.0', 'stddev': '0.0', 'min': '2020', 'max': '2020', 'missing': '0'}}, {'feature': 'CPTCode_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '2.18', 'stddev': '3.55', 'min': '0.0', 'max': '22.0', 'missing': '0'}}, {'feature': 'CptModifier_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '0.42', 'stddev': '0.98', 'min': '0.0', 'max': '6.0', 'missing': '0'}}, {'feature': 'pdiagnosis1_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '33.73', 'stddev': '38.52', 'min': '0.0', 'max': '148.0', 'missing': '0'}}, {'feature': 'pdiagnosis2_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '35.52', 'stddev': '41.1', 'min': '0.0', 'max': '155.0', 'missing': '0'}}, {'feature': 'Gender_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '500', 'mean': '0.45', 'stddev': '0.5', 'min': '0', 'max': '1', 'missing': '0'}}, {'feature': 'DepartmentCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '0.91', 'stddev': '1.2', 'min': '0.0', 'max': '7.0', 'missing': '0'}}, {'feature': 'SpecialityCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0'}}, {'feature': 'PayerId_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '4.89', 'stddev': '9.31', 'min': '0.0', 'max': '50.0', 'missing': '0'}}], 'path_AutoSample': '/FileStore/platform/sampleData/60891770a187c956b337cf50_AutoFE/part-00000-tid-3917072144446901006-f58791a3-edf4-48b1-a470-3dde18d61c22-172002-1-c000.csv', 'listOfGraph': [{'values': {'label': ['174DA322-870E-429F-A9A9-BE22340E9836', 'A1575911-0E83-4479-B182-D26EDAD439F6', 'A87EF1E9-E636-4742-8E2B-414477926782', '1FFF3E45-B9F4-4CFB-8580-C4B05D4822DD', '20BC5C22-2C34-494C-B75F-55BDD8B54263', '210DAE19-9257-4180-9057-E5817F555A88', '3FA21D4D-05B5-4C43-A33E-A01D20B45A99', 'DC65D7C3-0822-4E54-86EF-DC7639D38C62', '11327DC4-1599-46B2-A000-C05D135E1F91', '853B5BAB-0A5F-4B44-85E0-FEE0AA663796', '92F7294B-4F6C-41B5-A9BD-65CE72B27FC0', '97BB8C89-8E47-400B-B546-B662E9EDCA00', '9C7B2481-4469-4FB8-AC5D-18EB2D29F213', 'BCE331FC-BC9E-43B2-B9C3-1ADE95CF4081', '0391FBA9-7748-4F50-8CA2-FC4EB5116F6A', '88D923C8-5217-46AF-887E-CB88A2E6C3E3', '8AAE78A7-7725-4C1C-B55C-82B41A364A2B', 'B5C5BD20-64AE-4166-8115-14DA13624764', '343273F8-E598-4815-A700-0683D31F3291', '50F7ADC5-2166-4461-B01E-8D8FAA92304B', 'A9EC659A-ED07-4B6C-84CA-D4E45411C3C6', 'DCBDA7E0-A5F8-4D0F-836D-66187FEB5B6F', 'E907166F-CE0F-49D2-A04B-40734F1A6F0C', '6518F89E-CD31-4903-89E2-2F31BEB28508', '7B41BFB4-B1D7-47A9-9659-6E8319429423', '7E4D4049-B1BC-4135-8020-D69318B66B18', '94FEF155-5ED7-4DF9-836F-78557484B8E4', 'A6B1DE4F-2FBD-4AB9-B09E-77B3E8C44771', 'B9F344F0-CEE0-4EA2-A55C-274FAB39E9DB', 'BBE4E59F-B282-490E-8FCD-6C9F5E8069EB', 'EC206AC2-403A-4047-B82E-82C383E29CCE', '01867531-8532-4FD4-BE17-232F39CFA32D', '2ECD88B7-05A0-4959-8736-397F8CF74784', '524D7B98-2D7C-458D-8877-35CDE8F3D981', '54E3991B-1609-470D-8095-C14FF5A9444D', '8F8BDCC6-5B74-4EBA-A154-8576DE0FFC13', 'A28023B3-9D64-44D3-BD13-2A22A1350A24', '1B71D2B0-C473-4A8D-9FA0-76A0692C5102', '6CF3754A-BC46-430C-A116-9329959CD556', 'A6CED81D-7B1D-441A-8EFC-544E8D31FD26', 'A82270CF-0B59-4C62-877D-9FAF720B6E5D', 'D9A3771E-E97A-4EBE-BFA9-54577F445F21', '1AA261AC-C877-4F0D-8372-F268A4D5F90B', '2B385821-C105-42CA-84F0-CEF8BFD1681A', '9D5CF424-8DA6-4A2B-8716-FDB93B808F3C', 'A17DF3A2-8077-49B4-A114-11F77F67E7C9', '0E47C422-450E-4A04-9B90-680F296C9686', '46CB6A8C-362F-4345-941E-842F429B48B5', '8CD490B4-CEBE-4DFE-BA0A-C805ACDF0D34', '9F54BD25-7C67-454A-B100-CD9974B62C7D', 'A78BD078-2165-4C68-B398-9AB6E94C0FB6', 'C72EBDAD-1D1D-40D0-BA4D-A9CDD08430A6', 'DBFEC39B-37B2-4B37-9D95-A444C27A619C', 'E1C8F5F6-2DDF-4370-B40C-D86322919657', 'EB7C11AD-0A01-4359-B1AB-289373C669DC', '16062E35-A49E-4227-A3E6-E8DDBFEBA5D2', '20416F30-920B-4925-8642-C83F38ECFCEE', '642D42E4-275F-4E86-8378-21E2CD291224', 'FBB301BB-1560-415C-91B5-A6E8B0B26B6C', '4011D505-BE33-4A24-AD40-240BFDDB2766', '74FC2B0E-AEC7-4263-951F-14E72EF044DA', 'A5881100-AA89-4D5D-80AC-AE2419271C5D', 'D0667457-EF74-4FA4-91D9-7169C64A53DB', 'E16D36F3-C4D2-4C4B-A004-E06C823C7960', '20463C7A-45B7-4BD4-A3EA-53ACC03E93F1', '6E834EA0-8A83-4326-8FAE-80C6DEE3B045', '8607EC2B-26F1-4A40-9646-5990D37EC622', 'A050F893-D9BB-4FB6-9B4B-B7AA83355D60', 'C2F471A9-22FA-4A9A-A14C-49A5F292662D', 'D6F6C707-D42D-4485-A571-82EEF5002E64', 'E164830A-C381-4AEC-81EE-3B116E56B496', '2701DB53-B3A0-466C-BFCB-6684D7059DC4', '89D7AAB0-BF88-493E-A7AB-7D6D793273DB', '8A07AC76-56FD-4E13-BFEA-A724E4C2D603', '8E5FAFBE-87F5-4C26-88EA-002384A50B1D', 'E67C3D6F-A4D2-4EA9-B41A-E5BEB54E0209', 'AC68E17B-9EBB-45A3-BD22-60367A203515', 'D0BA312C-C42D-4695-A6E4-71096BE6854B', '50C6C060-08F2-4F75-91BE-EF1D34F6EDA9', 'C99C5CC6-DF6E-4024-88A6-04D326F6D6B2', '1A70E6C3-0B4F-4338-A15C-903F552EC30C', '279BE041-8831-43CF-A0EB-219F709A986F', '3E0A6D69-9B07-4BFC-B389-24E0C884701E', '9017B6B5-6C3F-42FC-ABD4-67A54E07D67F', 'F16597BA-3D84-47E3-AF6C-072ECBD33467', '2AA7F9ED-3694-4ADD-91A8-3EFB8F8054FB', '45F8D74C-DE28-4F0D-B781-AB81CCD5D5CD', 'D7B8BD7C-A3F9-422B-B491-59F236BADE4D', 'E9DB7A1D-67B8-4936-B7C7-FB071EA72907', '2B6E92D7-EA15-4F53-B133-2A1F9B1E958C', '86E8386B-2651-4928-8CD1-631CE24D0432', '97D21C13-7AFC-4865-888F-5E92CC51F2ED', 'A40F161A-A931-41D9-A65B-597347BC9D6A', 'C43925F7-AA9E-419B-B0DB-0A044E67C4B5', 'D05F93BC-037E-4BD7-8E65-FDA46391791F', '14167C30-8123-4DCF-B3C5-05D3F16B6391', '2022006D-F4D6-4F8A-B76D-1617C97083CA', '25EF819E-5F4C-43DA-BEBB-C4FAD71B0EF5', '8648E212-02DB-4D77-BB02-88B67C56A9DE', 'EDDC8ACA-6742-4C6C-A200-FA42DD89285D', '03E0DCDC-A071-4987-A7CE-268D9C3D017B', '161B0D6C-5A50-4EEE-9B2A-E4DE8D9EBC1C', '1BD4EAF6-F3F8-4369-93B3-4E6144A69CAD', '20AC5B30-ACB1-4714-9794-5454B92C1A23', '3FE9456A-15CB-4840-90CB-05F7DF34CACE', '42F5D00A-6F59-4F63-ACCA-9EFDA0DD9194', '6E80EF7D-E761-4720-A1BD-9E2CA7FDD21F', '734CA534-2CD7-4A77-9D03-CF2390E8B0D6', 'B0501331-13EF-4126-B052-44D67EDFE7F9', 'C0B29F0F-BBE1-47C7-B625-0989FDCC5288', '136261A4-52B0-43B5-B099-1EF929506A93', '252D8E17-A9E8-48F5-84D3-6DA9B452EE75', '5A8270A6-53BD-4916-8805-B762FF67BE20', '67841660-18E4-45DE-B386-285C53582762', '9C60FA6D-FAAF-46A9-A149-1F941986D089', 'D6FE7A69-6B2B-495F-A36A-789DF588F6D5', 'D9C0CCE1-6D5D-4DB7-8376-D8562C150145', '42B818D0-170D-4838-BC4A-7D22039C0F53', '7DC8A130-8030-4915-8BF5-54015B27DAB6', 'A3555B0C-5A99-4619-B6E8-9831D99F0210', 'AECE54B4-2AE0-44A1-9409-2BFB52D07BD2', 'BE7612D0-7548-49C7-800E-173BAF9DF290', '0EB85789-CE25-4CCC-850E-44D2C3496272', '45F8ABD8-1D29-495F-B63D-866175B861A0', 'A69FF747-5E5E-44A3-A2AE-806C56D75743', 'B1C24F17-5A6A-4D76-A42E-A273CF94898F', '41D99DEA-233E-47A6-98C0-8B6F522D1B70', '53BD3672-31BA-4CE7-B2F3-2AFFC29C5F6F', '55F86AF5-F804-4A84-90ED-12E4125E1A8D', '8C94AF61-5EC3-44D7-B4B3-BBEB64AD2A23', 'B52B5682-9815-4EC1-A594-912A9724EB74', '079F8DAC-D8C1-48FA-9743-BC2AA08E287C', '2D60073D-3E0E-4347-B0A6-7E0CE91E92FA', '99772A77-B764-48FF-9242-5700C9572B65', 'E45AB682-6C1F-4B84-A1AC-3D1CD79AC3A9', '51526EAE-3FB7-4C7E-9A33-26990B70EFFE', '540B2A5F-11A9-4F0F-8038-B5A1ECE5E583', '91D48DF5-F63A-466A-AFB9-2E65C4F2D44D', 'ADF5D273-DB58-4F7F-8DF0-9D9D9647DA16', 'F672AF9B-E361-4572-A9C0-8B0731F9CD3E', '34803FD1-6C40-4323-8657-F1FCC3E34990', '5327CFC0-5C48-435B-98EB-A76AAE858941', 'D2DF16C4-D3AB-4E5C-9350-8F6FDE2EDA8D', 'E2DBCD71-B901-41F3-97B1-ED71030C7F2F', '049BA1F1-F362-48BE-9393-C05CCEA8694B', '13002A76-350E-4525-BDA1-9F181E056F74', '167E1195-CC7A-4F8C-8D91-9577BCBEE8FB', '1F264D61-D5D7-4505-9AC2-25ACF275E3BD', '23BCDE27-9F80-48E4-B29D-348C871829B3', 'A71A1D2F-3F1A-4CA6-AA98-E5D6A855C250', 'B62DA993-E320-4CEE-9D23-B3238CA5759A', 'B6375BF2-EFA5-44C9-A05C-F71DACD5F8A8', 'C924A1C9-2E9A-4BF3-8BC7-25BFAEB02CE1', 'D2300410-689E-453C-895D-FC7FDA559759', 'F12A6DBF-FBB4-4C62-8EE0-0F677D58EC3B', 'FA9EEE47-7F76-4264-9A82-9B370BCF9534', 'FBBC8CD5-C35A-42E5-AEBA-C5F61050B726', '045700E8-F572-417C-A8DD-41E76C5C159A', '1A088AE8-D8BA-404D-97E5-9DB1DFB972F8', '1C109ACE-6610-4098-BE1A-C64F7805EBCA', '25251236-8157-4EE2-89CD-C5A600E278BE', 'A1818C97-82BE-4115-8C3C-3532CB8DE838', 'AC9C3FBE-B694-4A6F-9A6F-6CC4563C5C73', 'B5E35D7B-A0F8-4A47-B72F-1D372236FC63', 'C1119267-721A-4606-9D06-09FF240505EC', 'C3339BC7-4E2E-4E0E-822E-7EE8E489451C', 'CDDFE064-998C-4CDD-97A2-04647A33EE7A', 'D4930936-02DA-4DBB-A3B2-5391AC5D7153', 'FD8029C5-4ACA-4F97-8CE6-144AE49C8BCB', '4C3D48F4-860F-49EF-812D-75B08DC1359C', '58207A31-753A-42B1-86D8-E750AEA10079', 'EAF0CF6B-9569-41B6-8B77-6335F34B3D7E', '693B32F4-ECA0-4844-80DF-6D9C94DD57BA', '7E2F8EA5-4A1D-4490-A208-49CE684BB427', '83A30AD9-1329-4675-99E9-AF5ACBE98CFA', '84F4A744-F946-4B1E-99DF-20E0538C5F4F', '90C2C4BC-4F1D-4730-8078-6CF025EB493F', 'AE214CAC-A747-4549-B4E0-957075E94F38', 'BD337B93-0D12-4AE2-9566-35705B577186', 'CE11AF94-7EB4-4B98-BC57-51593FD36B72', '1F6C5DD9-4889-43B7-8768-BA1D2873322F', '322A11BB-2B77-480A-BABA-2FD89CDD0305', 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', '3342F086-2D91-4081-BEE0-C18AEAB0C981', 'C4FFF996-0FB6-427D-9266-BE10E9F5DAA9', '510C2305-6CB3-460D-84C7-9EB6281F8E9D', 'FA7AC95E-1961-4A7B-AC57-3FD3212BAF20', '04CCB591-8B59-4CCB-9EB0-5A4AECB3C705', '0D66C132-CD95-442B-800E-E19F2651E018', '0ECEF203-5A40-428B-A19E-BFA699806763', '2A1A46A7-1E50-41C1-ADD1-EBC0F82168F9', '6C6C6230-52AE-4590-AFA7-42A14991BD69', '71B8A2A8-14EC-4B19-A4DA-6DE30E8A6E30', '2B3AACB7-A045-4E46-A2DB-8B841AD87F65', '2FCDBCD0-49FC-4AA6-B249-041E476D766A', '3DF7C0B1-EA0D-4014-916E-E76984AFDE2A', '8A1DE8B3-2FDE-4581-AC94-4EA74A997CB0', 'F49D6604-F705-480F-BCB9-E6F16B76DACF', '31C70405-0F52-4BD8-A785-13AFD15809A6', '323DACC7-ED74-42A3-A1E6-21BC104AAD51', '35841CCE-C3B1-4652-B011-C44F79BB442D', '8875D93D-866D-4897-81C3-67DB4570B102', 'A7A12FBC-72F0-4216-8360-126AF826214C', 'BDAADB25-9498-4D4F-809E-4590BAEE53B5', '0438621D-9EC2-4530-B3E8-F0C68FD8295F', '55E3B0D0-8E40-48F3-854B-A8E3C893E803', '793B38EA-690C-456B-9597-5444E6D1499F', '899B4439-339E-4CC1-AC1E-4C1F4BA312EB', 'A4CD837B-6B10-47C0-B9DC-E8D6ADF73B88', 'CC817F76-EE46-4AD5-9911-86A53FB1DBAE', 'F1C5E648-7E44-43B6-A884-48A861BA52BB', '3D2ECD44-776B-437D-9DD6-936DD5B7A4BF', 'AE3A61BE-5820-45D3-BCD1-9982D0BA93EF', 'C94A98D6-223E-47E8-9589-5631EBCB55E9', '05BBEB22-65F8-4773-89A4-452C6FD9DDE9', '0C7AF11E-16DB-461A-A412-B711F0810E4F', '8745504F-A5BD-470D-ABAD-CDA68F032BE0', '8B63DA7A-BB50-450B-BA2A-E68DFFF3F4F6', '0BC312A3-FBB5-4F2E-9A5C-6BAC5566A285', '1BA29057-DE25-4EF0-A06C-94A526C557A3', '8E5DABE9-6BE2-4D98-BF87-D87C6A70C35B', '9AE67B15-8F75-4CED-8129-80E003318733', 'D84605C6-4883-46E5-8BE3-4C82081FC5E0', 'EF5E604D-077C-4903-95D7-5568576B7F96', '14710135-067E-4E9D-91B1-E52FD6B72796', 'ADC1E05E-8451-4C66-976B-3DBBB7103A27', '1BDC9A8F-39D7-43D8-B3A8-5E9457AEB7CE', '45BD84A3-1EE3-4B60-8352-D372DFEB677C', '6F66C6AD-3D4F-4EC2-9C71-D7580A768CF5', '74E7B927-7E80-423E-8EBE-94FC0B7E514F', '82948A9C-6345-49ED-AB80-DD62B36CE11F', 'BA941CAA-6F1A-4BBD-9D4A-42C905C8C41A', 'ED3A209D-7398-4255-A9FB-3712B16F9DC5', 'FBCA9AD8-4FC2-48A9-B059-0347A0EDA4E1', '142DD0C0-133D-4FCB-934A-C6C50D564E61', '803ED239-BF70-4FCD-B02C-B5977404BA98', '967ABAFC-DBB1-4878-89E6-2B32E0E8387C', 'B3EE5A6F-6BC4-4F1D-B196-1127D5A7FED1', 'B77643EB-41F3-4EF0-818C-B4617862C124', 'CF7B7502-BFFD-479E-87EF-8C30089033C8', '04A3E774-1DF2-461B-8DCA-90EAB47932A0', '1054BD81-AB8A-4FDC-9A87-F41814188945', '202111D3-0B41-42EF-8113-161C06700D53', '23863534-0389-4F1E-BDDC-96C82F56F4C0', '906E67BB-7D18-4CBE-8802-172E4D09F677', 'A930D4B2-2DE6-4134-A58E-36E88012A437', '043D822F-B7D6-433B-B8F7-3A0F58BFFC22', '37409C70-626D-4D18-BFD4-749375A6D10F', 'C1DDC816-520D-4175-8613-E726EAF36CBA', 'CCC6C19C-C3C3-4F0B-8D42-083ABB99F0E9', 'FF11B590-D529-4372-A722-252F47C1AA76', 'AF3AFE4B-9BD1-42B1-9D03-5031CDB276F7', 'D45A198D-C1A5-4A54-8BCE-CD4AD0B2FBCF', 'E7C8D3FA-DCE9-4622-96F0-2047722D8564', '399470D8-4590-411F-8652-380D7FB48A6F', 'A8670196-A390-42F7-B247-84D40DCE973F', 'F39EE67E-0589-4138-9AEF-261777DFC2B1', 'FA571DDD-0422-48A6-BE3F-1DFAABE49A5D', '010ED4A3-EB64-4AE2-99CE-B458B59B538C', '1EA4A9FD-2C9B-4C54-9296-841FA5F9F23D', '3DAA771F-294A-4395-9B6B-30B898BC01E7', '411875F3-AAAD-48AC-92E6-112579D42F27', '7A0B900D-FB81-4652-AC59-4D27FB1BF2DA', 'D1CA4C65-B76E-4F88-991D-E39CDBCB9A25', 'D501A556-9400-4BBE-876D-4E2269A69E50', 'E44DA795-311C-464F-A98C-AF783E9A8687', '8800B6CD-48B2-414A-AA85-2FB6E2F94F62', '0C738DFF-CDE2-446B-8605-885F7FFEA814', '387FB6B8-8FE5-4483-A401-B595EF6BE611', '9E48B685-ED16-456A-9BDA-73F1CDE529D5', 'FF73DA08-83EA-4F38-8153-B686DFBD480C', '8E785A6C-A77D-46F3-B74F-701B18C72B2B', 'B881C1B6-9321-4EB8-901E-843FB577ECDE', '6C01513E-DC0D-48F4-85F7-5DE34C582CA7', '88E69525-5461-45A9-879A-21A99C906710', '980CDD7C-B8FF-4C24-9EB8-E66EF9502A25', '29761757-EC8E-4BC8-9BD4-764810C0D435', '2E9CCC26-EE83-4CE0-BBE6-78198DE26939', '3596B626-33E4-4438-A2D7-C7B7F2884A61', '722ED05A-EAA6-41C2-AB96-ED5E231D9CA5', '8D12EF16-4731-4FDD-802D-73518A27A384', 'BB4D3CAB-1412-4339-A438-A8AFB1ECCA82', '113F2B98-6796-4055-8712-8147E996D2FB', '19635F03-1921-4E20-A610-C32DCD0A86E0', '1EDBDDB5-5D98-4516-87A4-135EB82D051F', '28A393A5-A7B5-475F-8710-5DFB463F120E', '36009550-373F-4179-BD6D-42DC2B91652B', '8EEEA997-D2B5-4EC9-A50E-B2CBD0094405', 'C295E50E-9D1F-4967-B8F4-CE7ACB0DD858', 'F3A2BB7A-DCFC-4740-AB2C-51B4DE18A84F', '077A557E-846D-453A-AE7D-9B3003FFA94F', '0EDBE7EE-67FD-48BA-86E9-F70C9C504642', 'B06CBEAD-36DB-426E-84CE-89B56FD46159', '1DEC7009-90C7-4C0B-8617-D9E805BD39DC', '65AAE408-BFF7-45D0-98E7-78DFDAEC64EC', '823ACF06-0F8B-4DC7-AB68-4DFE535C2F92', 'B86E4CA3-AF97-467C-80C1-A05484D3A866', 'D525418D-529A-470D-B010-7B90CD096743', '15695896-D50A-4650-8607-86092DE66F86', '724A277E-2530-4A79-AC3F-6E4467BB2688', '7C334D9D-8773-4B52-871C-7F4CE8FED784', 'A15234F5-28B1-425F-8338-F05A4BF34B5A', '0CD457A8-5460-4F98-8059-107464922EFC', '1CE42B39-5F00-4C37-A276-6DFEE0CB161F', '2AD5B7D3-9824-461C-8E4C-F5088C5A5061', '6295FB65-95AE-465C-A7BE-93CDEA6FCD41', '9E709F1E-38BF-4B10-B126-7C344A21DD11', '9EB1990C-6484-48F0-9B12-F05AA4B6815E', 'E2272BDD-653A-47F9-A8BA-EEEDBC803C1B', 'FB109A88-81CE-4832-B4AC-56DD97251649', '53B02D9D-4160-42AA-936D-FCBB76F49B95', '99D51AF1-7AD3-4DD6-96F0-25AA1E59972F', '07A0559D-6D61-4D02-9F83-0730224B6C93', '4D06240B-F581-412B-B7B2-5F3A1060CB47', 'D9501B03-2C0C-4096-A3F1-A4EBEC12D7AE', 'EC536097-D4FA-4221-9C08-0C6168F80D46', '11077BFE-4169-4F6F-A104-75B18FD0FF87', '2EB8DC5F-2F79-46F0-A78F-A59491C148F6', '36EF7301-7E02-412A-9E23-47987397A484', '620495B1-AE1E-4BEA-881B-AA8645DDAAB3', '73A89B2A-DEAD-456A-95A2-EA941FF2DBFB', '83B3ABC6-1E23-4471-9F1E-78EA18B3CCE0', '9231D20B-208F-4B7A-BEEA-28718633282D', '9FE9533D-C993-42CC-A477-DA0FD4E5FF72', 'DA09EFC9-6D5E-4472-8CF3-C9017C22A30E', 'F9A0096A-9F38-42FA-B2FF-20B3F570FEB8', '116B1537-FBEA-479D-8725-F3F98CE8696A', '496D2DE6-BB81-4484-AC58-3BB0D13B4E8E', '8D6F6E45-269B-4E19-AB45-130E4C2F8EE5', 'BF7CB069-0FD9-44DA-A277-EDEB8C31E102', '02705462-64DD-4C6E-AB7A-2CDDB023FE7A', '3882B031-F286-4F77-8AAE-3ECC1E13433E', '9C0A75B2-5E85-479D-B614-3C97418C4570', 'F5C916EC-F2CA-4136-9312-5A71933D7C4C', '846A3024-1033-46EB-9F06-0F3E17DAD4AA', 'CBA90F0E-ADB0-4F8F-A849-CD75DB7DD2A1', 'D7A3A337-59AB-4474-8D56-FF4B8402E128', '15527B5F-A9E1-4797-A410-9E324DD82868', '5649E152-6876-444A-97D3-66E0C38D7661', 'BB2FFBE2-24D7-4618-B3FE-DC8B3ECA14AC', 'E760489F-6EE4-4B15-9B6D-5B12F6A5E3DC', 'FAD56DDC-6281-4433-94D8-23FCB65FD823', '74492123-CD52-45B6-AAFB-CDD634C4C9BC', '79956D3A-C616-4A7A-9F06-BA0DC2806695', '9C582B3F-8717-4E03-84C9-F6EAAC3F4948', '394DE176-F74B-4877-A640-EF3C34905AE0', '3F563146-EE95-44B0-B130-41E8E1DE3030', '48C82871-65CB-40BD-98D3-6EB4CB2E0F61', '49B6D9F9-0BB5-43E9-B453-A0D4616A3689', '5C1B73F4-3EDD-41BE-9FEC-7BAC25366092', '9D0125FC-58A9-4CD0-A823-57B6464A308F', '9E726301-D189-4224-945B-2FBA8FA26A03', 'A5EE1C1B-278D-478B-AC1F-980E200CB843', 'C8D69246-8334-4EBF-954E-5AAFA7C1F65F', 'CAABED88-1591-4E9D-82C4-7926EA41DD52', 'DD23176B-512C-49D2-AC85-CFAA4256C72D', 'FAD99EF5-F102-4FC6-BCCC-ECDEF99456BB', '2653D41E-11AE-432B-993D-8CC4D4DAA1FA', '71E5F463-049A-42A7-A510-EACEC14E8301', '86D2CB7E-E54C-40E4-BCDD-C6D0DE7A6D7C', 'AB9BAF44-7980-4636-B844-488341E49BA9', 'F66CDB59-6144-4776-B129-5FEED24970C3', 'F829CBA3-AD5D-4CF8-BE1F-3FDC44044AF7', '244D35E3-5D74-4F5C-BB72-5860E1D4561C', '2C3E981A-853E-45E7-87BD-799325176251', '3B1A2F21-FFBF-4454-B0FB-BCBF1CEA4FE3', '49FF4EFE-6A51-44EA-BDDA-27F6A4663C04', '6BD754C1-07F9-4C5D-A1B8-B77B284AFC7F', '71A70101-0BDF-43F3-B580-4DB046CB1EBD', '8076FF62-D8A1-4309-9EAD-F4B83E85DB1D', 'DB05D823-A5CC-4AD0-87B0-6FBF69B7493E', 'DC1BE341-E3CB-4EBF-B968-F5764D826DC7', 'DCF767F0-7167-41FC-B981-296FBE63588B', '46445DC3-6A22-4664-8F2F-C73C4F9B7010', '9F250BA0-F703-4F1D-8AD6-82F2F4E58B70', 'BCCDC778-B59A-44F9-B1B6-9B67462C4B0A', 'CF8CDC47-5AD3-442E-9F3C-07CF84EC35A5', 'EADDE82D-5C40-4BF2-8C07-A91EEAAF8EC5', 'F32643CC-08F3-4108-B5D1-4A881B590328', 'C9139FC7-C65B-4F97-B7CC-56F810FD45DF', 'ECA40D88-F3E1-4286-8064-FD546E3C844E', 'FDBE8690-4E2C-4BD4-991A-74F5F3CC2872', '04A27415-9D35-49E4-A5F5-305DFD917D5F', '04EA280C-12AB-4842-9913-E97DFDDF508F', '0CB92950-EE3F-424C-AA0A-445F2D738518', '29737DE0-22D6-4D41-870C-DA59BE4A4E75', 'AA77712E-61BA-45A8-8E10-28057191D585', 'D271CCF9-CBA9-4A48-8202-5ED6FE135BDA', 'F488446B-42F0-4C47-B313-AE5AD0AD4C08', '0DC89466-2E6B-4DC7-B913-E5BBE5B168A0', '27DEFE26-1A3A-4654-A273-799CEA02BB33', '3D48B549-0C2E-4EC1-AA68-CD7DE250D65A', '790CB21B-A420-4C48-8EBC-67AAC5413D69', '9DD3CB15-B27B-4F87-A3C8-A9FC8D1F56B5', '2F71902A-768E-4A58-BE77-2AB583872514', '305A3131-EFD4-420C-88B8-361D358D9857', '65920EF9-C672-4D37-9A6B-795B0F656EDD', '736AECC6-FFE6-48E7-9F0D-4AF0CDC06DED', '9DEA15FC-C28D-42CB-8283-AC5B84B7FCEF', '0B1BA7BE-21C6-4B0E-A25F-C198051FD597', '0E9EF90A-1AEA-403E-9058-D6F60A449238', '235B02A5-8AC3-47BD-86BB-CAA1A63C317B', '48A25AFB-573A-4213-9B1C-E3654825F634', '49C499CF-DD82-4195-B4F3-F889BE7E0BC4', '691933B0-0D3D-4355-B453-79E791D6DC36', '83B52DD3-F13E-450B-84A7-CAA79A556718', '842AC936-21A9-4F15-8BF5-ACF6D4756EB0', 'A3CC38B5-605E-476B-A987-2AC63C81F539', 'C3A37CEA-E1D8-4DB2-837C-D1041863EC7F', 'CF82712C-91A7-46D2-9F6D-19A1BC8A1A0D', '485494F4-D20A-4303-89BB-F09C5646A9EA', '4FEA26C5-DD7F-4A65-B859-D5FECFF66684', 'B48620D1-DD13-471B-936F-AE9970B850B6', 'BBF82897-F57E-484D-8159-C4E14BBECB5C', 'F0B25A0F-D965-460E-AA37-EA676B8BE27F', '279D822D-CECC-407C-8F7D-DA683C10C7B1', '27E0E710-3B33-4DEC-BED0-FDB55F95EEE1', '8793C097-7DC2-47B0-8639-CBD108B34F5F', '1D2CA217-3B28-4C61-89C2-7C55BAB37BDD', '3A4437B5-45EE-4734-BF9C-C7BA808C123B', '13872B75-E219-4B25-9F7E-24F021CF0B48', '377FADE6-202C-4841-BDD4-82367D36097C', '6A29D415-E299-4812-B8CD-8CD471F28115', '84A66A4F-CB48-4BA8-B11C-F2EA34C4A1A4', 'D7B0792D-0B5F-4CAA-9787-237DFECCA98B', '6D22E643-C022-4637-A81F-22A0DB410A33', '7C771536-6EAB-4FEA-B848-E077662307D7', 'DD0C3E18-53F4-4001-9334-BEC408540A3E', 'EA11E355-7A8A-4BAD-9066-1FC3E824CBC5', '267E6CA7-6B3A-4582-B01B-9DD2C4AC8DEE', '43B4ED70-4EAA-4C2A-AB60-F4A3848BF5E9', '7D59A1A0-3C61-4F7A-AD1E-0EF39B392E83', 'FF80B585-D64F-45DC-9CE9-59C51F3001D2', '324408CC-B86D-49E1-A0DE-99A7EE03276E', '3E505BF0-6673-4C9F-996F-64E07E72E60C', '5FFAD31C-B0EF-4A88-B7EA-2F02D7B336D5', '8CC0B44A-2E34-417F-A084-3E2F067F6966', 'B29EEE24-30E4-4B06-9FA3-777B1E99463D', 'C84DCC7E-7D84-433A-8D23-74F8BB79EE40', 'E776DC1F-10CF-4EC1-A151-05EC29E6C03C', '40F1314D-EBF6-49BC-B1A8-674F07F92039', '66E28365-DDB9-4F59-916F-2B171C5D93AB', '88539DEA-C521-4D06-A758-7FEBC99D5C5E', '8CCEE88C-C36B-4DE3-88D7-00D78C1D9B0A', 'C0AC5031-0382-4350-82C5-5B80634DEEAC', 'E5775C4C-F86F-49D2-8B58-22ACA6668352', '08683FBF-CD1F-42D9-A442-EF696A6D7C83', '289AA716-A1D3-48FF-B830-4BF9DD4A4FB7', '28D97A0D-03AE-4405-BFD3-66E00EDAE377', '55F62337-F077-4BA9-87D8-4C7AC0736913', '56A0F3CD-F2AA-4492-A3F3-D8694240FF34', 'B16FA6E7-E985-411C-AA06-A1F7E6BBCEFB', '0BB8A5C4-7444-4DEE-A3AC-2E0CFF7A3477', '1DB8D73F-61E5-4065-9342-592BE88E5981', '4C323EC0-5AB8-4CAF-9381-0011A0141AD7', '07BDF3AA-3B6E-4650-BDFC-596FE687C33F', '3F664203-58B4-411D-835A-667F32E1AF1E', '94F9C8EF-40F1-4CB4-B8F1-71EA8B1482D9', 'A56274A7-1DCD-46CB-84FC-83FD3C26472D', 'BDB2ADB4-356A-40AF-9E57-C6ECA2EB1235', 'F28706C5-DDB5-4FC3-9F7D-4896DB86AF18', '3DA05017-5FD5-4CBE-9CC5-956876EECBE6', '6AD7EB61-BE8E-49A3-929C-CE2CA6C95779', '8B79B6F2-FE78-4775-8668-2A35ECA963E5', '98AEC5FB-BE93-4E59-A4A9-9533419B73B6', 'A24C0F04-7684-4CC9-BFBF-CD7E01137187', 'C22535F6-D4C8-4B58-B0C4-BA24CE26B9E7', 'D9115251-3465-4EF5-B9B3-54B0EAF82045', '24253A09-E06B-4C51-A3AD-0709BD0DA4B5', '449D29CA-9C67-4DA9-95FE-C1531DA5DE4C', '77C723D8-92FE-487C-825B-93C8605DC74E', '8BAEB725-2E28-4DE9-B911-836846DF6F11', 'D89C117A-4B7C-471A-94A0-1F6F022193BB', 'E0D39756-5A76-4FA1-BA20-C957C3486DFC', '0A6DE37A-0F94-43B3-84E3-7590769F820E', '0F43FFB3-7DF2-4156-BD39-A6C0667C7392', '08AD40B0-B651-4069-AA92-9A64FC246B17', '0EC59014-45A0-45B7-AB3D-12367ECD4174', '13E519C2-73BD-451A-A651-55B67924A9B6', '26C382E6-F716-4F7F-A72C-A6C7150C2066', '989C36A9-F0D6-48B5-8D26-F5A56E610792', 'C4ADE72E-AD16-4711-886D-AB997CCB5E71', 'DE90AF3B-500B-42BC-B1F8-D4E5971DD853', '03A51C92-3977-42BE-B158-53B124029BCC', '176F9200-4582-49AB-80EB-558534EC297C', '2861F7ED-8D48-436D-9AD0-FEF67A864566', '7157C3A4-F8EE-42C2-864E-C9E7FD5C62C6', 'FB5D5A33-BA7C-4140-AB6B-9AC8E94FABA6', '3AAEBB1A-A489-4699-A7B4-F237F130B7EE', 'A3646A59-090D-4D1F-AE88-16B127AF81ED', 'A90B0781-610E-4FE4-80ED-BD9F6ABDE174', 'AF591C35-31A5-45DD-9399-292D82D794AF', 'D06DE332-BDE9-4D63-8959-BF68C5A40297', '23AEEFFE-19BA-487C-A1D5-A88964FF3BC1', '341B166A-3402-4A75-AD95-D57FDDA27DC9', '45D50939-3539-478D-AABE-C57C6791CF1F', '4A1D1603-893E-46B7-92C1-8C3069A4B63D', '502690F0-D587-4D27-B3A7-5B9679F7E217', '97E40160-D786-4460-B2D1-FCFFE2DB599F', 'A372D150-FF7E-4BA6-9349-DDD6451A1084'], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'ChargeDetailID', 'count': 500, 'min': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'max': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192'}, {'values': {'label': ['2020-12-18 00:00:00', '2020-12-13 00:00:00', '2020-11-16 00:00:00', '2020-11-27 00:00:00', '2020-12-03 00:00:00', '2020-11-02 00:00:00', '2020-12-14 00:00:00', '2020-11-28 00:00:00', '2020-11-30 00:00:00', '2020-11-26 00:00:00', '2020-11-09 00:00:00', '2020-12-19 00:00:00', '2020-12-04 00:00:00', '2020-11-15 00:00:00', '2020-11-20 00:00:00', '2020-12-05 00:00:00', '2020-12-06 00:00:00', '2020-12-07 00:00:00', '2020-11-06 00:00:00', '2020-11-13 00:00:00', '2020-12-11 00:00:00', '2020-12-17 00:00:00', '2020-11-12 00:00:00', '2020-11-07 00:00:00', '2020-11-14 00:00:00', '2020-11-29 00:00:00', '2020-12-12 00:00:00', '2020-12-10 00:00:00', '2020-12-21 00:00:00', '2020-11-21 00:00:00', '2020-12-24 00:00:00', '2020-11-05 00:00:00', '2020-11-19 00:00:00', '2020-12-20 00:00:00', '2020-11-08 00:00:00', '2020-12-27 00:00:00', '2020-11-01 00:00:00', '2020-12-26 00:00:00', '2020-12-28 00:00:00', '2020-12-31 00:00:00'], 'data': [15, 15, 23, 16, 19, 12, 17, 14, 17, 10, 11, 13, 12, 14, 12, 9, 14, 21, 16, 19, 18, 7, 15, 14, 14, 17, 12, 8, 10, 8, 5, 9, 14, 12, 15, 5, 9, 1, 5, 3]}, 'label': 'DateofService', 'count': 40, 'min': '', 'max': ''}, {'values': {'label': ['45.0 - 709.3', '709.3 - 1373.6', '1373.6 - 2037.9', '2037.9 - 2702.2', '2702.2 - 3366.5', '3366.5 - 4030.8', '4030.8 - 4695.1', '4695.1 - 5359.4', '5359.4 - 6023.7', '6023.7 - 6688.0'], 'data': [467, 13, 5, 0, 3, 5, 0, 0, 6, 1]}, 'label': 'Amount', 'count': 10, 'min': 45, 'max': 6688}, {'values': {'label': ['-250.0 - -212.7', '-212.7 - -175.4', '-175.4 - -138.1', '-138.1 - -100.8', '-100.8 - -63.5', '-63.5 - -26.2', '-26.2 - 11.1', '11.1 - 48.4', '48.4 - 85.7', '85.7 - 123.0'], 'data': [1, 0, 2, 0, 2, 3, 489, 0, 2, 1]}, 'label': 'PatientBal', 'count': 10, 'min': -250, 'max': 123}, {'values': {'label': ['0.0', '24.0', '76.0', '36.0'], 'data': [497, 1, 1, 1]}, 'label': 'InsuranceBal', 'count': 4, 'min': '0.0', 'max': '76.68'}, {'values': {'label': ['0.0', '76.0'], 'data': [499, 1]}, 'label': 'PrimaryInsuranceBal', 'count': 2, 'min': '0.0', 'max': '76.68'}, {'values': {'label': ['92014', '92134', '92250', '67028', '99204', '99354', '99215', '92012', 'J0178', '92225', '92226', '92004', '76512', '67036', '92235', 'J2778PF3', '99213', '99203', '67113', 'J7312', '92018', 'J9035', '99205'], 'data': [156, 185, 35, 22, 28, 2, 3, 12, 7, 7, 6, 6, 3, 2, 10, 9, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'CPTCode', 'count': 23, 'min': '67028', 'max': 'J9035'}, {'values': {'label': ['25', '0000', 'LT', 'RT', '50', '26', '57'], 'data': [70, 385, 13, 15, 13, 1, 3]}, 'label': 'CptModifier', 'count': 7, 'min': '0000', 'max': 'RT'}, {'values': {'label': ['H35.3112', 'H33.312', 'H35.711', 'H35.3211', 'E11.3313', 'H33.311', 'E11.3413', 'H33.101', 'H35.3121', 'H30.021', 'H35.3132', 'E11.3512', 'H30.91', 'E11.3513', 'E11.3391', 'H35.3231', 'H35.033', 'H35.413', 'H35.61', 'E10.3511', 'C69.32', 'H34.8110', 'H35.372', 'E11.3511', 'H33.301', 'C69.31', 'E11.3411', 'D31.31', 'H35.352', 'H43.813', 'H35.3134', 'H20.013', 'H35.3221', 'H35.3122', 'H35.351', 'B58.01', 'H34.231', 'H34.8310', 'H25.813', 'H33.001', 'H35.3113', 'H43.821', 'H35.3130', 'H34.8120', 'H35.411', 'H44.2A2', 'H33.002', 'H35.373', 'H35.371', 'H35.3222', 'H35.3212', 'H25.13', 'E10.3513', 'H35.051', 'H35.362', 'D31.22', 'H44.113', 'H43.11', 'H35.343', 'H43.811', 'H30.23', 'H33.313', 'H53.9', 'H30.133', 'E11.3311', 'H47.012', 'H34.8330', 'H47.10', 'H35.353', 'H20.9', 'H30.93', 'H35.3131', 'H35.3124', 'H31.113', 'H35.341', 'E10.3292', 'H33.22', 'H43.812', 'H35.053', 'H40.051', 'E11.3213', 'H35.342', 'H35.3133', 'H35.052', 'H20.12', 'H43.10', 'H35.3213', 'H35.3114', 'H53.8', 'E11.3212', 'H33.321', 'C79.32', 'H35.89', 'H30.22', 'E11.3211', 'H34.8312', 'H35.721', 'H35.433', 'H35.412', 'H40.033', 'H34.8320', 'H43.13', 'H47.211', 'H35.461', 'H43.823', 'H57.11', 'E11.3312', 'H15.121', 'H35.3111', 'H31.091', 'H44.2D2', 'H33.322', 'E11.3292', 'H44.111', 'H43.12', 'H33.103', 'H21.01', 'H04.123', 'H31.011', 'H35.54', 'H43.392', 'H35.3110', 'C69.21', 'H59.021', 'H43.22', 'H11.31', 'H20.012', 'H34.8130', 'H35.073', 'H35.023', 'E10.3391', 'H35.713', 'H31.8', 'H35.63', 'D31.32', 'H33.042', 'T85.29XA', 'H33.323', 'H40.053', 'H44.23', 'H10.45', 'E11.9', 'T66.XXXA', 'H43.391', 'H35.101', 'H30.013', 'H35.50', 'Q14.1', 'H35.712'], 'data': [21, 3, 5, 20, 14, 2, 8, 1, 1, 1, 13, 5, 2, 38, 1, 30, 15, 2, 1, 2, 5, 7, 11, 14, 2, 1, 4, 3, 3, 6, 5, 1, 3, 7, 8, 1, 1, 5, 1, 6, 2, 3, 3, 4, 2, 1, 5, 11, 9, 2, 8, 2, 1, 1, 1, 1, 1, 2, 1, 4, 2, 1, 2, 1, 4, 1, 1, 1, 6, 2, 2, 10, 2, 1, 3, 1, 1, 4, 1, 3, 3, 3, 3, 2, 2, 1, 2, 3, 1, 2, 3, 1, 2, 1, 5, 3, 2, 1, 3, 1, 3, 1, 1, 1, 3, 1, 4, 1, 4, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'pdiagnosis1', 'count': 149, 'min': 'B58.01', 'max': 'T85.29XA'}, {'values': {'label': ['H35.373', 'H35.411', 'H35.721', 'H35.3122', 'H43.811', 'E11.9', 'Z79.4', 'H43.823', 'H43.813', 'H35.351', 'H35.70', 'H43.822', 'E11.3312', 'H35.723', 'H34.8320', 'H52.13', 'H35.3221', 'Z71.89', 'H35.432', 'H43.821', 'H35.371', 'H35.61', 'H35.412', 'H35.372', 'H20.9', 'H21.543', 'H43.812', 'H35.3211', 'H30.101', 'H43.89', 'H35.033', 'H04.123', 'H25.811', 'H35.463', 'H35.3212', 'H35.3132', 'H35.461', 'H44.22', 'H35.032', 'H40.021', 'H35.341', 'H35.352', 'Z98.890', 'H34.8322', 'H35.711', 'E11.3311', 'H35.3133', 'H30.113', 'H35.3124', 'S01.111D', 'H47.211', 'E11.3412', 'Z79.899', 'D31.31', 'H35.353', 'Z96.1', 'H40.052', 'H35.3130', 'H35.3134', 'H40.013', 'H33.312', 'H43.12', 'D31.32', 'H34.8120', 'H43.11', 'H33.311', 'H31.092', 'H33.331', 'H30.93', 'H33.42', 'H47.10', 'H40.1110', 'H21.542', 'H35.3223', 'E11.3513', 'H59.022', 'E11.3292', 'E11.3592', 'H35.3131', 'H47.011', 'H35.3220', 'H35.022', 'H25.813', 'H35.3120', 'E11.3293', 'H35.342', 'H35.3112', 'H35.52', 'H25.11', 'H43.392', 'H43.23', 'H35.413', 'H40.053', 'H35.722', 'H43.13', 'H35.3121', 'E11.3511', 'H43.393', 'H40.1130', 'E11.3291', 'H33.41', 'H33.321', 'E11.3213', 'G45.9', 'E11.3532', 'H35.22', 'I10', 'H31.012', 'H31.011', 'H30.92', 'H47.091', 'H11.32', 'H33.43', 'H35.433', 'K50.918', 'H35.40', 'H40.043', 'H35.052', 'T85.398A', 'H11.31', 'H35.81', 'E11.3391', 'H44.111', 'H25.13', 'E11.3212', 'H01.003', 'T66.XXXS', 'H40.051', 'H33.322', 'H43.391', 'G89.11', 'H35.713', 'C69.22', 'H27.131', 'H35.89', 'S05.11XA', 'H26.222', 'H33.002', 'H35.3222', 'E11.3392', 'H40.42X0', 'E10.3312', 'H44.701', 'E11.3512', 'H44.23', 'H25.9', 'H33.003', 'T66.XXXA', 'H11.001', 'H35.3231', 'H31.091', 'H33.001', 'H31.422', 'H44.2A3', 'H47.233', 'H31.002'], 'data': [20, 2, 3, 9, 14, 6, 47, 7, 22, 7, 4, 6, 5, 4, 9, 2, 23, 4, 2, 4, 8, 3, 4, 10, 4, 1, 12, 2, 1, 3, 25, 2, 2, 2, 1, 2, 2, 1, 1, 1, 3, 8, 3, 3, 1, 1, 1, 1, 3, 1, 1, 4, 2, 5, 10, 3, 1, 1, 3, 1, 3, 4, 3, 4, 4, 3, 1, 1, 2, 1, 1, 1, 1, 3, 3, 1, 2, 3, 3, 1, 1, 1, 1, 2, 2, 4, 1, 1, 1, 3, 1, 7, 1, 2, 2, 3, 2, 2, 2, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'pdiagnosis2', 'count': 156, 'min': 'C69.22', 'max': 'Z98.890'}, {'values': {'label': ['1.0'], 'data': [500]}, 'label': 'IsBillable', 'count': 1, 'min': '1.0', 'max': '1.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'Isauthorized', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'PredeterminationIndicator', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['0.0'], 'data': [500]}, 'label': 'MedNecessityRequiredIndicator', 'count': 1, 'min': '0.0', 'max': '0.0'}, {'values': {'label': ['5.0 - 14.7', '14.7 - 24.4', '24.4 - 34.1', '34.1 - 43.8', '43.8 - 53.5', '53.5 - 63.2', '63.2 - 72.9', '72.9 - 82.6', '82.6 - 92.3', '92.3 - 102.0'], 'data': [4, 6, 11, 20, 44, 75, 134, 120, 72, 14]}, 'label': 'Age', 'count': 10, 'min': 5, 'max': 102}, {'values': {'label': ['M', 'F'], 'data': [225, 275]}, 'label': 'Gender', 'count': 2, 'min': 'F', 'max': 'M'}, {'values': {'label': ['Pulmonary', 'Emergency', 'Cardiology', 'Orthopedic', 'Pediatrics', 'Physical Therapy', 'Ophtamology', 'Gastroenterology'], 'data': [172, 233, 39, 36, 3, 7, 9, 1]}, 'label': 'DepartmentCategory', 'count': 8, 'min': 'Cardiology', 'max': 'Pulmonary'}, {'values': {'label': ['Ophtamology'], 'data': [500]}, 'label': 'SpecialityCategory', 'count': 1, 'min': 'Ophtamology', 'max': 'Ophtamology'}, {'values': {'label': ['1.0', '0.0'], 'data': [455, 45]}, 'label': 'label', 'count': 2, 'min': '0.0', 'max': '1.0'}, {'values': {'label': ['B03B74CE-C223-4B1E-802B-5F5F76BFE64C', '105C1090-6ED0-4189-B07E-4C22940D948F', 'AD96772F-54C7-4750-86B1-1BD565B55244', '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', '92A93FB0-CEC4-4CCE-A905-62C0C73F482E', 'D91D11FE-B441-4585-9CC0-6A1F80FF8FF4', '4502FB31-5B01-4EEF-BB9E-2D396D96A6FC', '32E7EC13-C493-40B1-A999-6AC94E7FA3B9', '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', '0A471D4C-3977-4BF5-8039-5064519FDCD4', '7A54693F-FA82-4B81-BCB2-089EE6928B0D', 'BCA515C2-1B14-4060-9E24-B80BF9260648', '2CD6534B-F8F6-4CF9-8DEB-CCEF9EF87F9A', 'A7CD19DF-074A-47D3-958D-2B5E00671A3D', '71A0FEDA-7791-4DA1-84FE-FF0AC1D040BD', '84222BD3-605E-43EA-9A84-7C0169BE0B0D', '4CED3627-B1D6-4530-87D7-9609525FB29C', '041FE834-D4EA-40F1-A495-4B94A97ACD4D', 'F3FB074E-423D-4F7F-A817-90C0551007A4', '1382B639-5E23-4CFF-B643-FEB94CD6A7D1', 'A7C641BC-17FF-488C-8BAE-7046F63E8880', 'F4AE93A8-4386-486D-A12C-36C4C2032A0D', '8351B681-269A-4AE6-936A-FF10AFAFD629', 'E459B9E9-8650-4DA2-91F6-00B77900FC50', '2EAF4989-97B8-4F1B-9259-03E37314202F', 'AC7D1C02-4A39-4BDB-8B96-2F053F645CB4', '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', '9DFF1BC1-F4E1-4202-B9E9-1E440CC81851', 'F473817F-7515-46B7-97F9-348C88D7C348', '0BC878EE-8F48-415E-9233-25A7C28264DB', '789050C0-D1C3-40AD-8942-E67329E2F890', '193BA5E7-F797-4FD1-A9AB-27590E630FCB', '5EA62C5D-D9C0-4AFE-ACAF-4AD36F6B1057', '72F5084C-9DF9-4F0A-8BA1-A9B40DFCC9C6', '330E2DD7-352E-4E5C-9669-AE796ABFE5DC', 'E0FE1EC8-51D4-430D-8B8B-E2F9966875F8', 'A4D682CE-4D10-4765-AE88-0D5D9E1862E7', '4A91BB61-C5A3-4821-931A-1FCF4D3BCCB4', 'C4A6C186-B867-46C5-A163-5AF23FE61DED', 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'BCCB5D33-476A-4FAC-BB0C-5C7479803EB4', '3BEA3D9A-B5EF-4B67-A4AB-868B7BE2C0A1', '0779204D-FC67-444E-97E3-83CB62A9D0A2', 'EDD37436-0E9D-45B5-8C46-A22CA763970A', '8CA1878F-1BD4-4595-9AFF-3FF6967CD5CB', '315C773B-4EF1-4164-AB43-6817D993329F', 'AAB72015-9704-40ED-A2E7-87468E793D48', '20C2761F-16B5-42B7-8C93-82A112735F02', '835BC6C3-FEDE-4AF9-8B88-91628A98389C', 'A8A42CA9-168A-44ED-B0F4-14AA767DA83E', 'D1C4AE8C-46B5-464F-91E1-A39C5C76CC25'], 'data': [15, 1, 30, 199, 25, 81, 2, 2, 16, 6, 37, 3, 1, 1, 1, 6, 3, 1, 2, 5, 12, 3, 3, 2, 2, 4, 2, 6, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'label': 'PayerId', 'count': 51, 'min': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52'}, {'values': {'label': ['2.0', '1.0'], 'data': [225, 275]}, 'label': 'Gender_Recoded', 'count': 2, 'min': '1', 'max': '2'}, {'values': {'label': ['0.0 - 49.9', '49.9 - 99.8', '99.8 - 149.7', '149.7 - 199.6', '199.6 - 249.5', '249.5 - 299.4', '299.4 - 349.3', '349.3 - 399.2', '399.2 - 449.1', '449.1 - 499.0'], 'data': [50, 50, 50, 50, 50, 50, 50, 50, 50, 50]}, 'label': 'ChargeDetailID_stringindexer_transform', 'count': 10, 'min': 0, 'max': 499}, {'values': {'label': ['1 - 4', '4 - 7', '7 - 10', '10 - 13', '13 - 16', '16 - 19', '19 - 22', '22 - 25', '25 - 28', '28 - 31'], 'data': [40, 60, 61, 53, 79, 45, 69, 5, 32, 56]}, 'label': 'DateofService_dayofmonth', 'count': 10, 'min': 1, 'max': 31}, {'values': {'label': ['12', '11'], 'data': [221, 279]}, 'label': 'DateofService_month', 'count': 2, 'min': '11', 'max': '12'}, {'values': {'label': ['2020'], 'data': [500]}, 'label': 'DateofService_year', 'count': 1, 'min': '2020', 'max': '2020'}, {'values': {'label': ['0.0 - 2.2', '2.2 - 4.4', '4.4 - 6.6', '6.6 - 8.8', '8.8 - 11.0', '11.0 - 13.2', '13.2 - 15.4', '15.4 - 17.6', '17.6 - 19.8', '19.8 - 22.0'], 'data': [376, 50, 22, 16, 13, 12, 4, 2, 2, 3]}, 'label': 'CPTCode_stringindexer_transform', 'count': 10, 'min': 0, 'max': 22}, {'values': {'label': ['1.0', '0.0', '4.0', '2.0', '3.0', '6.0', '5.0'], 'data': [70, 385, 13, 15, 13, 1, 3]}, 'label': 'CptModifier_stringindexer_transform', 'count': 7, 'min': '0.0', 'max': '6.0'}, {'values': {'label': ['0.0 - 14.8', '14.8 - 29.6', '29.6 - 44.4', '44.4 - 59.2', '59.2 - 74.0', '74.0 - 88.8', '88.8 - 103.6', '103.6 - 118.4', '118.4 - 133.2', '133.2 - 148.0'], 'data': [230, 79, 49, 36, 28, 18, 15, 15, 15, 15]}, 'label': 'pdiagnosis1_stringindexer_transform', 'count': 10, 'min': 0, 'max': 148}, {'values': {'label': ['0.0 - 15.5', '15.5 - 31.0', '31.0 - 46.5', '46.5 - 62.0', '62.0 - 77.5', '77.5 - 93.0', '93.0 - 108.5', '108.5 - 124.0', '124.0 - 139.5', '139.5 - 155.0'], 'data': [238, 66, 51, 35, 32, 15, 16, 15, 16, 16]}, 'label': 'pdiagnosis2_stringindexer_transform', 'count': 10, 'min': 0, 'max': 155}, {'values': {'label': ['1', '0'], 'data': [225, 275]}, 'label': 'Gender_stringindexer_transform', 'count': 2, 'min': '0', 'max': '1'}, {'values': {'label': ['1.0', '0.0', '2.0', '3.0', '6.0', '5.0', '4.0', '7.0'], 'data': [172, 233, 39, 36, 3, 7, 9, 1]}, 'label': 'DepartmentCategory_stringindexer_transform', 'count': 8, 'min': '0.0', 'max': '7.0'}, {'values': {'label': ['0'], 'data': [500]}, 'label': 'SpecialityCategory_stringindexer_transform', 'count': 1, 'min': '0', 'max': '0'}, {'values': {'label': ['0.0 - 5.0', '5.0 - 10.0', '10.0 - 15.0', '15.0 - 20.0', '20.0 - 25.0', '25.0 - 30.0', '30.0 - 35.0', '35.0 - 40.0', '40.0 - 45.0', '45.0 - 50.0'], 'data': [372, 55, 21, 12, 10, 9, 5, 5, 5, 6]}, 'label': 'PayerId_stringindexer_transform', 'count': 10, 'min': 0, 'max': 50}], 'path_ManualSample': '/FileStore/platform/sampleData/60891770a187c956b337cf50_ManualFE/part-00000-tid-7570618811626959673-94990ba1-2b39-4e6c-ac98-d7d939c2d4f4-172005-1-c000.csv'}, 'selectedFE': [{'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '010ED4A3-EB64-4AE2-99CE-B458B59B538C', 'max': 'FFE0153F-9565-4FBB-9B1E-FBBA52624192', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'transformationsData': [{'feature_label': 'DateofService', 'transformation_label': 'Extract Date'}], 'feature': 'DateofService', 'type': 'date', 'selected': True, 'replaceby': 'random', 'stats': {'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': False, 'updatedLabel': 'DateofService'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Amount', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '357.38', 'stddev': '823.57', 'min': '45.0', 'max': '6688.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'Amount'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PatientBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '-1.54', 'stddev': '18.4', 'min': '-250.1', 'max': '123.41', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientBal'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'InsuranceBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.27', 'stddev': '3.94', 'min': '0.0', 'max': '76.68', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'InsuranceBal'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PrimaryInsuranceBal', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.15', 'stddev': '3.43', 'min': '0.0', 'max': '76.68', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryInsuranceBal'}, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '91232.81', 'stddev': '6100.5', 'min': '67028', 'max': 'J9035', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '5.5', 'stddev': '12.4', 'min': '0000', 'max': 'RT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'B58.01', 'max': 'T85.29XA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'C69.22', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'IsBillable', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '1.0', 'stddev': '0.0', 'min': '1.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'IsBillable'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Isauthorized', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'Isauthorized'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'PredeterminationIndicator', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'PredeterminationIndicator'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'MedNecessityRequiredIndicator', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0.0', 'max': '0.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'MedNecessityRequiredIndic...'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Age', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '67.76', 'stddev': '16.27', 'min': '5.0', 'max': '102.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'Age'}, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'label', 'type': 'real', 'selected': True, 'replaceby': 'mean', 'stats': {'count': '500', 'mean': '0.91', 'stddev': '0.29', 'min': '0.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'updatedLabel': 'label'}, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': True, 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '0375F08F-43D4-44BF-AB09-D3A109DAEAC2', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'transformationsData': [{'transformation_label': 'novalue'}], 'feature': 'Gender_Recoded', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': True, 'stats': {'count': '500', 'mean': '1.45', 'stddev': '0.5', 'min': '1', 'max': '2', 'missing': '0'}, 'updatedLabel': 'Gender_Recoded'}, {'feature': 'ChargeDetailID_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '249.5', 'stddev': '144.48', 'min': '0.0', 'max': '499.0', 'missing': '0'}, 'updatedLabel': 'ChargeDetailID_stringinde...'}, {'feature': 'DateofService_dayofmonth', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '14.58', 'stddev': '8.27', 'min': '1', 'max': '31', 'missing': '0'}, 'updatedLabel': 'DateofService_dayofmonth'}, {'feature': 'DateofService_month', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '11.44', 'stddev': '0.5', 'min': '11', 'max': '12', 'missing': '0'}, 'updatedLabel': 'DateofService_month'}, {'feature': 'DateofService_year', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'generated': True, 'selected': True, 'stats': {'count': '500', 'mean': '2020.0', 'stddev': '0.0', 'min': '2020', 'max': '2020', 'missing': '0'}, 'updatedLabel': 'DateofService_year'}, {'feature': 'CPTCode_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '2.18', 'stddev': '3.55', 'min': '0.0', 'max': '22.0', 'missing': '0'}, 'updatedLabel': 'CPTCode_stringindexer_tra...'}, {'feature': 'CptModifier_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '0.42', 'stddev': '0.98', 'min': '0.0', 'max': '6.0', 'missing': '0'}, 'updatedLabel': 'CptModifier_stringindexer...'}, {'feature': 'pdiagnosis1_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '33.73', 'stddev': '38.52', 'min': '0.0', 'max': '148.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis1_stringindexer...'}, {'feature': 'pdiagnosis2_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '35.52', 'stddev': '41.1', 'min': '0.0', 'max': '155.0', 'missing': '0'}, 'updatedLabel': 'pdiagnosis2_stringindexer...'}, {'feature': 'Gender_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '500', 'mean': '0.45', 'stddev': '0.5', 'min': '0', 'max': '1', 'missing': '0'}, 'updatedLabel': 'Gender_stringindexer_tran...'}, {'feature': 'DepartmentCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '0.91', 'stddev': '1.2', 'min': '0.0', 'max': '7.0', 'missing': '0'}, 'updatedLabel': 'DepartmentCategory_string...'}, {'feature': 'SpecialityCategory_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'numeric', 'selected': True, 'stats': {'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0'}, 'updatedLabel': 'SpecialityCategory_string...'}, {'feature': 'PayerId_stringindexer_transform', 'transformation': [{'transformation': 'novalue', 'selectedAsDefault': 0}], 'transformationsData': [{'transformation_label': 'novalue'}], 'type': 'real', 'selected': True, 'stats': {'count': '500', 'mean': '4.89', 'stddev': '9.31', 'min': '0.0', 'max': '50.0', 'missing': '0'}, 'updatedLabel': 'PayerId_stringindexer_tra...'}], 'autoFE_flag': True, 'source_attributes': {'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'Demodb2020', 'optionalDB': 'Demodb2020', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT * FROM\r\n(\r\n\t\t\tSELECT DISTINCT\r\n\t\t\tcd.ChargeDetailID, \r\n\t\t\tcd.DateofService, \r\n\t\t\tcd.Amount,\r\n\t\t\tcd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\n\t\t\tcptc.CPTCode,\r\n\t\t\tCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\t\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\t\tIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END), \r\n\t\t\tIsauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\t\t\tPredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\t\t\tMedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\t\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\t\t\td.DepartmentCategory,\r\n\t\t\ts.SpecialityCategory,\r\n\t\t\tlabel = 0 , \r\n\t\t\tt.PayerId\r\n\t\t\tfrom ChargeDetail cd\r\n\t\t\tinner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\n\t\t\tleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = e.PayerID  \r\n\t\t\tinner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\t\t\tinner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\n\t\t\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\t\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\t\tWHERE\r\n\t\t\tcd.DateOfService between '11/01/2020' and '12/31/2020' \r\n\t\t\tand cd.Amount >.01 and t.CloseDate is not null\r\n\t\t\tand trc.reasonType = 'Denial'  \r\n\t\t\tand tt.TransactionTypeCategory = 'Charges'\r\n\t\t\t\r\nUNION\r\n\r\nSELECT DISTINCT \r\ncd.ChargeDetailID, cd.DateOfService,  \r\ncd.Amount,\r\ncd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\ncptc.CPTCode,\r\nCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\ndc.DiagnosisCode as pdiagnosis1,\r\ndc2.DiagnosisCode as pdiagnosis2,\r\nIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END), \r\nIsauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END),   \r\nPredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\nMedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\nFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\nd.DepartmentCategory,\r\ns.SpecialityCategory,\r\nlabel = 1,\r\nt.PayerId\r\nfrom ChargeDetail cd\r\ninner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\nleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerID = e.PayerID\r\ninner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID  \r\ninner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\ninner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\ninner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\ninner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\ninner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\ninner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\ninner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\ninner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\ninner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\ninner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\nWHERE\r\ncd.DateOfService between '11/01/2020' and '12/31/2020' \r\nand cd.Amount >.01 and t.CloseDate is not null \r\nand trc.reasonType != 'Denial' \r\nand tt.TransactionTypeCategory = 'Charges' \r\n) as DND", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}, 'modeltrainingpercent': '100', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}, 'dbfsFileName': 'TestDatapipeline28Apr', 'directory_path': '/Surekha_DataPipe'}")
	#sinkPostExecutionHook(dbfssink)

except Exception as ex: 
	logging.error(ex)
